# Redbus Webscraping 

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime
import pytz

driver = webdriver.Chrome()

# 🔗 Choose one state (uncomment APSRTC to test for Andhra Pradesh)
state_links = [
    "https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/astc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile"
]

india = pytz.timezone("Asia/Kolkata")
today_dt = datetime.now(india)
date_of_scrape = today_dt.strftime("%d-%m-%Y")

all_buses_data = []

def safe_text(by, value, context):
    try:
        return context.find_element(by, value).text
    except:
        return None

def scroll_stepwise_private_buses():
    scraped_count = 0
    stable_rounds = 0

    while True:
        buses = driver.find_elements(By.CLASS_NAME, "tupleWrapper___aa6a16")
        current_count = len(buses)

        if current_count > scraped_count:
            scraped_count = current_count
            stable_rounds = 0
        else:
            stable_rounds += 1
            time.sleep(4)

        if stable_rounds >= 2:
            break

        if current_count > 0:
            try:
                last_bus = buses[min(current_count, len(buses)) - 1]
                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", last_bus)
            except:
                break

        if current_count % 4 == 0:
            time.sleep(4)
        else:
            time.sleep(2)

    return driver.find_elements(By.CLASS_NAME, "tupleWrapper___aa6a16")

def scroll_stepwise_government_buses():
    scraped_count = 0
    stable_rounds = 0

    while True:
        buses = driver.find_elements(By.CLASS_NAME, "tupleWrapper___aa6a16")
        current_count = len(buses)

        if current_count > scraped_count:
            scraped_count = current_count
            stable_rounds = 0
        else:
            stable_rounds += 1
            time.sleep(5)

        if stable_rounds >= 2:
            break

        if current_count > 0:
            try:
                last_bus = buses[min(current_count, len(buses)) - 1]
                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", last_bus)
            except:
                break

        if current_count % 4 == 0:
            time.sleep(5)
        else:
            time.sleep(3)

    return driver.find_elements(By.CLASS_NAME, "tupleWrapper___aa6a16")

for state_url in state_links:
    print(f"\n🔄 Scraping state page: {state_url}")
    driver.get(state_url)
    time.sleep(6)

    try:
        state_name = driver.find_element(By.CLASS_NAME, "D120_search_h1").text.strip()
    except:
        state_name = "Unknown Transport Corp"

    print(f"📍 State Transport Corp: {state_name}")

    routes = []
    while True:
        time.sleep(2)
        for r in driver.find_elements(By.CLASS_NAME, "route"):
            route_name = r.text
            route_link = r.get_attribute("href")
            if route_link:
                routes.append({"route_name": route_name, "route_link": route_link})

        try:
            pages = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs")
            current_active = driver.find_element(By.CLASS_NAME, "DC_117_pageActive")
            current_index = pages.index(current_active)

            if current_index + 1 < len(pages):
                next_page = pages[current_index + 1]
                driver.execute_script("arguments[0].click();", next_page)
            else:
                break
        except:
            break

    for route in routes:
        print(f"\n➡️ Scraping route: {route['route_name']}")
        driver.get(route["route_link"])
        time.sleep(6)

        private_buses = scroll_stepwise_private_buses()
        for bus in private_buses:
            try:
                busname = safe_text(By.CLASS_NAME, "travelsName___495898", bus)
                price = safe_text(By.CLASS_NAME, "finalFare___898bb7", bus)
                bustype = safe_text(By.CLASS_NAME, "busType___13ff4b", bus)
                departing_time = safe_text(By.CLASS_NAME, "boardingTime___aced27", bus)
                duration = safe_text(By.CLASS_NAME, "duration___5b44b1", bus)
                reaching_time = safe_text(By.CLASS_NAME, "droppingTime___616c2f", bus)
                star_rating = safe_text(By.CLASS_NAME, "rating___7724f1", bus)
                seats_available = safe_text(By.CLASS_NAME, "totalSeats___ba48cf", bus)

                all_buses_data.append({
                    "state": state_name,
                    "route": route["route_name"],
                    "busname": busname,
                    "price": price,
                    "bustype": bustype,
                    "departing_time": departing_time,
                    "duration": duration,
                    "reaching_time": reaching_time,
                    "star_rating": star_rating,
                    "seats_available": seats_available,
                    "type": "Private",
                    "date_of_scrape": date_of_scrape
                })
            except:
                continue

        try:
            tabs = driver.find_elements(By.CLASS_NAME, "rtcName___f9dd9e")
            for tab in tabs:
                gov_tab_text = tab.text.strip().lower()
                if "rtc" in gov_tab_text or "buses" in gov_tab_text:
                    print(f"🚌 Clicking Government tab: {tab.text}")
                    driver.execute_script("arguments[0].click();", tab)
                    time.sleep(4)

                    gov_buses = scroll_stepwise_government_buses()
                    for bus in gov_buses:
                        try:
                            busname = safe_text(By.CLASS_NAME, "travelsName___495898", bus)
                            price = safe_text(By.CLASS_NAME, "finalFare___898bb7", bus)
                            bustype = safe_text(By.CLASS_NAME, "busType___13ff4b", bus)
                            departing_time = safe_text(By.CLASS_NAME, "boardingTime___aced27", bus)
                            duration = safe_text(By.CLASS_NAME, "duration___5b44b1", bus)
                            reaching_time = safe_text(By.CLASS_NAME, "droppingTime___616c2f", bus)
                            star_rating = safe_text(By.CLASS_NAME, "rating___7724f1", bus)
                            seats_available = safe_text(By.CLASS_NAME, "totalSeats___ba48cf", bus)

                            all_buses_data.append({
                                "state": state_name,
                                "route": route["route_name"],
                                "busname": busname,
                                "price": price,
                                "bustype": bustype,
                                "departing_time": departing_time,
                                "duration": duration,
                                "reaching_time": reaching_time,
                                "star_rating": star_rating,
                                "seats_available": seats_available,
                                "type": "Government",
                                "date_of_scrape": date_of_scrape
                            })
                        except:
                            continue
                    break
        except:
            print("⚠️ Government tab not found")

print(f"\n✅ Total Buses Collected: {len(all_buses_data)}")
for row in all_buses_data:
    print(row)

driver.quit()


# Webscraping Output Data

In [ ]:
🔄 Scraping state page: https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile
📍 State Transport Corp: Kerala RTC Online Ticket Booking

➡️ Scraping route: Bangalore to Kozhikode
🚌 Clicking Government tab: KSRTC Kerala Buses

➡️ Scraping route: Kozhikode to Ernakulam
🚌 Clicking Government tab: KSRTC Kerala Buses

➡️ Scraping route: Kozhikode to Bangalore
🚌 Clicking Government tab: KSRTC Kerala Buses

➡️ Scraping route: Ernakulam to Kozhikode
🚌 Clicking Government tab: KSRTC Kerala Buses

➡️ Scraping route: Mysore to Kozhikode
🚌 Clicking Government tab: KSRTC Kerala Buses

➡️ Scraping route: Kozhikode to Mysore
🚌 Clicking Government tab: KSRTC Kerala Buses

➡️ Scraping route: Kalpetta (kerala) to Bangalore
🚌 Clicking Government tab: KSRTC Kerala Buses

➡️ Scraping route: Kozhikode to Thrissur
🚌 Clicking Government tab: KSRTC Kerala Buses

🔄 Scraping state page: https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile
📍 State Transport Corp: APSRTC

➡️ Scraping route: Hyderabad to Vijayawada
🚌 Clicking Government tab: APSRTC Buses

➡️ Scraping route: Vijayawada to Hyderabad
🚌 Clicking Government tab: APSRTC Buses

➡️ Scraping route: Chittoor (Andhra Pradesh) to Bangalore
🚌 Clicking Government tab: APSRTC Buses

➡️ Scraping route: Kakinada to Visakhapatnam
🚌 Clicking Government tab: APSRTC Buses

➡️ Scraping route: Bangalore to Tirupati
🚌 Clicking Government tab: APSRTC Buses

➡️ Scraping route: Ongole to Hyderabad
🚌 Clicking Government tab: APSRTC Buses

➡️ Scraping route: Hyderabad to Ongole
🚌 Clicking Government tab: APSRTC Buses

➡️ Scraping route: Bangalore to Kadapa
🚌 Clicking Government tab: APSRTC Buses

➡️ Scraping route: Madanapalli to Bangalore
🚌 Clicking Government tab: APSRTC Buses

➡️ Scraping route: Bangalore to Madanapalli
🚌 Clicking Government tab: APSRTC Buses

➡️ Scraping route: Kadapa to Bangalore
🚌 Clicking Government tab: APSRTC Buses

➡️ Scraping route: Bangalore to Chittoor (Andhra Pradesh)
🚌 Clicking Government tab: APSRTC Buses

➡️ Scraping route: Anantapur (andhra pradesh) to Bangalore
🚌 Clicking Government tab: APSRTC Buses

➡️ Scraping route: Bangalore to Kadiri
🚌 Clicking Government tab: APSRTC Buses

➡️ Scraping route: Hyderabad to Narasaraopet
🚌 Clicking Government tab: APSRTC Buses

➡️ Scraping route: Narasaraopet to Hyderabad
🚌 Clicking Government tab: APSRTC Buses

➡️ Scraping route: Bangalore to Anantapur (andhra pradesh)
🚌 Clicking Government tab: APSRTC Buses

➡️ Scraping route: Kadiri to Bangalore
🚌 Clicking Government tab: APSRTC Buses

➡️ Scraping route: Visakhapatnam to Kakinada
🚌 Clicking Government tab: APSRTC Buses

➡️ Scraping route: Tirupati to Bangalore
🚌 Clicking Government tab: APSRTC Buses

🔄 Scraping state page: https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile
📍 State Transport Corp: TSRTC Online Bus Ticket Booking

➡️ Scraping route: Khammam to Hyderabad
🚌 Clicking Government tab: TGSRTC Buses

➡️ Scraping route: Hyderabad to Srisailam

➡️ Scraping route: Hyderabad to Khammam
🚌 Clicking Government tab: TGSRTC Buses

➡️ Scraping route: Hyderabad to Vijayawada
🚌 Clicking Government tab: APSRTC Buses

➡️ Scraping route: Karimnagar to Hyderabad
🚌 Clicking Government tab: TGSRTC Buses

➡️ Scraping route: Hyderabad to Adilabad
🚌 Clicking Government tab: TGSRTC Buses

➡️ Scraping route: Hyderabad to Mancherial
🚌 Clicking Government tab: TGSRTC Buses

➡️ Scraping route: Kothagudem to Hyderabad
🚌 Clicking Government tab: TGSRTC Buses

➡️ Scraping route: Guntur (Andhra Pradesh) to Hyderabad
🚌 Clicking Government tab: APSRTC Buses

➡️ Scraping route: Hyderabad to Ongole
🚌 Clicking Government tab: APSRTC Buses

➡️ Scraping route: Hyderabad to Kothagudem
🚌 Clicking Government tab: TGSRTC Buses

🔄 Scraping state page: https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile
📍 State Transport Corp: Kadamba Transport Corporation Limited (KTCL)

➡️ Scraping route: Bangalore to Goa

➡️ Scraping route: Pune to Goa

➡️ Scraping route: Mumbai to Goa

➡️ Scraping route: Goa to Bangalore

➡️ Scraping route: Goa to Pune

➡️ Scraping route: Hyderabad to Goa

➡️ Scraping route: Goa to Mumbai

➡️ Scraping route: Goa to Hyderabad

➡️ Scraping route: Belagavi to Goa

➡️ Scraping route: Goa to Belagavi

➡️ Scraping route: Hyderabad to Belagavi

➡️ Scraping route: Pandharpur to Goa

➡️ Scraping route: Goa to Pandharpur

➡️ Scraping route: Belagavi to Marcel

➡️ Scraping route: Solapur to Goa

➡️ Scraping route: Marcel to Belagavi

➡️ Scraping route: Goa to Kolhapur(Maharashtra)

➡️ Scraping route: Calangute (goa) to Mopa Airport

🔄 Scraping state page: https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile
📍 State Transport Corp: RSRTC

➡️ Scraping route: Jaipur (Rajasthan) to Jodhpur
🚌 Clicking Government tab: RSRTC Buses

➡️ Scraping route: Jodhpur to Ajmer
🚌 Clicking Government tab: RSRTC Buses

➡️ Scraping route: Beawar (Rajasthan) to Jaipur (Rajasthan)
🚌 Clicking Government tab: RSRTC Buses

➡️ Scraping route: Udaipur to Jodhpur
🚌 Clicking Government tab: RSRTC Buses

➡️ Scraping route: Sikar to Jaipur (Rajasthan)
🚌 Clicking Government tab: RSRTC Buses

➡️ Scraping route: Aligarh (uttar pradesh) to Jaipur (Rajasthan)
🚌 Clicking Government tab: RSRTC Buses

➡️ Scraping route: Kishangarh (Rajasthan) to Jaipur (Rajasthan)
🚌 Clicking Government tab: RSRTC Buses

➡️ Scraping route: Jaipur (Rajasthan) to Aligarh (uttar pradesh)
🚌 Clicking Government tab: RSRTC Buses

➡️ Scraping route: Kota(Rajasthan) to Jaipur (Rajasthan)
🚌 Clicking Government tab: RSRTC Buses

🔄 Scraping state page: https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile
📍 State Transport Corp: South Bengal State Transport Corporation (SBSTC)

➡️ Scraping route: Durgapur (West Bengal) to Kolkata
🚌 Clicking Government tab: SBSTC Buses

➡️ Scraping route: Kolkata to Burdwan
🚌 Clicking Government tab: SBSTC Buses

➡️ Scraping route: Midnapore to Kolkata

➡️ Scraping route: Kolkata to Durgapur (West Bengal)
🚌 Clicking Government tab: SBSTC Buses

➡️ Scraping route: Haldia to Kolkata

➡️ Scraping route: Kolkata to Haldia
🚌 Clicking Government tab: SBSTC Buses

➡️ Scraping route: Kolkata to Midnapore
🚌 Clicking Government tab: SBSTC Buses

➡️ Scraping route: Kolkata to Arambagh (West Bengal)

➡️ Scraping route: Kolkata to Digha
🚌 Clicking Government tab: SBSTC Buses

➡️ Scraping route: Jhargram to Kolkata

➡️ Scraping route: Kolkata to Bankura

➡️ Scraping route: Digha to Kolkata

➡️ Scraping route: Kolkata to Asansol (West Bengal)

➡️ Scraping route: Kolkata to Contai (Kanthi)

➡️ Scraping route: Kolkata to Nandakumar (west bengal)

➡️ Scraping route: Digha to Durgapur (West Bengal)

🔄 Scraping state page: https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile
📍 State Transport Corp: HRTC

➡️ Scraping route: Hamirpur (Himachal Pradesh) to Chandigarh

➡️ Scraping route: Chandigarh to Hamirpur (Himachal Pradesh)

➡️ Scraping route: Delhi to Shimla
🚌 Clicking Government tab: HRTC Buses

➡️ Scraping route: Shimla to Delhi
🚌 Clicking Government tab: HRTC Buses

➡️ Scraping route: Chamba (Himachal Pradesh) to Chandigarh

➡️ Scraping route: Delhi to Chandigarh
🚌 Clicking Government tab: HRTC Buses

➡️ Scraping route: Ghumarwin to Chandigarh
🚌 Clicking Government tab: HRTC Buses

➡️ Scraping route: Delhi to Chamba (Himachal Pradesh)

➡️ Scraping route: Shimla to Chandigarh
🚌 Clicking Government tab: HRTC Buses

➡️ Scraping route: Hamirpur (Himachal Pradesh) to Delhi
🚌 Clicking Government tab: HRTC Buses

➡️ Scraping route: Chamba (Himachal Pradesh) to Delhi

➡️ Scraping route: Delhi to Hamirpur (Himachal Pradesh)
🚌 Clicking Government tab: HRTC Buses

➡️ Scraping route: Sunder Nagar (Himachal Pradesh) to Chandigarh
🚌 Clicking Government tab: HRTC Buses

➡️ Scraping route: Solan to Delhi
🚌 Clicking Government tab: HRTC Buses

➡️ Scraping route: Dharamshala (Himachal Pradesh) to Chandigarh
🚌 Clicking Government tab: HRTC Buses

🔄 Scraping state page: https://www.redbus.in/online-booking/astc/?utm_source=rtchometile
📍 State Transport Corp: Assam State Transport Corporation (ASTC)

➡️ Scraping route: Guwahati to Tezpur

➡️ Scraping route: Guwahati to Nagaon (Assam)

➡️ Scraping route: Dhekiajuli to Guwahati

➡️ Scraping route: Jorhat to North Lakhimpur

➡️ Scraping route: North Lakhimpur to Sibsagar (Assam)

➡️ Scraping route: Guwahati to Dhubri

➡️ Scraping route: Dhubri to Guwahati

➡️ Scraping route: Sibsagar (Assam) to North Lakhimpur

➡️ Scraping route: North Lakhimpur to Jorhat

➡️ Scraping route: Goalpara to Guwahati

➡️ Scraping route: Guwahati to Bilasipara

🔄 Scraping state page: https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile
📍 State Transport Corp: Uttar Pradesh State Road Transport Corporation UPSRTC

➡️ Scraping route: Bareilly (Uttar Pradesh) to Delhi
🚌 Clicking Government tab: UPSRTC Buses

➡️ Scraping route: Aligarh (uttar pradesh) to Delhi

➡️ Scraping route: Delhi to Bareilly (Uttar Pradesh)
🚌 Clicking Government tab: UPSRTC Buses

➡️ Scraping route: Delhi to Aligarh (uttar pradesh)
🚌 Clicking Government tab: UPSRTC Buses

➡️ Scraping route: Lucknow to Prayagraj(Uttar Pradesh)
🚌 Clicking Government tab: UPSRTC Buses

➡️ Scraping route: Badaun to Delhi

➡️ Scraping route: Delhi to Badaun

➡️ Scraping route: Lucknow to Agra
🚌 Clicking Government tab: UPSRTC Buses

➡️ Scraping route: Prayagraj(Uttar Pradesh) to Lucknow
🚌 Clicking Government tab: UPSRTC Buses

➡️ Scraping route: Lucknow to Bareilly (Uttar Pradesh)
🚌 Clicking Government tab: UPSRTC Buses

➡️ Scraping route: Delhi to Farrukhabad (Uttar Pradesh)
🚌 Clicking Government tab: UPSRTC Buses

➡️ Scraping route: Farrukhabad (Uttar Pradesh) to Delhi
🚌 Clicking Government tab: UPSRTC Buses

➡️ Scraping route: Agra to Lucknow
🚌 Clicking Government tab: UPSRTC Buses

➡️ Scraping route: Delhi to Sitapur (Uttar Pradesh)
🚌 Clicking Government tab: UPSRTC Buses

➡️ Scraping route: Lucknow to Mathura
🚌 Clicking Government tab: UPSRTC Buses

➡️ Scraping route: Gorakhpur (uttar pradesh) to Lucknow
🚌 Clicking Government tab: UPSRTC Buses

➡️ Scraping route: Lucknow to Varanasi
🚌 Clicking Government tab: UPSRTC Buses

➡️ Scraping route: Lucknow to Aligarh (uttar pradesh)
🚌 Clicking Government tab: UPSRTC Buses

➡️ Scraping route: Shahjahanpur (Uttar Pradesh) to Delhi
🚌 Clicking Government tab: UPSRTC Buses

➡️ Scraping route: Aligarh (uttar pradesh) to Greater noida (uttar pradesh)

🔄 Scraping state page: https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile
📍 State Transport Corp: WBTC (CTC)

➡️ Scraping route: Kolkata to Digha
🚌 Clicking Government tab: SBSTC Buses

➡️ Scraping route: Digha to Kolkata

➡️ Scraping route: Digha to Barasat (West Bengal)

➡️ Scraping route: Barasat (West Bengal) to Digha

➡️ Scraping route: Midnapore to Kolkata

➡️ Scraping route: Durgapur (West Bengal) to Kolkata

➡️ Scraping route: Barasat (West Bengal) to Midnapore

➡️ Scraping route: Midnapore to Barasat (West Bengal)

➡️ Scraping route: Kolkata to Durgapur (West Bengal)
🚌 Clicking Government tab: SBSTC Buses

➡️ Scraping route: Barasat (West Bengal) to Kolaghat

➡️ Scraping route: Barasat (West Bengal) to Nandakumar (west bengal)

➡️ Scraping route: Barasat (West Bengal) to Contai (Kanthi)

➡️ Scraping route: Barasat (West Bengal) to Chowrangee(Kharagpur)

➡️ Scraping route: Kolkata to Suri

➡️ Scraping route: Habra to Digha

➡️ Scraping route: Chowrangee(Kharagpur) to Barasat (West Bengal)

➡️ Scraping route: Kolkata to Mayapur ISKCON

✅ Total Buses Collected: 1667
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Bangalore to Kozhikode', 'busname': 'MMK Travels', 'price': '₹700', 'bustype': 'A/C Semi Sleeper / Sleeper (2+1)', 'departing_time': '22:30', 'duration': '9h', 'reaching_time': '07:30', 'star_rating': '4.3', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Bangalore to Kozhikode', 'busname': 'SKS Tours & Travels', 'price': '₹899', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '21:15', 'duration': '8h 30m', 'reaching_time': '05:45', 'star_rating': '4.3', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Bangalore to Kozhikode', 'busname': 'NS Transports', 'price': '₹854', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '22:45', 'duration': '8h 30m', 'reaching_time': '07:15', 'star_rating': '4.2', 'seats_available': '7 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Bangalore to Kozhikode', 'busname': 'Highline Transports', 'price': '₹889', 'bustype': 'VE A/C Sleeper (2+1)', 'departing_time': '21:25', 'duration': '9h', 'reaching_time': '06:25', 'star_rating': '4.2', 'seats_available': '13 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Bangalore to Kozhikode', 'busname': 'DLT BUS', 'price': '₹999', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '22:30', 'duration': '9h', 'reaching_time': '07:30', 'star_rating': '4.1', 'seats_available': '9 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Bangalore to Kozhikode', 'busname': 'Kyros Connect', 'price': '₹799', 'bustype': 'Bharat Benz A/C Semi Sleeper (2+2)', 'departing_time': '23:00', 'duration': '10h', 'reaching_time': '09:00', 'star_rating': '4.1', 'seats_available': '30 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Bangalore to Kozhikode', 'busname': 'SKS Tourist Corporation (REGD)', 'price': '₹599', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '21:30', 'duration': '8h 30m', 'reaching_time': '06:00', 'star_rating': '4.1', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Bangalore to Kozhikode', 'busname': 'SAAM BUS', 'price': '₹800', 'bustype': 'VE A/C Sleeper (2+1)', 'departing_time': '21:45', 'duration': '9h', 'reaching_time': '06:45', 'star_rating': '3.8', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Bangalore to Kozhikode', 'busname': 'DA Transports', 'price': '₹719', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '18:55', 'duration': '12h', 'reaching_time': '06:55', 'star_rating': '3.8', 'seats_available': '21 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Bangalore to Kozhikode', 'busname': 'NS Transports', 'price': '₹629', 'bustype': 'NON A/C Semi Sleeper / Sleeper (2+1)', 'departing_time': '22:15', 'duration': '8h 15m', 'reaching_time': '06:30', 'star_rating': '3.7', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Bangalore to Kozhikode', 'busname': 'National travels and logistics', 'price': '₹749', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '22:30', 'duration': '8h 45m', 'reaching_time': '07:15', 'star_rating': '3.6', 'seats_available': '14 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Bangalore to Kozhikode', 'busname': 'SAAM BUS', 'price': '₹650', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '21:45', 'duration': '8h 20m', 'reaching_time': '06:05', 'star_rating': '3.6', 'seats_available': '13 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Bangalore to Kozhikode', 'busname': 'NS Transports', 'price': '₹629', 'bustype': 'NON A/C Semi Sleeper / Sleeper (2+1)', 'departing_time': '21:45', 'duration': '8h 25m', 'reaching_time': '06:10', 'star_rating': '3.5', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Bangalore to Kozhikode', 'busname': 'A1 Travels', 'price': '₹500', 'bustype': 'NON A/C Seater / Sleeper (3+1)', 'departing_time': '21:00', 'duration': '9h 45m', 'reaching_time': '06:45', 'star_rating': '3.5', 'seats_available': '34 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Bangalore to Kozhikode', 'busname': 'P K Travels', 'price': '₹900', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '22:15', 'duration': '8h 30m', 'reaching_time': '06:45', 'star_rating': '4.2', 'seats_available': '13 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Bangalore to Kozhikode', 'busname': 'KSRTC (Kerala) - 553', 'price': '₹762', 'bustype': 'Super Express Non AC Seater Air Bus (2+2)', 'departing_time': '16:45', 'duration': '8h 55m', 'reaching_time': '01:40', 'star_rating': '3.2', 'seats_available': '6 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Bangalore to Kozhikode', 'busname': 'KSRTC (Kerala) - 789', 'price': '₹474', 'bustype': 'Super Fast Non AC Seater (2+3)', 'departing_time': '20:00', 'duration': '7h', 'reaching_time': '03:00', 'star_rating': '3.6', 'seats_available': '6 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Bangalore to Kozhikode', 'busname': 'KSRTC (Kerala) - 1585', 'price': '₹762', 'bustype': 'Swift Deluxe Non AC Air Bus (2+2)', 'departing_time': '20:40', 'duration': '9h 5m', 'reaching_time': '05:45', 'star_rating': '4.2', 'seats_available': '25 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Bangalore to Kozhikode', 'busname': 'KSRTC (Kerala) - 1586', 'price': '₹762', 'bustype': 'Swift Deluxe Non AC Air Bus (2+2)', 'departing_time': '21:30', 'duration': '8h 55m', 'reaching_time': '06:25', 'star_rating': '3.3', 'seats_available': '30 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Bangalore to Kozhikode', 'busname': 'KSRTC (Kerala) - 2133', 'price': '₹762', 'bustype': 'Swift Deluxe Non AC Air Bus (2+2)', 'departing_time': '22:10', 'duration': '9h 10m', 'reaching_time': '07:20', 'star_rating': '3.7', 'seats_available': '23 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Bangalore to Kozhikode', 'busname': 'KSRTC (Kerala) - 54', 'price': '₹914', 'bustype': 'Garuda Premium', 'departing_time': '22:25', 'duration': '6h 55m', 'reaching_time': '05:20', 'star_rating': '4.4', 'seats_available': '12 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Bangalore to Kozhikode', 'busname': 'KSRTC (Kerala) - 574', 'price': '₹677', 'bustype': 'Super Express Non AC Seater Air Bus (2+2)', 'departing_time': '23:00', 'duration': '10h 35m', 'reaching_time': '09:35', 'star_rating': '3.7', 'seats_available': '30 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Bangalore to Kozhikode', 'busname': 'KSRTC (Kerala) - 583', 'price': '₹677', 'bustype': 'Swift Deluxe Non AC Air Bus (2+2)', 'departing_time': '23:30', 'duration': '10h 15m', 'reaching_time': '09:45', 'star_rating': '4.2', 'seats_available': '25 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Bangalore to Kozhikode', 'busname': 'KSRTC (Kerala) - 56', 'price': '₹863', 'bustype': 'SWIFT-GARUDA A/C SEATER BUS', 'departing_time': '23:40', 'duration': '10h 10m', 'reaching_time': '09:50', 'star_rating': '4.2', 'seats_available': '32 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Ernakulam', 'busname': 'A1 Travels', 'price': '₹650', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '20:10', 'duration': '3h 54m', 'reaching_time': '00:04', 'star_rating': '3.3', 'seats_available': '28 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Ernakulam', 'busname': 'A1 Travels', 'price': '₹550', 'bustype': 'NON A/C Semi Sleeper (2+2)', 'departing_time': '21:00', 'duration': '4h 30m', 'reaching_time': '01:30', 'star_rating': '4', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Ernakulam', 'busname': 'EMERALD TRAVELS', 'price': '₹800', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '22:10', 'duration': '4h 30m', 'reaching_time': '02:40', 'star_rating': '3', 'seats_available': '28 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Ernakulam', 'busname': 'EMERALD TRAVELS', 'price': '₹800', 'bustype': 'Bharat Benz A/C Semi Sleeper (2+2)', 'departing_time': '22:10', 'duration': '4h 30m', 'reaching_time': '02:40', 'star_rating': '3', 'seats_available': '30 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Ernakulam', 'busname': 'A1 Travels', 'price': '₹750', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '22:15', 'duration': '4h 30m', 'reaching_time': '02:45', 'star_rating': '3', 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Ernakulam', 'busname': 'A1 Travels', 'price': '₹700', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '22:30', 'duration': '4h', 'reaching_time': '02:30', 'star_rating': '3.4', 'seats_available': '29 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Ernakulam', 'busname': 'CITI TOURS', 'price': '₹600', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '22:45', 'duration': '4h 15m', 'reaching_time': '03:00', 'star_rating': '3.3', 'seats_available': '27 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Ernakulam', 'busname': 'Achayans Holidays', 'price': '₹560', 'bustype': 'VE A/C Semi Sleeper (2+2)', 'departing_time': '23:00', 'duration': '4h 50m', 'reaching_time': '03:50', 'star_rating': '4', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Ernakulam', 'busname': 'LIYAS TOURS', 'price': '₹949', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '23:15', 'duration': '4h', 'reaching_time': '03:15', 'star_rating': '3.8', 'seats_available': '3 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Ernakulam', 'busname': 'Sonia Travels', 'price': '₹680', 'bustype': 'A/C Seater/Sleeper (2+1)', 'departing_time': '23:30', 'duration': '4h 59m', 'reaching_time': '04:29', 'star_rating': '4', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Ernakulam', 'busname': 'KSRTC (Kerala) - 4161', 'price': '₹230', 'bustype': 'Super Fast Non AC Seater (2+3)', 'departing_time': '16:45', 'duration': '4h 54m', 'reaching_time': '21:39', 'star_rating': '3.2', 'seats_available': '29 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Ernakulam', 'busname': 'KSRTC (Kerala) - 2900', 'price': '₹383', 'bustype': 'Low Floor AC Seater 2+2', 'departing_time': '18:00', 'duration': '6h 15m', 'reaching_time': '00:15', 'star_rating': '2.4', 'seats_available': '5 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Ernakulam', 'busname': 'KSRTC (Kerala) - 1872', 'price': '₹383', 'bustype': 'Low Floor AC Seater 2+2', 'departing_time': '18:30', 'duration': '5h 14m', 'reaching_time': '23:44', 'star_rating': '3.8', 'seats_available': '3 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Ernakulam', 'busname': 'KSRTC (Kerala) - 2453', 'price': '₹383', 'bustype': 'Low Floor AC Seater 2+2', 'departing_time': '19:00', 'duration': '4h 55m', 'reaching_time': '23:55', 'star_rating': '3.4', 'seats_available': '14 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Ernakulam', 'busname': 'KSRTC (Kerala) - 3991', 'price': '₹309', 'bustype': 'Swift Deluxe Non AC Air Bus (2+2)', 'departing_time': '19:40', 'duration': '4h 45m', 'reaching_time': '00:25', 'star_rating': '2.5', 'seats_available': '5 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Ernakulam', 'busname': 'KSRTC (Kerala) - 2393', 'price': '₹383', 'bustype': 'Low Floor AC Seater 2+2', 'departing_time': '20:30', 'duration': '5h 14m', 'reaching_time': '01:44', 'star_rating': '3.9', 'seats_available': '7 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Ernakulam', 'busname': 'KSRTC (Kerala) - 3981', 'price': '₹309', 'bustype': 'Swift Deluxe Non AC Air Bus (2+2)', 'departing_time': '20:40', 'duration': '4h 50m', 'reaching_time': '01:30', 'star_rating': '3.2', 'seats_available': '5 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Ernakulam', 'busname': 'KSRTC (Kerala) - 827', 'price': '₹269', 'bustype': 'Super Fast Non AC Seater (2+3)', 'departing_time': '21:05', 'duration': '4h 55m', 'reaching_time': '02:00', 'star_rating': '4.4', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Ernakulam', 'busname': 'KSRTC (Kerala) - 3901', 'price': '₹255', 'bustype': 'Super Fast Non AC Seater (2+3)', 'departing_time': '21:30', 'duration': '5h 29m', 'reaching_time': '02:59', 'star_rating': '3.7', 'seats_available': '39 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Ernakulam', 'busname': 'KSRTC (Kerala) - 2454', 'price': '₹383', 'bustype': 'Low Floor AC Seater 2+2', 'departing_time': '21:40', 'duration': '4h 20m', 'reaching_time': '02:00', 'star_rating': '3.5', 'seats_available': '21 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Bangalore', 'busname': 'ESTEEM TOURS & TRAVELS', 'price': '₹700', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '20:05', 'duration': '9h 55m', 'reaching_time': '06:00', 'star_rating': '4.4', 'seats_available': '5 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Bangalore', 'busname': 'DLT BUS', 'price': '₹900', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '21:45', 'duration': '8h 45m', 'reaching_time': '06:30', 'star_rating': '4.3', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Bangalore', 'busname': 'MMK Travels', 'price': '₹700', 'bustype': 'Bharat Benz A/C Seater / Sleeper (2+2)', 'departing_time': '21:30', 'duration': '8h 30m', 'reaching_time': '06:00', 'star_rating': '4.2', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Bangalore', 'busname': 'SKS Tours & Travels', 'price': '₹899', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '21:00', 'duration': '9h', 'reaching_time': '06:00', 'star_rating': '4.3', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Bangalore', 'busname': 'SKS Tourist Corporation (REGD)', 'price': '₹599', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '20:30', 'duration': '9h', 'reaching_time': '05:30', 'star_rating': '4.3', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Bangalore', 'busname': 'NS Transports', 'price': '₹809', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '20:15', 'duration': '9h 15m', 'reaching_time': '05:30', 'star_rating': '4.2', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Bangalore', 'busname': 'Highline Transports', 'price': '₹889', 'bustype': 'VE A/C Sleeper (2+1)', 'departing_time': '21:00', 'duration': '9h 45m', 'reaching_time': '06:45', 'star_rating': '4', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Bangalore', 'busname': 'NS Transports', 'price': '₹629', 'bustype': 'NON A/C Semi Sleeper / Sleeper (2+1)', 'departing_time': '19:30', 'duration': '9h', 'reaching_time': '04:30', 'star_rating': '4', 'seats_available': '7 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Bangalore', 'busname': 'NS Transports', 'price': '₹674', 'bustype': 'NON A/C Semi Sleeper / Sleeper (2+1)', 'departing_time': '20:30', 'duration': '8h 50m', 'reaching_time': '05:20', 'star_rating': '3.9', 'seats_available': '7 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Bangalore', 'busname': 'National travels and logistics', 'price': '₹579', 'bustype': 'NON A/C Sleeper (2+1)', 'departing_time': '20:45', 'duration': '9h 30m', 'reaching_time': '06:15', 'star_rating': '3.9', 'seats_available': '8 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Bangalore', 'busname': 'KSRTC (Kerala) - 578', 'price': '₹762', 'bustype': 'Swift Deluxe Non AC Air Bus (2+2)', 'departing_time': '18:00', 'duration': '9h 5m', 'reaching_time': '03:05', 'star_rating': '3.9', 'seats_available': '17 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Bangalore', 'busname': 'KSRTC (Kerala) - 72', 'price': '₹976', 'bustype': 'SWIFT-GARUDA A/C SEATER BUS', 'departing_time': '19:05', 'duration': '9h 20m', 'reaching_time': '04:25', 'star_rating': '3.8', 'seats_available': '29 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Bangalore', 'busname': 'KSRTC (Kerala) - 1583', 'price': '₹762', 'bustype': 'Swift Deluxe Non AC Air Bus (2+2)', 'departing_time': '20:00', 'duration': '8h 50m', 'reaching_time': '04:50', 'star_rating': '3.2', 'seats_available': '22 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Bangalore', 'busname': 'KSRTC (Kerala) - 234', 'price': '₹742', 'bustype': 'Swift Deluxe Non AC Air Bus (2+2)', 'departing_time': '21:00', 'duration': '9h 10m', 'reaching_time': '06:10', 'star_rating': '3.5', 'seats_available': '22 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Bangalore', 'busname': 'KSRTC (Kerala) - 1584', 'price': '₹762', 'bustype': 'Swift Deluxe Non AC Air Bus (2+2)', 'departing_time': '21:30', 'duration': '8h 45m', 'reaching_time': '06:15', 'star_rating': '3.7', 'seats_available': '24 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Bangalore', 'busname': 'KSRTC (Kerala) - 50', 'price': '₹762', 'bustype': 'Swift Deluxe Non AC Air Bus (2+2)', 'departing_time': '22:00', 'duration': '8h 55m', 'reaching_time': '06:55', 'star_rating': '4.1', 'seats_available': '14 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'Sheraton Tours and Travels', 'price': '₹600', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '20:00', 'duration': '5h 45m', 'reaching_time': '01:45', 'star_rating': '3.3', 'seats_available': '4 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'Adrin Travels', 'price': '₹900', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '20:00', 'duration': '4h 55m', 'reaching_time': '00:55', 'star_rating': '3.4', 'seats_available': '23 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'Crownline Travels', 'price': '₹699', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '20:45', 'duration': '5h', 'reaching_time': '01:45', 'star_rating': '4.1', 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'Kalpaka Travels 1', 'price': '₹700', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '20:45', 'duration': '5h 45m', 'reaching_time': '02:30', 'star_rating': '4', 'seats_available': '8 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'Adrin Travels', 'price': '₹900', 'bustype': 'A/C Seater/Sleeper (2+1)', 'departing_time': '21:00', 'duration': '5h', 'reaching_time': '02:00', 'star_rating': '3.4', 'seats_available': '24 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'Cochin Bus', 'price': '₹730', 'bustype': 'Volvo Multi-Axle B9R A/c Semi Sleeper (2+2)', 'departing_time': '21:15', 'duration': '5h 15m', 'reaching_time': '02:30', 'star_rating': '4.8', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'UFO Bus', 'price': '₹799', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '21:15', 'duration': '5h 30m', 'reaching_time': '02:45', 'star_rating': '3.5', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'Global Transport', 'price': '₹499', 'bustype': 'A/C Semi Sleeper (2+2)', 'departing_time': '21:15', 'duration': '5h 31m', 'reaching_time': '02:46', 'star_rating': '4.7', 'seats_available': '28 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'Madhavi Travels', 'price': '₹549', 'bustype': 'A/C Semi Sleeper (2+2)', 'departing_time': '21:30', 'duration': '4h 45m', 'reaching_time': '02:15', 'star_rating': '4', 'seats_available': '9 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'Cochin Bus', 'price': '₹730', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '21:30', 'duration': '5h 15m', 'reaching_time': '02:45', 'star_rating': '4.8', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'Cochin Bus', 'price': '₹900', 'bustype': 'Volvo Multi Axle B9R A/C Sleeper (2+1)', 'departing_time': '21:30', 'duration': '5h', 'reaching_time': '02:30', 'star_rating': '4.8', 'seats_available': '3 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'Cochin Bus', 'price': '₹900', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '21:30', 'duration': '5h', 'reaching_time': '02:30', 'star_rating': '4.8', 'seats_available': '4 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'UFO Bus', 'price': '₹649', 'bustype': 'Volvo Multi-Axle B9R A/c Semi Sleeper (2+2)', 'departing_time': '21:30', 'duration': '5h 15m', 'reaching_time': '02:45', 'star_rating': '4.3', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'SONIA LUXURY LINES', 'price': '₹730', 'bustype': 'VE A/C Seater / Sleeper (2+1)', 'departing_time': '21:35', 'duration': '4h 35m', 'reaching_time': '02:10', 'star_rating': '4.5', 'seats_available': '17 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'Achayans Holidays', 'price': '₹560', 'bustype': 'VE A/C Semi Sleeper (2+2)', 'departing_time': '21:35', 'duration': '4h 40m', 'reaching_time': '02:15', 'star_rating': '4', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'Madhavi Travels', 'price': '₹799', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '21:40', 'duration': '4h 15m', 'reaching_time': '01:55', 'star_rating': '4.5', 'seats_available': '7 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'Madhavi Travels', 'price': '₹599', 'bustype': 'A/C Semi Sleeper (2+2)', 'departing_time': '21:40', 'duration': '5h 55m', 'reaching_time': '03:35', 'star_rating': '3.8', 'seats_available': '8 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'SG Travels', 'price': '₹789', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '21:45', 'duration': '4h 45m', 'reaching_time': '02:30', 'star_rating': '4.8', 'seats_available': '5 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'Arya Travels', 'price': '₹695', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '21:45', 'duration': '5h 30m', 'reaching_time': '03:15', 'star_rating': '3.6', 'seats_available': '17 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'Madhavi Travels', 'price': '₹549', 'bustype': 'A/C Semi Sleeper (2+2)', 'departing_time': '21:50', 'duration': '5h 45m', 'reaching_time': '03:35', 'star_rating': '3.9', 'seats_available': '13 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'Madhavi Travels', 'price': '₹849', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '22:00', 'duration': '5h', 'reaching_time': '03:00', 'star_rating': '3.1', 'seats_available': '8 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'Kalpaka Travels 1', 'price': '₹549', 'bustype': 'A/C Semi Sleeper (2+2)', 'departing_time': '22:00', 'duration': '5h', 'reaching_time': '03:00', 'star_rating': '4.3', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'New Friends Travels', 'price': '₹929', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '23:57', 'duration': '4h 53m', 'reaching_time': '04:50', 'star_rating': '3.8', 'seats_available': '19 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'A1 Travels', 'price': '₹800', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '23:59', 'duration': '5h 31m', 'reaching_time': '05:30', 'star_rating': '3.7', 'seats_available': '21 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'Shajees Motors', 'price': '₹495', 'bustype': 'A/C Semi Sleeper (2+2)', 'departing_time': '19:05', 'duration': '6h 10m', 'reaching_time': '01:15', 'star_rating': '2.9', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'Holy Maria Travels', 'price': '₹549', 'bustype': 'A/C Semi Sleeper (2+2)', 'departing_time': '20:50', 'duration': '5h 30m', 'reaching_time': '02:20', 'star_rating': '2.1', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'Ubc travels', 'price': '₹900', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '21:00', 'duration': '4h 45m', 'reaching_time': '01:45', 'star_rating': '2.5', 'seats_available': '41 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'Sonia Travels', 'price': '₹550', 'bustype': 'A/C Semi Sleeper (2+2)', 'departing_time': '22:00', 'duration': '4h 45m', 'reaching_time': '02:45', 'star_rating': '2.8', 'seats_available': '27 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'Sonia Travels', 'price': '₹680', 'bustype': 'A/C Seater/Sleeper (2+1)', 'departing_time': '22:00', 'duration': '4h 29m', 'reaching_time': '02:29', 'star_rating': '2.8', 'seats_available': '19 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'A1 Travels', 'price': '₹700', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '23:59', 'duration': '5h 1m', 'reaching_time': '05:00', 'star_rating': '2.2', 'seats_available': '13 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'KSRTC (Kerala) - 2819', 'price': '₹269', 'bustype': 'Super Fast Non AC Seater (2+3)', 'departing_time': '16:50', 'duration': '5h 50m', 'reaching_time': '22:40', 'star_rating': '1.6', 'seats_available': '40 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'KSRTC (Kerala) - 2572', 'price': '₹261', 'bustype': 'Fast Passenger Non AC Seater (2+3)', 'departing_time': '17:10', 'duration': '6h 40m', 'reaching_time': '23:50', 'star_rating': '2.8', 'seats_available': '33 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'KSRTC (Kerala) - 723', 'price': '₹269', 'bustype': 'Super Fast Non AC Seater (2+3)', 'departing_time': '17:40', 'duration': '5h 25m', 'reaching_time': '23:05', 'star_rating': '3.7', 'seats_available': '40 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'KSRTC (Kerala) - 2569', 'price': '₹261', 'bustype': 'Fast Passenger Non AC Seater (2+3)', 'departing_time': '18:00', 'duration': '6h 35m', 'reaching_time': '00:35', 'star_rating': '1.8', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'KSRTC (Kerala) - 2517', 'price': '₹309', 'bustype': 'Swift Deluxe Non AC Air Bus (2+2)', 'departing_time': '18:36', 'duration': '4h 40m', 'reaching_time': '23:16', 'star_rating': '2.4', 'seats_available': '4 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'KSRTC (Kerala) - 3900', 'price': '₹255', 'bustype': 'Super Fast Non AC Seater (2+3)', 'departing_time': '19:00', 'duration': '5h 50m', 'reaching_time': '00:50', 'star_rating': '2.8', 'seats_available': '30 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'KSRTC (Kerala) - 915', 'price': '₹261', 'bustype': 'Super Fast Non AC Seater (2+3)', 'departing_time': '21:35', 'duration': '5h 35m', 'reaching_time': '03:10', 'star_rating': '3.6', 'seats_available': '22 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'KSRTC (Kerala) - 352', 'price': '₹269', 'bustype': 'Super Fast Non AC Seater (2+3)', 'departing_time': '21:45', 'duration': '5h 10m', 'reaching_time': '02:55', 'star_rating': '3', 'seats_available': '27 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'KSRTC (Kerala) - 1028', 'price': '₹269', 'bustype': 'Super Fast Non AC Seater (2+3)', 'departing_time': '22:05', 'duration': '4h 55m', 'reaching_time': '03:00', 'star_rating': '2.8', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Ernakulam to Kozhikode', 'busname': 'KSRTC (Kerala) - 4036', 'price': '₹524', 'bustype': 'AC MULTI AXLE', 'departing_time': '22:25', 'duration': '4h 25m', 'reaching_time': '02:50', 'star_rating': '2.8', 'seats_available': '20 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Mysore to Kozhikode', 'busname': 'LIYAS TOURS', 'price': '₹649', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '17:25', 'duration': '5h 50m', 'reaching_time': '23:15', 'star_rating': '4.7', 'seats_available': '5 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Mysore to Kozhikode', 'busname': 'Cochin Bus', 'price': '₹600', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '18:00', 'duration': '5h 55m', 'reaching_time': '23:55', 'star_rating': '4.9', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Mysore to Kozhikode', 'busname': 'Suraj Holidays', 'price': '₹649', 'bustype': 'NON A/C Seater/ Sleeper (2+1)', 'departing_time': '23:00', 'duration': '7h', 'reaching_time': '06:00', 'star_rating': '3.8', 'seats_available': '21 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Mysore to Kozhikode', 'busname': 'A1 Travels', 'price': '₹500', 'bustype': 'NON A/C Seater / Sleeper (3+1)', 'departing_time': '23:40', 'duration': '7h 5m', 'reaching_time': '06:45', 'star_rating': '4.5', 'seats_available': '38 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Mysore to Kozhikode', 'busname': 'SKS Tourist Corporation (REGD)', 'price': '₹599', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '23:55', 'duration': '6h 5m', 'reaching_time': '06:00', 'star_rating': '3.8', 'seats_available': '21 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Mysore to Kozhikode', 'busname': 'NS Transports', 'price': '₹629', 'bustype': 'NON A/C Semi Sleeper / Sleeper (2+1)', 'departing_time': '23:57', 'duration': '6h 33m', 'reaching_time': '06:30', 'star_rating': '4.2', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Mysore to Kozhikode', 'busname': 'ESTEEM TOURS & TRAVELS', 'price': '₹750', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '23:59', 'duration': '6h 56m', 'reaching_time': '06:55', 'star_rating': '5', 'seats_available': '25 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Mysore to Kozhikode', 'busname': 'NS Transports', 'price': '₹629', 'bustype': 'NON A/C Semi Sleeper / Sleeper (2+1)', 'departing_time': '23:59', 'duration': '6h 11m', 'reaching_time': '06:10', 'star_rating': '4.2', 'seats_available': '17 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Mysore to Kozhikode', 'busname': 'Galaxy Tours', 'price': '₹600', 'bustype': 'A/C Seater/Sleeper (2+1)', 'departing_time': '01:00', 'duration': '5h 30m', 'reaching_time': '06:30', 'star_rating': '3', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Mysore to Kozhikode', 'busname': 'Go Link', 'price': '₹650', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '01:30', 'duration': '6h', 'reaching_time': '07:30', 'star_rating': '3.7', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Mysore to Kozhikode', 'busname': 'KSRTC (Kerala) - 3771', 'price': '₹293', 'bustype': 'Super Fast Non AC Seater (2+3)', 'departing_time': '16:45', 'duration': '5h 45m', 'reaching_time': '22:30', 'star_rating': '3.8', 'seats_available': '41 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Mysore to Kozhikode', 'busname': 'KSRTC (Kerala) - 798', 'price': '₹293', 'bustype': 'Super Fast Non AC Seater (2+3)', 'departing_time': '17:00', 'duration': '6h', 'reaching_time': '23:00', 'star_rating': '1.9', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Mysore to Kozhikode', 'busname': 'KSRTC (Kerala) - 3772', 'price': '₹293', 'bustype': 'Super Fast Non AC Seater (2+3)', 'departing_time': '18:30', 'duration': '5h 45m', 'reaching_time': '00:15', 'star_rating': '3.1', 'seats_available': '30 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Mysore to Kozhikode', 'busname': 'KSRTC (Kerala) - 380', 'price': '₹390', 'bustype': 'Swift Deluxe Non AC Air Bus (2+2)', 'departing_time': '18:35', 'duration': '5h 45m', 'reaching_time': '00:20', 'star_rating': '3.8', 'seats_available': '2 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Mysore to Kozhikode', 'busname': 'KSRTC (Kerala) - 789', 'price': '₹474', 'bustype': 'Super Fast Non AC Seater (2+3)', 'departing_time': '22:20', 'duration': '4h 40m', 'reaching_time': '03:00', 'star_rating': '3.8', 'seats_available': '7 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Mysore to Kozhikode', 'busname': 'KSRTC (Kerala) - 1077', 'price': '₹348', 'bustype': 'Super Fast Non AC Seater (2+3)', 'departing_time': '22:30', 'duration': '7h 20m', 'reaching_time': '05:50', 'star_rating': '3.7', 'seats_available': '33 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Mysore', 'busname': 'Bharathi Travels', 'price': '₹550', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '15:40', 'duration': '5h 20m', 'reaching_time': '21:00', 'star_rating': '3.5', 'seats_available': '17 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Mysore', 'busname': 'NS Transports', 'price': '₹629', 'bustype': 'NON A/C Semi Sleeper / Sleeper (2+1)', 'departing_time': '19:30', 'duration': '7h', 'reaching_time': '02:30', 'star_rating': '4.2', 'seats_available': '7 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Mysore', 'busname': 'ESTEEM TOURS & TRAVELS', 'price': '₹700', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '20:05', 'duration': '7h 35m', 'reaching_time': '03:40', 'star_rating': '4.4', 'seats_available': '5 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Mysore', 'busname': 'NS Transports', 'price': '₹809', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '20:15', 'duration': '6h 25m', 'reaching_time': '02:40', 'star_rating': '4.2', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Mysore', 'busname': 'SAAM BUS', 'price': '₹800', 'bustype': 'VE A/C Sleeper (2+1)', 'departing_time': '20:30', 'duration': '7h 30m', 'reaching_time': '04:00', 'star_rating': '3.6', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Mysore', 'busname': 'Suraj Holidays', 'price': '₹599', 'bustype': 'NON A/C Seater/ Sleeper (2+1)', 'departing_time': '20:30', 'duration': '6h', 'reaching_time': '02:30', 'star_rating': '4.4', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Mysore', 'busname': 'NS Transports', 'price': '₹674', 'bustype': 'NON A/C Semi Sleeper / Sleeper (2+1)', 'departing_time': '20:30', 'duration': '6h', 'reaching_time': '02:30', 'star_rating': '4.2', 'seats_available': '7 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Mysore', 'busname': 'PADIKKAL TRAVELS PRIVATE LIMITED', 'price': '₹810', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '20:30', 'duration': '6h 20m', 'reaching_time': '02:50', 'star_rating': '3.2', 'seats_available': '7 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Mysore', 'busname': 'MMK Travels', 'price': '₹699', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '20:30', 'duration': '6h 30m', 'reaching_time': '03:00', 'star_rating': '4.2', 'seats_available': '27 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Mysore', 'busname': 'SKS Tourist Corporation (REGD)', 'price': '₹599', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '20:30', 'duration': '7h', 'reaching_time': '03:30', 'star_rating': '3.6', 'seats_available': '23 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Mysore', 'busname': 'KSRTC (Kerala) - 578', 'price': '₹474', 'bustype': 'Swift Deluxe Non AC Air Bus (2+2)', 'departing_time': '18:00', 'duration': '6h 45m', 'reaching_time': '00:45', 'star_rating': '4.5', 'seats_available': '25 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Mysore', 'busname': 'KSRTC (Kerala) - 72', 'price': '₹596', 'bustype': 'SWIFT-GARUDA A/C SEATER BUS', 'departing_time': '19:05', 'duration': '7h', 'reaching_time': '02:05', 'star_rating': '4.5', 'seats_available': '30 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Mysore', 'busname': 'KSRTC (Kerala) - 1583', 'price': '₹474', 'bustype': 'Swift Deluxe Non AC Air Bus (2+2)', 'departing_time': '20:00', 'duration': '6h 30m', 'reaching_time': '02:30', 'star_rating': '4', 'seats_available': '25 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Mysore', 'busname': 'KSRTC (Kerala) - 234', 'price': '₹453', 'bustype': 'Swift Deluxe Non AC Air Bus (2+2)', 'departing_time': '21:00', 'duration': '6h 50m', 'reaching_time': '03:50', 'star_rating': '2.4', 'seats_available': '29 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Mysore', 'busname': 'KSRTC (Kerala) - 1584', 'price': '₹474', 'bustype': 'Swift Deluxe Non AC Air Bus (2+2)', 'departing_time': '21:30', 'duration': '6h 25m', 'reaching_time': '03:55', 'star_rating': '3.8', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Mysore', 'busname': 'KSRTC (Kerala) - 50', 'price': '₹474', 'bustype': 'Swift Deluxe Non AC Air Bus (2+2)', 'departing_time': '22:00', 'duration': '6h 35m', 'reaching_time': '04:35', 'star_rating': '4.3', 'seats_available': '15 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Mysore', 'busname': 'KSRTC (Kerala) - 719', 'price': '₹348', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '23:00', 'duration': '6h 40m', 'reaching_time': '05:40', 'star_rating': '3.8', 'seats_available': '41 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Mysore', 'busname': 'KSRTC (Kerala) - 96', 'price': '₹390', 'bustype': 'Swift Deluxe Non AC Air Bus (2+2)', 'departing_time': '23:50', 'duration': '4h 55m', 'reaching_time': '04:45', 'star_rating': '3.4', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kalpetta (kerala) to Bangalore', 'busname': 'MMK Travels', 'price': '₹700', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '23:45', 'duration': '6h 15m', 'reaching_time': '06:00', 'star_rating': '4.6', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kalpetta (kerala) to Bangalore', 'busname': 'Highline Transports', 'price': '₹889', 'bustype': 'VE A/C Sleeper (2+1)', 'departing_time': '23:15', 'duration': '7h 30m', 'reaching_time': '06:45', 'star_rating': '4.2', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kalpetta (kerala) to Bangalore', 'busname': 'A1 Travels', 'price': '₹500', 'bustype': 'NON A/C Seater / Sleeper (3+1)', 'departing_time': '22:55', 'duration': '7h 20m', 'reaching_time': '06:15', 'star_rating': '4.2', 'seats_available': '24 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kalpetta (kerala) to Bangalore', 'busname': 'DLT BUS', 'price': '₹900', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '23:59', 'duration': '6h 31m', 'reaching_time': '06:30', 'star_rating': '4.6', 'seats_available': '13 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kalpetta (kerala) to Bangalore', 'busname': 'Kyros Connect', 'price': '₹799', 'bustype': 'Bharat Benz A/C Semi Sleeper (2+2)', 'departing_time': '15:45', 'duration': '7h 30m', 'reaching_time': '23:15', 'star_rating': '3.9', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kalpetta (kerala) to Bangalore', 'busname': 'MCK Travels', 'price': '₹649', 'bustype': 'A/C Seater/Sleeper (2+1)', 'departing_time': '23:05', 'duration': '7h 45m', 'reaching_time': '06:50', 'star_rating': None, 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kalpetta (kerala) to Bangalore', 'busname': 'DA Transports', 'price': '₹719', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '23:59', 'duration': '6h 46m', 'reaching_time': '06:45', 'star_rating': '3.6', 'seats_available': '14 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kalpetta (kerala) to Bangalore', 'busname': 'A1 Travels', 'price': '₹550', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '22:15', 'duration': '7h 15m', 'reaching_time': '05:30', 'star_rating': '2.7', 'seats_available': '22 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kalpetta (kerala) to Bangalore', 'busname': 'A1 Travels', 'price': '₹500', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '22:30', 'duration': '7h', 'reaching_time': '05:30', 'star_rating': '4.3', 'seats_available': '14 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kalpetta (kerala) to Bangalore', 'busname': 'SKS Tourist Corporation (REGD)', 'price': '₹749', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '23:16', 'duration': '6h 29m', 'reaching_time': '05:45', 'star_rating': '4.1', 'seats_available': '5 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kalpetta (kerala) to Bangalore', 'busname': 'KSRTC (Kerala) - 578', 'price': '₹670', 'bustype': 'Swift Deluxe Non AC Air Bus (2+2)', 'departing_time': '20:10', 'duration': '6h 55m', 'reaching_time': '03:05', 'star_rating': '3.1', 'seats_available': '26 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kalpetta (kerala) to Bangalore', 'busname': 'KSRTC (Kerala) - 72', 'price': '₹863', 'bustype': 'SWIFT-GARUDA A/C SEATER BUS', 'departing_time': '20:45', 'duration': '7h 40m', 'reaching_time': '04:25', 'star_rating': '2.6', 'seats_available': '30 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kalpetta (kerala) to Bangalore', 'busname': 'KSRTC (Kerala) - 1583', 'price': '₹670', 'bustype': 'Swift Deluxe Non AC Air Bus (2+2)', 'departing_time': '21:35', 'duration': '7h 15m', 'reaching_time': '04:50', 'star_rating': '2.5', 'seats_available': '22 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kalpetta (kerala) to Bangalore', 'busname': 'KSRTC (Kerala) - 1584', 'price': '₹670', 'bustype': 'Swift Deluxe Non AC Air Bus (2+2)', 'departing_time': '23:05', 'duration': '7h 10m', 'reaching_time': '06:15', 'star_rating': '4.1', 'seats_available': '24 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Thrissur', 'busname': 'Galaxy Tours', 'price': '₹650', 'bustype': '2+1 Air Suspension A/C Seater / Sleeper', 'departing_time': '21:45', 'duration': '3h', 'reaching_time': '00:45', 'star_rating': '3', 'seats_available': '30 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Thrissur', 'busname': 'YBM Travels(BLM)', 'price': '₹1,321', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '22:00', 'duration': '3h', 'reaching_time': '01:00', 'star_rating': '4.2', 'seats_available': '33 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Thrissur', 'busname': 'New Friends Travels', 'price': '₹999', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '22:00', 'duration': '3h 15m', 'reaching_time': '01:15', 'star_rating': '3.8', 'seats_available': '21 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Thrissur', 'busname': 'EMERALD TRAVELS', 'price': '₹800', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '22:10', 'duration': '2h 20m', 'reaching_time': '00:30', 'star_rating': '3.3', 'seats_available': '28 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Thrissur', 'busname': 'EMERALD TRAVELS', 'price': '₹800', 'bustype': 'Bharat Benz A/C Semi Sleeper (2+2)', 'departing_time': '22:10', 'duration': '2h 20m', 'reaching_time': '00:30', 'star_rating': '3.3', 'seats_available': '30 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Thrissur', 'busname': 'CITI TOURS', 'price': '₹600', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '22:45', 'duration': '2h 45m', 'reaching_time': '01:30', 'star_rating': '3.2', 'seats_available': '27 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Thrissur', 'busname': 'Kalpaka Travels 1', 'price': '₹800', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '23:00', 'duration': '1h 30m', 'reaching_time': '00:30', 'star_rating': '3.8', 'seats_available': '4 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Thrissur', 'busname': 'Madhavi Travels', 'price': '₹849', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '23:40', 'duration': '3h 10m', 'reaching_time': '02:50', 'star_rating': '3.9', 'seats_available': '13 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Thrissur', 'busname': 'UFO Bus', 'price': '₹749', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '23:55', 'duration': '3h 35m', 'reaching_time': '03:30', 'star_rating': '4.1', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Thrissur', 'busname': 'UFO Bus', 'price': '₹648', 'bustype': 'Volvo Multi-Axle B9R A/c Semi Sleeper (2+2)', 'departing_time': '23:55', 'duration': '3h 35m', 'reaching_time': '03:30', 'star_rating': '4.1', 'seats_available': '39 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Thrissur', 'busname': 'Cochin Bus', 'price': '₹700', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '23:55', 'duration': '3h 35m', 'reaching_time': '03:30', 'star_rating': '4.3', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Thrissur', 'busname': 'MERLIN CONNECT', 'price': '₹799', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '23:57', 'duration': '3h 18m', 'reaching_time': '03:15', 'star_rating': '3.4', 'seats_available': '30 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Thrissur', 'busname': 'Shajees Motors', 'price': '₹550', 'bustype': 'A/C Semi Sleeper (2+2)', 'departing_time': '23:58', 'duration': '2h 52m', 'reaching_time': '02:50', 'star_rating': '3.7', 'seats_available': '30 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Thrissur', 'busname': 'Kalpaka Travels 1', 'price': '₹699', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '23:59', 'duration': '3h 46m', 'reaching_time': '03:45', 'star_rating': '3.8', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Thrissur', 'busname': 'Kalpaka Travels 1', 'price': '₹549', 'bustype': 'A/C Semi Sleeper (2+2)', 'departing_time': '23:59', 'duration': '3h 41m', 'reaching_time': '03:40', 'star_rating': '3.8', 'seats_available': '34 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Thrissur', 'busname': 'SG Travels', 'price': '₹650', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '23:59', 'duration': '3h 31m', 'reaching_time': '03:30', 'star_rating': '4.6', 'seats_available': '13 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Thrissur', 'busname': 'Cochin Bus', 'price': '₹700', 'bustype': 'Volvo Multi-Axle B9R A/c Semi Sleeper (2+2)', 'departing_time': '23:59', 'duration': '3h 31m', 'reaching_time': '03:30', 'star_rating': '4.3', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Thrissur', 'busname': 'Cochin Bus', 'price': '₹900', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '23:59', 'duration': '3h 31m', 'reaching_time': '03:30', 'star_rating': '4.3', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Thrissur', 'busname': 'Sonia Travels', 'price': '₹550', 'bustype': 'A/C Semi Sleeper (2+2)', 'departing_time': '23:59', 'duration': '2h 46m', 'reaching_time': '02:45', 'star_rating': '4.1', 'seats_available': '28 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Thrissur', 'busname': 'SONIA LUXURY LINES', 'price': '₹799', 'bustype': 'VE A/C Seater / Sleeper (2+1)', 'departing_time': '23:59', 'duration': '2h 31m', 'reaching_time': '02:30', 'star_rating': '4.5', 'seats_available': '30 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Thrissur', 'busname': 'KSRTC (Kerala) - 726', 'price': '₹181', 'bustype': 'Super Fast Non AC Seater (2+3)', 'departing_time': '17:05', 'duration': '3h 20m', 'reaching_time': '20:25', 'star_rating': '2.6', 'seats_available': '48 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Thrissur', 'busname': 'KSRTC (Kerala) - 2390', 'price': '₹253', 'bustype': 'Low Floor AC Seater 2+2', 'departing_time': '17:30', 'duration': '3h 30m', 'reaching_time': '21:00', 'star_rating': '3.8', 'seats_available': '1 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Thrissur', 'busname': 'KSRTC (Kerala) - 2900', 'price': '₹253', 'bustype': 'Low Floor AC Seater 2+2', 'departing_time': '18:00', 'duration': '3h 20m', 'reaching_time': '21:20', 'star_rating': '2.6', 'seats_available': '2 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Thrissur', 'busname': 'KSRTC (Kerala) - 1872', 'price': '₹253', 'bustype': 'Low Floor AC Seater 2+2', 'departing_time': '18:30', 'duration': '3h 5m', 'reaching_time': '21:35', 'star_rating': '4.2', 'seats_available': '5 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Thrissur', 'busname': 'KSRTC (Kerala) - 2453', 'price': '₹253', 'bustype': 'Low Floor AC Seater 2+2', 'departing_time': '19:00', 'duration': '2h 50m', 'reaching_time': '21:50', 'star_rating': '4.5', 'seats_available': '13 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Thrissur', 'busname': 'KSRTC (Kerala) - 3991', 'price': '₹216', 'bustype': 'Swift Deluxe Non AC Air Bus (2+2)', 'departing_time': '19:40', 'duration': '3h 5m', 'reaching_time': '22:45', 'star_rating': '3.6', 'seats_available': '7 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Thrissur', 'busname': 'KSRTC (Kerala) - 3294', 'price': '₹181', 'bustype': 'Super Fast Non AC Seater (2+3)', 'departing_time': '20:20', 'duration': '3h 30m', 'reaching_time': '23:50', 'star_rating': '3.6', 'seats_available': '40 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Thrissur', 'busname': 'KSRTC (Kerala) - 2393', 'price': '₹253', 'bustype': 'Low Floor AC Seater 2+2', 'departing_time': '20:30', 'duration': '3h 5m', 'reaching_time': '23:35', 'star_rating': '3.1', 'seats_available': '11 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Thrissur', 'busname': 'KSRTC (Kerala) - 424', 'price': '₹181', 'bustype': 'Super Fast Non AC Seater (2+3)', 'departing_time': '20:35', 'duration': '3h 25m', 'reaching_time': '00:00', 'star_rating': '3.6', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kerala RTC Online Ticket Booking', 'route': 'Kozhikode to Thrissur', 'busname': 'KSRTC (Kerala) - 3981', 'price': '₹216', 'bustype': 'Swift Deluxe Non AC Air Bus (2+2)', 'departing_time': '20:40', 'duration': '3h 10m', 'reaching_time': '23:50', 'star_rating': '3.6', 'seats_available': '5 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Vijayawada', 'busname': 'IntrCity SmartBus', 'price': '₹539', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '23:05', 'duration': '7h 25m', 'reaching_time': '06:30', 'star_rating': '4.7', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Vijayawada', 'busname': 'IntrCity SmartBus', 'price': '₹649', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '23:35', 'duration': '6h 30m', 'reaching_time': '06:05', 'star_rating': '4.6', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Vijayawada', 'busname': 'FRESHBUS', 'price': '₹395', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '23:20', 'duration': '6h 40m', 'reaching_time': '06:00', 'star_rating': '4.6', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Vijayawada', 'busname': 'FRESHBUS', 'price': '₹575', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '23:59', 'duration': '6h 36m', 'reaching_time': '06:35', 'star_rating': '4.5', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Vijayawada', 'busname': 'FRESHBUS', 'price': '₹539', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '22:35', 'duration': '6h 50m', 'reaching_time': '05:25', 'star_rating': '4.6', 'seats_available': '31 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Vijayawada', 'busname': 'FRESHBUS', 'price': '₹323', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '16:10', 'duration': '7h 55m', 'reaching_time': '00:05', 'star_rating': '4.8', 'seats_available': '19 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Vijayawada', 'busname': 'FRESHBUS', 'price': '₹323', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '17:35', 'duration': '7h 30m', 'reaching_time': '01:05', 'star_rating': '4.5', 'seats_available': '32 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Vijayawada', 'busname': 'zingbus plus', 'price': '₹473', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '21:55', 'duration': '7h 35m', 'reaching_time': '05:30', 'star_rating': '4.3', 'seats_available': '17 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Vijayawada', 'busname': 'NueGo', 'price': '₹474', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '21:30', 'duration': '7h 25m', 'reaching_time': '04:55', 'star_rating': '4.5', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Vijayawada', 'busname': 'zingbus plus', 'price': '₹527', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '23:10', 'duration': '6h 45m', 'reaching_time': '05:55', 'star_rating': '4.4', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Vijayawada', 'busname': 'APSRTC - 3376', 'price': '₹436', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '16:45', 'duration': '6h 30m', 'reaching_time': '23:15', 'star_rating': '3.8', 'seats_available': '27 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Vijayawada', 'busname': 'APSRTC - 3570', 'price': '₹670', 'bustype': 'AMARAVATHI (VOLVO / SCANIA A.C Multi Axle)', 'departing_time': '17:15', 'duration': '6h 40m', 'reaching_time': '23:55', 'star_rating': '3.7', 'seats_available': '41 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Vijayawada', 'busname': 'APSRTC - 3366', 'price': '₹469', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '17:55', 'duration': '7h 50m', 'reaching_time': '01:45', 'star_rating': '2.6', 'seats_available': '29 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Vijayawada', 'busname': 'APSRTC - 3360', 'price': '₹720', 'bustype': 'AMARAVATHI (VOLVO / SCANIA A.C Multi Axle)', 'departing_time': '18:25', 'duration': '7h 5m', 'reaching_time': '01:30', 'star_rating': '3.6', 'seats_available': '38 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Vijayawada', 'busname': 'APSRTC - 2575', 'price': '₹639', 'bustype': 'NIGHT RIDER (SEATER CUM SLEEPER)', 'departing_time': '18:55', 'duration': '6h 50m', 'reaching_time': '01:45', 'star_rating': '2.5', 'seats_available': '30 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Vijayawada', 'busname': 'APSRTC - 2674', 'price': '₹469', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:15', 'duration': '7h 50m', 'reaching_time': '03:05', 'star_rating': '3.4', 'seats_available': '27 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Vijayawada', 'busname': 'APSRTC - 2898', 'price': '₹607', 'bustype': 'STAR LINER(NON-AC SLEEPER 2+1)', 'departing_time': '19:25', 'duration': '6h 20m', 'reaching_time': '01:45', 'star_rating': '4.8', 'seats_available': '22 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Vijayawada', 'busname': 'APSRTC - 4916', 'price': '₹469', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:25', 'duration': '6h 50m', 'reaching_time': '02:15', 'star_rating': '1.2', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Vijayawada', 'busname': 'APSRTC - 24262', 'price': '₹469', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:30', 'duration': '7h', 'reaching_time': '02:30', 'star_rating': '3.8', 'seats_available': '17 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Vijayawada', 'busname': 'APSRTC - 98050', 'price': '₹436', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:30', 'duration': '7h', 'reaching_time': '02:30', 'star_rating': '4.9', 'seats_available': '32 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Vijayawada to Hyderabad', 'busname': 'FRESHBUS', 'price': '₹404', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '16:40', 'duration': '6h 50m', 'reaching_time': '23:30', 'star_rating': '4.7', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Vijayawada to Hyderabad', 'busname': 'FRESHBUS', 'price': '₹323', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '17:40', 'duration': '6h 50m', 'reaching_time': '00:30', 'star_rating': '4.7', 'seats_available': '27 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Vijayawada to Hyderabad', 'busname': 'FRESHBUS', 'price': '₹449', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '22:50', 'duration': '6h 42m', 'reaching_time': '05:32', 'star_rating': '4.6', 'seats_available': '28 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Vijayawada to Hyderabad', 'busname': 'FRESHBUS', 'price': '₹458', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '23:42', 'duration': '6h 50m', 'reaching_time': '06:32', 'star_rating': '4.5', 'seats_available': '30 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Vijayawada to Hyderabad', 'busname': 'FRESHBUS', 'price': '₹428', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '22:00', 'duration': '6h 32m', 'reaching_time': '04:32', 'star_rating': '4.6', 'seats_available': '32 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Vijayawada to Hyderabad', 'busname': 'FRESHBUS', 'price': '₹449', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '23:59', 'duration': '7h 33m', 'reaching_time': '07:32', 'star_rating': '4.6', 'seats_available': '38 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Vijayawada to Hyderabad', 'busname': 'IntrCity SmartBus', 'price': '₹429', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '23:05', 'duration': '6h 55m', 'reaching_time': '06:00', 'star_rating': '4.5', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Vijayawada to Hyderabad', 'busname': 'Sai RK Travels', 'price': '₹510', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '22:10', 'duration': '7h 20m', 'reaching_time': '05:30', 'star_rating': '3.6', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Vijayawada to Hyderabad', 'busname': 'IntrCity SmartBus', 'price': '₹569', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '23:45', 'duration': '6h 25m', 'reaching_time': '06:10', 'star_rating': '4.6', 'seats_available': '19 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Vijayawada to Hyderabad', 'busname': 'zingbus plus', 'price': '₹474', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '23:40', 'duration': '6h 30m', 'reaching_time': '06:10', 'star_rating': '4.5', 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Vijayawada to Hyderabad', 'busname': 'APSRTC - 35271', 'price': '₹538', 'bustype': 'INDRA(A.C. Seater)', 'departing_time': '17:00', 'duration': '6h 10m', 'reaching_time': '23:10', 'star_rating': '3.1', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Vijayawada to Hyderabad', 'busname': 'APSRTC - 35135', 'price': '₹670', 'bustype': 'AMARAVATHI (VOLVO / SCANIA A.C Multi Axle)', 'departing_time': '17:45', 'duration': '6h 22m', 'reaching_time': '00:07', 'star_rating': '4.4', 'seats_available': '38 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Vijayawada to Hyderabad', 'busname': 'APSRTC - 35085', 'price': '₹720', 'bustype': 'AMARAVATHI (VOLVO / SCANIA A.C Multi Axle)', 'departing_time': '18:30', 'duration': '6h 55m', 'reaching_time': '01:25', 'star_rating': '3.7', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Vijayawada to Hyderabad', 'busname': 'APSRTC - 23507', 'price': '₹528', 'bustype': 'INDRA(A.C. Seater)', 'departing_time': '19:00', 'duration': '6h 5m', 'reaching_time': '01:05', 'star_rating': '2.1', 'seats_available': '18 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Vijayawada to Hyderabad', 'busname': 'APSRTC - 3613', 'price': '₹436', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:00', 'duration': '7h 42m', 'reaching_time': '02:42', 'star_rating': '4.2', 'seats_available': '28 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Vijayawada to Hyderabad', 'busname': 'APSRTC - 9316', 'price': '₹436', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '20:20', 'duration': '7h 50m', 'reaching_time': '04:10', 'star_rating': '4.5', 'seats_available': '22 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Vijayawada to Hyderabad', 'busname': 'APSRTC - 35189', 'price': '₹670', 'bustype': 'AMARAVATHI (VOLVO / SCANIA A.C Multi Axle)', 'departing_time': '20:30', 'duration': '5h 5m', 'reaching_time': '01:35', 'star_rating': '4.6', 'seats_available': '36 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Vijayawada to Hyderabad', 'busname': 'APSRTC - 4046', 'price': '₹446', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '21:00', 'duration': '6h 40m', 'reaching_time': '03:40', 'star_rating': '3.8', 'seats_available': '18 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Vijayawada to Hyderabad', 'busname': 'APSRTC - 9651', 'price': '₹469', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '21:00', 'duration': '7h 41m', 'reaching_time': '04:41', 'star_rating': '4.3', 'seats_available': '15 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Vijayawada to Hyderabad', 'busname': 'APSRTC - 9654', 'price': '₹446', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '21:40', 'duration': '7h 40m', 'reaching_time': '05:20', 'star_rating': '3.9', 'seats_available': '7 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Chittoor (Andhra Pradesh) to Bangalore', 'busname': 'NueGo', 'price': '₹342', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '16:50', 'duration': '6h', 'reaching_time': '22:50', 'star_rating': '3.9', 'seats_available': '1 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Chittoor (Andhra Pradesh) to Bangalore', 'busname': 'Yolo Bus', 'price': '₹399', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '17:17', 'duration': '4h 49m', 'reaching_time': '22:06', 'star_rating': '3.6', 'seats_available': '1 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Chittoor (Andhra Pradesh) to Bangalore', 'busname': 'KTC Travels', 'price': '₹500', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '17:35', 'duration': '4h 25m', 'reaching_time': '22:00', 'star_rating': '3.8', 'seats_available': '19 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Chittoor (Andhra Pradesh) to Bangalore', 'busname': 'KTC Travels', 'price': '₹395', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '17:55', 'duration': '4h 40m', 'reaching_time': '22:35', 'star_rating': '3.8', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Chittoor (Andhra Pradesh) to Bangalore', 'busname': 'FRESHBUS', 'price': '₹395', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '18:35', 'duration': '3h 55m', 'reaching_time': '22:30', 'star_rating': '4.7', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Chittoor (Andhra Pradesh) to Bangalore', 'busname': 'NueGo', 'price': '₹332', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '19:20', 'duration': '28h 25m', 'reaching_time': '23:45', 'star_rating': '4.4', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Chittoor (Andhra Pradesh) to Bangalore', 'busname': 'FRESHBUS', 'price': '₹350', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '19:50', 'duration': '4h 15m', 'reaching_time': '00:05', 'star_rating': '5', 'seats_available': '40 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Chittoor (Andhra Pradesh) to Bangalore', 'busname': 'Yolo Bus', 'price': '₹499', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '20:05', 'duration': '4h 37m', 'reaching_time': '00:42', 'star_rating': '4', 'seats_available': '17 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Chittoor (Andhra Pradesh) to Bangalore', 'busname': 'Yolo Bus', 'price': '₹429', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '20:21', 'duration': '4h 37m', 'reaching_time': '00:58', 'star_rating': '3.8', 'seats_available': '17 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Chittoor (Andhra Pradesh) to Bangalore', 'busname': 'NueGo', 'price': '₹332', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '20:50', 'duration': '5h 25m', 'reaching_time': '02:15', 'star_rating': '4.3', 'seats_available': '36 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Chittoor (Andhra Pradesh) to Bangalore', 'busname': 'APSRTC - 9049', 'price': '₹239', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '17:15', 'duration': '4h 25m', 'reaching_time': '21:40', 'star_rating': '4.2', 'seats_available': '3 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Chittoor (Andhra Pradesh) to Bangalore', 'busname': 'APSRTC - 9888', 'price': '₹202', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '18:45', 'duration': '4h 35m', 'reaching_time': '23:20', 'star_rating': '3.8', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Chittoor (Andhra Pradesh) to Bangalore', 'busname': 'APSRTC - 9003', 'price': '₹218', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '20:00', 'duration': '4h 25m', 'reaching_time': '00:25', 'star_rating': '3.8', 'seats_available': '44 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Chittoor (Andhra Pradesh) to Bangalore', 'busname': 'APSRTC - 9041', 'price': '₹239', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '21:00', 'duration': '4h 25m', 'reaching_time': '01:25', 'star_rating': '4.3', 'seats_available': '32 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Chittoor (Andhra Pradesh) to Bangalore', 'busname': 'APSRTC - 9080', 'price': '₹239', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '21:45', 'duration': '4h 30m', 'reaching_time': '02:15', 'star_rating': '4', 'seats_available': '33 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Chittoor (Andhra Pradesh) to Bangalore', 'busname': 'APSRTC - 5524', 'price': '₹202', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '22:45', 'duration': '4h 10m', 'reaching_time': '02:55', 'star_rating': '4', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Chittoor (Andhra Pradesh) to Bangalore', 'busname': 'APSRTC - 9061', 'price': '₹239', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '23:15', 'duration': '4h 30m', 'reaching_time': '03:45', 'star_rating': '4.4', 'seats_available': '44 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Chittoor (Andhra Pradesh) to Bangalore', 'busname': 'APSRTC - 5528', 'price': '₹334', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '23:59', 'duration': '4h 31m', 'reaching_time': '04:30', 'star_rating': '4.5', 'seats_available': '27 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kakinada to Visakhapatnam', 'busname': 'SRI VIJAYA KRISHNA TOURS and TRAVELS', 'price': '₹999', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '23:59', 'duration': '4h 31m', 'reaching_time': '04:30', 'star_rating': '3.5', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kakinada to Visakhapatnam', 'busname': 'Gaganasri Travels', 'price': '₹5,000', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '23:50', 'duration': '3h 40m', 'reaching_time': '03:30', 'star_rating': None, 'seats_available': '1 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kakinada to Visakhapatnam', 'busname': 'APSRTC - 3406', 'price': '₹281', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '16:45', 'duration': '3h 45m', 'reaching_time': '20:30', 'star_rating': '3.9', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kakinada to Visakhapatnam', 'busname': 'APSRTC - 2891', 'price': '₹340', 'bustype': 'INDRA(A.C. Seater)', 'departing_time': '17:00', 'duration': '3h', 'reaching_time': '20:00', 'star_rating': '3.6', 'seats_available': '3 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kakinada to Visakhapatnam', 'busname': 'APSRTC - 2534', 'price': '₹281', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '17:15', 'duration': '4h 30m', 'reaching_time': '21:45', 'star_rating': '3.9', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kakinada to Visakhapatnam', 'busname': 'APSRTC - 2815', 'price': '₹281', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '17:30', 'duration': '3h', 'reaching_time': '20:30', 'star_rating': '4.1', 'seats_available': '29 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kakinada to Visakhapatnam', 'busname': 'APSRTC - 23527', 'price': '₹281', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '17:45', 'duration': '4h 30m', 'reaching_time': '22:15', 'star_rating': '3.9', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kakinada to Visakhapatnam', 'busname': 'APSRTC - 27083', 'price': '₹383', 'bustype': 'METRO LUXURY A/C', 'departing_time': '18:00', 'duration': '3h', 'reaching_time': '21:00', 'star_rating': '3.5', 'seats_available': '17 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kakinada to Visakhapatnam', 'busname': 'APSRTC - 3332', 'price': '₹281', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '18:00', 'duration': '4h', 'reaching_time': '22:00', 'star_rating': '4.3', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kakinada to Visakhapatnam', 'busname': 'APSRTC - 2869', 'price': '₹281', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '18:30', 'duration': '3h', 'reaching_time': '21:30', 'star_rating': '3', 'seats_available': '28 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kakinada to Visakhapatnam', 'busname': 'APSRTC - 2857', 'price': '₹269', 'bustype': 'ULTRA DELUXE (NON-AC, 2+2 PUSH BACK)', 'departing_time': '18:30', 'duration': '5h', 'reaching_time': '23:30', 'star_rating': '3.9', 'seats_available': '29 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kakinada to Visakhapatnam', 'busname': 'APSRTC - 3324', 'price': '₹281', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:00', 'duration': '4h', 'reaching_time': '23:00', 'star_rating': '3.3', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Tirupati', 'busname': 'FlixBus', 'price': '₹535', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '15:45', 'duration': '4h 55m', 'reaching_time': '20:40', 'star_rating': '4.7', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Tirupati', 'busname': 'FlixBus', 'price': '₹535', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '15:50', 'duration': '6h 30m', 'reaching_time': '22:20', 'star_rating': '4.2', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Tirupati', 'busname': 'FRESHBUS', 'price': '₹314', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '15:50', 'duration': '5h 10m', 'reaching_time': '21:00', 'star_rating': '4.6', 'seats_available': '4 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Tirupati', 'busname': 'HYBUS', 'price': '₹494', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '22:30', 'duration': '6h 30m', 'reaching_time': '05:00', 'star_rating': '4.5', 'seats_available': '23 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Tirupati', 'busname': 'Yolo Bus', 'price': '₹199', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '15:59', 'duration': '5h 40m', 'reaching_time': '21:39', 'star_rating': '3.7', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Tirupati', 'busname': 'GMC Travels', 'price': '₹780', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '16:15', 'duration': '4h 55m', 'reaching_time': '21:10', 'star_rating': '4', 'seats_available': '29 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Tirupati', 'busname': 'Shree Manikanta Travels', 'price': '₹699', 'bustype': 'Volvo 9600 Multi-Axle A/C Sleeper (2+1)', 'departing_time': '16:30', 'duration': '4h 50m', 'reaching_time': '21:20', 'star_rating': '4', 'seats_available': '6 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Tirupati', 'busname': 'HYBUS', 'price': '₹585', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '22:00', 'duration': '6h 30m', 'reaching_time': '04:30', 'star_rating': '4.4', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Tirupati', 'busname': 'FRESHBUS', 'price': '₹350', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '19:35', 'duration': '5h 40m', 'reaching_time': '01:15', 'star_rating': '4.7', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Tirupati', 'busname': 'BSRM Travels', 'price': '₹404', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '21:00', 'duration': '7h', 'reaching_time': '04:00', 'star_rating': '4.4', 'seats_available': '17 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Tirupati', 'busname': 'APSRTC - 4157', 'price': '₹426', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '16:45', 'duration': '6h 15m', 'reaching_time': '23:00', 'star_rating': '3.9', 'seats_available': '30 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Tirupati', 'busname': 'APSRTC - 4310', 'price': '₹426', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '17:15', 'duration': '5h 45m', 'reaching_time': '23:00', 'star_rating': '3.6', 'seats_available': '30 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Tirupati', 'busname': 'APSRTC - 9020', 'price': '₹312', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '17:30', 'duration': '6h 25m', 'reaching_time': '23:55', 'star_rating': '3.7', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Tirupati', 'busname': 'APSRTC - 3871', 'price': '₹731', 'bustype': 'VENNELA (A.C. SLEEPER)', 'departing_time': '17:30', 'duration': '6h 25m', 'reaching_time': '23:55', 'star_rating': '4.3', 'seats_available': '17 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Tirupati', 'busname': 'APSRTC - 4020', 'price': '₹426', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '18:15', 'duration': '5h 35m', 'reaching_time': '23:50', 'star_rating': '2.9', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Tirupati', 'busname': 'APSRTC - 47211', 'price': '₹312', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '18:30', 'duration': '6h 20m', 'reaching_time': '00:50', 'star_rating': '2.3', 'seats_available': '28 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Tirupati', 'busname': 'APSRTC - 4610', 'price': '₹426', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:00', 'duration': '6h', 'reaching_time': '01:00', 'star_rating': '3.7', 'seats_available': '26 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Tirupati', 'busname': 'APSRTC - 5082', 'price': '₹426', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '20:20', 'duration': '6h 5m', 'reaching_time': '02:25', 'star_rating': '3.8', 'seats_available': '25 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Tirupati', 'busname': 'APSRTC - 5501', 'price': '₹312', 'bustype': 'SAPTAGIRI EXPRESS', 'departing_time': '20:30', 'duration': '6h', 'reaching_time': '02:30', 'star_rating': '3.8', 'seats_available': '36 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Tirupati', 'busname': 'APSRTC - 5497', 'price': '₹312', 'bustype': 'SAPTAGIRI EXPRESS', 'departing_time': '21:00', 'duration': '6h', 'reaching_time': '03:00', 'star_rating': '4.1', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Ongole to Hyderabad', 'busname': 'Vikram Travels', 'price': '₹499', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '23:15', 'duration': '7h 3m', 'reaching_time': '06:18', 'star_rating': '4.7', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Ongole to Hyderabad', 'busname': 'Sri Vengamamba Bus Transport (SVBT)', 'price': '₹650', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '23:00', 'duration': '7h 35m', 'reaching_time': '06:35', 'star_rating': '4.7', 'seats_available': '14 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Ongole to Hyderabad', 'busname': 'FlixBus', 'price': '₹901', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '23:55', 'duration': '7h 15m', 'reaching_time': '07:10', 'star_rating': '4.7', 'seats_available': '8 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Ongole to Hyderabad', 'busname': 'IntrCity SmartBus', 'price': '₹1,079', 'bustype': 'AC Sleeper (2+1)', 'departing_time': '23:40', 'duration': '6h 40m', 'reaching_time': '06:20', 'star_rating': '4.7', 'seats_available': '6 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Ongole to Hyderabad', 'busname': 'Kaveri Travels', 'price': '₹500', 'bustype': 'NON AC Seater / Sleeper 2+1', 'departing_time': '22:20', 'duration': '7h 25m', 'reaching_time': '05:45', 'star_rating': '4.6', 'seats_available': '17 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Ongole to Hyderabad', 'busname': 'V Kaveri Travels', 'price': '₹999', 'bustype': 'Volvo 9600 Multi-Axle A/C Sleeper (2+1)', 'departing_time': '23:20', 'duration': '7h 5m', 'reaching_time': '06:25', 'star_rating': '4.5', 'seats_available': '14 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Ongole to Hyderabad', 'busname': 'Sri Vengamamba Bus Transport (SVBT)', 'price': '₹1,090', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '22:30', 'duration': '7h 35m', 'reaching_time': '06:05', 'star_rating': '4.5', 'seats_available': '7 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Ongole to Hyderabad', 'busname': 'V Kaveri Travels', 'price': '₹800', 'bustype': 'NON A/C Sleeper (2+1)', 'departing_time': '22:00', 'duration': '7h 55m', 'reaching_time': '05:55', 'star_rating': '4.5', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Ongole to Hyderabad', 'busname': 'Kaveri Travels', 'price': '₹799', 'bustype': 'Volvo Multi Axle A/C Sleeper I-Shift B11R (2+1)', 'departing_time': '23:35', 'duration': '7h 5m', 'reaching_time': '06:40', 'star_rating': '4.5', 'seats_available': '14 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Ongole to Hyderabad', 'busname': 'Kaveri Travels', 'price': '₹979', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '21:50', 'duration': '8h 5m', 'reaching_time': '05:55', 'star_rating': '4.3', 'seats_available': '14 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Ongole to Hyderabad', 'busname': 'APSRTC - 5380', 'price': '₹624', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:00', 'duration': '10h 6m', 'reaching_time': '05:06', 'star_rating': '3.9', 'seats_available': '13 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Ongole to Hyderabad', 'busname': 'APSRTC - 5866', 'price': '₹602', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:45', 'duration': '10h 1m', 'reaching_time': '05:46', 'star_rating': '4.7', 'seats_available': '11 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Ongole to Hyderabad', 'busname': 'APSRTC - 5913', 'price': '₹590', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '20:30', 'duration': '9h 6m', 'reaching_time': '05:36', 'star_rating': '4.5', 'seats_available': '17 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Ongole to Hyderabad', 'busname': 'APSRTC - 45901', 'price': '₹555', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '21:00', 'duration': '7h 20m', 'reaching_time': '04:20', 'star_rating': '3.9', 'seats_available': '8 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Ongole to Hyderabad', 'busname': 'APSRTC - 5915', 'price': '₹605', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '21:15', 'duration': '8h 5m', 'reaching_time': '05:20', 'star_rating': '4', 'seats_available': '1 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Ongole to Hyderabad', 'busname': 'APSRTC - 5957', 'price': '₹719', 'bustype': 'STAR LINER(NON-AC SLEEPER 2+1)', 'departing_time': '21:30', 'duration': '6h 30m', 'reaching_time': '04:00', 'star_rating': '4.7', 'seats_available': '4 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Ongole to Hyderabad', 'busname': 'APSRTC - 48820', 'price': '₹555', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '21:30', 'duration': '7h 25m', 'reaching_time': '04:55', 'star_rating': '3.8', 'seats_available': '13 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Ongole to Hyderabad', 'busname': 'APSRTC - 5927', 'price': '₹555', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '21:45', 'duration': '6h 35m', 'reaching_time': '04:20', 'star_rating': '4.4', 'seats_available': '11 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Ongole to Hyderabad', 'busname': 'APSRTC - 5378', 'price': '₹671', 'bustype': 'INDRA(A.C. Seater)', 'departing_time': '22:00', 'duration': '6h 40m', 'reaching_time': '04:40', 'star_rating': '4.2', 'seats_available': '15 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Ongole to Hyderabad', 'busname': 'APSRTC - 5895', 'price': '₹555', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '22:00', 'duration': '7h 20m', 'reaching_time': '05:20', 'star_rating': '4.6', 'seats_available': '13 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Ongole', 'busname': 'Vikram Travels', 'price': '₹539', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '21:55', 'duration': '7h 35m', 'reaching_time': '05:30', 'star_rating': '4.7', 'seats_available': '13 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Ongole', 'busname': 'IntrCity SmartBus', 'price': '₹869', 'bustype': 'AC Sleeper (2+1)', 'departing_time': '23:40', 'duration': '7h 15m', 'reaching_time': '06:55', 'star_rating': '4.7', 'seats_available': '19 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Ongole', 'busname': 'IntrCity SmartBus', 'price': '₹579', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '23:50', 'duration': '6h 35m', 'reaching_time': '06:25', 'star_rating': '4.5', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Ongole', 'busname': 'Yolo Bus', 'price': '₹599', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '21:45', 'duration': '8h 33m', 'reaching_time': '06:18', 'star_rating': '4.3', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Ongole', 'busname': 'Sri Vengamamba Bus Transport (SVBT)', 'price': '₹550', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '22:00', 'duration': '8h 15m', 'reaching_time': '06:15', 'star_rating': '4.3', 'seats_available': '14 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Ongole', 'busname': 'V Kaveri Travels', 'price': '₹1,079', 'bustype': 'Volvo 9600 Multi-Axle A/C Sleeper (2+1)', 'departing_time': '21:30', 'duration': '8h 15m', 'reaching_time': '05:45', 'star_rating': '4.3', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Ongole', 'busname': 'Kaveri Travels', 'price': '₹760', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '22:30', 'duration': '8h', 'reaching_time': '06:30', 'star_rating': '4.2', 'seats_available': '9 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Ongole', 'busname': 'Kamakshi Tours And Travels', 'price': '₹549', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '21:30', 'duration': '8h 30m', 'reaching_time': '06:00', 'star_rating': '4.2', 'seats_available': '9 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Ongole', 'busname': 'FlixBus', 'price': '₹1,401', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '20:35', 'duration': '8h 40m', 'reaching_time': '05:15', 'star_rating': '4.3', 'seats_available': '5 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Ongole', 'busname': 'IntrCity SmartBus', 'price': '₹949', 'bustype': 'AC Sleeper (2+1)', 'departing_time': '23:45', 'duration': '5h 55m', 'reaching_time': '05:40', 'star_rating': '4.4', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Ongole', 'busname': 'APSRTC - 5870', 'price': '₹522', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '18:30', 'duration': '9h 30m', 'reaching_time': '04:00', 'star_rating': '4.8', 'seats_available': '16 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Ongole', 'busname': 'APSRTC - 5070', 'price': '₹556', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '18:45', 'duration': '8h 40m', 'reaching_time': '03:25', 'star_rating': '1.7', 'seats_available': '12 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Ongole', 'busname': 'APSRTC - 5859', 'price': '₹555', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:00', 'duration': '9h', 'reaching_time': '04:00', 'star_rating': '4', 'seats_available': '6 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Ongole', 'busname': 'APSRTC - 48821', 'price': '₹522', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:00', 'duration': '9h 15m', 'reaching_time': '04:15', 'star_rating': '4.2', 'seats_available': '14 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Ongole', 'busname': 'APSRTC - 5054', 'price': '₹556', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:15', 'duration': '9h 45m', 'reaching_time': '05:00', 'star_rating': '3.3', 'seats_available': '14 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Ongole', 'busname': 'APSRTC - 5994', 'price': '₹522', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:30', 'duration': '10h 15m', 'reaching_time': '05:45', 'star_rating': '4.5', 'seats_available': '12 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Ongole', 'busname': 'APSRTC - 5867', 'price': '₹602', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:30', 'duration': '11h 30m', 'reaching_time': '07:00', 'star_rating': '3.8', 'seats_available': '1 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Ongole', 'busname': 'APSRTC - 5181', 'price': '₹556', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:40', 'duration': '9h', 'reaching_time': '04:40', 'star_rating': '2.7', 'seats_available': '12 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Ongole', 'busname': 'APSRTC - 5922', 'price': '₹555', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:45', 'duration': '9h 20m', 'reaching_time': '05:05', 'star_rating': '3.7', 'seats_available': '5 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Ongole', 'busname': 'APSRTC - 5916', 'price': '₹605', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:50', 'duration': '9h 10m', 'reaching_time': '05:00', 'star_rating': '3.4', 'seats_available': '3 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadapa', 'busname': 'Shiva Parvathi Tours And Travels', 'price': '₹500', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '19:00', 'duration': '7h 30m', 'reaching_time': '02:30', 'star_rating': '3.4', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadapa', 'busname': 'BHAIRAVA TOURS AND TRAVELS', 'price': '₹499', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '19:15', 'duration': '7h 45m', 'reaching_time': '03:00', 'star_rating': '3.5', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadapa', 'busname': 'S R Travels', 'price': '₹560', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '19:15', 'duration': '8h', 'reaching_time': '03:15', 'star_rating': '3.1', 'seats_available': '6 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadapa', 'busname': 'Royal Voyage Travels (RVT)', 'price': '₹500', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '19:20', 'duration': '8h 5m', 'reaching_time': '03:25', 'star_rating': '4.1', 'seats_available': '17 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadapa', 'busname': 'Giridhar Tours and Travels', 'price': '₹700', 'bustype': 'A/C Seater/Sleeper (2+1)', 'departing_time': '19:24', 'duration': '7h 35m', 'reaching_time': '02:59', 'star_rating': '4.7', 'seats_available': '23 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadapa', 'busname': 'S R Travels', 'price': '₹500', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '19:30', 'duration': '8h', 'reaching_time': '03:30', 'star_rating': '3.4', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadapa', 'busname': 'Amara Tours And Travels', 'price': '₹499', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '19:30', 'duration': '7h 30m', 'reaching_time': '03:00', 'star_rating': '4.3', 'seats_available': '21 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadapa', 'busname': 'Sri Vinayaka Travels', 'price': '₹650', 'bustype': 'NON A/C Sleeper (2+1)', 'departing_time': '19:45', 'duration': '9h', 'reaching_time': '04:45', 'star_rating': '4', 'seats_available': '9 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadapa', 'busname': 'CGR Travels', 'price': '₹450', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '19:55', 'duration': '8h 30m', 'reaching_time': '04:25', 'star_rating': '4.1', 'seats_available': '1 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadapa', 'busname': 'Sri Vengamamba Bus Transport (SVBT)', 'price': '₹400', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '20:00', 'duration': '6h 10m', 'reaching_time': '02:10', 'star_rating': '4.4', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadapa', 'busname': 'APSRTC - 5226', 'price': '₹472', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '17:45', 'duration': '8h', 'reaching_time': '01:45', 'star_rating': '2.1', 'seats_available': '29 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadapa', 'busname': 'APSRTC - 5296', 'price': '₹472', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '18:00', 'duration': '8h', 'reaching_time': '02:00', 'star_rating': '2.9', 'seats_available': '25 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadapa', 'busname': 'APSRTC - 7745', 'price': '₹472', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '18:20', 'duration': '8h 10m', 'reaching_time': '02:30', 'star_rating': '3.5', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadapa', 'busname': 'APSRTC - 48484', 'price': '₹472', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:20', 'duration': '7h 25m', 'reaching_time': '02:45', 'star_rating': '3.7', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadapa', 'busname': 'APSRTC - 46152', 'price': '₹472', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '20:00', 'duration': '7h 15m', 'reaching_time': '03:15', 'star_rating': '3.7', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadapa', 'busname': 'APSRTC - 50079', 'price': '₹472', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '20:15', 'duration': '7h', 'reaching_time': '03:15', 'star_rating': '3.9', 'seats_available': '29 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadapa', 'busname': 'APSRTC - 6558', 'price': '₹472', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '20:30', 'duration': '6h 30m', 'reaching_time': '03:00', 'star_rating': '4.4', 'seats_available': '20 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadapa', 'busname': 'APSRTC - 6017', 'price': '₹472', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '21:00', 'duration': '7h', 'reaching_time': '04:00', 'star_rating': '4.1', 'seats_available': '32 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadapa', 'busname': 'APSRTC - 50062', 'price': '₹472', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '21:45', 'duration': '7h', 'reaching_time': '04:45', 'star_rating': '4.1', 'seats_available': '20 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadapa', 'busname': 'APSRTC - 6241', 'price': '₹412', 'bustype': 'ULTRA DELUXE (NON-AC, 2+2 PUSH BACK)', 'departing_time': '22:00', 'duration': '6h 30m', 'reaching_time': '04:30', 'star_rating': '3.1', 'seats_available': '30 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Madanapalli to Bangalore', 'busname': 'HARITHA TRAVELS', 'price': '₹5,000', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '23:59', 'duration': '6h 1m', 'reaching_time': '06:00', 'star_rating': '3.7', 'seats_available': '41 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Madanapalli to Bangalore', 'busname': 'HARITHA TRAVELS', 'price': '₹498', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '01:15', 'duration': '4h 20m', 'reaching_time': '05:35', 'star_rating': '3.7', 'seats_available': '24 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Madanapalli to Bangalore', 'busname': 'Jaysri Travels', 'price': '₹664', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '03:15', 'duration': '4h 45m', 'reaching_time': '08:00', 'star_rating': None, 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Madanapalli to Bangalore', 'busname': 'APSRTC - 5759', 'price': '₹173', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '17:45', 'duration': '4h', 'reaching_time': '21:45', 'star_rating': '3.7', 'seats_available': '40 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Madanapalli to Bangalore', 'busname': 'APSRTC - 7590', 'price': '₹173', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '19:00', 'duration': '3h 15m', 'reaching_time': '22:15', 'star_rating': '4.1', 'seats_available': '41 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Madanapalli', 'busname': 'Sri Vengamamba Bus Transport (SVBT)', 'price': '₹400', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '20:00', 'duration': '3h 59m', 'reaching_time': '23:59', 'star_rating': '4.5', 'seats_available': '22 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Madanapalli', 'busname': 'Maheswari Travels', 'price': '₹399', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '20:15', 'duration': '5h 45m', 'reaching_time': '02:00', 'star_rating': '4.5', 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Madanapalli', 'busname': 'SREE RUDRA TRAVELS', 'price': '₹400', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '20:15', 'duration': '5h 20m', 'reaching_time': '01:35', 'star_rating': '4.2', 'seats_available': '25 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Madanapalli', 'busname': 'Sree Rudra Travels', 'price': '₹400', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '20:15', 'duration': '5h 20m', 'reaching_time': '01:35', 'star_rating': '3.9', 'seats_available': '25 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Madanapalli', 'busname': 'Maheswari Travels', 'price': '₹399', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '20:30', 'duration': '6h 5m', 'reaching_time': '02:35', 'star_rating': '4.4', 'seats_available': '19 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Madanapalli', 'busname': 'Sri Vengamamba Bus Transport (SVBT)', 'price': '₹410', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '20:30', 'duration': '4h', 'reaching_time': '00:30', 'star_rating': '4.5', 'seats_available': '23 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Madanapalli', 'busname': 'Maheswari Travels', 'price': '₹349', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '20:35', 'duration': '5h 55m', 'reaching_time': '02:30', 'star_rating': '4.5', 'seats_available': '7 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Madanapalli', 'busname': 'AR & BCVR Travels', 'price': '₹450', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '21:30', 'duration': '4h 30m', 'reaching_time': '02:00', 'star_rating': '4.1', 'seats_available': '32 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Madanapalli', 'busname': 'BHAIRAVA TOURS AND TRAVELS', 'price': '₹5,555', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '15:50', 'duration': '3h 40m', 'reaching_time': '19:30', 'star_rating': '3.1', 'seats_available': '39 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Madanapalli', 'busname': 'Jathin travels', 'price': '₹650', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '18:20', 'duration': '5h 10m', 'reaching_time': '23:30', 'star_rating': '4.1', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Madanapalli', 'busname': 'APSRTC - 5769', 'price': '₹173', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '17:30', 'duration': '3h 30m', 'reaching_time': '21:00', 'star_rating': '3.6', 'seats_available': '17 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Madanapalli', 'busname': 'APSRTC - 7587', 'price': '₹173', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '18:30', 'duration': '4h', 'reaching_time': '22:30', 'star_rating': '3.3', 'seats_available': '32 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Madanapalli', 'busname': 'APSRTC - 47135', 'price': '₹173', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '19:30', 'duration': '4h', 'reaching_time': '23:30', 'star_rating': '3.8', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Madanapalli', 'busname': 'APSRTC - 7595', 'price': '₹173', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '20:30', 'duration': '4h', 'reaching_time': '00:30', 'star_rating': '4.5', 'seats_available': '33 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadapa to Bangalore', 'busname': 'RK Travels', 'price': '₹500', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '22:25', 'duration': '7h 5m', 'reaching_time': '05:30', 'star_rating': '4.2', 'seats_available': '14 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadapa to Bangalore', 'busname': 'Sri Vinayaka Travels', 'price': '₹750', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '22:30', 'duration': '6h 30m', 'reaching_time': '05:00', 'star_rating': '4.4', 'seats_available': '9 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadapa to Bangalore', 'busname': 'Sri Vinayaka Travels', 'price': '₹699', 'bustype': 'NON A/C Sleeper (2+1)', 'departing_time': '22:30', 'duration': '6h 15m', 'reaching_time': '04:45', 'star_rating': '3.5', 'seats_available': '3 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadapa to Bangalore', 'busname': 'S R Travels', 'price': '₹750', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '22:40', 'duration': '6h 35m', 'reaching_time': '05:15', 'star_rating': '4', 'seats_available': '4 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadapa to Bangalore', 'busname': 'S R Travels', 'price': '₹590', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '22:40', 'duration': '6h 20m', 'reaching_time': '05:00', 'star_rating': '3.7', 'seats_available': '7 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadapa to Bangalore', 'busname': 'Maheswari Travels', 'price': '₹450', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '22:40', 'duration': '7h', 'reaching_time': '05:40', 'star_rating': '4.2', 'seats_available': '14 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadapa to Bangalore', 'busname': 'AR & BCVR Travels', 'price': '₹700', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '22:45', 'duration': '7h 15m', 'reaching_time': '06:00', 'star_rating': '3.3', 'seats_available': '4 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadapa to Bangalore', 'busname': 'PRL BUS', 'price': '₹500', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '22:45', 'duration': '6h 15m', 'reaching_time': '05:00', 'star_rating': '4', 'seats_available': '19 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadapa to Bangalore', 'busname': 'SUVARNAMUKHI TRAVELS', 'price': '₹800', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '22:45', 'duration': '5h 50m', 'reaching_time': '04:35', 'star_rating': '3.9', 'seats_available': '6 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadapa to Bangalore', 'busname': 'Maheswari Travels', 'price': '₹349', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '22:50', 'duration': '7h', 'reaching_time': '05:50', 'star_rating': '3.9', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadapa to Bangalore', 'busname': 'APSRTC - 6018', 'price': '₹472', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '17:15', 'duration': '5h 55m', 'reaching_time': '23:10', 'star_rating': '3.7', 'seats_available': '32 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadapa to Bangalore', 'busname': 'APSRTC - 6010', 'price': '₹472', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '20:00', 'duration': '6h 20m', 'reaching_time': '02:20', 'star_rating': '3.7', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadapa to Bangalore', 'busname': 'APSRTC - 6240', 'price': '₹412', 'bustype': 'ULTRA DELUXE (NON-AC, 2+2 PUSH BACK)', 'departing_time': '21:00', 'duration': '7h', 'reaching_time': '04:00', 'star_rating': '4.1', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadapa to Bangalore', 'busname': 'APSRTC - 5281', 'price': '₹472', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '21:15', 'duration': '5h 55m', 'reaching_time': '03:10', 'star_rating': '3.7', 'seats_available': '33 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadapa to Bangalore', 'busname': 'APSRTC - 6205', 'price': '₹472', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '21:30', 'duration': '5h 45m', 'reaching_time': '03:15', 'star_rating': '3.5', 'seats_available': '32 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadapa to Bangalore', 'busname': 'APSRTC - 5295', 'price': '₹472', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '22:00', 'duration': '5h 5m', 'reaching_time': '03:05', 'star_rating': '3.1', 'seats_available': '33 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadapa to Bangalore', 'busname': 'APSRTC - 6073', 'price': '₹526', 'bustype': 'INDRA(A.C. Seater)', 'departing_time': '22:00', 'duration': '5h', 'reaching_time': '03:00', 'star_rating': '3.8', 'seats_available': '33 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadapa to Bangalore', 'busname': 'APSRTC - 50181', 'price': '₹472', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '22:15', 'duration': '6h 35m', 'reaching_time': '04:50', 'star_rating': '3.2', 'seats_available': '26 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadapa to Bangalore', 'busname': 'APSRTC - 6207', 'price': '₹472', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '22:30', 'duration': '5h 15m', 'reaching_time': '03:45', 'star_rating': '4.3', 'seats_available': '27 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadapa to Bangalore', 'busname': 'APSRTC - 6065', 'price': '₹472', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '22:45', 'duration': '5h 50m', 'reaching_time': '04:35', 'star_rating': '4.4', 'seats_available': '18 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Chittoor (Andhra Pradesh)', 'busname': 'FlixBus', 'price': '₹535', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '15:50', 'duration': '5h 30m', 'reaching_time': '21:20', 'star_rating': '3.6', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Chittoor (Andhra Pradesh)', 'busname': 'FRESHBUS', 'price': '₹314', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '15:50', 'duration': '3h 40m', 'reaching_time': '19:30', 'star_rating': '4.8', 'seats_available': '4 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Chittoor (Andhra Pradesh)', 'busname': 'SVD Vedansh Travels', 'price': '₹500', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '16:00', 'duration': '3h 30m', 'reaching_time': '19:30', 'star_rating': '3.8', 'seats_available': '7 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Chittoor (Andhra Pradesh)', 'busname': 'Yolo Bus', 'price': '₹199', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '16:09', 'duration': '4h 34m', 'reaching_time': '20:43', 'star_rating': '3.9', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Chittoor (Andhra Pradesh)', 'busname': 'Yolo Bus', 'price': '₹399', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '16:35', 'duration': '2h 19m', 'reaching_time': '18:54', 'star_rating': '3', 'seats_available': '1 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Chittoor (Andhra Pradesh)', 'busname': 'NueGo', 'price': '₹342', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '18:00', 'duration': '5h 40m', 'reaching_time': '23:40', 'star_rating': '4.2', 'seats_available': '22 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Chittoor (Andhra Pradesh)', 'busname': 'FRESHBUS', 'price': '₹341', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '19:35', 'duration': '4h 40m', 'reaching_time': '00:15', 'star_rating': '4.7', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Chittoor (Andhra Pradesh)', 'busname': 'Mayuri Travels', 'price': '₹504', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '18:00', 'duration': '3h 30m', 'reaching_time': '21:30', 'star_rating': '3.8', 'seats_available': '24 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Chittoor (Andhra Pradesh)', 'busname': 'OurBus India Limited', 'price': '₹449', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '18:45', 'duration': '5h 50m', 'reaching_time': '00:35', 'star_rating': '4', 'seats_available': '30 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Chittoor (Andhra Pradesh)', 'busname': 'Sai Balaji Travels', 'price': '₹549', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '18:50', 'duration': '3h 55m', 'reaching_time': '22:45', 'star_rating': '4.2', 'seats_available': '21 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Chittoor (Andhra Pradesh)', 'busname': 'APSRTC - 9046', 'price': '₹218', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '17:00', 'duration': '4h 25m', 'reaching_time': '21:25', 'star_rating': '3.4', 'seats_available': '30 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Chittoor (Andhra Pradesh)', 'busname': 'APSRTC - 9020', 'price': '₹218', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '17:30', 'duration': '4h 25m', 'reaching_time': '21:55', 'star_rating': '3.2', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Chittoor (Andhra Pradesh)', 'busname': 'APSRTC - 47211', 'price': '₹218', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '18:30', 'duration': '4h 25m', 'reaching_time': '22:55', 'star_rating': '3.6', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Chittoor (Andhra Pradesh)', 'busname': 'APSRTC - 5515', 'price': '₹239', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '20:30', 'duration': '5h 30m', 'reaching_time': '02:00', 'star_rating': '4', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Chittoor (Andhra Pradesh)', 'busname': 'APSRTC - 47216', 'price': '₹218', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '21:00', 'duration': '5h 25m', 'reaching_time': '02:25', 'star_rating': '3.2', 'seats_available': '45 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Chittoor (Andhra Pradesh)', 'busname': 'APSRTC - 5672', 'price': '₹202', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '21:30', 'duration': '4h 40m', 'reaching_time': '02:10', 'star_rating': '4.2', 'seats_available': '13 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Chittoor (Andhra Pradesh)', 'busname': 'APSRTC - 47006', 'price': '₹202', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '21:30', 'duration': '4h 30m', 'reaching_time': '02:00', 'star_rating': '3.8', 'seats_available': '32 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Chittoor (Andhra Pradesh)', 'busname': 'APSRTC - 5449', 'price': '₹304', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '21:30', 'duration': '5h 45m', 'reaching_time': '03:15', 'star_rating': '4.7', 'seats_available': '25 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Chittoor (Andhra Pradesh)', 'busname': 'APSRTC - 9887', 'price': '₹218', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '22:00', 'duration': '4h 15m', 'reaching_time': '02:15', 'star_rating': '4.2', 'seats_available': '9 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Chittoor (Andhra Pradesh)', 'busname': 'APSRTC - 9008', 'price': '₹239', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '22:00', 'duration': '5h', 'reaching_time': '03:00', 'star_rating': '3.5', 'seats_available': '21 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Anantapur (andhra pradesh) to Bangalore', 'busname': 'Sri Vengamamba Bus Transport (SVBT)', 'price': '₹450', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '23:55', 'duration': '4h 35m', 'reaching_time': '04:30', 'star_rating': '4.8', 'seats_available': '17 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Anantapur (andhra pradesh) to Bangalore', 'busname': 'Shyamoli Paribahan Pvt Ltd', 'price': '₹700', 'bustype': 'Volvo 9600 Multi-Axle A/C Sleeper (2+1)', 'departing_time': '23:40', 'duration': '2h 45m', 'reaching_time': '02:25', 'star_rating': '4.5', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Anantapur (andhra pradesh) to Bangalore', 'busname': 'Royal Rich India', 'price': '₹1,000', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '18:30', 'duration': '4h 45m', 'reaching_time': '23:15', 'star_rating': '4.8', 'seats_available': '6 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Anantapur (andhra pradesh) to Bangalore', 'busname': 'BHAIRAVA TOURS AND TRAVELS', 'price': '₹600', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '23:59', 'duration': '5h 46m', 'reaching_time': '05:45', 'star_rating': '3.6', 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Anantapur (andhra pradesh) to Bangalore', 'busname': 'HANUMAN TRANSPORTS', 'price': '₹390', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '23:45', 'duration': '4h', 'reaching_time': '03:45', 'star_rating': '4', 'seats_available': '13 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Anantapur (andhra pradesh) to Bangalore', 'busname': 'OVR Travels', 'price': '₹650', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '23:40', 'duration': '5h 50m', 'reaching_time': '05:30', 'star_rating': '4', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Anantapur (andhra pradesh) to Bangalore', 'busname': 'Kallada Travels (Suresh Kallada)', 'price': '₹800', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '20:05', 'duration': '3h 45m', 'reaching_time': '23:50', 'star_rating': '3.8', 'seats_available': '2 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Anantapur (andhra pradesh) to Bangalore', 'busname': 'Dream Line Travels Pvt Ltd', 'price': '₹850', 'bustype': 'VE A/C Sleeper (2+1)', 'departing_time': '04:20', 'duration': '5h 25m', 'reaching_time': '09:45', 'star_rating': '4.3', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Anantapur (andhra pradesh) to Bangalore', 'busname': 'OVR Travels', 'price': '₹800', 'bustype': 'NON A/C Sleeper (2+1)', 'departing_time': '23:40', 'duration': '5h 50m', 'reaching_time': '05:30', 'star_rating': '3.1', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Anantapur (andhra pradesh) to Bangalore', 'busname': 'Delta Transports Pvt Ltd', 'price': '₹1,200', 'bustype': 'Volvo 9600 Multi-Axle A/C Sleeper (2+1)', 'departing_time': '00:35', 'duration': '2h 55m', 'reaching_time': '03:30', 'star_rating': '3.8', 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Anantapur (andhra pradesh) to Bangalore', 'busname': 'APSRTC - 6745', 'price': '₹365', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '17:00', 'duration': '5h', 'reaching_time': '22:00', 'star_rating': '3.3', 'seats_available': '33 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Anantapur (andhra pradesh) to Bangalore', 'busname': 'APSRTC - 52921', 'price': '₹365', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '18:00', 'duration': '4h 50m', 'reaching_time': '22:50', 'star_rating': '2.4', 'seats_available': '30 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Anantapur (andhra pradesh) to Bangalore', 'busname': 'APSRTC - 52937', 'price': '₹307', 'bustype': 'ULTRA DELUXE (NON-AC, 2+2 PUSH BACK)', 'departing_time': '21:45', 'duration': '5h 15m', 'reaching_time': '03:00', 'star_rating': '3.9', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Anantapur (andhra pradesh) to Bangalore', 'busname': 'APSRTC - 6985', 'price': '₹365', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '22:30', 'duration': '5h', 'reaching_time': '03:30', 'star_rating': '4.2', 'seats_available': '32 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Anantapur (andhra pradesh) to Bangalore', 'busname': 'APSRTC - 6781', 'price': '₹307', 'bustype': 'ULTRA DELUXE (NON-AC, 2+2 PUSH BACK)', 'departing_time': '22:45', 'duration': '4h 45m', 'reaching_time': '03:30', 'star_rating': '3.8', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Anantapur (andhra pradesh) to Bangalore', 'busname': 'APSRTC - 6960', 'price': '₹365', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '23:15', 'duration': '3h 47m', 'reaching_time': '03:02', 'star_rating': '3.4', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Anantapur (andhra pradesh) to Bangalore', 'busname': 'APSRTC - 6937', 'price': '₹294', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '23:25', 'duration': '5h 20m', 'reaching_time': '04:45', 'star_rating': '3.6', 'seats_available': '10 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Anantapur (andhra pradesh) to Bangalore', 'busname': 'APSRTC - 6703', 'price': '₹365', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '23:45', 'duration': '3h 50m', 'reaching_time': '03:35', 'star_rating': '3', 'seats_available': '33 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Anantapur (andhra pradesh) to Bangalore', 'busname': 'APSRTC - 6962', 'price': '₹365', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '23:58', 'duration': '3h 52m', 'reaching_time': '03:50', 'star_rating': '3.5', 'seats_available': '18 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Anantapur (andhra pradesh) to Bangalore', 'busname': 'APSRTC - 6986', 'price': '₹365', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '23:59', 'duration': '4h 20m', 'reaching_time': '04:19', 'star_rating': '4.2', 'seats_available': '27 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadiri', 'busname': 'Maheswari Travels', 'price': '₹499', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '20:00', 'duration': '6h 40m', 'reaching_time': '02:40', 'star_rating': '3.8', 'seats_available': '17 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadiri', 'busname': 'Indu Travels', 'price': '₹449', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '20:00', 'duration': '6h 15m', 'reaching_time': '02:15', 'star_rating': '3.5', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadiri', 'busname': 'SEVEN HILLS TRAVELS', 'price': '₹399', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '20:25', 'duration': '6h 5m', 'reaching_time': '02:30', 'star_rating': '3.2', 'seats_available': '19 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadiri', 'busname': 'Indu Travels', 'price': '₹399', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '20:30', 'duration': '5h 45m', 'reaching_time': '02:15', 'star_rating': '3.5', 'seats_available': '21 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadiri', 'busname': 'Vasavi travels', 'price': '₹499', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '20:30', 'duration': '5h 30m', 'reaching_time': '02:00', 'star_rating': '3.8', 'seats_available': '33 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadiri', 'busname': 'AR & BCVR Travels', 'price': '₹450', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '21:15', 'duration': '5h 45m', 'reaching_time': '03:00', 'star_rating': '4.1', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadiri', 'busname': 'Ahobila Travels', 'price': '₹750', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '20:00', 'duration': '6h', 'reaching_time': '02:00', 'star_rating': '3.2', 'seats_available': '40 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadiri', 'busname': 'AR & BCVR Travels', 'price': '₹600', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '20:30', 'duration': '6h 30m', 'reaching_time': '03:00', 'star_rating': '3.9', 'seats_available': '22 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadiri', 'busname': 'SREE LEKYAA TRAVELS', 'price': '₹5,000', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '20:20', 'duration': '6h 40m', 'reaching_time': '03:00', 'star_rating': None, 'seats_available': '36 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadiri', 'busname': 'APSRTC - 52532', 'price': '₹224', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '17:30', 'duration': '4h 30m', 'reaching_time': '22:00', 'star_rating': '2.6', 'seats_available': '45 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadiri', 'busname': 'APSRTC - 52533', 'price': '₹224', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '18:45', 'duration': '5h', 'reaching_time': '23:45', 'star_rating': '3.7', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadiri', 'busname': 'APSRTC - 6160', 'price': '₹321', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:15', 'duration': '4h 44m', 'reaching_time': '23:59', 'star_rating': '3.2', 'seats_available': '14 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadiri', 'busname': 'APSRTC - 6182', 'price': '₹321', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:45', 'duration': '5h', 'reaching_time': '00:45', 'star_rating': '3.3', 'seats_available': '5 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadiri', 'busname': 'APSRTC - 52535', 'price': '₹224', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '21:30', 'duration': '5h', 'reaching_time': '02:30', 'star_rating': '2.9', 'seats_available': '44 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadiri', 'busname': 'APSRTC - 6112', 'price': '₹321', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '22:00', 'duration': '4h', 'reaching_time': '02:00', 'star_rating': '4.2', 'seats_available': '1 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadiri', 'busname': 'APSRTC - 52596', 'price': '₹205', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '22:30', 'duration': '5h', 'reaching_time': '03:30', 'star_rating': '3.9', 'seats_available': '22 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadiri', 'busname': 'APSRTC - 6129', 'price': '₹321', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '22:30', 'duration': '4h', 'reaching_time': '02:30', 'star_rating': '2.4', 'seats_available': '18 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadiri', 'busname': 'APSRTC - 52542', 'price': '₹226', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '23:00', 'duration': '5h 30m', 'reaching_time': '04:30', 'star_rating': '3.9', 'seats_available': '23 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Kadiri', 'busname': 'APSRTC - 52537', 'price': '₹224', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '23:05', 'duration': '4h 55m', 'reaching_time': '04:00', 'star_rating': '3.7', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Narasaraopet', 'busname': 'KKaveri Travels', 'price': '₹500', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '21:00', 'duration': '7h 10m', 'reaching_time': '04:10', 'star_rating': '3.5', 'seats_available': '5 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Narasaraopet', 'busname': 'V Kaveri Travels', 'price': '₹949', 'bustype': 'Volvo Multi-Axle A/C Sleeper (2+1)', 'departing_time': '21:30', 'duration': '6h 30m', 'reaching_time': '04:00', 'star_rating': '4.1', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Narasaraopet', 'busname': 'Kamakshi Tours And Travels', 'price': '₹549', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '21:30', 'duration': '6h 45m', 'reaching_time': '04:15', 'star_rating': '4.3', 'seats_available': '9 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Narasaraopet', 'busname': 'JSR Bus Tours & Travels', 'price': '₹290', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '22:35', 'duration': '5h 55m', 'reaching_time': '04:30', 'star_rating': '4.4', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Narasaraopet', 'busname': 'Kaveri Travels', 'price': '₹450', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '21:30', 'duration': '7h 15m', 'reaching_time': '04:45', 'star_rating': '4', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Narasaraopet', 'busname': 'Kundana Travels', 'price': '₹499', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '21:30', 'duration': '6h 55m', 'reaching_time': '04:25', 'star_rating': '4.6', 'seats_available': '4 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Narasaraopet', 'busname': 'Pspk Bus', 'price': '₹450', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '21:30', 'duration': '6h 40m', 'reaching_time': '04:10', 'star_rating': '4.2', 'seats_available': '21 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Narasaraopet', 'busname': 'V Kaveri Travels', 'price': '₹700', 'bustype': 'NON A/C Sleeper (2+1)', 'departing_time': '21:30', 'duration': '6h 30m', 'reaching_time': '04:00', 'star_rating': '4.3', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Narasaraopet', 'busname': 'VSN Travels', 'price': '₹590', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '21:40', 'duration': '6h 15m', 'reaching_time': '03:55', 'star_rating': '4.2', 'seats_available': '9 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Narasaraopet', 'busname': 'Sai Balaji Travels', 'price': '₹499', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '21:40', 'duration': '7h', 'reaching_time': '04:40', 'star_rating': '4.6', 'seats_available': '17 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Narasaraopet', 'busname': 'APSRTC - 5815', 'price': '₹402', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '18:50', 'duration': '7h 40m', 'reaching_time': '02:30', 'star_rating': '3.6', 'seats_available': '10 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Narasaraopet', 'busname': 'APSRTC - 4538', 'price': '₹427', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '20:45', 'duration': '7h 15m', 'reaching_time': '04:00', 'star_rating': '4', 'seats_available': '15 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Narasaraopet', 'busname': 'APSRTC - 4529', 'price': '₹427', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '20:45', 'duration': '7h 15m', 'reaching_time': '04:00', 'star_rating': '4.2', 'seats_available': '15 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Narasaraopet', 'busname': 'APSRTC - 5831', 'price': '₹526', 'bustype': 'INDRA(A.C. Seater)', 'departing_time': '21:00', 'duration': '7h', 'reaching_time': '04:00', 'star_rating': '3.7', 'seats_available': '9 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Narasaraopet', 'busname': 'APSRTC - 4527', 'price': '₹427', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '21:40', 'duration': '6h 45m', 'reaching_time': '04:25', 'star_rating': '3.9', 'seats_available': '9 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Narasaraopet', 'busname': 'APSRTC - 4515', 'price': '₹427', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '21:40', 'duration': '6h 50m', 'reaching_time': '04:30', 'star_rating': '3.9', 'seats_available': '7 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Narasaraopet', 'busname': 'APSRTC - 4556', 'price': '₹526', 'bustype': 'INDRA(A.C. Seater)', 'departing_time': '22:00', 'duration': '6h 40m', 'reaching_time': '04:40', 'star_rating': '4.4', 'seats_available': '18 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Narasaraopet', 'busname': 'APSRTC - 4448', 'price': '₹427', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '22:00', 'duration': '7h', 'reaching_time': '05:00', 'star_rating': '3.9', 'seats_available': '11 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Narasaraopet', 'busname': 'APSRTC - 4546', 'price': '₹427', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '22:00', 'duration': '6h 35m', 'reaching_time': '04:35', 'star_rating': '3.8', 'seats_available': '11 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Hyderabad to Narasaraopet', 'busname': 'APSRTC - 4484', 'price': '₹427', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '22:15', 'duration': '7h', 'reaching_time': '05:15', 'star_rating': '4.3', 'seats_available': '12 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Narasaraopet to Hyderabad', 'busname': 'VSN Travels', 'price': '₹699', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '22:30', 'duration': '7h', 'reaching_time': '05:30', 'star_rating': '4.2', 'seats_available': '7 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Narasaraopet to Hyderabad', 'busname': 'Siva Sai Balaji Tours and Travels', 'price': '₹574', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '22:30', 'duration': '6h', 'reaching_time': '04:30', 'star_rating': '4.2', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Narasaraopet to Hyderabad', 'busname': 'Siva Sai Balaji Tours and Travels', 'price': '₹449', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '22:35', 'duration': '5h 15m', 'reaching_time': '03:50', 'star_rating': '4', 'seats_available': '9 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Narasaraopet to Hyderabad', 'busname': 'Kundana Travels', 'price': '₹499', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '23:30', 'duration': '5h 30m', 'reaching_time': '05:00', 'star_rating': '3.4', 'seats_available': '3 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Narasaraopet to Hyderabad', 'busname': 'SRKT Travels', 'price': '₹400', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '22:40', 'duration': '5h 35m', 'reaching_time': '04:15', 'star_rating': '3.7', 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Narasaraopet to Hyderabad', 'busname': 'Vikram sri krishna travels', 'price': '₹130', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '22:45', 'duration': '6h 15m', 'reaching_time': '05:00', 'star_rating': '4.5', 'seats_available': '13 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Narasaraopet to Hyderabad', 'busname': 'Kundana Travels', 'price': '₹299', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '22:45', 'duration': '7h 10m', 'reaching_time': '05:55', 'star_rating': '3.9', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Narasaraopet to Hyderabad', 'busname': 'Kundana Travels', 'price': '₹990', 'bustype': 'NON A/C Seater (2+2)', 'departing_time': '22:45', 'duration': '7h 10m', 'reaching_time': '05:55', 'star_rating': '3.5', 'seats_available': '32 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Narasaraopet to Hyderabad', 'busname': 'JSR Bus Tours & Travels', 'price': '₹290', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '22:50', 'duration': '5h 25m', 'reaching_time': '04:15', 'star_rating': '4.1', 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Narasaraopet to Hyderabad', 'busname': 'PALURI GOWTHAMI TOURS AND TRAVELS', 'price': '₹490', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '22:55', 'duration': '6h 35m', 'reaching_time': '05:30', 'star_rating': '4.5', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Narasaraopet to Hyderabad', 'busname': 'APSRTC - 4482', 'price': '₹427', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '22:00', 'duration': '4h 55m', 'reaching_time': '02:55', 'star_rating': '3.8', 'seats_available': '8 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Narasaraopet to Hyderabad', 'busname': 'APSRTC - 4537', 'price': '₹427', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '22:05', 'duration': '5h 5m', 'reaching_time': '03:10', 'star_rating': '3.9', 'seats_available': '8 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Narasaraopet to Hyderabad', 'busname': 'APSRTC - 4553', 'price': '₹427', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '22:15', 'duration': '5h 10m', 'reaching_time': '03:25', 'star_rating': '4.5', 'seats_available': '8 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Narasaraopet to Hyderabad', 'busname': 'APSRTC - 4526', 'price': '₹427', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '22:35', 'duration': '5h 5m', 'reaching_time': '03:40', 'star_rating': '3.2', 'seats_available': '10 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Narasaraopet to Hyderabad', 'busname': 'APSRTC - 5816', 'price': '₹435', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '22:40', 'duration': '4h 45m', 'reaching_time': '03:25', 'star_rating': '4.6', 'seats_available': '8 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Narasaraopet to Hyderabad', 'busname': 'APSRTC - 4447', 'price': '₹427', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '23:00', 'duration': '4h 45m', 'reaching_time': '03:45', 'star_rating': '4.1', 'seats_available': '11 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Narasaraopet to Hyderabad', 'busname': 'APSRTC - 4516', 'price': '₹427', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '23:05', 'duration': '5h 15m', 'reaching_time': '04:20', 'star_rating': '3.6', 'seats_available': '9 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Narasaraopet to Hyderabad', 'busname': 'APSRTC - 4557', 'price': '₹516', 'bustype': 'INDRA(A.C. Seater)', 'departing_time': '23:15', 'duration': '4h 26m', 'reaching_time': '03:41', 'star_rating': '4.1', 'seats_available': '13 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Narasaraopet to Hyderabad', 'busname': 'APSRTC - 4530', 'price': '₹427', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '23:35', 'duration': '5h 5m', 'reaching_time': '04:40', 'star_rating': '4.2', 'seats_available': '8 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Narasaraopet to Hyderabad', 'busname': 'APSRTC - 5834', 'price': '₹563', 'bustype': 'STAR LINER(NON-AC SLEEPER 2+1)', 'departing_time': '23:50', 'duration': '4h 25m', 'reaching_time': '04:15', 'star_rating': '3.7', 'seats_available': '2 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Anantapur (andhra pradesh)', 'busname': 'Sri Vengamamba Bus Transport (SVBT)', 'price': '₹500', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '22:30', 'duration': '5h 10m', 'reaching_time': '03:40', 'star_rating': '4.5', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Anantapur (andhra pradesh)', 'busname': 'IntrCity SmartBus', 'price': '₹399', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '19:00', 'duration': '4h', 'reaching_time': '23:00', 'star_rating': '4.1', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Anantapur (andhra pradesh)', 'busname': 'IntrCity SmartBus', 'price': '₹399', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '20:55', 'duration': '3h 50m', 'reaching_time': '00:45', 'star_rating': '4.7', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Anantapur (andhra pradesh)', 'busname': 'IntrCity SmartBus', 'price': '₹509', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '19:05', 'duration': '3h 50m', 'reaching_time': '22:55', 'star_rating': '4.4', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Anantapur (andhra pradesh)', 'busname': 'IntrCity SmartBus', 'price': '₹639', 'bustype': 'VE A/C Seater / Sleeper (2+1)', 'departing_time': '22:50', 'duration': '4h', 'reaching_time': '02:50', 'star_rating': '4.7', 'seats_available': '8 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Anantapur (andhra pradesh)', 'busname': 'IntrCity SmartBus', 'price': '₹749', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '20:00', 'duration': '4h 10m', 'reaching_time': '00:10', 'star_rating': '4.8', 'seats_available': '4 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Anantapur (andhra pradesh)', 'busname': 'HANUMAN TRANSPORTS', 'price': '₹440', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '23:00', 'duration': '4h', 'reaching_time': '03:00', 'star_rating': '4.3', 'seats_available': '13 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Anantapur (andhra pradesh)', 'busname': 'IntrCity SmartBus', 'price': '₹399', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '20:25', 'duration': '4h', 'reaching_time': '00:25', 'star_rating': '4.3', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Anantapur (andhra pradesh)', 'busname': 'Jabbar Travels', 'price': '₹610', 'bustype': 'NON A/C Semi Sleeper / Sleeper (2+1)', 'departing_time': '22:30', 'duration': '4h', 'reaching_time': '02:30', 'star_rating': '4.7', 'seats_available': '8 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Anantapur (andhra pradesh)', 'busname': 'OVR Travels', 'price': '₹650', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '20:00', 'duration': '7h', 'reaching_time': '03:00', 'star_rating': '4.3', 'seats_available': '37 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Anantapur (andhra pradesh)', 'busname': 'APSRTC - 52219', 'price': '₹307', 'bustype': 'ULTRA DELUXE (NON-AC, 2+2 PUSH BACK)', 'departing_time': '17:00', 'duration': '4h 40m', 'reaching_time': '21:40', 'star_rating': '4.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Anantapur (andhra pradesh)', 'busname': 'APSRTC - 52846', 'price': '₹307', 'bustype': 'ULTRA DELUXE (NON-AC, 2+2 PUSH BACK)', 'departing_time': '17:30', 'duration': '5h', 'reaching_time': '22:30', 'star_rating': '3.7', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Anantapur (andhra pradesh)', 'busname': 'APSRTC - 52227', 'price': '₹365', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '17:45', 'duration': '4h 50m', 'reaching_time': '22:35', 'star_rating': '2.7', 'seats_available': '28 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Anantapur (andhra pradesh)', 'busname': 'APSRTC - 52220', 'price': '₹307', 'bustype': 'ULTRA DELUXE (NON-AC, 2+2 PUSH BACK)', 'departing_time': '18:15', 'duration': '4h 45m', 'reaching_time': '23:00', 'star_rating': '4.3', 'seats_available': '33 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Anantapur (andhra pradesh)', 'busname': 'APSRTC - 6598', 'price': '₹365', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:30', 'duration': '4h 25m', 'reaching_time': '23:55', 'star_rating': '2.3', 'seats_available': '18 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Anantapur (andhra pradesh)', 'busname': 'APSRTC - 6586', 'price': '₹365', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '20:00', 'duration': '4h 35m', 'reaching_time': '00:35', 'star_rating': '3.8', 'seats_available': '18 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Anantapur (andhra pradesh)', 'busname': 'APSRTC - 6319', 'price': '₹365', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '21:00', 'duration': '4h 45m', 'reaching_time': '01:45', 'star_rating': '2', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Anantapur (andhra pradesh)', 'busname': 'APSRTC - 6542', 'price': '₹434', 'bustype': 'STAR LINER(NON-AC SLEEPER 2+1)', 'departing_time': '21:00', 'duration': '4h 30m', 'reaching_time': '01:30', 'star_rating': '2.1', 'seats_available': '12 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Anantapur (andhra pradesh)', 'busname': 'APSRTC - 6345', 'price': '₹365', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '21:15', 'duration': '4h 45m', 'reaching_time': '02:00', 'star_rating': '5', 'seats_available': '20 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Bangalore to Anantapur (andhra pradesh)', 'busname': 'APSRTC - 51501', 'price': '₹433', 'bustype': 'INDRA(A.C. Seater)', 'departing_time': '21:15', 'duration': '4h 45m', 'reaching_time': '02:00', 'star_rating': '2.1', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadiri to Bangalore', 'busname': 'AR & BCVR Travels', 'price': '₹500', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '23:55', 'duration': '5h 5m', 'reaching_time': '05:00', 'star_rating': '3.3', 'seats_available': '23 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadiri to Bangalore', 'busname': 'Indu Travels', 'price': '₹449', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '23:59', 'duration': '5h 1m', 'reaching_time': '05:00', 'star_rating': '3.8', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadiri to Bangalore', 'busname': 'SEVEN HILLS TRAVELS', 'price': '₹499', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '23:59', 'duration': '5h 1m', 'reaching_time': '05:00', 'star_rating': '3.3', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadiri to Bangalore', 'busname': 'Vasavi travels', 'price': '₹500', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '23:59', 'duration': '5h 31m', 'reaching_time': '05:30', 'star_rating': '3.8', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadiri to Bangalore', 'busname': 'Maheswari Travels', 'price': '₹399', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '23:45', 'duration': '6h 5m', 'reaching_time': '05:50', 'star_rating': '1', 'seats_available': '6 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadiri to Bangalore', 'busname': 'Indu Travels', 'price': '₹449', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '23:55', 'duration': '5h 35m', 'reaching_time': '05:30', 'star_rating': '2.6', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadiri to Bangalore', 'busname': 'Ahobila Travels', 'price': '₹700', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '23:30', 'duration': '5h 30m', 'reaching_time': '05:00', 'star_rating': '3.2', 'seats_available': '1 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadiri to Bangalore', 'busname': 'APSRTC - 6824', 'price': '₹224', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '17:15', 'duration': '4h 45m', 'reaching_time': '22:00', 'star_rating': '2.6', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadiri to Bangalore', 'busname': 'APSRTC - 52511', 'price': '₹205', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '18:00', 'duration': '3h 30m', 'reaching_time': '21:30', 'star_rating': '4.4', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadiri to Bangalore', 'busname': 'APSRTC - 52545', 'price': '₹246', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '20:30', 'duration': '4h 15m', 'reaching_time': '00:45', 'star_rating': '4.1', 'seats_available': '44 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadiri to Bangalore', 'busname': 'APSRTC - 6846', 'price': '₹224', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '21:45', 'duration': '4h 30m', 'reaching_time': '02:15', 'star_rating': '4.1', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadiri to Bangalore', 'busname': 'APSRTC - 52509', 'price': '₹246', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '23:00', 'duration': '4h 20m', 'reaching_time': '03:20', 'star_rating': '3.8', 'seats_available': '41 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Kadiri to Bangalore', 'busname': 'APSRTC - 6149', 'price': '₹321', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '23:20', 'duration': '4h 55m', 'reaching_time': '04:15', 'star_rating': '4.7', 'seats_available': '17 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Visakhapatnam to Kakinada', 'busname': 'SRI VIJAYA KRISHNA TOURS and TRAVELS', 'price': '₹999', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '21:45', 'duration': '4h 45m', 'reaching_time': '02:30', 'star_rating': '3.5', 'seats_available': '1 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Visakhapatnam to Kakinada', 'busname': 'Gaganasri Travels', 'price': '₹5,000', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '23:10', 'duration': '3h 50m', 'reaching_time': '03:00', 'star_rating': None, 'seats_available': '1 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Visakhapatnam to Kakinada', 'busname': 'APSRTC - 2542', 'price': '₹281', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '17:00', 'duration': '4h 35m', 'reaching_time': '21:35', 'star_rating': '2', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Visakhapatnam to Kakinada', 'busname': 'APSRTC - 3437', 'price': '₹281', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '17:15', 'duration': '4h 15m', 'reaching_time': '21:30', 'star_rating': '2.8', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Visakhapatnam to Kakinada', 'busname': 'APSRTC - 3715', 'price': '₹281', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '17:30', 'duration': '5h 10m', 'reaching_time': '22:40', 'star_rating': '3.7', 'seats_available': '36 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Visakhapatnam to Kakinada', 'busname': 'APSRTC - 2215', 'price': '₹281', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '17:30', 'duration': '4h 35m', 'reaching_time': '22:05', 'star_rating': '3.7', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Visakhapatnam to Kakinada', 'busname': 'APSRTC - 2818', 'price': '₹281', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '18:00', 'duration': '3h 40m', 'reaching_time': '21:40', 'star_rating': '1.6', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Visakhapatnam to Kakinada', 'busname': 'APSRTC - 2834', 'price': '₹281', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '18:15', 'duration': '4h 15m', 'reaching_time': '22:30', 'star_rating': '2', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Visakhapatnam to Kakinada', 'busname': 'APSRTC - 2668', 'price': '₹281', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '18:30', 'duration': '3h 40m', 'reaching_time': '22:10', 'star_rating': '1.6', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Visakhapatnam to Kakinada', 'busname': 'APSRTC - 3713', 'price': '₹281', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:00', 'duration': '4h 59m', 'reaching_time': '23:59', 'star_rating': '4.6', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Visakhapatnam to Kakinada', 'busname': 'APSRTC - 2820', 'price': '₹340', 'bustype': 'INDRA(A.C. Seater)', 'departing_time': '19:15', 'duration': '3h 40m', 'reaching_time': '22:55', 'star_rating': '2.8', 'seats_available': '29 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Visakhapatnam to Kakinada', 'busname': 'APSRTC - 2579', 'price': '₹281', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:40', 'duration': '4h 50m', 'reaching_time': '00:30', 'star_rating': '3.7', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Tirupati to Bangalore', 'busname': 'Vibhav Holidays', 'price': '₹499', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '16:00', 'duration': '5h', 'reaching_time': '21:00', 'star_rating': '3.3', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Tirupati to Bangalore', 'busname': 'Sea Bird Tourist', 'price': '₹450', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '16:00', 'duration': '6h', 'reaching_time': '22:00', 'star_rating': '3.2', 'seats_available': '7 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Tirupati to Bangalore', 'busname': 'NueGo', 'price': '₹342', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '16:05', 'duration': '6h 45m', 'reaching_time': '22:50', 'star_rating': '4.3', 'seats_available': '1 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Tirupati to Bangalore', 'busname': 'HYBUS', 'price': '₹585', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '23:45', 'duration': '5h 30m', 'reaching_time': '05:15', 'star_rating': '4.6', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Tirupati to Bangalore', 'busname': 'HYBUS', 'price': '₹558', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '16:40', 'duration': '3h 50m', 'reaching_time': '20:30', 'star_rating': '4.5', 'seats_available': '13 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Tirupati to Bangalore', 'busname': 'KTC Travels', 'price': '₹395', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '16:45', 'duration': '5h 50m', 'reaching_time': '22:35', 'star_rating': '3.9', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Tirupati to Bangalore', 'busname': 'FRESHBUS', 'price': '₹422', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '17:00', 'duration': '5h 30m', 'reaching_time': '22:30', 'star_rating': '4.6', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Tirupati to Bangalore', 'busname': 'Bharathi Travels', 'price': '₹550', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '17:30', 'duration': '5h 45m', 'reaching_time': '23:15', 'star_rating': '3.8', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Tirupati to Bangalore', 'busname': 'NueGo', 'price': '₹332', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '18:00', 'duration': '29h 45m', 'reaching_time': '23:45', 'star_rating': '4.5', 'seats_available': '21 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Tirupati to Bangalore', 'busname': 'FRESHBUS', 'price': '₹359', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '18:00', 'duration': '6h 5m', 'reaching_time': '00:05', 'star_rating': '4.6', 'seats_available': '38 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Tirupati to Bangalore', 'busname': 'APSRTC - 9003', 'price': '₹312', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '18:00', 'duration': '6h 25m', 'reaching_time': '00:25', 'star_rating': '3.8', 'seats_available': '43 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Tirupati to Bangalore', 'busname': 'APSRTC - 9041', 'price': '₹333', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '19:00', 'duration': '6h 25m', 'reaching_time': '01:25', 'star_rating': '2.8', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Tirupati to Bangalore', 'busname': 'APSRTC - 9080', 'price': '₹333', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '19:45', 'duration': '6h 30m', 'reaching_time': '02:15', 'star_rating': '2.4', 'seats_available': '43 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Tirupati to Bangalore', 'busname': 'APSRTC - 5524', 'price': '₹296', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '20:45', 'duration': '6h 10m', 'reaching_time': '02:55', 'star_rating': '3.8', 'seats_available': '39 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Tirupati to Bangalore', 'busname': 'APSRTC - 9061', 'price': '₹333', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '21:15', 'duration': '6h 30m', 'reaching_time': '03:45', 'star_rating': '4.1', 'seats_available': '45 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Tirupati to Bangalore', 'busname': 'APSRTC - 3764', 'price': '₹426', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '21:30', 'duration': '4h 50m', 'reaching_time': '02:20', 'star_rating': '3.8', 'seats_available': '32 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Tirupati to Bangalore', 'busname': 'APSRTC - 3878', 'price': '₹426', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '21:40', 'duration': '4h 50m', 'reaching_time': '02:30', 'star_rating': '3.9', 'seats_available': '33 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Tirupati to Bangalore', 'busname': 'APSRTC - 5504', 'price': '₹312', 'bustype': 'SAPTAGIRI EXPRESS', 'departing_time': '21:45', 'duration': '6h', 'reaching_time': '03:45', 'star_rating': '3.6', 'seats_available': '45 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Tirupati to Bangalore', 'busname': 'APSRTC - 5528', 'price': '₹456', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '22:30', 'duration': '6h', 'reaching_time': '04:30', 'star_rating': '4.6', 'seats_available': '28 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'APSRTC', 'route': 'Tirupati to Bangalore', 'busname': 'APSRTC - 5314', 'price': '₹426', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '22:30', 'duration': '5h 35m', 'reaching_time': '04:05', 'star_rating': '4.1', 'seats_available': '33 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Khammam to Hyderabad', 'busname': 'Shyamoli Paribahan Pvt Ltd', 'price': '₹650', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '18:15', 'duration': '4h 20m', 'reaching_time': '22:35', 'star_rating': '4.3', 'seats_available': '5 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Khammam to Hyderabad', 'busname': 'IRA TRANSPORTS', 'price': '₹799', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '19:00', 'duration': '2h 30m', 'reaching_time': '21:30', 'star_rating': '4', 'seats_available': '24 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Khammam to Hyderabad', 'busname': 'Bharathi Tours & Travels', 'price': '₹590', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '23:29', 'duration': '5h 1m', 'reaching_time': '04:30', 'star_rating': '3.4', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Khammam to Hyderabad', 'busname': 'Bharathi Tours & Travels', 'price': '₹405', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '23:40', 'duration': '4h 20m', 'reaching_time': '04:00', 'star_rating': '4.2', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Khammam to Hyderabad', 'busname': 'LSR Travels', 'price': '₹300', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '23:40', 'duration': '6h 25m', 'reaching_time': '06:05', 'star_rating': None, 'seats_available': '23 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Khammam to Hyderabad', 'busname': 'Sri KVR Travels', 'price': '₹510', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '23:45', 'duration': '5h 15m', 'reaching_time': '05:00', 'star_rating': '4.2', 'seats_available': '32 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Khammam to Hyderabad', 'busname': 'Sri Venkateswara Travels', 'price': '₹350', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '23:45', 'duration': '6h 15m', 'reaching_time': '06:00', 'star_rating': '3.2', 'seats_available': '19 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Khammam to Hyderabad', 'busname': 'RRR Travels', 'price': '₹700', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '23:45', 'duration': '5h 30m', 'reaching_time': '05:15', 'star_rating': '3.9', 'seats_available': '31 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Khammam to Hyderabad', 'busname': 'Sri Krishna Travels(VGN)', 'price': '₹450', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '23:55', 'duration': '6h 10m', 'reaching_time': '06:05', 'star_rating': '3.7', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Khammam to Hyderabad', 'busname': 'Sri Krishna Travels(VGN)', 'price': '₹400', 'bustype': 'NON A/C Hi-Tech (2+2)', 'departing_time': '23:55', 'duration': '5h 30m', 'reaching_time': '05:25', 'star_rating': '3.7', 'seats_available': '30 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Khammam to Hyderabad', 'busname': 'TGSRTC - 8657', 'price': '₹389', 'bustype': 'Rajdhani (AC Semi Sleeper 2+2)', 'departing_time': '16:45', 'duration': '3h', 'reaching_time': '19:45', 'star_rating': '3.6', 'seats_available': '28 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Khammam to Hyderabad', 'busname': 'TGSRTC - 64567', 'price': '₹389', 'bustype': 'Rajdhani (AC Semi Sleeper 2+2)', 'departing_time': '16:50', 'duration': '3h 30m', 'reaching_time': '20:20', 'star_rating': '3.8', 'seats_available': '26 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Khammam to Hyderabad', 'busname': 'TGSRTC - 1319', 'price': '₹389', 'bustype': 'LAHARI A/C SLEEPER CUM SEATER', 'departing_time': '17:10', 'duration': '2h 40m', 'reaching_time': '19:50', 'star_rating': '3.4', 'seats_available': '23 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Khammam to Hyderabad', 'busname': 'TGSRTC - 8613', 'price': '₹320', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '17:20', 'duration': '5h 10m', 'reaching_time': '22:30', 'star_rating': '3.8', 'seats_available': '30 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Khammam to Hyderabad', 'busname': 'TGSRTC - 8675', 'price': '₹389', 'bustype': 'Rajdhani (AC Semi Sleeper 2+2)', 'departing_time': '17:30', 'duration': '3h 30m', 'reaching_time': '21:00', 'star_rating': '4.7', 'seats_available': '28 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Khammam to Hyderabad', 'busname': 'TGSRTC - 8822', 'price': '₹320', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '17:45', 'duration': '3h 55m', 'reaching_time': '21:40', 'star_rating': '3.8', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Khammam to Hyderabad', 'busname': 'TGSRTC - 8859', 'price': '₹320', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '18:00', 'duration': '4h 10m', 'reaching_time': '22:10', 'star_rating': '3.8', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Khammam to Hyderabad', 'busname': 'TGSRTC - 1952', 'price': '₹389', 'bustype': 'LAHARI A/C SLEEPER CUM SEATER', 'departing_time': '18:10', 'duration': '2h 10m', 'reaching_time': '20:20', 'star_rating': '2.6', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Khammam to Hyderabad', 'busname': 'TGSRTC - 64089', 'price': '₹389', 'bustype': 'Rajdhani (AC Semi Sleeper 2+2)', 'departing_time': '18:15', 'duration': '3h 5m', 'reaching_time': '21:20', 'star_rating': '4.1', 'seats_available': '30 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Khammam to Hyderabad', 'busname': 'TGSRTC - 8919', 'price': '₹320', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '18:30', 'duration': '3h 40m', 'reaching_time': '22:10', 'star_rating': '3.8', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Srisailam', 'busname': 'TGSRTC - 4322', 'price': '₹431', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:45', 'duration': '12h 35m', 'reaching_time': '08:20', 'star_rating': '2.8', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Srisailam', 'busname': 'TGSRTC - 1252', 'price': '₹431', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '20:15', 'duration': '12h 5m', 'reaching_time': '08:20', 'star_rating': '2.5', 'seats_available': '14 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Srisailam', 'busname': 'TGSRTC - 1280', 'price': '₹431', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '20:45', 'duration': '11h 35m', 'reaching_time': '08:20', 'star_rating': '1.8', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Srisailam', 'busname': 'TGSRTC - 1045', 'price': '₹431', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '21:30', 'duration': '11h', 'reaching_time': '08:30', 'star_rating': '4.1', 'seats_available': '2 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Srisailam', 'busname': 'TGSRTC - 1250', 'price': '₹431', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '21:45', 'duration': '10h 35m', 'reaching_time': '08:20', 'star_rating': '3.7', 'seats_available': '8 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Srisailam', 'busname': 'TGSRTC - 1286', 'price': '₹431', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '22:15', 'duration': '10h 5m', 'reaching_time': '08:20', 'star_rating': '4.3', 'seats_available': '5 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Khammam', 'busname': 'Siri Tours and Travels', 'price': '₹500', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '18:30', 'duration': '6h', 'reaching_time': '00:30', 'star_rating': '3.1', 'seats_available': '23 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Khammam', 'busname': 'Puri Jagannadh Tours And Travels', 'price': '₹555', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '18:30', 'duration': '8h', 'reaching_time': '02:30', 'star_rating': '3.9', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Khammam', 'busname': 'Gupta Travels Galaxy', 'price': '₹800', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '18:45', 'duration': '4h 25m', 'reaching_time': '23:10', 'star_rating': '3.3', 'seats_available': '23 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Khammam', 'busname': 'Sri Krishna Travels', 'price': '₹600', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '19:30', 'duration': '4h', 'reaching_time': '23:30', 'star_rating': '3.7', 'seats_available': '14 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Khammam', 'busname': 'SBS Tours and Travels', 'price': '₹444', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '19:50', 'duration': '6h 34m', 'reaching_time': '02:24', 'star_rating': '3.7', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Khammam', 'busname': 'Sri Krishna Travels(VGN)', 'price': '₹450', 'bustype': 'NON A/C Hi-Tech (2+2)', 'departing_time': '20:20', 'duration': '7h 10m', 'reaching_time': '03:30', 'star_rating': '3.7', 'seats_available': '17 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Khammam', 'busname': 'Sri Krishna Travels(VGN)', 'price': '₹450', 'bustype': 'NON A/C Hi-Tech (2+2)', 'departing_time': '20:20', 'duration': '7h 10m', 'reaching_time': '03:30', 'star_rating': '3.7', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Khammam', 'busname': 'Sri KVR Travels', 'price': '₹510', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '20:30', 'duration': '6h 30m', 'reaching_time': '03:00', 'star_rating': '4.2', 'seats_available': '5 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Khammam', 'busname': 'DMR Travels', 'price': '₹900', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '20:50', 'duration': '5h 40m', 'reaching_time': '02:30', 'star_rating': '3.6', 'seats_available': '24 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Khammam', 'busname': 'Sri KVR Travels', 'price': '₹450', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '21:00', 'duration': '6h 15m', 'reaching_time': '03:15', 'star_rating': '4.3', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Khammam', 'busname': 'TGSRTC - 8687', 'price': '₹389', 'bustype': 'Rajdhani (AC Semi Sleeper 2+2)', 'departing_time': '16:45', 'duration': '3h 30m', 'reaching_time': '20:15', 'star_rating': '3.4', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Khammam', 'busname': 'TGSRTC - 64021', 'price': '₹389', 'bustype': 'Rajdhani (AC Semi Sleeper 2+2)', 'departing_time': '17:30', 'duration': '3h 30m', 'reaching_time': '21:00', 'star_rating': '1.7', 'seats_available': '26 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Khammam', 'busname': 'TGSRTC - 64363', 'price': '₹320', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '18:00', 'duration': '4h 20m', 'reaching_time': '22:20', 'star_rating': '3.9', 'seats_available': '26 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Khammam', 'busname': 'TGSRTC - 8610', 'price': '₹389', 'bustype': 'Rajdhani (AC Semi Sleeper 2+2)', 'departing_time': '18:15', 'duration': '3h 30m', 'reaching_time': '21:45', 'star_rating': '4.2', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Khammam', 'busname': 'TGSRTC - 7645', 'price': '₹320', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '18:30', 'duration': '7h 15m', 'reaching_time': '01:45', 'star_rating': '3.9', 'seats_available': '28 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Khammam', 'busname': 'TGSRTC - 8928', 'price': '₹320', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '18:30', 'duration': '4h 40m', 'reaching_time': '23:10', 'star_rating': '3.9', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Khammam', 'busname': 'TGSRTC - 8618', 'price': '₹320', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '18:40', 'duration': '4h 20m', 'reaching_time': '23:00', 'star_rating': '3.9', 'seats_available': '30 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Khammam', 'busname': 'TGSRTC - 8699', 'price': '₹389', 'bustype': 'Rajdhani (AC Semi Sleeper 2+2)', 'departing_time': '19:00', 'duration': '3h 30m', 'reaching_time': '22:30', 'star_rating': '2.3', 'seats_available': '32 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Khammam', 'busname': 'TGSRTC - 8933', 'price': '₹320', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:15', 'duration': '4h 50m', 'reaching_time': '00:05', 'star_rating': '3.9', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Khammam', 'busname': 'TGSRTC - 8643', 'price': '₹389', 'bustype': 'Rajdhani (AC Semi Sleeper 2+2)', 'departing_time': '19:45', 'duration': '3h 30m', 'reaching_time': '23:15', 'star_rating': '2.3', 'seats_available': '26 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Vijayawada', 'busname': 'IntrCity SmartBus', 'price': '₹539', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '23:05', 'duration': '7h 25m', 'reaching_time': '06:30', 'star_rating': '4.7', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Vijayawada', 'busname': 'IntrCity SmartBus', 'price': '₹649', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '23:35', 'duration': '6h 30m', 'reaching_time': '06:05', 'star_rating': '4.6', 'seats_available': '13 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Vijayawada', 'busname': 'FRESHBUS', 'price': '₹395', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '23:20', 'duration': '6h 40m', 'reaching_time': '06:00', 'star_rating': '4.6', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Vijayawada', 'busname': 'FRESHBUS', 'price': '₹602', 'bustype': 'Electric A/C Seater/Sleeper (2+1)', 'departing_time': '23:40', 'duration': '6h 40m', 'reaching_time': '06:20', 'star_rating': '4.6', 'seats_available': '37 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Vijayawada', 'busname': 'FRESHBUS', 'price': '₹539', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '22:35', 'duration': '6h 50m', 'reaching_time': '05:25', 'star_rating': '4.6', 'seats_available': '31 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Vijayawada', 'busname': 'FRESHBUS', 'price': '₹299', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '16:10', 'duration': '7h 55m', 'reaching_time': '00:05', 'star_rating': '4.8', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Vijayawada', 'busname': 'FRESHBUS', 'price': '₹300', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '17:35', 'duration': '7h 30m', 'reaching_time': '01:05', 'star_rating': '4.5', 'seats_available': '31 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Vijayawada', 'busname': 'zingbus plus', 'price': '₹527', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '23:10', 'duration': '6h 45m', 'reaching_time': '05:55', 'star_rating': '4.4', 'seats_available': '25 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Vijayawada', 'busname': 'NueGo', 'price': '₹474', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '21:30', 'duration': '7h 25m', 'reaching_time': '04:55', 'star_rating': '4.5', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Vijayawada', 'busname': 'FRESHBUS', 'price': '₹575', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '23:59', 'duration': '6h 36m', 'reaching_time': '06:35', 'star_rating': '4.5', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Vijayawada', 'busname': 'APSRTC - 3570', 'price': '₹670', 'bustype': 'AMARAVATHI (VOLVO / SCANIA A.C Multi Axle)', 'departing_time': '17:15', 'duration': '6h 40m', 'reaching_time': '23:55', 'star_rating': '3.7', 'seats_available': '36 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Vijayawada', 'busname': 'APSRTC - 3366', 'price': '₹469', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '17:55', 'duration': '7h 50m', 'reaching_time': '01:45', 'star_rating': '2.6', 'seats_available': '29 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Vijayawada', 'busname': 'APSRTC - 3360', 'price': '₹720', 'bustype': 'AMARAVATHI (VOLVO / SCANIA A.C Multi Axle)', 'departing_time': '18:25', 'duration': '7h 5m', 'reaching_time': '01:30', 'star_rating': '3.6', 'seats_available': '38 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Vijayawada', 'busname': 'APSRTC - 2575', 'price': '₹639', 'bustype': 'NIGHT RIDER (SEATER CUM SLEEPER)', 'departing_time': '18:55', 'duration': '6h 50m', 'reaching_time': '01:45', 'star_rating': '2.5', 'seats_available': '30 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Vijayawada', 'busname': 'APSRTC - 2674', 'price': '₹469', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:15', 'duration': '7h 50m', 'reaching_time': '03:05', 'star_rating': '3.4', 'seats_available': '27 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Vijayawada', 'busname': 'APSRTC - 2898', 'price': '₹607', 'bustype': 'STAR LINER(NON-AC SLEEPER 2+1)', 'departing_time': '19:25', 'duration': '6h 20m', 'reaching_time': '01:45', 'star_rating': '4.8', 'seats_available': '22 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Vijayawada', 'busname': 'APSRTC - 4916', 'price': '₹469', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:25', 'duration': '6h 50m', 'reaching_time': '02:15', 'star_rating': '1.2', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Vijayawada', 'busname': 'APSRTC - 98050', 'price': '₹436', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:30', 'duration': '7h', 'reaching_time': '02:30', 'star_rating': '4.9', 'seats_available': '32 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Vijayawada', 'busname': 'APSRTC - 2563', 'price': '₹528', 'bustype': 'INDRA(A.C. Seater)', 'departing_time': '19:30', 'duration': '5h 35m', 'reaching_time': '01:05', 'star_rating': '2.7', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Vijayawada', 'busname': 'APSRTC - 24262', 'price': '₹469', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:30', 'duration': '7h', 'reaching_time': '02:30', 'star_rating': '3.8', 'seats_available': '17 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Karimnagar to Hyderabad', 'busname': 'Mythri Tours And Travels', 'price': '₹449', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '18:40', 'duration': '3h 40m', 'reaching_time': '22:20', 'star_rating': '3.5', 'seats_available': '2 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Karimnagar to Hyderabad', 'busname': 'Harikrishna Travels', 'price': '₹700', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '20:30', 'duration': '2h 30m', 'reaching_time': '23:00', 'star_rating': '2.2', 'seats_available': '4 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Karimnagar to Hyderabad', 'busname': 'IRA TRANSPORTS', 'price': '₹1,286', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '18:30', 'duration': '3h 15m', 'reaching_time': '21:45', 'star_rating': '4', 'seats_available': '5 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Karimnagar to Hyderabad', 'busname': 'IRA TRANSPORTS', 'price': '₹1,574', 'bustype': 'VE A/C Sleeper (2+1)', 'departing_time': '18:45', 'duration': '3h 45m', 'reaching_time': '22:30', 'star_rating': '4', 'seats_available': '4 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Karimnagar to Hyderabad', 'busname': 'Harikrishna Travels', 'price': '₹3,999', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '20:00', 'duration': '3h', 'reaching_time': '23:00', 'star_rating': '2.2', 'seats_available': '36 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Karimnagar to Hyderabad', 'busname': 'TGSRTC - 9443', 'price': '₹260', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '16:30', 'duration': '3h 20m', 'reaching_time': '19:50', 'star_rating': '3.2', 'seats_available': '27 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Karimnagar to Hyderabad', 'busname': 'TGSRTC - 7053', 'price': '₹260', 'bustype': 'e-SUP LUXURY (2+2)', 'departing_time': '16:30', 'duration': '3h', 'reaching_time': '19:30', 'star_rating': '4.4', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Karimnagar to Hyderabad', 'busname': 'TGSRTC - 7122', 'price': '₹315', 'bustype': 'Rajdhani (AC Semi Sleeper 2+2)', 'departing_time': '16:30', 'duration': '3h', 'reaching_time': '19:30', 'star_rating': '3.8', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Karimnagar to Hyderabad', 'busname': 'TGSRTC - 7055', 'price': '₹260', 'bustype': 'e-SUP LUXURY (2+2)', 'departing_time': '17:10', 'duration': '3h', 'reaching_time': '20:10', 'star_rating': '4', 'seats_available': '36 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Karimnagar to Hyderabad', 'busname': 'TGSRTC - 7395', 'price': '₹260', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '17:20', 'duration': '3h', 'reaching_time': '20:20', 'star_rating': '3.5', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Karimnagar to Hyderabad', 'busname': 'TGSRTC - 7808', 'price': '₹315', 'bustype': 'LAHARI A/C SLEEPER CUM SEATER', 'departing_time': '17:30', 'duration': '3h 45m', 'reaching_time': '21:15', 'star_rating': '4.4', 'seats_available': '26 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Karimnagar to Hyderabad', 'busname': 'TGSRTC - 7057', 'price': '₹260', 'bustype': 'e-SUP LUXURY (2+2)', 'departing_time': '17:50', 'duration': '3h', 'reaching_time': '20:50', 'star_rating': '5', 'seats_available': '36 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Karimnagar to Hyderabad', 'busname': 'TGSRTC - 7326', 'price': '₹260', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '17:55', 'duration': '3h', 'reaching_time': '20:55', 'star_rating': '2.5', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Karimnagar to Hyderabad', 'busname': 'TGSRTC - 7059', 'price': '₹260', 'bustype': 'e-SUP LUXURY (2+2)', 'departing_time': '18:30', 'duration': '3h', 'reaching_time': '21:30', 'star_rating': '4.8', 'seats_available': '36 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Karimnagar to Hyderabad', 'busname': 'TGSRTC - 7061', 'price': '₹260', 'bustype': 'e-SUP LUXURY (2+2)', 'departing_time': '18:50', 'duration': '3h', 'reaching_time': '21:50', 'star_rating': '3.3', 'seats_available': '36 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Adilabad', 'busname': 'Muskan Travels', 'price': '₹344', 'bustype': 'NON A/C Semi Sleeper (2+2)', 'departing_time': '22:00', 'duration': '8h 30m', 'reaching_time': '06:30', 'star_rating': '3.9', 'seats_available': '14 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Adilabad', 'busname': 'zingbus plus', 'price': '₹862', 'bustype': 'VE A/C Sleeper (2+1)', 'departing_time': '22:45', 'duration': '6h 25m', 'reaching_time': '05:10', 'star_rating': '4.6', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Adilabad', 'busname': 'Sitara Travels', 'price': '₹349', 'bustype': 'NON A/C Airbus (2+2)', 'departing_time': '22:00', 'duration': '7h 15m', 'reaching_time': '05:15', 'star_rating': '3.7', 'seats_available': '24 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Adilabad', 'busname': 'Diamond Travels', 'price': '₹400', 'bustype': 'NON A/C Seater (2+1)', 'departing_time': '22:30', 'duration': '6h 30m', 'reaching_time': '05:00', 'star_rating': '3.7', 'seats_available': '21 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Adilabad', 'busname': 'Orange Tours and Travels', 'price': '₹450', 'bustype': 'Volvo Multi-Axle A/C (2+2)', 'departing_time': '18:00', 'duration': '5h 15m', 'reaching_time': '23:15', 'star_rating': '3.6', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Adilabad', 'busname': 'Sahara Tours and Travels', 'price': '₹400', 'bustype': 'Bharat Benz A/C Seater (2+2)', 'departing_time': '18:00', 'duration': '5h 15m', 'reaching_time': '23:15', 'star_rating': '3.6', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Adilabad', 'busname': 'Savera Tours and Travels', 'price': '₹569', 'bustype': 'NON A/C Sleeper (2+1)', 'departing_time': '22:15', 'duration': '7h 15m', 'reaching_time': '05:30', 'star_rating': '3.4', 'seats_available': '1 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Adilabad', 'busname': 'MDK TOURS AND TRAVELS', 'price': '₹387', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '22:00', 'duration': '7h 30m', 'reaching_time': '05:30', 'star_rating': '3.5', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Adilabad', 'busname': 'Sahara Tours and Travels', 'price': '₹350', 'bustype': 'Non AC Seater (2+2)', 'departing_time': '22:00', 'duration': '6h 30m', 'reaching_time': '04:30', 'star_rating': '3.5', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Adilabad', 'busname': 'Hans Travels (I) Private Limited', 'price': '₹999', 'bustype': 'Volvo 9600 Multi-Axle A/C Sleeper (2+1)', 'departing_time': '16:00', 'duration': '6h 30m', 'reaching_time': '22:30', 'star_rating': '3.9', 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Adilabad', 'busname': 'TGSRTC - 1135', 'price': '₹509', 'bustype': 'Lahari Non A/C Sleeper Cum Seater', 'departing_time': '16:30', 'duration': '5h 40m', 'reaching_time': '22:10', 'star_rating': '1.8', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Adilabad', 'busname': 'TGSRTC - 1139', 'price': '₹584', 'bustype': 'Rajdhani (AC Semi Sleeper 2+2)', 'departing_time': '17:15', 'duration': '5h 45m', 'reaching_time': '23:00', 'star_rating': '3.6', 'seats_available': '28 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Adilabad', 'busname': 'TGSRTC - 1155', 'price': '₹509', 'bustype': 'Lahari Non A/C Sleeper Cum Seater', 'departing_time': '17:30', 'duration': '7h 40m', 'reaching_time': '01:10', 'star_rating': '3.6', 'seats_available': '36 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Adilabad', 'busname': 'TGSRTC - 7954', 'price': '₹402', 'bustype': 'SEMI DELUXE', 'departing_time': '18:00', 'duration': '6h 40m', 'reaching_time': '00:40', 'star_rating': '3.6', 'seats_available': '54 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Adilabad', 'busname': 'TGSRTC - 7932', 'price': '₹481', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:00', 'duration': '7h 30m', 'reaching_time': '02:30', 'star_rating': '4', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Adilabad', 'busname': 'TGSRTC - 7908', 'price': '₹481', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '20:30', 'duration': '7h 25m', 'reaching_time': '03:55', 'star_rating': '4.1', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Adilabad', 'busname': 'TGSRTC - 7920', 'price': '₹481', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '21:00', 'duration': '7h 25m', 'reaching_time': '04:25', 'star_rating': '3.7', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Adilabad', 'busname': 'TGSRTC - 7906', 'price': '₹483', 'bustype': 'Lahari Non A/C Sleeper Cum Seater', 'departing_time': '21:30', 'duration': '7h 25m', 'reaching_time': '04:55', 'star_rating': '3.2', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Adilabad', 'busname': 'TGSRTC - 7912', 'price': '₹481', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '21:50', 'duration': '7h 25m', 'reaching_time': '05:15', 'star_rating': '4.9', 'seats_available': '26 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Adilabad', 'busname': 'TGSRTC - 1656', 'price': '₹584', 'bustype': 'Rajdhani (AC Semi Sleeper 2+2)', 'departing_time': '22:15', 'duration': '5h 40m', 'reaching_time': '03:55', 'star_rating': '3.2', 'seats_available': '32 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Mancherial', 'busname': 'Bharathi Tours & Travels', 'price': '₹441', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '21:20', 'duration': '6h 40m', 'reaching_time': '04:00', 'star_rating': '3.3', 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Mancherial', 'busname': 'DNR Express', 'price': '₹800', 'bustype': 'AC Sleeper (2+1)', 'departing_time': '21:30', 'duration': '4h 30m', 'reaching_time': '02:00', 'star_rating': '3.2', 'seats_available': '9 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Mancherial', 'busname': 'TGSRTC - 8099', 'price': '₹391', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '16:45', 'duration': '5h 15m', 'reaching_time': '22:00', 'star_rating': '3.7', 'seats_available': '30 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Mancherial', 'busname': 'TGSRTC - 8092', 'price': '₹391', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '17:15', 'duration': '5h 15m', 'reaching_time': '22:30', 'star_rating': '3.7', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Mancherial', 'busname': 'TGSRTC - 8034', 'price': '₹391', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '18:00', 'duration': '6h 5m', 'reaching_time': '00:05', 'star_rating': '3.7', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Mancherial', 'busname': 'TGSRTC - 8019', 'price': '₹474', 'bustype': 'Rajdhani (AC Semi Sleeper 2+2)', 'departing_time': '18:00', 'duration': '5h 15m', 'reaching_time': '23:15', 'star_rating': '3.7', 'seats_available': '30 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Mancherial', 'busname': 'TGSRTC - 8180', 'price': '₹406', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '20:00', 'duration': '5h 25m', 'reaching_time': '01:25', 'star_rating': '2.6', 'seats_available': '29 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Mancherial', 'busname': 'TGSRTC - 8158', 'price': '₹391', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '21:15', 'duration': '5h 50m', 'reaching_time': '03:05', 'star_rating': '3.7', 'seats_available': '30 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Mancherial', 'busname': 'TGSRTC - 8042', 'price': '₹406', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '21:20', 'duration': '6h 15m', 'reaching_time': '03:35', 'star_rating': '4.7', 'seats_available': '26 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Mancherial', 'busname': 'TGSRTC - 8011', 'price': '₹474', 'bustype': 'Rajdhani (AC Semi Sleeper 2+2)', 'departing_time': '21:45', 'duration': '5h 15m', 'reaching_time': '03:00', 'star_rating': '3.7', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Mancherial', 'busname': 'TGSRTC - 8032', 'price': '₹424', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '21:45', 'duration': '6h 15m', 'reaching_time': '04:00', 'star_rating': None, 'seats_available': '20 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Mancherial', 'busname': 'TGSRTC - 8028', 'price': '₹386', 'bustype': 'Lahari Non A/C Sleeper Cum Seater', 'departing_time': '22:30', 'duration': '5h 55m', 'reaching_time': '04:25', 'star_rating': '3.3', 'seats_available': '16 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Kothagudem to Hyderabad', 'busname': 'Bharathi Tours & Travels', 'price': '₹690', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '23:00', 'duration': '6h 30m', 'reaching_time': '05:30', 'star_rating': '4.4', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Kothagudem to Hyderabad', 'busname': 'VSR Tours and Travels', 'price': '₹649', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '23:45', 'duration': '6h 15m', 'reaching_time': '06:00', 'star_rating': '4.5', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Kothagudem to Hyderabad', 'busname': 'BSR Tours And Travels', 'price': '₹680', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '23:15', 'duration': '6h 45m', 'reaching_time': '06:00', 'star_rating': '4.3', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Kothagudem to Hyderabad', 'busname': 'BSR Tours And Travels', 'price': '₹680', 'bustype': 'Volvo Multi-Axle A/C Sleeper (2+1)', 'departing_time': '23:30', 'duration': '6h 30m', 'reaching_time': '06:00', 'star_rating': '4.3', 'seats_available': '5 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Kothagudem to Hyderabad', 'busname': 'Bharathi Tours & Travels', 'price': '₹405', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '23:10', 'duration': '5h 50m', 'reaching_time': '05:00', 'star_rating': '3.9', 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Kothagudem to Hyderabad', 'busname': 'Bharathi Tours & Travels', 'price': '₹531', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '23:25', 'duration': '5h 35m', 'reaching_time': '05:00', 'star_rating': '4.3', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Kothagudem to Hyderabad', 'busname': 'Bharathi Tours & Travels', 'price': '₹450', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '23:00', 'duration': '6h', 'reaching_time': '05:00', 'star_rating': '3.8', 'seats_available': '22 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Kothagudem to Hyderabad', 'busname': 'SAIRAJ TRAVELS', 'price': '₹600', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '23:00', 'duration': '6h 30m', 'reaching_time': '05:30', 'star_rating': '4.7', 'seats_available': '5 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Kothagudem to Hyderabad', 'busname': 'SAIRAJ TRAVELS', 'price': '₹500', 'bustype': 'NON A/C Seater (2+2)', 'departing_time': '23:00', 'duration': '7h 15m', 'reaching_time': '06:15', 'star_rating': '4.5', 'seats_available': '8 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Kothagudem to Hyderabad', 'busname': 'Nani’s Sai Krishna Travels', 'price': '₹400', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '23:09', 'duration': '5h 51m', 'reaching_time': '05:00', 'star_rating': '3.3', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Kothagudem to Hyderabad', 'busname': 'TGSRTC - 6914', 'price': '₹451', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '16:50', 'duration': '6h 15m', 'reaching_time': '23:05', 'star_rating': '3.7', 'seats_available': '28 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Kothagudem to Hyderabad', 'busname': 'TGSRTC - 8791', 'price': '₹548', 'bustype': 'Rajdhani (AC Semi Sleeper 2+2)', 'departing_time': '19:00', 'duration': '6h', 'reaching_time': '01:00', 'star_rating': '3.5', 'seats_available': '32 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Kothagudem to Hyderabad', 'busname': 'TGSRTC - 6631', 'price': '₹470', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '20:30', 'duration': '7h', 'reaching_time': '03:30', 'star_rating': '3.7', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Kothagudem to Hyderabad', 'busname': 'TGSRTC - 8754', 'price': '₹451', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '20:45', 'duration': '6h', 'reaching_time': '02:45', 'star_rating': '3.7', 'seats_available': '29 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Kothagudem to Hyderabad', 'busname': 'TGSRTC - 1224', 'price': '₹392', 'bustype': 'Deluxe (Non AC Seater 2+2)', 'departing_time': '21:00', 'duration': '7h 53m', 'reaching_time': '04:53', 'star_rating': '3.7', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Kothagudem to Hyderabad', 'busname': 'TGSRTC - 8797', 'price': '₹548', 'bustype': 'Rajdhani (AC Semi Sleeper 2+2)', 'departing_time': '21:15', 'duration': '6h', 'reaching_time': '03:15', 'star_rating': '3.7', 'seats_available': '32 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Kothagudem to Hyderabad', 'busname': 'TGSRTC - 8925', 'price': '₹497', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '21:30', 'duration': '7h 30m', 'reaching_time': '05:00', 'star_rating': '3.6', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Kothagudem to Hyderabad', 'busname': 'TGSRTC - 8718', 'price': '₹451', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '21:40', 'duration': '6h 5m', 'reaching_time': '03:45', 'star_rating': '4.3', 'seats_available': '30 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Kothagudem to Hyderabad', 'busname': 'TGSRTC - 7646', 'price': '₹451', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '22:00', 'duration': '6h', 'reaching_time': '04:00', 'star_rating': '2.9', 'seats_available': '29 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Kothagudem to Hyderabad', 'busname': 'TGSRTC - 1323', 'price': '₹548', 'bustype': 'LAHARI A/C SLEEPER CUM SEATER', 'departing_time': '22:05', 'duration': '4h 15m', 'reaching_time': '02:20', 'star_rating': '3.7', 'seats_available': '33 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Guntur (Andhra Pradesh) to Hyderabad', 'busname': 'IntrCity SmartBus', 'price': '₹569', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '22:45', 'duration': '7h', 'reaching_time': '05:45', 'star_rating': '4.6', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Guntur (Andhra Pradesh) to Hyderabad', 'busname': 'IntrCity SmartBus', 'price': '₹679', 'bustype': 'Scania AC Multi Axle Sleeper (2+1)', 'departing_time': '22:35', 'duration': '6h 45m', 'reaching_time': '05:20', 'star_rating': '4.6', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Guntur (Andhra Pradesh) to Hyderabad', 'busname': 'IntrCity SmartBus', 'price': '₹779', 'bustype': 'AC Sleeper (2+1)', 'departing_time': '22:25', 'duration': '7h 15m', 'reaching_time': '05:40', 'star_rating': '4.6', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Guntur (Andhra Pradesh) to Hyderabad', 'busname': 'FRESHBUS', 'price': '₹413', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '16:10', 'duration': '7h 25m', 'reaching_time': '23:35', 'star_rating': '4.6', 'seats_available': '29 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Guntur (Andhra Pradesh) to Hyderabad', 'busname': 'Sri Sanvi Tours and Travels', 'price': '₹764', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '22:20', 'duration': '7h 10m', 'reaching_time': '05:30', 'star_rating': '4.4', 'seats_available': '13 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Guntur (Andhra Pradesh) to Hyderabad', 'busname': 'Sri Sanvi Tours and Travels', 'price': '₹764', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '22:40', 'duration': '6h 50m', 'reaching_time': '05:30', 'star_rating': '4.5', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Guntur (Andhra Pradesh) to Hyderabad', 'busname': 'zingbus plus', 'price': '₹474', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '22:35', 'duration': '7h 5m', 'reaching_time': '05:40', 'star_rating': '4.3', 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Guntur (Andhra Pradesh) to Hyderabad', 'busname': 'zingbus plus', 'price': '₹459', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '22:55', 'duration': '7h 10m', 'reaching_time': '06:05', 'star_rating': '4.3', 'seats_available': '25 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Guntur (Andhra Pradesh) to Hyderabad', 'busname': 'Teja Tours and Travels', 'price': '₹432', 'bustype': 'A/C Seater/Sleeper (2+1)', 'departing_time': '23:00', 'duration': '7h', 'reaching_time': '06:00', 'star_rating': '4.4', 'seats_available': '9 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Guntur (Andhra Pradesh) to Hyderabad', 'busname': 'Pavan Tours and Travels', 'price': '₹540', 'bustype': 'VE A/C Sleeper (2+1)', 'departing_time': '22:00', 'duration': '6h 55m', 'reaching_time': '04:55', 'star_rating': '4.3', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Guntur (Andhra Pradesh) to Hyderabad', 'busname': 'APSRTC - 4256', 'price': '₹451', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '20:30', 'duration': '6h 25m', 'reaching_time': '02:55', 'star_rating': '3.9', 'seats_available': '29 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Guntur (Andhra Pradesh) to Hyderabad', 'busname': 'APSRTC - 4737', 'price': '₹483', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '21:00', 'duration': '5h 40m', 'reaching_time': '02:40', 'star_rating': '3.1', 'seats_available': '29 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Guntur (Andhra Pradesh) to Hyderabad', 'busname': 'APSRTC - 4250', 'price': '₹483', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '21:15', 'duration': '5h 50m', 'reaching_time': '03:05', 'star_rating': '3.4', 'seats_available': '20 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Guntur (Andhra Pradesh) to Hyderabad', 'busname': 'APSRTC - 4620', 'price': '₹483', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '21:45', 'duration': '6h 5m', 'reaching_time': '03:50', 'star_rating': '3.2', 'seats_available': '11 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Guntur (Andhra Pradesh) to Hyderabad', 'busname': 'APSRTC - 4198', 'price': '₹585', 'bustype': 'INDRA(A.C. Seater)', 'departing_time': '22:00', 'duration': '5h 50m', 'reaching_time': '03:50', 'star_rating': '4.3', 'seats_available': '23 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Guntur (Andhra Pradesh) to Hyderabad', 'busname': 'APSRTC - 4252', 'price': '₹483', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '22:05', 'duration': '6h 20m', 'reaching_time': '04:25', 'star_rating': '3.6', 'seats_available': '24 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Guntur (Andhra Pradesh) to Hyderabad', 'busname': 'APSRTC - 4611', 'price': '₹483', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '22:15', 'duration': '5h 45m', 'reaching_time': '04:00', 'star_rating': '4.5', 'seats_available': '21 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Guntur (Andhra Pradesh) to Hyderabad', 'busname': 'APSRTC - 4178', 'price': '₹817', 'bustype': 'AMARAVATHI (VOLVO / SCANIA A.C Multi Axle)', 'departing_time': '22:15', 'duration': '6h 5m', 'reaching_time': '04:20', 'star_rating': '3.9', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Guntur (Andhra Pradesh) to Hyderabad', 'busname': 'APSRTC - 4628', 'price': '₹483', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '22:30', 'duration': '5h 40m', 'reaching_time': '04:10', 'star_rating': '3.5', 'seats_available': '15 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Guntur (Andhra Pradesh) to Hyderabad', 'busname': 'APSRTC - 4214', 'price': '₹743', 'bustype': 'AMARAVATHI (VOLVO / SCANIA A.C Multi Axle)', 'departing_time': '22:30', 'duration': '5h 25m', 'reaching_time': '03:55', 'star_rating': '3.6', 'seats_available': '40 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Ongole', 'busname': 'Vikram Travels', 'price': '₹539', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '21:55', 'duration': '7h 35m', 'reaching_time': '05:30', 'star_rating': '4.7', 'seats_available': '13 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Ongole', 'busname': 'IntrCity SmartBus', 'price': '₹829', 'bustype': 'AC Sleeper (2+1)', 'departing_time': '23:40', 'duration': '7h 15m', 'reaching_time': '06:55', 'star_rating': '4.7', 'seats_available': '19 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Ongole', 'busname': 'IntrCity SmartBus', 'price': '₹599', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '23:50', 'duration': '6h 35m', 'reaching_time': '06:25', 'star_rating': '4.5', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Ongole', 'busname': 'Yolo Bus', 'price': '₹599', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '21:45', 'duration': '8h 33m', 'reaching_time': '06:18', 'star_rating': '4.3', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Ongole', 'busname': 'Sri Vengamamba Bus Transport (SVBT)', 'price': '₹550', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '22:00', 'duration': '8h 15m', 'reaching_time': '06:15', 'star_rating': '4.3', 'seats_available': '14 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Ongole', 'busname': 'V Kaveri Travels', 'price': '₹1,079', 'bustype': 'Volvo 9600 Multi-Axle A/C Sleeper (2+1)', 'departing_time': '21:30', 'duration': '8h 15m', 'reaching_time': '05:45', 'star_rating': '4.3', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Ongole', 'busname': 'Kaveri Travels', 'price': '₹749', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '22:30', 'duration': '8h', 'reaching_time': '06:30', 'star_rating': '4.2', 'seats_available': '9 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Ongole', 'busname': 'Kamakshi Tours And Travels', 'price': '₹549', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '21:30', 'duration': '8h 30m', 'reaching_time': '06:00', 'star_rating': '4.2', 'seats_available': '9 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Ongole', 'busname': 'FlixBus', 'price': '₹1,401', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '20:35', 'duration': '8h 40m', 'reaching_time': '05:15', 'star_rating': '4.3', 'seats_available': '5 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Ongole', 'busname': 'IntrCity SmartBus', 'price': '₹949', 'bustype': 'AC Sleeper (2+1)', 'departing_time': '23:45', 'duration': '5h 55m', 'reaching_time': '05:40', 'star_rating': '4.4', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Ongole', 'busname': 'APSRTC - 5870', 'price': '₹522', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '18:30', 'duration': '9h 30m', 'reaching_time': '04:00', 'star_rating': '4.8', 'seats_available': '16 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Ongole', 'busname': 'APSRTC - 5070', 'price': '₹556', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '18:45', 'duration': '8h 40m', 'reaching_time': '03:25', 'star_rating': '1.7', 'seats_available': '12 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Ongole', 'busname': 'APSRTC - 48821', 'price': '₹522', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:00', 'duration': '9h 15m', 'reaching_time': '04:15', 'star_rating': '4.2', 'seats_available': '14 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Ongole', 'busname': 'APSRTC - 5859', 'price': '₹555', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:00', 'duration': '9h', 'reaching_time': '04:00', 'star_rating': '4', 'seats_available': '6 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Ongole', 'busname': 'APSRTC - 5054', 'price': '₹556', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:15', 'duration': '9h 45m', 'reaching_time': '05:00', 'star_rating': '3.3', 'seats_available': '14 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Ongole', 'busname': 'APSRTC - 5867', 'price': '₹602', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:30', 'duration': '11h 30m', 'reaching_time': '07:00', 'star_rating': '3.8', 'seats_available': '1 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Ongole', 'busname': 'APSRTC - 5994', 'price': '₹522', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:30', 'duration': '10h 15m', 'reaching_time': '05:45', 'star_rating': '4.5', 'seats_available': '12 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Ongole', 'busname': 'APSRTC - 5181', 'price': '₹556', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:40', 'duration': '9h', 'reaching_time': '04:40', 'star_rating': '2.7', 'seats_available': '12 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Ongole', 'busname': 'APSRTC - 5922', 'price': '₹555', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:45', 'duration': '9h 20m', 'reaching_time': '05:05', 'star_rating': '3.7', 'seats_available': '5 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Ongole', 'busname': 'APSRTC - 5916', 'price': '₹605', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:50', 'duration': '9h 10m', 'reaching_time': '05:00', 'star_rating': '3.4', 'seats_available': '3 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Kothagudem', 'busname': 'BSR Tours And Travels', 'price': '₹780', 'bustype': 'Volvo Multi-Axle A/C Sleeper (2+1)', 'departing_time': '22:00', 'duration': '6h 30m', 'reaching_time': '04:30', 'star_rating': '4.6', 'seats_available': '3 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Kothagudem', 'busname': 'BSR Tours And Travels', 'price': '₹680', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '22:00', 'duration': '6h 30m', 'reaching_time': '04:30', 'star_rating': '4.5', 'seats_available': '5 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Kothagudem', 'busname': 'VSR Tours and Travels', 'price': '₹699', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '23:00', 'duration': '5h 50m', 'reaching_time': '04:50', 'star_rating': '4.3', 'seats_available': '9 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Kothagudem', 'busname': 'Morning Star Travels', 'price': '₹590', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '23:20', 'duration': '5h', 'reaching_time': '04:20', 'star_rating': '4', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Kothagudem', 'busname': 'SAIRAJ TRAVELS', 'price': '₹700', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '22:15', 'duration': '6h 45m', 'reaching_time': '05:00', 'star_rating': '4.7', 'seats_available': '1 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Kothagudem', 'busname': 'Bharathi Tours & Travels', 'price': '₹621', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '21:20', 'duration': '8h 10m', 'reaching_time': '05:30', 'star_rating': '4.4', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Kothagudem', 'busname': 'Bharathi Tours & Travels', 'price': '₹450', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '21:20', 'duration': '8h 10m', 'reaching_time': '05:30', 'star_rating': '4', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Kothagudem', 'busname': 'Morning Star Travels', 'price': '₹590', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '22:50', 'duration': '5h 40m', 'reaching_time': '04:30', 'star_rating': '3.5', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Kothagudem', 'busname': 'Bharathi Tours & Travels', 'price': '₹702', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '22:20', 'duration': '6h 30m', 'reaching_time': '04:50', 'star_rating': '4', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Kothagudem', 'busname': 'Bharathi Tours & Travels', 'price': '₹490', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '22:20', 'duration': '7h 35m', 'reaching_time': '05:55', 'star_rating': '3.8', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Kothagudem', 'busname': 'TGSRTC - 8715', 'price': '₹451', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '17:00', 'duration': '30h 10m', 'reaching_time': '23:10', 'star_rating': '3.8', 'seats_available': '30 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Kothagudem', 'busname': 'TGSRTC - 8869', 'price': '₹451', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '17:45', 'duration': '30h 5m', 'reaching_time': '23:50', 'star_rating': '3.8', 'seats_available': '30 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Kothagudem', 'busname': 'TGSRTC - 8928', 'price': '₹451', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '18:30', 'duration': '30h 30m', 'reaching_time': '01:00', 'star_rating': '3.8', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Kothagudem', 'busname': 'TGSRTC - 7645', 'price': '₹451', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '18:30', 'duration': '9h 10m', 'reaching_time': '03:40', 'star_rating': '3.8', 'seats_available': '27 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Kothagudem', 'busname': 'TGSRTC - 8933', 'price': '₹451', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '19:15', 'duration': '6h 30m', 'reaching_time': '01:45', 'star_rating': '3.8', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Kothagudem', 'busname': 'TGSRTC - 8708', 'price': '₹451', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '20:15', 'duration': '30h 10m', 'reaching_time': '02:25', 'star_rating': '3.8', 'seats_available': '30 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Kothagudem', 'busname': 'TGSRTC - 8774', 'price': '₹451', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '20:40', 'duration': '6h 10m', 'reaching_time': '02:50', 'star_rating': '3.8', 'seats_available': '27 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Kothagudem', 'busname': 'TGSRTC - 8902', 'price': '₹451', 'bustype': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', 'departing_time': '20:45', 'duration': '7h 40m', 'reaching_time': '04:25', 'star_rating': '2.9', 'seats_available': '17 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Kothagudem', 'busname': 'TGSRTC - 9984', 'price': '₹451', 'bustype': 'Lahari Non A/C Sleeper Cum Seater', 'departing_time': '21:10', 'duration': '7h 5m', 'reaching_time': '04:15', 'star_rating': '4.4', 'seats_available': '20 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'TSRTC Online Bus Ticket Booking', 'route': 'Hyderabad to Kothagudem', 'busname': 'TGSRTC - 34340', 'price': '₹548', 'bustype': 'Rajdhani (AC Semi Sleeper 2+2)', 'departing_time': '21:15', 'duration': '6h 15m', 'reaching_time': '03:30', 'star_rating': '3.8', 'seats_available': '29 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Bangalore to Goa', 'busname': 'Kadamba Transport Corporation Limited (KTCL) - 180251', 'price': '₹900', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '20:05', 'duration': '10h 55m', 'reaching_time': '07:00', 'star_rating': '3.8', 'seats_available': '9 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Bangalore to Goa', 'busname': 'zingbus plus', 'price': '₹813', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '20:05', 'duration': '12h 55m', 'reaching_time': '09:00', 'star_rating': '4.8', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Bangalore to Goa', 'busname': 'IntrCity SmartBus', 'price': '₹869', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '20:15', 'duration': '14h', 'reaching_time': '10:15', 'star_rating': '4.6', 'seats_available': '8 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Bangalore to Goa', 'busname': 'Sea Bird Tourist', 'price': '₹720', 'bustype': 'NON A/C Sleeper (2+1)', 'departing_time': '18:00', 'duration': '13h 30m', 'reaching_time': '07:30', 'star_rating': '3.9', 'seats_available': '7 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Bangalore to Goa', 'busname': 'Intercity travels', 'price': '₹999', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '21:00', 'duration': '13h', 'reaching_time': '10:00', 'star_rating': '4.5', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Bangalore to Goa', 'busname': 'Sunrise Tours and Travels', 'price': '₹999', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '22:00', 'duration': '12h 40m', 'reaching_time': '10:40', 'star_rating': '4.3', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Bangalore to Goa', 'busname': 'VKV Travels', 'price': '₹1,131', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '22:55', 'duration': '11h 5m', 'reaching_time': '10:00', 'star_rating': '4.3', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Bangalore to Goa', 'busname': 'IntrCity SmartBus', 'price': '₹799', 'bustype': 'AC Sleeper (2+1)', 'departing_time': '21:30', 'duration': '14h 20m', 'reaching_time': '11:50', 'star_rating': '3.9', 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Bangalore to Goa', 'busname': 'IntrCity SmartBus', 'price': '₹799', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '21:00', 'duration': '14h', 'reaching_time': '11:00', 'star_rating': '4.2', 'seats_available': '14 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Bangalore to Goa', 'busname': 'FlixBus', 'price': '₹1,017', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '20:00', 'duration': '13h 35m', 'reaching_time': '09:35', 'star_rating': '4.3', 'seats_available': '19 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Pune to Goa', 'busname': 'Kadamba Transport Corporation Limited (KTCL) - 180183', 'price': '₹900', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '20:30', 'duration': '9h 30m', 'reaching_time': '06:00', 'star_rating': '3.8', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Pune to Goa', 'busname': 'Atmaram Gobus', 'price': '₹900', 'bustype': 'VE A/C Sleeper (2+1)', 'departing_time': '21:00', 'duration': '10h 30m', 'reaching_time': '07:30', 'star_rating': '4.7', 'seats_available': '8 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Pune to Goa', 'busname': 'Ashray Travels Benz', 'price': '₹950', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '21:35', 'duration': '11h 30m', 'reaching_time': '09:05', 'star_rating': '4.6', 'seats_available': '5 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Pune to Goa', 'busname': 'zingbus plus', 'price': '₹900', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '22:50', 'duration': '10h 5m', 'reaching_time': '08:55', 'star_rating': '4.5', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Pune to Goa', 'busname': 'Atmaram Gobus', 'price': '₹900', 'bustype': 'VE A/C Sleeper (2+1)', 'departing_time': '22:30', 'duration': '10h 15m', 'reaching_time': '08:45', 'star_rating': '4.6', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Pune to Goa', 'busname': 'Laxmi holidays', 'price': '₹600', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '21:25', 'duration': '10h 5m', 'reaching_time': '07:30', 'star_rating': '4.5', 'seats_available': '43 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Pune to Goa', 'busname': 'Ashray National Express', 'price': '₹700', 'bustype': 'NON A/C Sleeper (2+1)', 'departing_time': '20:00', 'duration': '11h 15m', 'reaching_time': '07:15', 'star_rating': '4.6', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Pune to Goa', 'busname': 'Prasanna - Purple Bus', 'price': '₹918', 'bustype': 'VE A/C Sleeper (2+1)', 'departing_time': '22:30', 'duration': '11h', 'reaching_time': '09:30', 'star_rating': '4.1', 'seats_available': '21 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Pune to Goa', 'busname': 'Atmaram Gobus', 'price': '₹900', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '23:30', 'duration': '9h 15m', 'reaching_time': '08:45', 'star_rating': '4.5', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Pune to Goa', 'busname': 'zingbus plus', 'price': '₹1,055', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '23:55', 'duration': '10h', 'reaching_time': '09:55', 'star_rating': '4.5', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Mumbai to Goa', 'busname': 'Kadamba Transport Corporation Limited (KTCL) - 97386', 'price': '₹800', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '17:05', 'duration': '12h 55m', 'reaching_time': '06:00', 'star_rating': '4.1', 'seats_available': '28 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Mumbai to Goa', 'busname': 'Laxmi holidays', 'price': '₹600', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '16:40', 'duration': '16h 20m', 'reaching_time': '09:00', 'star_rating': '4.5', 'seats_available': '21 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Mumbai to Goa', 'busname': 'IntrCity SmartBus', 'price': '₹800', 'bustype': 'VE A/C Sleeper (2+1)', 'departing_time': '17:00', 'duration': '15h 35m', 'reaching_time': '08:35', 'star_rating': '4.5', 'seats_available': '6 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Mumbai to Goa', 'busname': 'Atmaram Travels', 'price': '₹800', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '17:30', 'duration': '16h 15m', 'reaching_time': '09:45', 'star_rating': '3.8', 'seats_available': '13 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Mumbai to Goa', 'busname': 'zingbus plus', 'price': '₹800', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '17:00', 'duration': '15h 55m', 'reaching_time': '08:55', 'star_rating': '4.5', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Mumbai to Goa', 'busname': 'Rajdhani Travels Corporation', 'price': '₹800', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '20:50', 'duration': '12h 25m', 'reaching_time': '09:15', 'star_rating': '4.5', 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Mumbai to Goa', 'busname': 'zingbus plus', 'price': '₹896', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '18:00', 'duration': '15h 55m', 'reaching_time': '09:55', 'star_rating': '4.4', 'seats_available': '13 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Mumbai to Goa', 'busname': 'IntrCity SmartBus', 'price': '₹800', 'bustype': 'VE A/C Sleeper (2+1)', 'departing_time': '19:30', 'duration': '14h 45m', 'reaching_time': '10:15', 'star_rating': '4.3', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Mumbai to Goa', 'busname': 'IntrCity SmartBus', 'price': '₹750', 'bustype': 'AC Sleeper (2+1)', 'departing_time': '17:45', 'duration': '15h 35m', 'reaching_time': '09:20', 'star_rating': '4.2', 'seats_available': '7 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Mumbai to Goa', 'busname': 'Dolphin Travel House (38)', 'price': '₹750', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '16:10', 'duration': '16h 5m', 'reaching_time': '08:15', 'star_rating': '4.2', 'seats_available': '9 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Bangalore', 'busname': 'Kadamba Transport Corporation Limited (KTCL) - 180243', 'price': '₹1,000', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '18:30', 'duration': '13h', 'reaching_time': '07:30', 'star_rating': '3.6', 'seats_available': '6 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Bangalore', 'busname': 'zingbus plus', 'price': '₹800', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '17:25', 'duration': '12h 35m', 'reaching_time': '06:00', 'star_rating': '4.7', 'seats_available': '13 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Bangalore', 'busname': 'Intercity travels', 'price': '₹1,099', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '18:00', 'duration': '12h 30m', 'reaching_time': '06:30', 'star_rating': '4.6', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Bangalore', 'busname': 'Sea Bird Tourist', 'price': '₹989', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '20:30', 'duration': '13h', 'reaching_time': '09:30', 'star_rating': '3', 'seats_available': '21 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Bangalore', 'busname': 'IntrCity SmartBus', 'price': '₹799', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '17:00', 'duration': '13h 25m', 'reaching_time': '06:25', 'star_rating': '4.5', 'seats_available': '4 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Bangalore', 'busname': 'IntrCity SmartBus', 'price': '₹799', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '17:00', 'duration': '12h 30m', 'reaching_time': '05:30', 'star_rating': '4.4', 'seats_available': '4 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Bangalore', 'busname': 'Sunrise Tours and Travels', 'price': '₹1,099', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '20:00', 'duration': '11h 30m', 'reaching_time': '07:30', 'star_rating': '4.4', 'seats_available': '3 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Bangalore', 'busname': 'Sea Bird Tourist', 'price': '₹900', 'bustype': 'NON A/C Sleeper (2+1)', 'departing_time': '18:30', 'duration': '14h', 'reaching_time': '08:30', 'star_rating': '3.9', 'seats_available': '7 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Bangalore', 'busname': 'VKV Travels', 'price': '₹988', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '16:25', 'duration': '9h 30m', 'reaching_time': '01:55', 'star_rating': '4.4', 'seats_available': '13 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Bangalore', 'busname': 'FlixBus', 'price': '₹588', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '17:30', 'duration': '13h 25m', 'reaching_time': '06:55', 'star_rating': '4.4', 'seats_available': '9 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Pune', 'busname': 'Kadamba Transport Corporation Limited (KTCL) - 180182', 'price': '₹900', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '17:00', 'duration': '11h 45m', 'reaching_time': '04:45', 'star_rating': '4', 'seats_available': '5 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Pune', 'busname': 'Atmaram Gobus', 'price': '₹900', 'bustype': 'VE A/C Sleeper (2+1)', 'departing_time': '21:32', 'duration': '9h 30m', 'reaching_time': '07:02', 'star_rating': '4.7', 'seats_available': '6 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Pune', 'busname': 'Atmaram Gobus', 'price': '₹900', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '23:00', 'duration': '9h', 'reaching_time': '08:00', 'star_rating': '4.7', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Pune', 'busname': 'Anand Ashray Travels', 'price': '₹999', 'bustype': 'Volvo Multi-Axle A/C Sleeper (2+1)', 'departing_time': '20:15', 'duration': '10h 15m', 'reaching_time': '06:30', 'star_rating': '4.3', 'seats_available': '14 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Pune', 'busname': 'Atmaram Gobus', 'price': '₹900', 'bustype': 'VE A/C Sleeper (2+1)', 'departing_time': '22:30', 'duration': '9h 30m', 'reaching_time': '08:00', 'star_rating': '4.6', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Pune', 'busname': 'zingbus plus', 'price': '₹900', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '18:40', 'duration': '9h 50m', 'reaching_time': '04:30', 'star_rating': '4.7', 'seats_available': '9 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Pune', 'busname': 'Laxmi holidays', 'price': '₹750', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '19:30', 'duration': '11h 50m', 'reaching_time': '07:20', 'star_rating': '4.4', 'seats_available': '27 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Pune', 'busname': 'Prasanna - Purple Bus', 'price': '₹905', 'bustype': 'VE A/C Sleeper (2+1)', 'departing_time': '22:00', 'duration': '9h 30m', 'reaching_time': '07:30', 'star_rating': '4.3', 'seats_available': '19 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Pune', 'busname': 'Ashray Travels Club', 'price': '₹500', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '20:00', 'duration': '10h 30m', 'reaching_time': '06:30', 'star_rating': '4.3', 'seats_available': '23 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Pune', 'busname': 'IntrCity SmartBus', 'price': '₹900', 'bustype': 'AC Sleeper (2+1)', 'departing_time': '18:30', 'duration': '10h 55m', 'reaching_time': '05:25', 'star_rating': '4.5', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Hyderabad to Goa', 'busname': 'FlixBus', 'price': '₹1,169', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '18:00', 'duration': '18h 20m', 'reaching_time': '12:20', 'star_rating': '4.6', 'seats_available': '5 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Hyderabad to Goa', 'busname': 'IntrCity SmartBus', 'price': '₹902', 'bustype': 'AC Sleeper (2+1)', 'departing_time': '19:00', 'duration': '16h 55m', 'reaching_time': '11:55', 'star_rating': '4.4', 'seats_available': '3 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Hyderabad to Goa', 'busname': 'zingbus plus', 'price': '₹1,131', 'bustype': 'AC Sleeper (2+1)', 'departing_time': '18:00', 'duration': '17h 40m', 'reaching_time': '11:40', 'star_rating': '4.1', 'seats_available': '4 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Hyderabad to Goa', 'busname': 'PSR Travels Goa (SOARES)', 'price': '₹899', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '18:30', 'duration': '16h 15m', 'reaching_time': '10:45', 'star_rating': '4', 'seats_available': '3 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Hyderabad to Goa', 'busname': 'IntrCity SmartBus', 'price': '₹712', 'bustype': 'AC Sleeper (2+1)', 'departing_time': '18:30', 'duration': '16h 55m', 'reaching_time': '11:25', 'star_rating': '3.7', 'seats_available': '4 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Hyderabad to Goa', 'busname': 'PSR Travels Goa (Parrikar)', 'price': '₹899', 'bustype': 'VE A/C Sleeper (2+1)', 'departing_time': '20:25', 'duration': '16h 20m', 'reaching_time': '12:45', 'star_rating': '4.1', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Hyderabad to Goa', 'busname': 'PSR Travels Goa', 'price': '₹799', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '21:45', 'duration': '15h 30m', 'reaching_time': '13:15', 'star_rating': '3.2', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Hyderabad to Goa', 'busname': 'Elegance Tours And Travels Pvt Ltd', 'price': '₹2,638', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '17:45', 'duration': '17h 45m', 'reaching_time': '11:30', 'star_rating': '2.4', 'seats_available': '27 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Hyderabad to Goa', 'busname': 'Raja Express', 'price': '₹1,330', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '23:55', 'duration': '15h 50m', 'reaching_time': '15:45', 'star_rating': '2.6', 'seats_available': '23 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Hyderabad to Goa', 'busname': 'VRL Travels', 'price': '₹1,200', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '21:00', 'duration': '16h 30m', 'reaching_time': '13:30', 'star_rating': '3.6', 'seats_available': '13 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Mumbai', 'busname': 'Kadamba Transport Corporation Limited (KTCL) - 180926', 'price': '₹800', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '18:30', 'duration': '13h 30m', 'reaching_time': '08:00', 'star_rating': '4.2', 'seats_available': '21 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Mumbai', 'busname': 'zingbus plus', 'price': '₹1,100', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '18:00', 'duration': '14h 30m', 'reaching_time': '08:30', 'star_rating': '4.7', 'seats_available': '9 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Mumbai', 'busname': 'Laxmi holidays', 'price': '₹800', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '20:00', 'duration': '14h 15m', 'reaching_time': '10:15', 'star_rating': '4.5', 'seats_available': '21 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Mumbai', 'busname': 'IntrCity SmartBus', 'price': '₹1,000', 'bustype': 'VE A/C Sleeper (2+1)', 'departing_time': '21:00', 'duration': '13h 30m', 'reaching_time': '10:30', 'star_rating': '4.3', 'seats_available': '8 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Mumbai', 'busname': 'zingbus plus', 'price': '₹1,000', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '19:25', 'duration': '14h 35m', 'reaching_time': '10:00', 'star_rating': '4.5', 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Mumbai', 'busname': 'IntrCity SmartBus', 'price': '₹1,000', 'bustype': 'AC Sleeper (2+1)', 'departing_time': '18:30', 'duration': '14h 30m', 'reaching_time': '09:00', 'star_rating': '4.5', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Mumbai', 'busname': 'IntrCity SmartBus', 'price': '₹1,000', 'bustype': 'VE A/C Sleeper (2+1)', 'departing_time': '17:30', 'duration': '14h 30m', 'reaching_time': '08:00', 'star_rating': '4.5', 'seats_available': '14 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Mumbai', 'busname': 'Naik Tours And Travels', 'price': '₹1,000', 'bustype': 'VE A/C Sleeper (2+1)', 'departing_time': '18:45', 'duration': '11h 35m', 'reaching_time': '06:20', 'star_rating': '4', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Mumbai', 'busname': 'National Travels,Goa', 'price': '₹1,200', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '19:40', 'duration': '12h 30m', 'reaching_time': '08:10', 'star_rating': '4.2', 'seats_available': '8 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Mumbai', 'busname': 'VAILANKANNI TRAVELS', 'price': '₹1,000', 'bustype': 'VE A/C Sleeper (2+1)', 'departing_time': '16:25', 'duration': '14h 30m', 'reaching_time': '06:55', 'star_rating': '4.2', 'seats_available': '7 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Hyderabad', 'busname': 'Atmaram Gobus', 'price': '₹1,000', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '18:00', 'duration': '13h 30m', 'reaching_time': '07:30', 'star_rating': '4.6', 'seats_available': '2 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Hyderabad', 'busname': 'FlixBus', 'price': '₹1,169', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '16:35', 'duration': '14h 45m', 'reaching_time': '07:20', 'star_rating': '4.6', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Hyderabad', 'busname': 'zingbus plus', 'price': '₹850', 'bustype': 'AC Sleeper (2+1)', 'departing_time': '16:45', 'duration': '15h 55m', 'reaching_time': '08:40', 'star_rating': '4.4', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Hyderabad', 'busname': 'PSR Travels Goa (Parrikar)', 'price': '₹1,099', 'bustype': 'VE A/C Sleeper (2+1)', 'departing_time': '16:45', 'duration': '14h 45m', 'reaching_time': '07:30', 'star_rating': '4.3', 'seats_available': '9 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Hyderabad', 'busname': 'IntrCity SmartBus', 'price': '₹899', 'bustype': 'AC Sleeper (2+1)', 'departing_time': '16:30', 'duration': '15h 50m', 'reaching_time': '08:20', 'star_rating': '3.9', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Hyderabad', 'busname': 'PSR Travels Goa (SOARES)', 'price': '₹1,099', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '18:00', 'duration': '15h', 'reaching_time': '09:00', 'star_rating': '3.8', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Hyderabad', 'busname': 'PVS Tours and Travels', 'price': '₹900', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '19:00', 'duration': '16h', 'reaching_time': '11:00', 'star_rating': '3.2', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Hyderabad', 'busname': 'PSR Travels Goa', 'price': '₹900', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '19:30', 'duration': '15h 15m', 'reaching_time': '10:45', 'star_rating': '3.8', 'seats_available': '2 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Hyderabad', 'busname': 'VRL Travels', 'price': '₹1,000', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '18:00', 'duration': '15h 30m', 'reaching_time': '09:30', 'star_rating': '2.6', 'seats_available': '22 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Hyderabad', 'busname': 'PVS Tours and Travels', 'price': '₹800', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '18:00', 'duration': '17h 59m', 'reaching_time': '11:59', 'star_rating': '3.3', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Belagavi to Goa', 'busname': 'AR Group Vijaylaxmi Travels', 'price': '₹383', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '03:00', 'duration': '3h', 'reaching_time': '06:00', 'star_rating': '4.3', 'seats_available': '21 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Belagavi', 'busname': 'PSR Travels Goa (Parrikar)', 'price': '₹1,000', 'bustype': 'VE A/C Sleeper (2+1)', 'departing_time': '16:45', 'duration': '4h 40m', 'reaching_time': '21:25', 'star_rating': '4.1', 'seats_available': '30 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Belagavi', 'busname': 'PSR Travels Goa (SOARES)', 'price': '₹1,000', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '18:00', 'duration': '4h 30m', 'reaching_time': '22:30', 'star_rating': '3.8', 'seats_available': '27 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Belagavi', 'busname': 'PVS Tours and Travels', 'price': '₹800', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '18:00', 'duration': '5h 45m', 'reaching_time': '23:45', 'star_rating': '3.2', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Belagavi', 'busname': 'PVS Tours and Travels', 'price': '₹700', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '19:00', 'duration': '4h 50m', 'reaching_time': '23:50', 'star_rating': '3.2', 'seats_available': '21 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Belagavi', 'busname': 'AR Group Vijaylaxmi Travels', 'price': '₹570', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '20:00', 'duration': '6h', 'reaching_time': '02:00', 'star_rating': '4.3', 'seats_available': '36 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Belagavi', 'busname': 'S R Travels', 'price': '₹700', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '20:30', 'duration': '5h 30m', 'reaching_time': '02:00', 'star_rating': '3', 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Belagavi', 'busname': 'Atmaram Travels( Vijaylaxmi )', 'price': '₹500', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '20:30', 'duration': '4h 30m', 'reaching_time': '01:00', 'star_rating': '4.1', 'seats_available': '36 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Belagavi', 'busname': 'Orange Tours and Travels', 'price': '₹800', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '16:30', 'duration': '4h 15m', 'reaching_time': '20:45', 'star_rating': '2.7', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Belagavi', 'busname': 'S R Travels', 'price': '₹399', 'bustype': 'NON AC Seater / Sleeper 2+1', 'departing_time': '19:45', 'duration': '3h 45m', 'reaching_time': '23:30', 'star_rating': '2.9', 'seats_available': '25 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Belagavi', 'busname': 'S R Travels', 'price': '₹600', 'bustype': 'NON A/C Sleeper (2+1)', 'departing_time': '20:00', 'duration': '5h', 'reaching_time': '01:00', 'star_rating': '2.9', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Hyderabad to Belagavi', 'busname': 'Kadamba Transport Corporation Limited (KTCL) - 181828', 'price': '₹1,000', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '17:00', 'duration': '12h', 'reaching_time': '05:00', 'star_rating': '3.3', 'seats_available': '1 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Hyderabad to Belagavi', 'busname': 'PSR Travels Goa (SOARES)', 'price': '₹899', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '18:30', 'duration': '11h', 'reaching_time': '05:30', 'star_rating': '4', 'seats_available': '3 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Hyderabad to Belagavi', 'busname': 'PSR Travels Goa (Parrikar)', 'price': '₹899', 'bustype': 'VE A/C Sleeper (2+1)', 'departing_time': '20:25', 'duration': '11h 50m', 'reaching_time': '08:15', 'star_rating': '4.4', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Hyderabad to Belagavi', 'busname': 'Elegance Tours And Travels Pvt Ltd', 'price': '₹2,638', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '17:45', 'duration': '13h 15m', 'reaching_time': '07:00', 'star_rating': '1.9', 'seats_available': '27 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Hyderabad to Belagavi', 'busname': 'Raja Express', 'price': '₹1,330', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '23:55', 'duration': '11h 5m', 'reaching_time': '11:00', 'star_rating': '2.6', 'seats_available': '23 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Hyderabad to Belagavi', 'busname': 'VRL Travels', 'price': '₹1,200', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '21:00', 'duration': '13h 30m', 'reaching_time': '10:30', 'star_rating': '3.3', 'seats_available': '13 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Hyderabad to Belagavi', 'busname': 'ISRTC BUS', 'price': '₹1,709', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '21:15', 'duration': '12h 15m', 'reaching_time': '09:30', 'star_rating': '1.5', 'seats_available': '25 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Hyderabad to Belagavi', 'busname': 'VRL Travels', 'price': '₹900', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '22:00', 'duration': '12h 30m', 'reaching_time': '10:30', 'star_rating': '3', 'seats_available': '7 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Hyderabad to Belagavi', 'busname': 'VRL Travels', 'price': '₹1,300', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '17:00', 'duration': '13h 30m', 'reaching_time': '06:30', 'star_rating': '4.1', 'seats_available': '3 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Pandharpur to Goa', 'busname': 'Kadamba Transport Corporation Limited (KTCL) - 61005', 'price': '₹790', 'bustype': 'Non AC Seater 2+2', 'departing_time': '21:15', 'duration': '12h 45m', 'reaching_time': '10:00', 'star_rating': '4.3', 'seats_available': '27 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Pandharpur', 'busname': 'Kadamba Transport Corporation Limited (KTCL) - 60996', 'price': '₹790', 'bustype': 'Non AC Seater 2+2', 'departing_time': '19:00', 'duration': '12h', 'reaching_time': '07:00', 'star_rating': '4', 'seats_available': '3 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Solapur to Goa', 'busname': 'Kadamba Transport Corporation Limited (KTCL) - 61005', 'price': '₹920', 'bustype': 'Non AC Seater 2+2', 'departing_time': '20:00', 'duration': '14h', 'reaching_time': '10:00', 'star_rating': '4', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Solapur to Goa', 'busname': 'Sharma Travels', 'price': '₹1,905', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '23:00', 'duration': '10h', 'reaching_time': '09:00', 'star_rating': '4.5', 'seats_available': '1 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Solapur to Goa', 'busname': 'Humsafar Travels', 'price': '₹1,050', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '23:15', 'duration': '8h 45m', 'reaching_time': '08:00', 'star_rating': '3.6', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Solapur to Goa', 'busname': 'S R Travels', 'price': '₹700', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '18:45', 'duration': '12h 15m', 'reaching_time': '07:00', 'star_rating': '2.4', 'seats_available': '19 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Solapur to Goa', 'busname': 'AR Group Vijaylaxmi Travels', 'price': '₹476', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '19:00', 'duration': '11h', 'reaching_time': '06:00', 'star_rating': '3.9', 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Solapur to Goa', 'busname': 'AR Group Vijaylaxmi Travels', 'price': '₹4,999', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '19:00', 'duration': '11h', 'reaching_time': '06:00', 'star_rating': '4.3', 'seats_available': '44 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Solapur to Goa', 'busname': 'S R Travels', 'price': '₹699', 'bustype': 'NON A/C Sleeper (2+1)', 'departing_time': '18:00', 'duration': '12h', 'reaching_time': '06:00', 'star_rating': '2.9', 'seats_available': '13 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Kolhapur(Maharashtra)', 'busname': 'Kadamba Transport Corporation Limited (KTCL) - 60996', 'price': '₹450', 'bustype': 'Non AC Seater 2+2', 'departing_time': '19:00', 'duration': '7h 30m', 'reaching_time': '02:30', 'star_rating': '4.2', 'seats_available': '3 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Kolhapur(Maharashtra)', 'busname': 'Atmaram Travels (Vishal)', 'price': '₹800', 'bustype': 'NON A/C Sleeper (2+1)', 'departing_time': '16:30', 'duration': '6h 25m', 'reaching_time': '22:55', 'star_rating': '4.1', 'seats_available': '24 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Kolhapur(Maharashtra)', 'busname': 'IntrCity SmartBus', 'price': '₹600', 'bustype': 'AC Sleeper (2+1)', 'departing_time': '16:30', 'duration': '5h 45m', 'reaching_time': '22:15', 'star_rating': '3.9', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Kolhapur(Maharashtra)', 'busname': 'FlixBus', 'price': '₹874', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '16:35', 'duration': '5h 45m', 'reaching_time': '22:20', 'star_rating': '4.6', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Kolhapur(Maharashtra)', 'busname': 'zingbus plus', 'price': '₹360', 'bustype': 'AC Sleeper (2+1)', 'departing_time': '16:45', 'duration': '5h 20m', 'reaching_time': '22:05', 'star_rating': '4.2', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Kolhapur(Maharashtra)', 'busname': 'Kelkar Travels Garuda', 'price': '₹360', 'bustype': 'NON A/C Seater/ Sleeper (2+1)', 'departing_time': '17:30', 'duration': '9h', 'reaching_time': '02:30', 'star_rating': '3.7', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Kolhapur(Maharashtra)', 'busname': 'IntrCity SmartBus', 'price': '₹550', 'bustype': 'VE A/C Sleeper (2+1)', 'departing_time': '17:30', 'duration': '6h 10m', 'reaching_time': '23:40', 'star_rating': '4.9', 'seats_available': '19 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Kolhapur(Maharashtra)', 'busname': 'Kelkar Travels Phoenix', 'price': '₹360', 'bustype': 'Non AC Seater (2+2)', 'departing_time': '17:45', 'duration': '7h 30m', 'reaching_time': '01:15', 'star_rating': '3.7', 'seats_available': '23 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Kolhapur(Maharashtra)', 'busname': 'IntrCity SmartBus', 'price': '₹360', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '18:00', 'duration': '5h 55m', 'reaching_time': '23:55', 'star_rating': '4.4', 'seats_available': '14 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Goa to Kolhapur(Maharashtra)', 'busname': 'Atmaram Gobus', 'price': '₹800', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '18:00', 'duration': '5h 40m', 'reaching_time': '23:40', 'star_rating': '4.8', 'seats_available': '2 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Kadamba Transport Corporation Limited (KTCL)', 'route': 'Calangute (goa) to Mopa Airport', 'busname': 'Kadamba Transport Corporation Limited (KTCL) - 183104', 'price': '₹200', 'bustype': 'AC Seater (2+2)', 'departing_time': '16:30', 'duration': '1h 15m', 'reaching_time': '17:45', 'star_rating': '3.8', 'seats_available': '33 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Jodhpur', 'busname': 'M R Travels', 'price': '₹600', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '22:30', 'duration': '7h', 'reaching_time': '05:30', 'star_rating': '4.6', 'seats_available': '23 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Jodhpur', 'busname': 'Jain travels regd', 'price': '₹600', 'bustype': 'AC Sleeper (2+1)', 'departing_time': '22:30', 'duration': '6h 35m', 'reaching_time': '05:05', 'star_rating': '4.6', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Jodhpur', 'busname': 'Gajraj Travels', 'price': '₹600', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '22:15', 'duration': '7h 20m', 'reaching_time': '05:35', 'star_rating': '4.5', 'seats_available': '13 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Jodhpur', 'busname': 'Jain travels regd', 'price': '₹450', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '23:00', 'duration': '7h', 'reaching_time': '06:00', 'star_rating': '4.2', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Jodhpur', 'busname': 'M R Travels', 'price': '₹510', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '23:35', 'duration': '6h 35m', 'reaching_time': '06:10', 'star_rating': '4.1', 'seats_available': '31 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Jodhpur', 'busname': 'Jain travels regd', 'price': '₹350', 'bustype': 'NON AC Seater / Sleeper 2+1', 'departing_time': '21:45', 'duration': '7h 15m', 'reaching_time': '05:00', 'star_rating': '3.9', 'seats_available': '38 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Jodhpur', 'busname': 'Gayatri Travels', 'price': '₹315', 'bustype': 'NON AC Seater / Sleeper 2+1', 'departing_time': '22:00', 'duration': '7h 30m', 'reaching_time': '05:30', 'star_rating': '3.1', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Jodhpur', 'busname': 'T.R. JANI TRAVELS', 'price': '₹430', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '23:00', 'duration': '6h 30m', 'reaching_time': '05:30', 'star_rating': '3.9', 'seats_available': '29 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Jodhpur', 'busname': 'Jain travels regd', 'price': '₹400', 'bustype': 'NON AC Seater/ Sleeper (2+1)', 'departing_time': '23:15', 'duration': '7h 15m', 'reaching_time': '06:30', 'star_rating': '3.4', 'seats_available': '32 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Jodhpur', 'busname': 'Jakhar Travels', 'price': '₹600', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '21:00', 'duration': '6h 50m', 'reaching_time': '03:50', 'star_rating': '3.2', 'seats_available': '9 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Jodhpur', 'busname': 'RSRTC - 157451', 'price': '₹740', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '17:30', 'duration': '6h 15m', 'reaching_time': '23:45', 'star_rating': '4.2', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Jodhpur', 'busname': 'RSRTC - 203315', 'price': '₹793', 'bustype': 'Deluxe AC Seater 2+2', 'departing_time': '17:40', 'duration': '9h 50m', 'reaching_time': '03:30', 'star_rating': '3.4', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Jodhpur', 'busname': 'RSRTC - 189582', 'price': '₹448', 'bustype': 'NON AC Seater / Sleeper 2+1', 'departing_time': '20:01', 'duration': '7h 29m', 'reaching_time': '03:30', 'star_rating': '3.4', 'seats_available': '36 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Jodhpur', 'busname': 'RSRTC - 198960', 'price': '₹448', 'bustype': 'NON AC Seater / Sleeper 2+1', 'departing_time': '20:40', 'duration': '8h', 'reaching_time': '04:40', 'star_rating': '4', 'seats_available': '36 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Jodhpur', 'busname': 'RSRTC - 163517', 'price': '₹356', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '21:05', 'duration': '7h 55m', 'reaching_time': '05:00', 'star_rating': '3.4', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Jodhpur', 'busname': 'RSRTC - 178948', 'price': '₹740', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '22:31', 'duration': '6h 14m', 'reaching_time': '04:45', 'star_rating': '4.5', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Jodhpur', 'busname': 'RSRTC - 170972', 'price': '₹740', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '23:59', 'duration': '6h 16m', 'reaching_time': '06:15', 'star_rating': '3.8', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jodhpur to Ajmer', 'busname': 'M R Travels', 'price': '₹650', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '16:15', 'duration': '3h 45m', 'reaching_time': '20:00', 'star_rating': '4.4', 'seats_available': '30 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jodhpur to Ajmer', 'busname': 'Jakhar Travels', 'price': '₹650', 'bustype': 'VE A/C Sleeper (2+1)', 'departing_time': '16:20', 'duration': '3h 10m', 'reaching_time': '19:30', 'star_rating': '3.9', 'seats_available': '23 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jodhpur to Ajmer', 'busname': 'Jain travels regd', 'price': '₹330', 'bustype': 'NON AC Seater/ Sleeper (2+1)', 'departing_time': '17:00', 'duration': '5h 55m', 'reaching_time': '22:55', 'star_rating': '3.3', 'seats_available': '27 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jodhpur to Ajmer', 'busname': 'Jakhar Travels', 'price': '₹450', 'bustype': 'NON A/C Seater/ Sleeper (2+1)', 'departing_time': '18:00', 'duration': '5h 15m', 'reaching_time': '23:15', 'star_rating': '3.9', 'seats_available': '32 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jodhpur to Ajmer', 'busname': 'M R Travels', 'price': '₹495', 'bustype': 'NON A/C Seater/ Sleeper (2+1)', 'departing_time': '18:30', 'duration': '5h 10m', 'reaching_time': '23:40', 'star_rating': '4.6', 'seats_available': '34 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jodhpur to Ajmer', 'busname': 'bhaiya travels', 'price': '₹580', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '18:36', 'duration': '5h 10m', 'reaching_time': '23:46', 'star_rating': '4.1', 'seats_available': '36 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jodhpur to Ajmer', 'busname': 'Jain travels regd', 'price': '₹330', 'bustype': 'NON AC Seater / Sleeper 2+1', 'departing_time': '19:00', 'duration': '4h 50m', 'reaching_time': '23:50', 'star_rating': '3.3', 'seats_available': '42 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jodhpur to Ajmer', 'busname': 'M R Travels', 'price': '₹612', 'bustype': 'Volvo Multi Axle A/C Sleeper I-Shift B11R (2+1)', 'departing_time': '19:30', 'duration': '4h 20m', 'reaching_time': '23:50', 'star_rating': '5', 'seats_available': '34 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jodhpur to Ajmer', 'busname': 'Jakhar Travels', 'price': '₹650', 'bustype': 'VE A/C Sleeper (2+1)', 'departing_time': '19:45', 'duration': '5h', 'reaching_time': '00:45', 'star_rating': '3.9', 'seats_available': '28 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jodhpur to Ajmer', 'busname': 'Chandra Raj Travels', 'price': '₹500', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '20:00', 'duration': '4h 15m', 'reaching_time': '00:15', 'star_rating': '3.4', 'seats_available': '31 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jodhpur to Ajmer', 'busname': 'RSRTC - 160785', 'price': '₹460', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '17:30', 'duration': '4h', 'reaching_time': '21:30', 'star_rating': '3.9', 'seats_available': '33 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jodhpur to Ajmer', 'busname': 'RSRTC - 190951', 'price': '₹229', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '18:00', 'duration': '4h 30m', 'reaching_time': '22:30', 'star_rating': '3.3', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jodhpur to Ajmer', 'busname': 'RSRTC - 193481', 'price': '₹229', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '18:30', 'duration': '4h 30m', 'reaching_time': '23:00', 'star_rating': '4', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jodhpur to Ajmer', 'busname': 'RSRTC - 176394', 'price': '₹460', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '20:00', 'duration': '3h 45m', 'reaching_time': '23:45', 'star_rating': '4', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jodhpur to Ajmer', 'busname': 'RSRTC - 198959', 'price': '₹302', 'bustype': 'NON AC Seater / Sleeper 2+1', 'departing_time': '21:00', 'duration': '4h 30m', 'reaching_time': '01:30', 'star_rating': '4', 'seats_available': '36 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jodhpur to Ajmer', 'busname': 'RSRTC - 203404', 'price': '₹229', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '21:00', 'duration': '4h 30m', 'reaching_time': '01:30', 'star_rating': None, 'seats_available': '45 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jodhpur to Ajmer', 'busname': 'RSRTC - 172841', 'price': '₹460', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '22:01', 'duration': '3h 29m', 'reaching_time': '01:30', 'star_rating': '2.9', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jodhpur to Ajmer', 'busname': 'RSRTC - 192299', 'price': '₹458', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '23:00', 'duration': '3h 45m', 'reaching_time': '02:45', 'star_rating': '4.2', 'seats_available': '36 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jodhpur to Ajmer', 'busname': 'RSRTC - 189583', 'price': '₹302', 'bustype': 'NON AC Seater / Sleeper 2+1', 'departing_time': '23:59', 'duration': '3h 46m', 'reaching_time': '03:45', 'star_rating': '4.4', 'seats_available': '17 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Beawar (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'Jain travels regd', 'price': '₹300', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '17:20', 'duration': '3h 25m', 'reaching_time': '20:45', 'star_rating': '3.7', 'seats_available': '30 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Beawar (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'Jain travels regd', 'price': '₹400', 'bustype': 'NON AC Seater / Sleeper 2+1', 'departing_time': '17:36', 'duration': '3h 49m', 'reaching_time': '21:25', 'star_rating': '3.7', 'seats_available': '27 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Beawar (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'Jain travels regd', 'price': '₹350', 'bustype': 'NON AC Seater/ Sleeper (2+1)', 'departing_time': '19:30', 'duration': '3h 30m', 'reaching_time': '23:00', 'star_rating': '3.7', 'seats_available': '36 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Beawar (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'Vijay Tour and Travels', 'price': '₹541', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '20:50', 'duration': '3h 5m', 'reaching_time': '23:55', 'star_rating': '3.6', 'seats_available': '31 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Beawar (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'Devraj Travels', 'price': '₹500', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '21:21', 'duration': '2h 54m', 'reaching_time': '00:15', 'star_rating': '3.9', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Beawar (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'Jain travels regd', 'price': '₹300', 'bustype': 'NON AC Seater/ Sleeper (2+1)', 'departing_time': '21:45', 'duration': '3h 30m', 'reaching_time': '01:15', 'star_rating': '3.7', 'seats_available': '39 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Beawar (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'Pandit Travels', 'price': '₹500', 'bustype': 'NON AC Seater / Sleeper 2+1', 'departing_time': '21:45', 'duration': '3h 30m', 'reaching_time': '01:15', 'star_rating': '3.3', 'seats_available': '22 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Beawar (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'Jain travels regd', 'price': '₹400', 'bustype': 'NON AC Seater / Sleeper 2+1', 'departing_time': '22:55', 'duration': '2h 50m', 'reaching_time': '01:45', 'star_rating': '3.7', 'seats_available': '42 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Beawar (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'Jain travels regd', 'price': '₹500', 'bustype': 'AC Sleeper (2+1)', 'departing_time': '23:00', 'duration': '3h 30m', 'reaching_time': '02:30', 'star_rating': '3.7', 'seats_available': '36 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Beawar (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'Dashmesh Travels', 'price': '₹570', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '23:30', 'duration': '3h 30m', 'reaching_time': '03:00', 'star_rating': '3.8', 'seats_available': '36 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Beawar (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'RSRTC - 169915', 'price': '₹211', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '17:30', 'duration': '4h', 'reaching_time': '21:30', 'star_rating': '3.8', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Beawar (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'RSRTC - 190595', 'price': '₹211', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '17:35', 'duration': '4h 25m', 'reaching_time': '22:00', 'star_rating': '3.8', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Beawar (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'RSRTC - 188046', 'price': '₹211', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '17:35', 'duration': '3h 55m', 'reaching_time': '21:30', 'star_rating': '3.8', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Beawar (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'RSRTC - 146140', 'price': '₹424', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '17:45', 'duration': '3h 15m', 'reaching_time': '21:00', 'star_rating': '4.5', 'seats_available': '16 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Beawar (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'RSRTC - 197983', 'price': '₹211', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '17:50', 'duration': '3h 55m', 'reaching_time': '21:45', 'star_rating': '3.8', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Beawar (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'RSRTC - 199305', 'price': '₹211', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '17:50', 'duration': '4h 30m', 'reaching_time': '22:20', 'star_rating': '3.8', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Beawar (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'RSRTC - 169078', 'price': '₹282', 'bustype': 'NON AC Seater / Sleeper 2+1', 'departing_time': '18:15', 'duration': '4h 30m', 'reaching_time': '22:45', 'star_rating': '3.8', 'seats_available': '36 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Beawar (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'RSRTC - 198032', 'price': '₹198', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '18:30', 'duration': '4h 30m', 'reaching_time': '23:00', 'star_rating': '3.8', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Beawar (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'RSRTC - 202661', 'price': '₹282', 'bustype': 'NON AC Seater / Sleeper 2+1', 'departing_time': '18:35', 'duration': '4h 25m', 'reaching_time': '23:00', 'star_rating': '3.8', 'seats_available': '36 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Beawar (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'RSRTC - 189844', 'price': '₹211', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '18:45', 'duration': '3h 45m', 'reaching_time': '22:30', 'star_rating': '3.8', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Udaipur to Jodhpur', 'busname': 'Jain travels regd', 'price': '₹460', 'bustype': 'VE A/C Seater / Sleeper (2+1)', 'departing_time': '22:30', 'duration': '6h', 'reaching_time': '04:30', 'star_rating': '4.7', 'seats_available': '27 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Udaipur to Jodhpur', 'busname': 'M R Travels', 'price': '₹612', 'bustype': 'Volvo Multi-Axle I-Shift B11R Semi Sleeper (2+2)', 'departing_time': '17:10', 'duration': '4h 45m', 'reaching_time': '21:55', 'star_rating': '4.5', 'seats_available': '33 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Udaipur to Jodhpur', 'busname': 'Shree Devnarayan Travels', 'price': '₹599', 'bustype': 'Bharat Benz A/C Seater (2+1)', 'departing_time': '17:00', 'duration': '4h 30m', 'reaching_time': '21:30', 'star_rating': '4.5', 'seats_available': '5 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Udaipur to Jodhpur', 'busname': 'M R Travels', 'price': '₹349', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '22:30', 'duration': '6h', 'reaching_time': '04:30', 'star_rating': '4.3', 'seats_available': '33 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Udaipur to Jodhpur', 'busname': 'Jakhar Travels', 'price': '₹457', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '22:00', 'duration': '6h 30m', 'reaching_time': '04:30', 'star_rating': '4.2', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Udaipur to Jodhpur', 'busname': 'Jain travels regd', 'price': '₹350', 'bustype': 'NON AC Seater/ Sleeper (2+1)', 'departing_time': '22:00', 'duration': '6h', 'reaching_time': '04:00', 'star_rating': '4.4', 'seats_available': '27 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Udaipur to Jodhpur', 'busname': 'Kalpana Travels', 'price': '₹500', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '22:00', 'duration': '6h 54m', 'reaching_time': '04:54', 'star_rating': '4.1', 'seats_available': '23 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Udaipur to Jodhpur', 'busname': 'Jain travels regd', 'price': '₹350', 'bustype': 'NON AC Seater/ Sleeper (2+1)', 'departing_time': '23:15', 'duration': '6h 45m', 'reaching_time': '06:00', 'star_rating': '3.5', 'seats_available': '27 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Udaipur to Jodhpur', 'busname': 'Jain Parshwanath Travels', 'price': '₹550', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '21:45', 'duration': '5h 33m', 'reaching_time': '03:18', 'star_rating': '5', 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Udaipur to Jodhpur', 'busname': 'Rishabh Travels Milan', 'price': '₹550', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '21:00', 'duration': '5h 15m', 'reaching_time': '02:15', 'star_rating': '3.5', 'seats_available': '17 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Udaipur to Jodhpur', 'busname': 'RSRTC - 197538', 'price': '₹294', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '17:15', 'duration': '7h', 'reaching_time': '00:15', 'star_rating': '4', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Udaipur to Jodhpur', 'busname': 'RSRTC - 193271', 'price': '₹294', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '19:30', 'duration': '6h 30m', 'reaching_time': '02:00', 'star_rating': '4.2', 'seats_available': '22 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Udaipur to Jodhpur', 'busname': 'RSRTC - 189622', 'price': '₹294', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '22:00', 'duration': '7h', 'reaching_time': '05:00', 'star_rating': '4', 'seats_available': '41 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Sikar to Jaipur (Rajasthan)', 'busname': 'BL Travels', 'price': '₹180', 'bustype': 'AC Seater (2+3)', 'departing_time': '16:30', 'duration': '2h 20m', 'reaching_time': '18:50', 'star_rating': '3.4', 'seats_available': '51 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Sikar to Jaipur (Rajasthan)', 'busname': 'Sagwan Travels', 'price': '₹300', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '17:40', 'duration': '2h 30m', 'reaching_time': '20:10', 'star_rating': '3.5', 'seats_available': '4 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Sikar to Jaipur (Rajasthan)', 'busname': 'Vijay Tour and Travels', 'price': '₹350', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '18:00', 'duration': '2h 50m', 'reaching_time': '20:50', 'star_rating': '3', 'seats_available': '22 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Sikar to Jaipur (Rajasthan)', 'busname': 'Vijay Tour and Travels', 'price': '₹350', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '18:45', 'duration': '2h 55m', 'reaching_time': '21:40', 'star_rating': '3', 'seats_available': '30 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Sikar to Jaipur (Rajasthan)', 'busname': 'Sagwan Travels', 'price': '₹300', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '19:30', 'duration': '2h 25m', 'reaching_time': '21:55', 'star_rating': '3.5', 'seats_available': '2 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Sikar to Jaipur (Rajasthan)', 'busname': 'Sagwan Travels', 'price': '₹300', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '19:50', 'duration': '2h 40m', 'reaching_time': '22:30', 'star_rating': '3.5', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Sikar to Jaipur (Rajasthan)', 'busname': 'Devraj Travels', 'price': '₹299', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '21:25', 'duration': '1h 25m', 'reaching_time': '22:50', 'star_rating': '3.9', 'seats_available': '9 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Sikar to Jaipur (Rajasthan)', 'busname': 'Devraj Travels', 'price': '₹300', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '22:31', 'duration': '1h 26m', 'reaching_time': '23:57', 'star_rating': '3.9', 'seats_available': '38 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Sikar to Jaipur (Rajasthan)', 'busname': 'Raj travels Jhunjhunu', 'price': '₹359', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '20:15', 'duration': '2h 30m', 'reaching_time': '22:45', 'star_rating': '2.9', 'seats_available': '40 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Sikar to Jaipur (Rajasthan)', 'busname': 'Raj travels Jhunjhunu', 'price': '₹270', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '20:35', 'duration': '2h 55m', 'reaching_time': '23:30', 'star_rating': '2.9', 'seats_available': '4 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Sikar to Jaipur (Rajasthan)', 'busname': 'RSRTC - 160960', 'price': '₹129', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '17:20', 'duration': '2h 25m', 'reaching_time': '19:45', 'star_rating': '3.5', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Sikar to Jaipur (Rajasthan)', 'busname': 'RSRTC - 135861', 'price': '₹129', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '17:20', 'duration': '2h 30m', 'reaching_time': '19:50', 'star_rating': '3.5', 'seats_available': '46 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Sikar to Jaipur (Rajasthan)', 'busname': 'RSRTC - 194578', 'price': '₹129', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '17:35', 'duration': '2h 25m', 'reaching_time': '20:00', 'star_rating': '3.5', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Sikar to Jaipur (Rajasthan)', 'busname': 'RSRTC - 168295', 'price': '₹129', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '17:40', 'duration': '2h 10m', 'reaching_time': '19:50', 'star_rating': '3.5', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Sikar to Jaipur (Rajasthan)', 'busname': 'RSRTC - 202688', 'price': '₹129', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '17:40', 'duration': '2h 20m', 'reaching_time': '20:00', 'star_rating': '3.5', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Sikar to Jaipur (Rajasthan)', 'busname': 'RSRTC - 193696', 'price': '₹228', 'bustype': 'Deluxe AC Seater 2+2', 'departing_time': '17:40', 'duration': '2h 5m', 'reaching_time': '19:45', 'star_rating': '2.9', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Sikar to Jaipur (Rajasthan)', 'busname': 'RSRTC - 177353', 'price': '₹129', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '18:15', 'duration': '2h 15m', 'reaching_time': '20:30', 'star_rating': '3.5', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Sikar to Jaipur (Rajasthan)', 'busname': 'RSRTC - 199449', 'price': '₹129', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '18:28', 'duration': '2h 42m', 'reaching_time': '21:10', 'star_rating': '3.5', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Sikar to Jaipur (Rajasthan)', 'busname': 'RSRTC - 199406', 'price': '₹129', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '18:28', 'duration': '2h 42m', 'reaching_time': '21:10', 'star_rating': '3.5', 'seats_available': '41 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Sikar to Jaipur (Rajasthan)', 'busname': 'RSRTC - 104469', 'price': '₹129', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '18:35', 'duration': '2h 15m', 'reaching_time': '20:50', 'star_rating': '3.5', 'seats_available': '46 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Aligarh (uttar pradesh) to Jaipur (Rajasthan)', 'busname': 'Rinku travels', 'price': '₹400', 'bustype': 'A/C Seater/Sleeper (2+1)', 'departing_time': '21:15', 'duration': '8h 7m', 'reaching_time': '05:22', 'star_rating': '3.7', 'seats_available': '8 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Aligarh (uttar pradesh) to Jaipur (Rajasthan)', 'busname': 'Mahalaxmi Travels', 'price': '₹499', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '23:55', 'duration': '7h 5m', 'reaching_time': '07:00', 'star_rating': '3.1', 'seats_available': '24 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Aligarh (uttar pradesh) to Jaipur (Rajasthan)', 'busname': 'MAHALA TRAVELS', 'price': '₹495', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '20:05', 'duration': '7h 8m', 'reaching_time': '03:13', 'star_rating': '3.3', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Aligarh (uttar pradesh) to Jaipur (Rajasthan)', 'busname': 'RSRTC - 160290', 'price': '₹369', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '19:30', 'duration': '7h', 'reaching_time': '02:30', 'star_rating': '3.5', 'seats_available': '41 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Aligarh (uttar pradesh) to Jaipur (Rajasthan)', 'busname': 'RSRTC - 197510', 'price': '₹591', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '21:30', 'duration': '6h 30m', 'reaching_time': '04:00', 'star_rating': '4.3', 'seats_available': '18 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Aligarh (uttar pradesh) to Jaipur (Rajasthan)', 'busname': 'RSRTC - 198017', 'price': '₹463', 'bustype': 'NON AC Seater / Sleeper 2+1', 'departing_time': '22:00', 'duration': '6h 15m', 'reaching_time': '04:15', 'star_rating': '2.9', 'seats_available': '26 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kishangarh (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'UPSRTC - KSN0275', 'price': '₹121', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:06', 'duration': '2h 24m', 'reaching_time': '20:30', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kishangarh (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'Mahalaxmi Travels', 'price': '₹199', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '18:30', 'duration': '3h', 'reaching_time': '21:30', 'star_rating': '3.7', 'seats_available': '17 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kishangarh (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'Jain travels regd', 'price': '₹300', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '18:45', 'duration': '2h', 'reaching_time': '20:45', 'star_rating': '4.1', 'seats_available': '40 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kishangarh (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'Jain travels regd', 'price': '₹250', 'bustype': 'NON AC Seater/ Sleeper (2+1)', 'departing_time': '23:15', 'duration': '2h', 'reaching_time': '01:15', 'star_rating': '4.1', 'seats_available': '39 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kishangarh (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'Garhwal Travels', 'price': '₹500', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '16:40', 'duration': '2h', 'reaching_time': '18:40', 'star_rating': '3.2', 'seats_available': '38 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kishangarh (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'Karwasra Travels', 'price': '₹950', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '18:00', 'duration': '3h', 'reaching_time': '21:00', 'star_rating': '4.1', 'seats_available': '37 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kishangarh (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'Krishna Travels', 'price': '₹1,260', 'bustype': 'NON A/C Sleeper (2+1)', 'departing_time': '18:40', 'duration': '2h', 'reaching_time': '20:40', 'star_rating': '5', 'seats_available': '36 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kishangarh (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'Shri Charbhuja Travels (A Unit Of Bala Agri..)', 'price': '₹1,100', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '18:44', 'duration': '1h 45m', 'reaching_time': '20:29', 'star_rating': '3.7', 'seats_available': '39 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kishangarh (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'Garhwal Travels', 'price': '₹540', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '18:55', 'duration': '1h 20m', 'reaching_time': '20:15', 'star_rating': '3.2', 'seats_available': '38 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kishangarh (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'Shri Krishna Travels ( Jai Shree Ganesh Yatra Co.)', 'price': '₹1,099', 'bustype': 'A/C Seater/Sleeper (2+1)', 'departing_time': '19:00', 'duration': '3h', 'reaching_time': '22:00', 'star_rating': '3.8', 'seats_available': '22 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kishangarh (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'RSRTC - 194408', 'price': '₹120', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '17:30', 'duration': '2h 15m', 'reaching_time': '19:45', 'star_rating': '3.9', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kishangarh (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'RSRTC - 157455', 'price': '₹239', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '17:45', 'duration': '1h 30m', 'reaching_time': '19:15', 'star_rating': '4.3', 'seats_available': '8 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kishangarh (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'RSRTC - 157466', 'price': '₹120', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '18:40', 'duration': '3h 5m', 'reaching_time': '21:45', 'star_rating': '3.9', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kishangarh (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'RSRTC - 193879', 'price': '₹120', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '18:45', 'duration': '2h 45m', 'reaching_time': '21:30', 'star_rating': '3.9', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kishangarh (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'RSRTC - 191024', 'price': '₹184', 'bustype': 'NON AC Seater / Sleeper 2+1', 'departing_time': '19:05', 'duration': '2h 20m', 'reaching_time': '21:25', 'star_rating': '3.9', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kishangarh (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'RSRTC - 146140', 'price': '₹239', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '19:30', 'duration': '1h 30m', 'reaching_time': '21:00', 'star_rating': '3.5', 'seats_available': '14 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kishangarh (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'RSRTC - 169915', 'price': '₹120', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '19:35', 'duration': '1h 55m', 'reaching_time': '21:30', 'star_rating': '3.9', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kishangarh (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'RSRTC - 194178', 'price': '₹239', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '21:15', 'duration': '1h 45m', 'reaching_time': '23:00', 'star_rating': '3.8', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kishangarh (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'RSRTC - 202954', 'price': '₹207', 'bustype': 'Deluxe AC Seater 2+2', 'departing_time': '21:45', 'duration': '1h 30m', 'reaching_time': '23:15', 'star_rating': '3.9', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kishangarh (Rajasthan) to Jaipur (Rajasthan)', 'busname': 'RSRTC - 188804', 'price': '₹120', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '22:00', 'duration': '1h 45m', 'reaching_time': '23:45', 'star_rating': '3.9', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Aligarh (uttar pradesh)', 'busname': 'UPSRTC - BDV0129', 'price': '₹361', 'bustype': 'Ordinary Non AC Seater 2+3', 'departing_time': '23:44', 'duration': '7h 41m', 'reaching_time': '07:25', 'star_rating': '3.3', 'seats_available': '52 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Aligarh (uttar pradesh)', 'busname': 'Rinku travels', 'price': '₹400', 'bustype': 'A/C Seater/Sleeper (2+1)', 'departing_time': '20:30', 'duration': '7h 15m', 'reaching_time': '03:45', 'star_rating': '3.7', 'seats_available': '31 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Aligarh (uttar pradesh)', 'busname': 'Mahalaxmi Travels', 'price': '₹499', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '21:00', 'duration': '7h', 'reaching_time': '04:00', 'star_rating': '3.2', 'seats_available': '29 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Aligarh (uttar pradesh)', 'busname': 'SHAGUN CARGO & TRAVELS', 'price': '₹600', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '22:30', 'duration': '5h 45m', 'reaching_time': '04:15', 'star_rating': '3', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Aligarh (uttar pradesh)', 'busname': 'KKT Travels', 'price': '₹450', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '23:30', 'duration': '8h 30m', 'reaching_time': '08:00', 'star_rating': '3.7', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Aligarh (uttar pradesh)', 'busname': 'SHAGUN CARGO & TRAVELS', 'price': '₹599', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '21:15', 'duration': '5h 45m', 'reaching_time': '03:00', 'star_rating': '2.9', 'seats_available': '33 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Aligarh (uttar pradesh)', 'busname': 'SHAGUN CARGO & TRAVELS', 'price': '₹600', 'bustype': 'NON A/C Seater/ Sleeper (2+1)', 'departing_time': '20:15', 'duration': '7h', 'reaching_time': '03:15', 'star_rating': '4.1', 'seats_available': '32 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Aligarh (uttar pradesh)', 'busname': 'Vijay Tour and Travels', 'price': '₹999', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '19:10', 'duration': '7h', 'reaching_time': '02:10', 'star_rating': '1', 'seats_available': '8 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Aligarh (uttar pradesh)', 'busname': 'Bhagwati Travel Agency', 'price': '₹1,299', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '23:15', 'duration': '8h 15m', 'reaching_time': '07:30', 'star_rating': '3.7', 'seats_available': '34 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Aligarh (uttar pradesh)', 'busname': 'MAHALA TRAVELS', 'price': '₹719', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '23:10', 'duration': '5h 49m', 'reaching_time': '04:59', 'star_rating': '3', 'seats_available': '24 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Aligarh (uttar pradesh)', 'busname': 'RSRTC - 178227', 'price': '₹369', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '19:50', 'duration': '7h 10m', 'reaching_time': '03:00', 'star_rating': '3.1', 'seats_available': '36 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Aligarh (uttar pradesh)', 'busname': 'RSRTC - 198749', 'price': '₹369', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '20:50', 'duration': '7h 10m', 'reaching_time': '04:00', 'star_rating': '3.1', 'seats_available': '41 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Aligarh (uttar pradesh)', 'busname': 'RSRTC - 190202', 'price': '₹369', 'bustype': 'Express Non AC Seater 2+3', 'departing_time': '21:50', 'duration': '7h 10m', 'reaching_time': '05:00', 'star_rating': '3.4', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Aligarh (uttar pradesh)', 'busname': 'RSRTC - 195689', 'price': '₹411', 'bustype': 'Semi Deluxe Non AC Seater 2+2', 'departing_time': '22:01', 'duration': '6h 29m', 'reaching_time': '04:30', 'star_rating': '2.2', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Jaipur (Rajasthan) to Aligarh (uttar pradesh)', 'busname': 'RSRTC - 189082', 'price': '₹591', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '23:00', 'duration': '5h 30m', 'reaching_time': '04:30', 'star_rating': '3.7', 'seats_available': '21 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kota(Rajasthan) to Jaipur (Rajasthan)', 'busname': 'Jakhar Travels', 'price': '₹700', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '23:59', 'duration': '6h 1m', 'reaching_time': '06:00', 'star_rating': '4.3', 'seats_available': '4 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kota(Rajasthan) to Jaipur (Rajasthan)', 'busname': 'Mahalaxmi Travels', 'price': '₹399', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '17:00', 'duration': '5h 15m', 'reaching_time': '22:15', 'star_rating': '4.1', 'seats_available': '8 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kota(Rajasthan) to Jaipur (Rajasthan)', 'busname': 'Vijay Tour and Travels', 'price': '₹474', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '18:00', 'duration': '4h 40m', 'reaching_time': '22:40', 'star_rating': '4.4', 'seats_available': '23 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kota(Rajasthan) to Jaipur (Rajasthan)', 'busname': 'Babu Travels', 'price': '₹349', 'bustype': 'NON A/C Seater/ Sleeper (2+1)', 'departing_time': '23:00', 'duration': '6h 5m', 'reaching_time': '05:05', 'star_rating': '3.8', 'seats_available': '19 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kota(Rajasthan) to Jaipur (Rajasthan)', 'busname': 'Babu Travels', 'price': '₹399', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '23:30', 'duration': '5h 40m', 'reaching_time': '05:10', 'star_rating': '4.4', 'seats_available': '19 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kota(Rajasthan) to Jaipur (Rajasthan)', 'busname': 'NueGo', 'price': '₹284', 'bustype': 'Electric A/C Semi Sleeper (2+2)', 'departing_time': '19:00', 'duration': '4h 35m', 'reaching_time': '23:35', 'star_rating': '3.9', 'seats_available': '23 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kota(Rajasthan) to Jaipur (Rajasthan)', 'busname': 'Jakhar Travels', 'price': '₹250', 'bustype': 'NON A/C Seater/ Sleeper (2+1)', 'departing_time': '23:00', 'duration': '6h 15m', 'reaching_time': '05:15', 'star_rating': '3.2', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kota(Rajasthan) to Jaipur (Rajasthan)', 'busname': 'Maa Gayatri Travels', 'price': '₹600', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '23:00', 'duration': '6h 30m', 'reaching_time': '05:30', 'star_rating': '3.7', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kota(Rajasthan) to Jaipur (Rajasthan)', 'busname': 'Navrang Travels', 'price': '₹405', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '23:00', 'duration': '5h 30m', 'reaching_time': '04:30', 'star_rating': '2.1', 'seats_available': '17 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kota(Rajasthan) to Jaipur (Rajasthan)', 'busname': 'Jakhar Travels', 'price': '₹300', 'bustype': 'NON A/C Seater/ Sleeper (2+1)', 'departing_time': '23:30', 'duration': '6h', 'reaching_time': '05:30', 'star_rating': '3.6', 'seats_available': '14 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kota(Rajasthan) to Jaipur (Rajasthan)', 'busname': 'RSRTC - 201742', 'price': '₹341', 'bustype': 'NON AC Seater / Sleeper 2+1', 'departing_time': '21:00', 'duration': '5h 10m', 'reaching_time': '02:10', 'star_rating': '4.6', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'RSRTC', 'route': 'Kota(Rajasthan) to Jaipur (Rajasthan)', 'busname': 'RSRTC - 197791', 'price': '₹344', 'bustype': 'NON AC Seater / Sleeper 2+1', 'departing_time': '23:15', 'duration': '5h 25m', 'reaching_time': '04:40', 'star_rating': '3.1', 'seats_available': '29 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Durgapur (West Bengal) to Kolkata', 'busname': 'SHYAMOLI Pvt. Ltd.(Karunamoyee)', 'price': '₹486', 'bustype': 'Scania Multi-Axle AC Semi Sleeper (2+2)', 'departing_time': '16:20', 'duration': '3h 50m', 'reaching_time': '20:10', 'star_rating': '4.3', 'seats_available': '6 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Durgapur (West Bengal) to Kolkata', 'busname': 'Express Line (Karunamoyee)', 'price': '₹486', 'bustype': 'Scania Multi-Axle AC Semi Sleeper (2+2)', 'departing_time': '16:30', 'duration': '3h 55m', 'reaching_time': '20:25', 'star_rating': '3.5', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Durgapur (West Bengal) to Kolkata', 'busname': 'Shyamoli Paribahan Pvt Ltd', 'price': '₹486', 'bustype': 'Scania Multi-Axle AC Semi Sleeper (2+2)', 'departing_time': '16:40', 'duration': '4h', 'reaching_time': '20:40', 'star_rating': '4.5', 'seats_available': '23 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Durgapur (West Bengal) to Kolkata', 'busname': 'Greenline (Karunamoyee)', 'price': '₹486', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '17:30', 'duration': '3h 50m', 'reaching_time': '21:20', 'star_rating': '4.2', 'seats_available': '7 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Durgapur (West Bengal) to Kolkata', 'busname': 'SHYAMOLI Pvt. Ltd.(Karunamoyee)', 'price': '₹486', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '16:55', 'duration': '4h', 'reaching_time': '20:55', 'star_rating': '4', 'seats_available': '14 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Durgapur (West Bengal) to Kolkata', 'busname': 'Greenline', 'price': '₹486', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '17:10', 'duration': '3h 50m', 'reaching_time': '21:00', 'star_rating': '4.5', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Durgapur (West Bengal) to Kolkata', 'busname': 'SHYAMOLI Pvt. Ltd.(Karunamoyee)', 'price': '₹486', 'bustype': 'Scania Multi-Axle AC Semi Sleeper (2+2)', 'departing_time': '18:00', 'duration': '3h 45m', 'reaching_time': '21:45', 'star_rating': '3.8', 'seats_available': '28 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Durgapur (West Bengal) to Kolkata', 'busname': 'Express Line', 'price': '₹486', 'bustype': 'Volvo 9600 A/C Semi Sleeper (2+2)', 'departing_time': '18:45', 'duration': '4h 5m', 'reaching_time': '22:50', 'star_rating': '4.5', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Durgapur (West Bengal) to Kolkata', 'busname': 'SBSTC-ASANSOL-KOLKATA- 16:30(HOWRAH DEPOT) - ASANSOL 8-U/16-30', 'price': '₹150', 'bustype': 'Express(Non AC Seater)', 'departing_time': '17:40', 'duration': '3h 30m', 'reaching_time': '21:10', 'star_rating': '4.3', 'seats_available': '45 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Durgapur (West Bengal) to Kolkata', 'busname': 'SBSTC-ASANSOL-KARUNAMOYEE 16:40 (ASN DEPOT) - K Moy-N/H3-U/16-40', 'price': '₹155', 'bustype': 'Express(Non AC Seater)', 'departing_time': '17:40', 'duration': '3h 55m', 'reaching_time': '21:35', 'star_rating': '4.3', 'seats_available': '26 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Burdwan', 'busname': 'SHYAMOLI Pvt. Ltd.(Karunamoyee)', 'price': '₹429', 'bustype': 'Scania Multi-Axle AC Semi Sleeper (2+2)', 'departing_time': '16:20', 'duration': '2h 10m', 'reaching_time': '18:30', 'star_rating': '4', 'seats_available': '32 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Burdwan', 'busname': 'Express Line (Karunamoyee)', 'price': '₹429', 'bustype': 'Scania Multi-Axle AC Semi Sleeper (2+2)', 'departing_time': '16:30', 'duration': '1h 30m', 'reaching_time': '18:00', 'star_rating': '4.6', 'seats_available': '9 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Burdwan', 'busname': 'Greenline', 'price': '₹429', 'bustype': 'Volvo 9600 Multi Axle Semi-Sleeper (2+2)', 'departing_time': '16:40', 'duration': '1h 10m', 'reaching_time': '17:50', 'star_rating': '4.7', 'seats_available': '28 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Burdwan', 'busname': 'Shyamoli Paribahan Pvt Ltd', 'price': '₹429', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '16:40', 'duration': '2h 15m', 'reaching_time': '18:55', 'star_rating': '4.2', 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Burdwan', 'busname': 'Greenline (Karunamoyee)', 'price': '₹429', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '16:45', 'duration': '2h 35m', 'reaching_time': '19:20', 'star_rating': '4.3', 'seats_available': '17 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Burdwan', 'busname': 'Express Line', 'price': '₹429', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '17:20', 'duration': '2h 10m', 'reaching_time': '19:30', 'star_rating': '4', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Burdwan', 'busname': 'SHYAMOLI Pvt. Ltd.(Karunamoyee)', 'price': '₹429', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '17:30', 'duration': '2h 30m', 'reaching_time': '20:00', 'star_rating': '4.5', 'seats_available': '25 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Burdwan', 'busname': 'Greenline', 'price': '₹429', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '18:05', 'duration': '2h 40m', 'reaching_time': '20:45', 'star_rating': '4.5', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Burdwan', 'busname': 'Greenline (Karunamoyee)', 'price': '₹429', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '18:15', 'duration': '2h 30m', 'reaching_time': '20:45', 'star_rating': '3.9', 'seats_available': '30 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Burdwan', 'busname': 'Royal Cruiser', 'price': '₹170', 'bustype': 'Scania Multi-Axle AC Semi Sleeper (2+2)', 'departing_time': '19:30', 'duration': '2h 45m', 'reaching_time': '22:15', 'star_rating': '3.4', 'seats_available': '47 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Burdwan', 'busname': 'SBSTC-KARUNAMOYEE - BURDWAN - 17:00 (BWN DEPOT) - KARUNAMOY4-D/17-00', 'price': '₹100', 'bustype': 'Express(Non AC Seater)', 'departing_time': '17:00', 'duration': '1h 59m', 'reaching_time': '18:59', 'star_rating': '3.8', 'seats_available': '9 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Burdwan', 'busname': 'SBSTC-KOLKATA - DINANATHPUR - VIA - BURDWAN - 17:30 (BWN - DINANATHPUR-U/17-30', 'price': '₹95', 'bustype': 'Express(Non AC Seater)', 'departing_time': '17:30', 'duration': '2h', 'reaching_time': '19:30', 'star_rating': '4.3', 'seats_available': '43 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Burdwan', 'busname': 'SBSTC-KARUNAMOYEE - BURDWAN - 17:30 (BWN DEPOT) - KARUNAMOY5-D/17-30', 'price': '₹100', 'bustype': 'Express(Non AC Seater)', 'departing_time': '17:30', 'duration': '1h 59m', 'reaching_time': '19:29', 'star_rating': '3.3', 'seats_available': '17 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Burdwan', 'busname': 'SBSTC-KARUNAMOYEE - BURDWAN - 18:15 (BWN DEPOT) - KARUNAMOY6-D/18-15', 'price': '₹100', 'bustype': 'Express(Non AC Seater)', 'departing_time': '18:15', 'duration': '1h 59m', 'reaching_time': '20:14', 'star_rating': '3.5', 'seats_available': '24 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Burdwan', 'busname': 'SBSTC-KOLKATA - BURDWAN - 18:15 (BWN DEPOT) - KOLKATA 5-D/18-15', 'price': '₹95', 'bustype': 'Express(Non AC Seater)', 'departing_time': '18:15', 'duration': '1h 59m', 'reaching_time': '20:14', 'star_rating': '4.1', 'seats_available': '36 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Burdwan', 'busname': 'SBSTC-KARUNAMOYEE - BURDWAN - A/C - 18:45 (BWN DEPOT) - K MOYEE AC-D/18-45', 'price': '₹200', 'bustype': 'A/C Seater (2+2)', 'departing_time': '18:45', 'duration': '1h 59m', 'reaching_time': '20:44', 'star_rating': '4.3', 'seats_available': '24 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Burdwan', 'busname': 'SBSTC-KARUNAMOYEE - BURDWAN - 19:30 (BWN DEPOT) - KARUNAMOY7-D/19-30', 'price': '₹100', 'bustype': 'Express(Non AC Seater)', 'departing_time': '19:30', 'duration': '1h 59m', 'reaching_time': '21:29', 'star_rating': '3.5', 'seats_available': '36 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Burdwan', 'busname': 'SBSTC-KOLKATA - BURDWAN - 20:00 (BWN DEPOT) - KOLKATA 9-D/20-30', 'price': '₹95', 'bustype': 'Express(Non AC Seater)', 'departing_time': '20:00', 'duration': '2h', 'reaching_time': '22:00', 'star_rating': '4.4', 'seats_available': '52 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Midnapore to Kolkata', 'busname': 'WBTC (CTC) Barasat - Midnapur - 208|16:50', 'price': '₹105', 'bustype': 'NON A/C Seater (2+3)', 'departing_time': '16:50', 'duration': '1h 50m', 'reaching_time': '18:40', 'star_rating': '3.4', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'SHYAMOLI Pvt. Ltd.(Karunamoyee)', 'price': '₹486', 'bustype': 'Scania Multi-Axle AC Semi Sleeper (2+2)', 'departing_time': '16:20', 'duration': '3h 40m', 'reaching_time': '20:00', 'star_rating': '4.2', 'seats_available': '25 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'Express Line (Karunamoyee)', 'price': '₹486', 'bustype': 'Scania Multi-Axle AC Semi Sleeper (2+2)', 'departing_time': '16:30', 'duration': '3h 15m', 'reaching_time': '19:45', 'star_rating': '4.6', 'seats_available': '9 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'Greenline', 'price': '₹486', 'bustype': 'Volvo 9600 Multi Axle Semi-Sleeper (2+2)', 'departing_time': '16:40', 'duration': '2h 35m', 'reaching_time': '19:15', 'star_rating': '4.6', 'seats_available': '7 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'Express Line', 'price': '₹486', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '17:20', 'duration': '4h', 'reaching_time': '21:20', 'star_rating': '4.6', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'Shyamoli Paribahan Pvt Ltd', 'price': '₹486', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '16:40', 'duration': '4h', 'reaching_time': '20:40', 'star_rating': '4', 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'SHYAMOLI Pvt. Ltd.(Karunamoyee)', 'price': '₹486', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '17:30', 'duration': '4h', 'reaching_time': '21:30', 'star_rating': '4.3', 'seats_available': '31 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'Greenline', 'price': '₹486', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '18:05', 'duration': '4h', 'reaching_time': '22:05', 'star_rating': '4.4', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'Greenline (Karunamoyee)', 'price': '₹486', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '18:15', 'duration': '4h', 'reaching_time': '22:15', 'star_rating': '4.1', 'seats_available': '29 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'Royal Cruiser', 'price': '₹350', 'bustype': 'Scania Multi-Axle AC Semi Sleeper (2+2)', 'departing_time': '19:30', 'duration': '3h 45m', 'reaching_time': '23:15', 'star_rating': '3.3', 'seats_available': '47 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'Greenline', 'price': '₹600', 'bustype': 'Volvo 9600 A/C Seater/Sleeper (2+1)', 'departing_time': '20:00', 'duration': '3h 58m', 'reaching_time': '23:58', 'star_rating': '4.5', 'seats_available': '36 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'SBSTC-KARUNAMOYEE - BANKURA - VIA - DURGAPUR - 17:00 - KMOYE 3-D/16-45', 'price': '₹155', 'bustype': 'Express(Non AC Seater)', 'departing_time': '17:00', 'duration': '3h 15m', 'reaching_time': '20:15', 'star_rating': '3.1', 'seats_available': '24 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'SBSTC-KOLKATA-ASANSOL17:00 (ASN DEPOT) - KOL-8-L/S-D/17-00', 'price': '₹150', 'bustype': 'Express(Non AC Seater)', 'departing_time': '17:00', 'duration': '3h 25m', 'reaching_time': '20:25', 'star_rating': '4', 'seats_available': '50 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'SBSTC-KOLKATA-DURGAPUR 17:15 (DGP DEPOT) - DGPKOLE-D/17-15', 'price': '₹150', 'bustype': 'Express(Non AC Seater)', 'departing_time': '17:15', 'duration': '2h 59m', 'reaching_time': '20:14', 'star_rating': '3.8', 'seats_available': '52 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'SBSTC-KOLKATA-JAMURIA VIA-DURGAPUR-17:30 (HOW DEPOT) - JAMURIA 1-U/17:30', 'price': '₹150', 'bustype': 'Express(Non AC Seater)', 'departing_time': '17:30', 'duration': '3h 30m', 'reaching_time': '21:00', 'star_rating': '3.9', 'seats_available': '52 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'SBSTC-KARUNAMOYEE-DURGAPUR 17:30 (DGP DEPOT) - L-63-D/17-30', 'price': '₹155', 'bustype': 'Express(Non AC Seater)', 'departing_time': '17:30', 'duration': '3h 59m', 'reaching_time': '21:29', 'star_rating': '3.5', 'seats_available': '45 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'SBSTC-KOLKATA-DURGAPUR 18:30 (DGP DEPOT) - DGPKOLE-D/17-45', 'price': '₹150', 'bustype': 'Express(Non AC Seater)', 'departing_time': '18:30', 'duration': '4h', 'reaching_time': '22:30', 'star_rating': '5', 'seats_available': '51 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Haldia to Kolkata', 'busname': 'WBTC (CTC) Habra- Barasat- Haldia - 180|16:45', 'price': '₹97', 'bustype': 'Non AC Seater (2+3)', 'departing_time': '16:45', 'duration': '2h 45m', 'reaching_time': '19:30', 'star_rating': '4.2', 'seats_available': '24 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Haldia to Kolkata', 'busname': 'SBSTC-HALDIA-KOLKATA 17:30 (HAL DEPOT) - H15 N/H-U/17-30', 'price': '₹110', 'bustype': 'Express(Non AC Seater)', 'departing_time': '17:30', 'duration': '3h', 'reaching_time': '20:30', 'star_rating': '3.9', 'seats_available': '30 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Haldia', 'busname': 'SS Travels', 'price': '₹153', 'bustype': 'Non AC Seater (2+2)', 'departing_time': '17:20', 'duration': '3h 55m', 'reaching_time': '21:15', 'star_rating': '3.4', 'seats_available': '22 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Haldia', 'busname': 'NANDALAL', 'price': '₹131', 'bustype': 'Non AC Seater (2+2)', 'departing_time': '17:45', 'duration': '3h', 'reaching_time': '20:45', 'star_rating': '2.5', 'seats_available': '44 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Haldia', 'busname': 'SBSTC-KOLKATA-HALDIA 17:00 (HAL DEPOT) - H33-D/17-00', 'price': '₹110', 'bustype': 'Express(Non AC Seater)', 'departing_time': '17:00', 'duration': '2h 59m', 'reaching_time': '19:59', 'star_rating': '4.1', 'seats_available': '40 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Haldia', 'busname': 'SBSTC-KOLATA-HALDIA 17:30 (HAL DEPOT) - H31-D/17-30', 'price': '₹110', 'bustype': 'Express(Non AC Seater)', 'departing_time': '17:30', 'duration': '2h 59m', 'reaching_time': '20:29', 'star_rating': '4', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Haldia', 'busname': 'SBSTC-KOLKATA-HALDIA 18:15 (HAL DEPOT) - H39-D/18-15', 'price': '₹110', 'bustype': 'Express(Non AC Seater)', 'departing_time': '18:15', 'duration': '2h 59m', 'reaching_time': '21:14', 'star_rating': '3.5', 'seats_available': '29 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Midnapore', 'busname': 'Maa Bhabatarini', 'price': '₹189', 'bustype': 'AC Seater (2+2)', 'departing_time': '17:20', 'duration': '3h 9m', 'reaching_time': '20:29', 'star_rating': '4.4', 'seats_available': '22 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Midnapore', 'busname': 'SBSTC-KOLKATA - MIDNAPUR - 18:00 (MID DEPOT) - KOL-08-D/19-00', 'price': '₹110', 'bustype': 'Express(Non AC Seater)', 'departing_time': '18:00', 'duration': '3h', 'reaching_time': '21:00', 'star_rating': '3.5', 'seats_available': '30 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Midnapore', 'busname': 'SBSTC-KOLKATA-JHARGRAM VIA MIDNAPUR 18:20 (JHAR ) - KOLKATA10 N/H-U/05-2', 'price': '₹110', 'bustype': 'Express(Non AC Seater)', 'departing_time': '18:20', 'duration': '3h 13m', 'reaching_time': '21:33', 'star_rating': '3.1', 'seats_available': '47 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Arambagh (West Bengal)', 'busname': 'SBSTC-KOLKATA -ARAMBAG 17:30 P.M (ARAM DEPOT) - KOLNS3-D/17-30', 'price': '₹95', 'bustype': 'Express(Non AC Seater)', 'departing_time': '17:30', 'duration': '1h 59m', 'reaching_time': '19:29', 'star_rating': '3.2', 'seats_available': '7 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Digha', 'busname': 'WBTC (CTC) HABRA-DIGHA via Esplanade - 176|23:10', 'price': '₹145', 'bustype': 'Non AC Seater (2+3)', 'departing_time': '23:10', 'duration': '4h 15m', 'reaching_time': '03:25', 'star_rating': '4.1', 'seats_available': '22 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Digha', 'busname': 'Raipur Cruiser', 'price': '₹400', 'bustype': 'Volvo 9600 Multi-Axle A/C Seater (2+2)', 'departing_time': '23:30', 'duration': '5h', 'reaching_time': '04:30', 'star_rating': '4.7', 'seats_available': '36 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Digha', 'busname': 'Raipur Cruiser', 'price': '₹399', 'bustype': 'Volvo 9600 Multi-Axle A/C Seater (2+2)', 'departing_time': '23:55', 'duration': '5h 5m', 'reaching_time': '05:00', 'star_rating': '4.7', 'seats_available': '51 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Digha', 'busname': 'Shyamoli Paribahan Pvt Ltd', 'price': '₹349', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '23:45', 'duration': '5h', 'reaching_time': '04:45', 'star_rating': '4.3', 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Digha', 'busname': 'Shyamoli Paribahan Pvt Ltd', 'price': '₹300', 'bustype': 'VE A/C Seater (2+2)', 'departing_time': '23:30', 'duration': '5h', 'reaching_time': '04:30', 'star_rating': '4.1', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Digha', 'busname': 'Bhorpet Travels', 'price': '₹300', 'bustype': 'A/C Seater/Sleeper (2+1)', 'departing_time': '23:50', 'duration': '4h 10m', 'reaching_time': '04:00', 'star_rating': '4.5', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Digha', 'busname': 'SANTOSH BUS SERVICE', 'price': '₹249', 'bustype': 'A/C Seater / Sleeper (2+2)', 'departing_time': '00:29', 'duration': '4h 36m', 'reaching_time': '05:05', 'star_rating': '3.9', 'seats_available': '49 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Digha', 'busname': 'Parameswar Travels', 'price': '₹289', 'bustype': 'A/C Seater Push Back (2+3)', 'departing_time': '23:59', 'duration': '5h 1m', 'reaching_time': '05:00', 'star_rating': '3.4', 'seats_available': '56 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Digha', 'busname': 'Parameswar Travels', 'price': '₹281', 'bustype': 'A/C Seater Push Back (2+3)', 'departing_time': '23:55', 'duration': '5h 5m', 'reaching_time': '05:00', 'star_rating': '3.5', 'seats_available': '47 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Digha', 'busname': 'Maa Chandi Travels', 'price': '₹306', 'bustype': 'A/C Seater / Sleeper (2+2)', 'departing_time': '23:57', 'duration': '5h 13m', 'reaching_time': '05:10', 'star_rating': '3.9', 'seats_available': '21 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Digha', 'busname': 'SBSTC-KOLKATA-DIGHA 23:30 (DIG DEPOT) - KOLKATA-N/S-D/23-00', 'price': '₹155', 'bustype': 'Express(Non AC Seater)', 'departing_time': '23:30', 'duration': '4h 59m', 'reaching_time': '04:29', 'star_rating': '3.9', 'seats_available': '32 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Digha', 'busname': 'SBSTC-GARIA-DIGHA -22:30 (DIGHA DEPOT) - DIGHA-3 N/H-D/07-00', 'price': '₹162', 'bustype': 'Express(Non AC Seater)', 'departing_time': '23:45', 'duration': '5h', 'reaching_time': '04:45', 'star_rating': '3.8', 'seats_available': '27 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Bankura', 'busname': 'SBSTC-KARUNAMOYEE - BANKURA - VIA - DURGAPUR - 17:00 - KMOYE 3-D/16-45', 'price': '₹185', 'bustype': 'Express(Non AC Seater)', 'departing_time': '17:00', 'duration': '5h 30m', 'reaching_time': '22:30', 'star_rating': '3.5', 'seats_available': '24 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Digha to Kolkata', 'busname': 'Satya Paribahan', 'price': '₹315', 'bustype': 'A/C Seater (2+3)', 'departing_time': '23:00', 'duration': '4h 45m', 'reaching_time': '03:45', 'star_rating': '3', 'seats_available': '33 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Digha to Kolkata', 'busname': 'Snemita Paribahan (Shinjini)', 'price': '₹275', 'bustype': 'AC Seater (2+3)', 'departing_time': '23:35', 'duration': '4h 40m', 'reaching_time': '04:15', 'star_rating': '2.9', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Digha to Kolkata', 'busname': 'Parameswar Travels', 'price': '₹285', 'bustype': 'A/C Seater Push Back (2+3)', 'departing_time': '21:00', 'duration': '6h', 'reaching_time': '03:00', 'star_rating': '2.9', 'seats_available': '51 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Digha to Kolkata', 'busname': 'Parameswar Travels', 'price': '₹285', 'bustype': 'A/C Seater Push Back (2+3)', 'departing_time': '17:50', 'duration': '5h 10m', 'reaching_time': '23:00', 'star_rating': '2.3', 'seats_available': '29 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Digha to Kolkata', 'busname': 'Aradhana Travel', 'price': '₹450', 'bustype': 'A/C Seater / Sleeper (2+2)', 'departing_time': '18:35', 'duration': '4h 25m', 'reaching_time': '23:00', 'star_rating': '1', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Digha to Kolkata', 'busname': 'Bhorpet Travels', 'price': '₹270', 'bustype': 'A/C Seater Push Back (2+2)', 'departing_time': '17:35', 'duration': '4h 55m', 'reaching_time': '22:30', 'star_rating': '3.9', 'seats_available': '29 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Digha to Kolkata', 'busname': 'Aradhana Bus Service', 'price': '₹630', 'bustype': 'A/C Seater / Sleeper (2+2)', 'departing_time': '22:55', 'duration': '5h 5m', 'reaching_time': '04:00', 'star_rating': '3.9', 'seats_available': '48 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Digha to Kolkata', 'busname': 'The Joy Ride', 'price': '₹650', 'bustype': 'A/C Seater / Sleeper (2+2)', 'departing_time': '21:15', 'duration': '5h 15m', 'reaching_time': '02:30', 'star_rating': None, 'seats_available': '45 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Digha to Kolkata', 'busname': 'Riya Bus', 'price': '₹450', 'bustype': 'Non AC Seater (2+3)', 'departing_time': '20:20', 'duration': '10h 10m', 'reaching_time': '06:30', 'star_rating': None, 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Digha to Kolkata', 'busname': 'City Express (Shinjini)', 'price': '₹333', 'bustype': 'AC Seater (2+3)', 'departing_time': '22:45', 'duration': '5h 25m', 'reaching_time': '04:10', 'star_rating': '3.1', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Asansol (West Bengal)', 'busname': 'SBSTC-KOLKATA-ASANSOL17:00 (ASN DEPOT) - KOL-8-L/S-D/17-00', 'price': '₹180', 'bustype': 'Express(Non AC Seater)', 'departing_time': '17:00', 'duration': '4h 30m', 'reaching_time': '21:30', 'star_rating': '3.1', 'seats_available': '46 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Asansol (West Bengal)', 'busname': 'Express Line (Karunamoyee)', 'price': '₹524', 'bustype': 'Scania Multi-Axle AC Semi Sleeper (2+2)', 'departing_time': '16:30', 'duration': '4h 15m', 'reaching_time': '20:45', 'star_rating': '4.4', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Asansol (West Bengal)', 'busname': 'SHYAMOLI Pvt. Ltd.(Karunamoyee)', 'price': '₹524', 'bustype': 'Scania Multi-Axle AC Semi Sleeper (2+2)', 'departing_time': '16:30', 'duration': '4h 30m', 'reaching_time': '21:00', 'star_rating': '3.7', 'seats_available': '32 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Asansol (West Bengal)', 'busname': 'Greenline', 'price': '₹524', 'bustype': 'Volvo 9600 Multi Axle Semi-Sleeper (2+2)', 'departing_time': '16:40', 'duration': '3h 20m', 'reaching_time': '20:00', 'star_rating': '4.3', 'seats_available': '6 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Asansol (West Bengal)', 'busname': 'Shyamoli Paribahan Pvt Ltd', 'price': '₹524', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '16:40', 'duration': '5h', 'reaching_time': '21:40', 'star_rating': '3.3', 'seats_available': '13 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Asansol (West Bengal)', 'busname': 'Greenline (Karunamoyee)', 'price': '₹524', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '16:45', 'duration': '5h', 'reaching_time': '21:45', 'star_rating': '4.1', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Asansol (West Bengal)', 'busname': 'Express Line', 'price': '₹524', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '17:20', 'duration': '5h', 'reaching_time': '22:20', 'star_rating': '4.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Asansol (West Bengal)', 'busname': 'SHYAMOLI Pvt. Ltd.(Karunamoyee)', 'price': '₹524', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '17:30', 'duration': '5h', 'reaching_time': '22:30', 'star_rating': '4.5', 'seats_available': '31 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Asansol (West Bengal)', 'busname': 'Greenline (Karunamoyee)', 'price': '₹524', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '18:15', 'duration': '5h', 'reaching_time': '23:15', 'star_rating': '4.3', 'seats_available': '29 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Asansol (West Bengal)', 'busname': 'Royal Cruiser', 'price': '₹450', 'bustype': 'Scania Multi-Axle AC Semi Sleeper (2+2)', 'departing_time': '19:30', 'duration': '4h 45m', 'reaching_time': '00:15', 'star_rating': '3.4', 'seats_available': '39 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Contai (Kanthi)', 'busname': 'SANTOSH BUS SERVICE', 'price': '₹305', 'bustype': 'AC Seater (2+3)', 'departing_time': '17:20', 'duration': '3h 35m', 'reaching_time': '20:55', 'star_rating': '3.6', 'seats_available': '40 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Contai (Kanthi)', 'busname': 'Samanta Travels', 'price': '₹370', 'bustype': 'A/C Seater (2+3)', 'departing_time': '18:10', 'duration': '4h', 'reaching_time': '22:10', 'star_rating': '3.3', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Contai (Kanthi)', 'busname': 'Snemita Parisheba', 'price': '₹352', 'bustype': 'AC Seater (2+3)', 'departing_time': '19:05', 'duration': '4h', 'reaching_time': '23:05', 'star_rating': '3.6', 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Contai (Kanthi)', 'busname': 'Snemita Parisheba', 'price': '₹352', 'bustype': 'AC Seater (2+3)', 'departing_time': '19:30', 'duration': '4h', 'reaching_time': '23:30', 'star_rating': '3.6', 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Contai (Kanthi)', 'busname': 'Parameswar Travels', 'price': '₹289', 'bustype': 'A/C Seater Push Back (2+3)', 'departing_time': '21:50', 'duration': '4h 40m', 'reaching_time': '02:30', 'star_rating': '4.1', 'seats_available': '52 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Contai (Kanthi)', 'busname': 'Rohit Travels', 'price': '₹650', 'bustype': 'A/C Seater / Sleeper (2+2)', 'departing_time': '23:39', 'duration': '4h 21m', 'reaching_time': '04:00', 'star_rating': '3.6', 'seats_available': '51 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Contai (Kanthi)', 'busname': 'Parameswar Travels', 'price': '₹298', 'bustype': 'A/C Seater Push Back (2+3)', 'departing_time': '23:45', 'duration': '4h 15m', 'reaching_time': '04:00', 'star_rating': '3.1', 'seats_available': '56 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Contai (Kanthi)', 'busname': 'Parameswar Travels', 'price': '₹281', 'bustype': 'A/C Seater Push Back (2+3)', 'departing_time': '23:55', 'duration': '4h 5m', 'reaching_time': '04:00', 'star_rating': '3.1', 'seats_available': '47 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Contai (Kanthi)', 'busname': 'Parameswar Travels', 'price': '₹289', 'bustype': 'A/C Seater Push Back (2+3)', 'departing_time': '23:59', 'duration': '4h 1m', 'reaching_time': '04:00', 'star_rating': '3.1', 'seats_available': '56 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Contai (Kanthi)', 'busname': 'SANTOSH BUS SERVICE', 'price': '₹249', 'bustype': 'A/C Seater / Sleeper (2+2)', 'departing_time': '00:29', 'duration': '3h 35m', 'reaching_time': '04:04', 'star_rating': '3.4', 'seats_available': '49 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Nandakumar (west bengal)', 'busname': 'SBSTC-KOLATA-HALDIA 17:30 (HAL DEPOT) - H31-D/17-30', 'price': '₹85', 'bustype': 'Express(Non AC Seater)', 'departing_time': '17:30', 'duration': '2h 14m', 'reaching_time': '19:44', 'star_rating': '4.6', 'seats_available': '32 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Nandakumar (west bengal)', 'busname': 'Basu Travels', 'price': '₹300', 'bustype': 'A/C Seater / Sleeper (2+2)', 'departing_time': '23:59', 'duration': '2h 1m', 'reaching_time': '02:00', 'star_rating': '3', 'seats_available': '34 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Nandakumar (west bengal)', 'busname': 'Sagufta Travels (Santosh)', 'price': '₹347', 'bustype': 'A/C Seater / Sleeper (2+2)', 'departing_time': '01:20', 'duration': '2h', 'reaching_time': '03:20', 'star_rating': '3.3', 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Nandakumar (west bengal)', 'busname': 'Basu Travels', 'price': '₹469', 'bustype': 'A/C Seater Push Back (2+3)', 'departing_time': '23:55', 'duration': '2h 20m', 'reaching_time': '02:15', 'star_rating': '3', 'seats_available': '21 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Nandakumar (west bengal)', 'busname': 'Basu Travels', 'price': '₹300', 'bustype': 'A/C Seater Push Back (2+3)', 'departing_time': '23:30', 'duration': '2h 15m', 'reaching_time': '01:45', 'star_rating': '3', 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Nandakumar (west bengal)', 'busname': 'Subhadip Travels(jahnabi)', 'price': '₹420', 'bustype': 'A/C Executive (2+3)', 'departing_time': '17:20', 'duration': '1h 30m', 'reaching_time': '18:50', 'star_rating': '2.4', 'seats_available': '46 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Kolkata to Nandakumar (west bengal)', 'busname': 'Subhadip Travels(Kojagar)', 'price': '₹420', 'bustype': 'Non AC Seater (2+3)', 'departing_time': '16:50', 'duration': '2h 25m', 'reaching_time': '19:15', 'star_rating': '2.4', 'seats_available': '31 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Digha to Durgapur (West Bengal)', 'busname': 'SBSTC-DIGHA - DURGAPUR-VIA-BISHNUPUR- 2/2 - 20:30 (DGP DEPOT) - L-13-D/20-00', 'price': '₹233', 'bustype': 'Express(Non AC Seater)', 'departing_time': '20:30', 'duration': '7h 56m', 'reaching_time': '04:26', 'star_rating': '3.2', 'seats_available': '19 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'South Bengal State Transport Corporation (SBSTC)', 'route': 'Digha to Durgapur (West Bengal)', 'busname': 'SBSTC-DIGHA-ASANSOL VIA DURGAPUR STN.20:30 (ASN DEPOT) - DIGHA N/S-D/21-30', 'price': '₹233', 'bustype': 'Express(Non AC Seater)', 'departing_time': '21:30', 'duration': '7h 55m', 'reaching_time': '05:25', 'star_rating': '2.7', 'seats_available': '48 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Hamirpur (Himachal Pradesh) to Chandigarh', 'busname': 'HRTC - 15', 'price': '₹609', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '20:00', 'duration': '4h 10m', 'reaching_time': '00:10', 'star_rating': '4.8', 'seats_available': '2 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Hamirpur (Himachal Pradesh) to Chandigarh', 'busname': 'Ram Dalal Holidays Pvt Ltd', 'price': '₹649', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '20:15', 'duration': '4h 20m', 'reaching_time': '00:35', 'star_rating': '3.9', 'seats_available': '6 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Hamirpur (Himachal Pradesh) to Chandigarh', 'busname': 'Laxmi holidays', 'price': '₹515', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '20:20', 'duration': '4h 10m', 'reaching_time': '00:30', 'star_rating': '4.2', 'seats_available': '4 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Hamirpur (Himachal Pradesh) to Chandigarh', 'busname': 'Northern Travels', 'price': '₹500', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '21:45', 'duration': '3h 45m', 'reaching_time': '01:30', 'star_rating': '3.2', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Hamirpur (Himachal Pradesh) to Chandigarh', 'busname': 'Ram Dalal Holidays Pvt Ltd', 'price': '₹649', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '22:00', 'duration': '4h 20m', 'reaching_time': '02:20', 'star_rating': '5', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Hamirpur (Himachal Pradesh) to Chandigarh', 'busname': 'Bedi Travels', 'price': '₹563', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '22:05', 'duration': '3h 50m', 'reaching_time': '01:55', 'star_rating': '4.8', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Hamirpur (Himachal Pradesh) to Chandigarh', 'busname': 'Laxmi holidays', 'price': '₹499', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '22:05', 'duration': '4h 35m', 'reaching_time': '02:40', 'star_rating': '4.9', 'seats_available': '5 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Hamirpur (Himachal Pradesh) to Chandigarh', 'busname': 'Himalayan Nomad Ktc', 'price': '₹567', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '22:05', 'duration': '3h 25m', 'reaching_time': '01:30', 'star_rating': '4.2', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Hamirpur (Himachal Pradesh) to Chandigarh', 'busname': 'Himalayan Nomad Ktc', 'price': '₹675', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '22:25', 'duration': '3h 20m', 'reaching_time': '01:45', 'star_rating': '4.5', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Chandigarh to Hamirpur (Himachal Pradesh)', 'busname': 'HRTC - 401670', 'price': '₹335', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '18:00', 'duration': '4h', 'reaching_time': '22:00', 'star_rating': '4', 'seats_available': '28 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Chandigarh to Hamirpur (Himachal Pradesh)', 'busname': 'HRTC - 1227', 'price': '₹392', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '21:40', 'duration': '5h 20m', 'reaching_time': '03:00', 'star_rating': '3.8', 'seats_available': '37 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Shimla', 'busname': 'Bedi Travels', 'price': '₹416', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '23:00', 'duration': '9h', 'reaching_time': '08:00', 'star_rating': '4.8', 'seats_available': '24 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Shimla', 'busname': 'zingbus plus', 'price': '₹499', 'bustype': 'Bharat Benz A/C Semi Sleeper (2+2)', 'departing_time': '23:30', 'duration': '9h 45m', 'reaching_time': '09:15', 'star_rating': '4.8', 'seats_available': '22 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Shimla', 'busname': 'Laxmi holidays', 'price': '₹500', 'bustype': 'Scania Multi-Axle AC Semi Sleeper (2+2)', 'departing_time': '22:30', 'duration': '8h 50m', 'reaching_time': '07:20', 'star_rating': '4.5', 'seats_available': '37 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Shimla', 'busname': 'Laxmi holidays', 'price': '₹519', 'bustype': 'Scania Multi-Axle AC Semi Sleeper (2+2)', 'departing_time': '23:30', 'duration': '8h 30m', 'reaching_time': '08:00', 'star_rating': '4.3', 'seats_available': '28 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Shimla', 'busname': 'zingbus plus', 'price': '₹471', 'bustype': 'Bharat Benz A/C Semi Sleeper (2+2)', 'departing_time': '22:55', 'duration': '9h 35m', 'reaching_time': '08:30', 'star_rating': '4.4', 'seats_available': '22 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Shimla', 'busname': 'FlixBus', 'price': '₹571', 'bustype': 'A/C Seater (2+2)', 'departing_time': '22:00', 'duration': '9h 40m', 'reaching_time': '07:40', 'star_rating': '4.4', 'seats_available': '40 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Shimla', 'busname': 'City Land Travels', 'price': '₹432', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '22:30', 'duration': '7h 30m', 'reaching_time': '06:00', 'star_rating': '4.1', 'seats_available': '21 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Shimla', 'busname': 'Deltin Travels Pvt Ltd', 'price': '₹500', 'bustype': 'Volvo 9600 A/C Semi Sleeper (2+2)', 'departing_time': '23:10', 'duration': '7h 30m', 'reaching_time': '06:40', 'star_rating': '4.1', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Shimla', 'busname': 'Ram Dalal Holidays Pvt Ltd', 'price': '₹448', 'bustype': 'Volvo 9600 A/C Semi Sleeper (2+2)', 'departing_time': '23:00', 'duration': '8h 40m', 'reaching_time': '07:40', 'star_rating': '4.3', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Shimla', 'busname': 'Laxmi holidays', 'price': '₹500', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '22:50', 'duration': '8h 30m', 'reaching_time': '07:20', 'star_rating': '4.3', 'seats_available': '28 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Shimla', 'busname': 'HRTC - 93', 'price': '₹1,051', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '19:35', 'duration': '9h 15m', 'reaching_time': '04:50', 'star_rating': '4.3', 'seats_available': '33 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Shimla', 'busname': 'HRTC - 556', 'price': '₹540', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '20:10', 'duration': '9h 35m', 'reaching_time': '05:45', 'star_rating': '4', 'seats_available': '16 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Shimla', 'busname': 'HRTC - 591', 'price': '₹660', 'bustype': 'A/C Executive (2+3)', 'departing_time': '21:00', 'duration': '10h 20m', 'reaching_time': '07:20', 'star_rating': '3.5', 'seats_available': '42 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Shimla', 'busname': 'HRTC - 8', 'price': '₹1,051', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '21:02', 'duration': '9h 28m', 'reaching_time': '06:30', 'star_rating': '3.8', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Shimla', 'busname': 'HRTC - 570', 'price': '₹540', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '21:25', 'duration': '9h 20m', 'reaching_time': '06:45', 'star_rating': '4.1', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Shimla', 'busname': 'HRTC - 20', 'price': '₹957', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '21:36', 'duration': '9h 29m', 'reaching_time': '07:05', 'star_rating': '4.3', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Shimla', 'busname': 'HRTC - 518', 'price': '₹540', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '21:58', 'duration': '9h 17m', 'reaching_time': '07:15', 'star_rating': '4', 'seats_available': '27 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Shimla', 'busname': 'HRTC - 9', 'price': '₹982', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '22:00', 'duration': '9h 30m', 'reaching_time': '07:30', 'star_rating': '4', 'seats_available': '24 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Shimla', 'busname': 'HRTC - 579', 'price': '₹540', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '22:30', 'duration': '9h 45m', 'reaching_time': '08:15', 'star_rating': '4.3', 'seats_available': '27 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Shimla', 'busname': 'HRTC - 22', 'price': '₹1,051', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '23:00', 'duration': '8h 50m', 'reaching_time': '07:50', 'star_rating': '4.5', 'seats_available': '15 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Delhi', 'busname': 'zingbus plus', 'price': '₹498', 'bustype': 'Bharat Benz A/C Semi Sleeper (2+2)', 'departing_time': '21:30', 'duration': '8h 45m', 'reaching_time': '06:15', 'star_rating': '4.7', 'seats_available': '31 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Delhi', 'busname': 'Laxmi holidays', 'price': '₹500', 'bustype': 'Scania Multi-Axle AC Semi Sleeper (2+2)', 'departing_time': '20:30', 'duration': '8h 15m', 'reaching_time': '04:45', 'star_rating': '4.5', 'seats_available': '38 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Delhi', 'busname': 'zingbus plus', 'price': '₹529', 'bustype': 'Bharat Benz A/C Semi Sleeper (2+2)', 'departing_time': '20:30', 'duration': '8h 35m', 'reaching_time': '05:05', 'star_rating': '4.3', 'seats_available': '30 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Delhi', 'busname': 'City Land Travels', 'price': '₹360', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '22:00', 'duration': '8h 30m', 'reaching_time': '06:30', 'star_rating': '4.2', 'seats_available': '23 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Delhi', 'busname': 'City Land Travels', 'price': '₹450', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '22:30', 'duration': '7h', 'reaching_time': '05:30', 'star_rating': '4', 'seats_available': '17 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Delhi', 'busname': 'FlixBus', 'price': '₹571', 'bustype': 'A/C Semi Sleeper (2+2)', 'departing_time': '21:00', 'duration': '8h 50m', 'reaching_time': '05:50', 'star_rating': '4.2', 'seats_available': '40 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Delhi', 'busname': 'Deltin Travels Pvt Ltd', 'price': '₹500', 'bustype': 'Volvo 9600 A/C Semi Sleeper (2+2)', 'departing_time': '21:20', 'duration': '7h 30m', 'reaching_time': '04:50', 'star_rating': '4.3', 'seats_available': '34 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Delhi', 'busname': 'Ram Dalal Holidays Pvt Ltd', 'price': '₹538', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '22:00', 'duration': '8h 40m', 'reaching_time': '06:40', 'star_rating': '4.1', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Delhi', 'busname': 'FlixBus', 'price': '₹571', 'bustype': 'Bharat Benz A/C Semi Sleeper (2+2)', 'departing_time': '21:30', 'duration': '8h 50m', 'reaching_time': '06:20', 'star_rating': '4.2', 'seats_available': '47 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Delhi', 'busname': 'Himalayan Nomad Ktc', 'price': '₹1,181', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '23:30', 'duration': '7h', 'reaching_time': '06:30', 'star_rating': '3.1', 'seats_available': '28 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Delhi', 'busname': 'HRTC - 104', 'price': '₹660', 'bustype': 'A/C Executive (2+3)', 'departing_time': '18:20', 'duration': '10h 10m', 'reaching_time': '04:30', 'star_rating': '2.7', 'seats_available': '43 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Delhi', 'busname': 'HRTC - 188', 'price': '₹540', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '19:25', 'duration': '9h 32m', 'reaching_time': '04:57', 'star_rating': '4.1', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Delhi', 'busname': 'HRTC - 2040093', 'price': '₹540', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '19:40', 'duration': '9h 32m', 'reaching_time': '05:12', 'star_rating': '2.8', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Delhi', 'busname': 'HRTC - 74', 'price': '₹982', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '20:00', 'duration': '9h', 'reaching_time': '05:00', 'star_rating': '4.1', 'seats_available': '22 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Delhi', 'busname': 'HRTC - 3', 'price': '₹1,051', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '21:00', 'duration': '9h', 'reaching_time': '06:00', 'star_rating': '4.2', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Delhi', 'busname': 'HRTC - 108', 'price': '₹660', 'bustype': 'A/C Executive (2+3)', 'departing_time': '21:20', 'duration': '9h 32m', 'reaching_time': '06:52', 'star_rating': '3.7', 'seats_available': '40 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Delhi', 'busname': 'HRTC - 19', 'price': '₹982', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '21:30', 'duration': '8h 30m', 'reaching_time': '06:00', 'star_rating': '2.9', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Delhi', 'busname': 'HRTC - 1712', 'price': '₹540', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '21:30', 'duration': '8h 30m', 'reaching_time': '06:00', 'star_rating': '3.9', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Delhi', 'busname': 'HRTC - 130', 'price': '₹540', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '21:45', 'duration': '8h 30m', 'reaching_time': '06:15', 'star_rating': '4', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Delhi', 'busname': 'HRTC - 179', 'price': '₹540', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '22:00', 'duration': '8h 30m', 'reaching_time': '06:30', 'star_rating': '3.2', 'seats_available': '27 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Chamba (Himachal Pradesh) to Chandigarh', 'busname': 'HRTC - 100', 'price': '₹1,328', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '17:40', 'duration': '10h 20m', 'reaching_time': '04:00', 'star_rating': '4.1', 'seats_available': '6 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Chamba (Himachal Pradesh) to Chandigarh', 'busname': 'HRTC - 1007', 'price': '₹691', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '19:00', 'duration': '10h 40m', 'reaching_time': '05:40', 'star_rating': '3.7', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Chamba (Himachal Pradesh) to Chandigarh', 'busname': 'HRTC - 1028', 'price': '₹734', 'bustype': 'A/C Executive (2+3)', 'departing_time': '20:15', 'duration': '11h', 'reaching_time': '07:15', 'star_rating': '3.3', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Chandigarh', 'busname': 'FlixBus', 'price': '₹346', 'bustype': 'A/C Semi Sleeper (2+2)', 'departing_time': '16:30', 'duration': '4h 55m', 'reaching_time': '21:25', 'star_rating': '4.2', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Chandigarh', 'busname': 'NueGo', 'price': '₹447', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '16:30', 'duration': '5h 25m', 'reaching_time': '21:55', 'star_rating': '3.9', 'seats_available': '17 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Chandigarh', 'busname': 'Laxmi holidays', 'price': '₹499', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '17:00', 'duration': '5h 40m', 'reaching_time': '22:40', 'star_rating': '4', 'seats_available': '33 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Chandigarh', 'busname': 'GULSHAN BUS', 'price': '₹699', 'bustype': 'A/C Seater / Sleeper (2+2)', 'departing_time': '17:00', 'duration': '7h', 'reaching_time': '00:00', 'star_rating': '3.9', 'seats_available': '24 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Chandigarh', 'busname': 'RK Holiday', 'price': '₹499', 'bustype': 'A/C Seater / Sleeper (2+2)', 'departing_time': '17:00', 'duration': '5h 15m', 'reaching_time': '22:15', 'star_rating': '3', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Chandigarh', 'busname': 'FlixBus', 'price': '₹267', 'bustype': 'Bharat Benz A/C Semi Sleeper (2+2)', 'departing_time': '17:30', 'duration': '6h 35m', 'reaching_time': '00:05', 'star_rating': '3.6', 'seats_available': '34 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Chandigarh', 'busname': 'NueGo', 'price': '₹447', 'bustype': 'Electric A/C Seater (2+2)', 'departing_time': '17:30', 'duration': '5h 5m', 'reaching_time': '22:35', 'star_rating': '3.5', 'seats_available': '37 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Chandigarh', 'busname': 'FlixBus', 'price': '₹620', 'bustype': 'Volvo 9600 A/C Semi Sleeper (2+2)', 'departing_time': '18:00', 'duration': '6h 40m', 'reaching_time': '00:40', 'star_rating': '4.2', 'seats_available': '24 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Chandigarh', 'busname': 'Laxmi holidays', 'price': '₹699', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '18:00', 'duration': '5h 30m', 'reaching_time': '23:30', 'star_rating': '4.2', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Chandigarh', 'busname': 'Big Bus', 'price': '₹499', 'bustype': 'A/C Seater / Sleeper (2+2)', 'departing_time': '18:15', 'duration': '5h 40m', 'reaching_time': '23:55', 'star_rating': '3.3', 'seats_available': '40 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Chandigarh', 'busname': 'HRTC - 549', 'price': '₹390', 'bustype': 'A/C Executive (2+3)', 'departing_time': '19:25', 'duration': '5h 15m', 'reaching_time': '00:40', 'star_rating': '4.3', 'seats_available': '28 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Chandigarh', 'busname': 'HRTC - 548', 'price': '₹290', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '19:31', 'duration': '5h 14m', 'reaching_time': '00:45', 'star_rating': '3.8', 'seats_available': '23 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Chandigarh', 'busname': 'HRTC - 545', 'price': '₹285', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '19:32', 'duration': '4h 43m', 'reaching_time': '00:15', 'star_rating': '4.2', 'seats_available': '33 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Chandigarh', 'busname': 'HRTC - 93', 'price': '₹543', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '19:35', 'duration': '5h 15m', 'reaching_time': '00:50', 'star_rating': '3.8', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Chandigarh', 'busname': 'HRTC - 301', 'price': '₹290', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '19:43', 'duration': '4h 37m', 'reaching_time': '00:20', 'star_rating': '3.8', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Chandigarh', 'busname': 'HRTC - 21', 'price': '₹543', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '20:00', 'duration': '5h 30m', 'reaching_time': '01:30', 'star_rating': None, 'seats_available': '27 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Chandigarh', 'busname': 'HRTC - 556', 'price': '₹290', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '20:10', 'duration': '5h 20m', 'reaching_time': '01:30', 'star_rating': '3.8', 'seats_available': '16 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Chandigarh', 'busname': 'HRTC - 557', 'price': '₹290', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '20:15', 'duration': '5h 15m', 'reaching_time': '01:30', 'star_rating': '3.8', 'seats_available': '23 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Chandigarh', 'busname': 'HRTC - 591', 'price': '₹400', 'bustype': 'A/C Executive (2+3)', 'departing_time': '21:00', 'duration': '4h 55m', 'reaching_time': '01:55', 'star_rating': '3.8', 'seats_available': '33 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Chandigarh', 'busname': 'HRTC - 8', 'price': '₹543', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '21:02', 'duration': '5h 3m', 'reaching_time': '02:05', 'star_rating': '5', 'seats_available': '21 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Ghumarwin to Chandigarh', 'busname': 'Laxmi holidays', 'price': '₹599', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '21:30', 'duration': '3h', 'reaching_time': '00:30', 'star_rating': '4.4', 'seats_available': '4 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Ghumarwin to Chandigarh', 'busname': 'Northern Travels', 'price': '₹500', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '22:50', 'duration': '2h 40m', 'reaching_time': '01:30', 'star_rating': '3.2', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Ghumarwin to Chandigarh', 'busname': 'Himalayan Nomad Ktc', 'price': '₹567', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '23:00', 'duration': '2h 30m', 'reaching_time': '01:30', 'star_rating': '4.2', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Ghumarwin to Chandigarh', 'busname': 'Bedi Travels', 'price': '₹563', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '23:15', 'duration': '2h 40m', 'reaching_time': '01:55', 'star_rating': '4.8', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Ghumarwin to Chandigarh', 'busname': 'JSR HOLIDAYS', 'price': '₹555', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '22:00', 'duration': '3h 45m', 'reaching_time': '01:45', 'star_rating': '2', 'seats_available': '32 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Ghumarwin to Chandigarh', 'busname': 'JSR HOLIDAYS', 'price': '₹463', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '23:30', 'duration': '2h 15m', 'reaching_time': '01:45', 'star_rating': '2', 'seats_available': '21 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Ghumarwin to Chandigarh', 'busname': 'Ram Dalal Holidays Pvt Ltd', 'price': '₹649', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '23:00', 'duration': '3h 20m', 'reaching_time': '02:20', 'star_rating': '4.4', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Ghumarwin to Chandigarh', 'busname': 'Himalayan Nomad Ktc', 'price': '₹675', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '23:20', 'duration': '2h 25m', 'reaching_time': '01:45', 'star_rating': '4.2', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Ghumarwin to Chandigarh', 'busname': 'JSR HOLIDAYS', 'price': '₹647', 'bustype': 'Bharat Benz A/C Semi Sleeper (2+2)', 'departing_time': '23:40', 'duration': '2h 20m', 'reaching_time': '02:00', 'star_rating': '2', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Ghumarwin to Chandigarh', 'busname': 'HRTC - 401270', 'price': '₹230', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '18:10', 'duration': '2h 55m', 'reaching_time': '21:05', 'star_rating': '3.5', 'seats_available': '27 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Ghumarwin to Chandigarh', 'busname': 'HRTC - 685', 'price': '₹255', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '19:10', 'duration': '4h 30m', 'reaching_time': '23:40', 'star_rating': '3.8', 'seats_available': '36 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Ghumarwin to Chandigarh', 'busname': 'HRTC - 763', 'price': '₹230', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '19:40', 'duration': '3h 10m', 'reaching_time': '22:50', 'star_rating': '3.8', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Ghumarwin to Chandigarh', 'busname': 'HRTC - 700', 'price': '₹535', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '20:00', 'duration': '3h 30m', 'reaching_time': '23:30', 'star_rating': '3.8', 'seats_available': '19 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Ghumarwin to Chandigarh', 'busname': 'HRTC - 750', 'price': '₹285', 'bustype': 'A/C Executive (2+3)', 'departing_time': '20:30', 'duration': '3h 55m', 'reaching_time': '00:25', 'star_rating': '3.8', 'seats_available': '33 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Ghumarwin to Chandigarh', 'busname': 'HRTC - 78', 'price': '₹502', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '21:15', 'duration': '3h 50m', 'reaching_time': '01:05', 'star_rating': '3.8', 'seats_available': '15 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Ghumarwin to Chandigarh', 'busname': 'HRTC - 15', 'price': '₹423', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '21:25', 'duration': '2h 45m', 'reaching_time': '00:10', 'star_rating': '3.8', 'seats_available': '2 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Chamba (Himachal Pradesh)', 'busname': 'HRTC - 531', 'price': '₹900', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '17:50', 'duration': '16h 10m', 'reaching_time': '10:00', 'star_rating': '3.6', 'seats_available': '23 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Chamba (Himachal Pradesh)', 'busname': 'HRTC - 590', 'price': '₹1,116', 'bustype': 'A/C Executive (2+3)', 'departing_time': '19:00', 'duration': '15h 15m', 'reaching_time': '10:15', 'star_rating': '3.3', 'seats_available': '23 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Chamba (Himachal Pradesh)', 'busname': 'HRTC - 84', 'price': '₹1,851', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '20:15', 'duration': '15h 15m', 'reaching_time': '11:30', 'star_rating': '4.4', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Chamba (Himachal Pradesh)', 'busname': 'HRTC - 578', 'price': '₹961', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '22:10', 'duration': '20h 25m', 'reaching_time': '18:35', 'star_rating': '3.8', 'seats_available': '37 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Chandigarh', 'busname': 'zingbus plus', 'price': '₹128', 'bustype': 'Bharat Benz A/C Semi Sleeper (2+2)', 'departing_time': '20:30', 'duration': '3h 15m', 'reaching_time': '23:45', 'star_rating': '4.4', 'seats_available': '33 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Chandigarh', 'busname': 'Laxmi holidays', 'price': '₹500', 'bustype': 'Scania Multi-Axle AC Semi Sleeper (2+2)', 'departing_time': '20:30', 'duration': '3h 5m', 'reaching_time': '23:35', 'star_rating': '4.6', 'seats_available': '40 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Chandigarh', 'busname': 'FlixBus', 'price': '₹454', 'bustype': 'A/C Semi Sleeper (2+2)', 'departing_time': '21:00', 'duration': '4h 25m', 'reaching_time': '01:25', 'star_rating': '4.3', 'seats_available': '40 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Chandigarh', 'busname': 'Deltin Travels Pvt Ltd', 'price': '₹500', 'bustype': 'Volvo 9600 A/C Semi Sleeper (2+2)', 'departing_time': '21:20', 'duration': '2h 39m', 'reaching_time': '23:59', 'star_rating': '4.2', 'seats_available': '24 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Chandigarh', 'busname': 'zingbus plus', 'price': '₹305', 'bustype': 'Bharat Benz A/C Semi Sleeper (2+2)', 'departing_time': '21:30', 'duration': '3h 15m', 'reaching_time': '00:45', 'star_rating': '4.8', 'seats_available': '32 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Chandigarh', 'busname': 'FlixBus', 'price': '₹454', 'bustype': 'Bharat Benz A/C Semi Sleeper (2+2)', 'departing_time': '21:30', 'duration': '4h 25m', 'reaching_time': '01:55', 'star_rating': '4.3', 'seats_available': '47 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Chandigarh', 'busname': 'City Land Travels', 'price': '₹360', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '22:00', 'duration': '3h 1m', 'reaching_time': '01:01', 'star_rating': '4.8', 'seats_available': '23 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Chandigarh', 'busname': 'Ram Dalal Holidays Pvt Ltd', 'price': '₹538', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '22:00', 'duration': '3h 45m', 'reaching_time': '01:45', 'star_rating': '4.5', 'seats_available': '22 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Chandigarh', 'busname': 'City Land Travels', 'price': '₹450', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '22:30', 'duration': '2h 50m', 'reaching_time': '01:20', 'star_rating': '3.9', 'seats_available': '14 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Chandigarh', 'busname': 'Vacation Express', 'price': '₹1,299', 'bustype': 'Volvo 9600 Multi Axle Semi-Sleeper (2+2)', 'departing_time': '21:20', 'duration': '3h', 'reaching_time': '00:20', 'star_rating': '3.9', 'seats_available': '45 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Chandigarh', 'busname': 'HRTC - 215', 'price': '₹270', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '18:00', 'duration': '4h 30m', 'reaching_time': '22:30', 'star_rating': '3.8', 'seats_available': '27 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Chandigarh', 'busname': 'HRTC - 104', 'price': '₹285', 'bustype': 'A/C Executive (2+3)', 'departing_time': '18:20', 'duration': '3h 1m', 'reaching_time': '21:21', 'star_rating': '2.6', 'seats_available': '43 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Chandigarh', 'busname': 'HRTC - 188', 'price': '₹270', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '19:25', 'duration': '3h 1m', 'reaching_time': '22:26', 'star_rating': '3.8', 'seats_available': '36 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Chandigarh', 'busname': 'HRTC - 2040093', 'price': '₹270', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '19:40', 'duration': '4h 35m', 'reaching_time': '00:15', 'star_rating': '3.8', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Chandigarh', 'busname': 'HRTC - 1711', 'price': '₹270', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '20:00', 'duration': '3h', 'reaching_time': '23:00', 'star_rating': '3.8', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Chandigarh', 'busname': 'HRTC - 74', 'price': '₹459', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '20:00', 'duration': '2h 45m', 'reaching_time': '22:45', 'star_rating': '3.3', 'seats_available': '22 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Chandigarh', 'busname': 'HRTC - 139', 'price': '₹270', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '20:05', 'duration': '4h 10m', 'reaching_time': '00:15', 'star_rating': '3.8', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Chandigarh', 'busname': 'HRTC - 230', 'price': '₹275', 'bustype': 'A/C Executive (2+3)', 'departing_time': '20:20', 'duration': '4h 40m', 'reaching_time': '01:00', 'star_rating': '3.8', 'seats_available': '44 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Chandigarh', 'busname': 'HRTC - 229', 'price': '₹270', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '20:40', 'duration': '3h 1m', 'reaching_time': '23:41', 'star_rating': '3.8', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Shimla to Chandigarh', 'busname': 'HRTC - 3', 'price': '₹528', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '21:00', 'duration': '3h 36m', 'reaching_time': '00:36', 'star_rating': '3.8', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Hamirpur (Himachal Pradesh) to Delhi', 'busname': 'Bedi Travels', 'price': '₹563', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '22:05', 'duration': '8h 25m', 'reaching_time': '06:30', 'star_rating': '4.8', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Hamirpur (Himachal Pradesh) to Delhi', 'busname': 'Ram Dalal Holidays Pvt Ltd', 'price': '₹649', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '20:15', 'duration': '9h 15m', 'reaching_time': '05:30', 'star_rating': '4.7', 'seats_available': '6 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Hamirpur (Himachal Pradesh) to Delhi', 'busname': 'Ram Dalal Holidays Pvt Ltd', 'price': '₹649', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '22:00', 'duration': '8h 40m', 'reaching_time': '06:40', 'star_rating': '4.7', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Hamirpur (Himachal Pradesh) to Delhi', 'busname': 'Laxmi holidays', 'price': '₹499', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '22:05', 'duration': '8h 55m', 'reaching_time': '07:00', 'star_rating': '4.5', 'seats_available': '5 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Hamirpur (Himachal Pradesh) to Delhi', 'busname': 'Himalayan Nomad Ktc', 'price': '₹675', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '22:25', 'duration': '7h 35m', 'reaching_time': '06:00', 'star_rating': '4.4', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Hamirpur (Himachal Pradesh) to Delhi', 'busname': 'Laxmi holidays', 'price': '₹599', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '20:20', 'duration': '8h 10m', 'reaching_time': '04:30', 'star_rating': '4.2', 'seats_available': '4 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Hamirpur (Himachal Pradesh) to Delhi', 'busname': 'Northern Travels', 'price': '₹500', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '21:45', 'duration': '7h 45m', 'reaching_time': '05:30', 'star_rating': '3.8', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Hamirpur (Himachal Pradesh) to Delhi', 'busname': 'Himalayan Nomad Ktc', 'price': '₹567', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '22:05', 'duration': '7h 25m', 'reaching_time': '05:30', 'star_rating': '4.1', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Hamirpur (Himachal Pradesh) to Delhi', 'busname': 'HRTC - 601', 'price': '₹605', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '18:30', 'duration': '10h', 'reaching_time': '04:30', 'star_rating': '4.3', 'seats_available': '32 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Hamirpur (Himachal Pradesh) to Delhi', 'busname': 'HRTC - 651', 'price': '₹605', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '19:40', 'duration': '9h 20m', 'reaching_time': '05:00', 'star_rating': '4', 'seats_available': '33 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Hamirpur (Himachal Pradesh) to Delhi', 'busname': 'HRTC - 15', 'price': '₹1,132', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '20:00', 'duration': '8h 30m', 'reaching_time': '04:30', 'star_rating': '4.4', 'seats_available': '4 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Hamirpur (Himachal Pradesh) to Delhi', 'busname': 'HRTC - 1106', 'price': '₹605', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '21:40', 'duration': '8h 50m', 'reaching_time': '06:30', 'star_rating': '4.7', 'seats_available': '26 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Chamba (Himachal Pradesh) to Delhi', 'busname': 'HRTC - 100', 'price': '₹1,851', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '17:40', 'duration': '15h 20m', 'reaching_time': '09:00', 'star_rating': '4', 'seats_available': '6 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Hamirpur (Himachal Pradesh)', 'busname': 'Bedi Travels', 'price': '₹509', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '21:30', 'duration': '8h 45m', 'reaching_time': '06:15', 'star_rating': '4.9', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Hamirpur (Himachal Pradesh)', 'busname': 'Ram Dalal Holidays Pvt Ltd', 'price': '₹600', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '22:00', 'duration': '7h 55m', 'reaching_time': '05:55', 'star_rating': '4.7', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Hamirpur (Himachal Pradesh)', 'busname': 'Ram Dalal Holidays Pvt Ltd', 'price': '₹600', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '21:20', 'duration': '8h 35m', 'reaching_time': '05:55', 'star_rating': '4.7', 'seats_available': '17 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Hamirpur (Himachal Pradesh)', 'busname': 'Laxmi holidays', 'price': '₹471', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '20:00', 'duration': '8h 30m', 'reaching_time': '04:30', 'star_rating': '4.4', 'seats_available': '9 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Hamirpur (Himachal Pradesh)', 'busname': 'Himalayan Nomad Ktc', 'price': '₹675', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '21:00', 'duration': '8h 20m', 'reaching_time': '05:20', 'star_rating': '4.5', 'seats_available': '19 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Hamirpur (Himachal Pradesh)', 'busname': 'Laxmi holidays', 'price': '₹629', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '22:00', 'duration': '8h 30m', 'reaching_time': '06:30', 'star_rating': '4', 'seats_available': '29 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Hamirpur (Himachal Pradesh)', 'busname': 'Himalayan Nomad Ktc', 'price': '₹675', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '20:45', 'duration': '8h 20m', 'reaching_time': '05:05', 'star_rating': '4.4', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Hamirpur (Himachal Pradesh)', 'busname': 'Northern Travels', 'price': '₹500', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '21:30', 'duration': '8h 10m', 'reaching_time': '05:40', 'star_rating': '4', 'seats_available': '17 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Hamirpur (Himachal Pradesh)', 'busname': 'Northern Travels', 'price': '₹500', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '21:00', 'duration': '8h 30m', 'reaching_time': '05:30', 'star_rating': '3.5', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Hamirpur (Himachal Pradesh)', 'busname': 'HRTC - 587', 'price': '₹605', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '18:25', 'duration': '9h 5m', 'reaching_time': '03:30', 'star_rating': '3.7', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Hamirpur (Himachal Pradesh)', 'busname': 'HRTC - 540', 'price': '₹772', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '19:00', 'duration': '13h', 'reaching_time': '08:00', 'star_rating': '4.8', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Hamirpur (Himachal Pradesh)', 'busname': 'HRTC - 548', 'price': '₹605', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '19:31', 'duration': '9h 14m', 'reaching_time': '04:45', 'star_rating': '3.9', 'seats_available': '27 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Hamirpur (Himachal Pradesh)', 'busname': 'HRTC - 552', 'price': '₹605', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '19:45', 'duration': '9h 15m', 'reaching_time': '05:00', 'star_rating': '4.2', 'seats_available': '36 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Hamirpur (Himachal Pradesh)', 'busname': 'HRTC - 559', 'price': '₹672', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '20:29', 'duration': '10h 36m', 'reaching_time': '07:05', 'star_rating': '2.9', 'seats_available': '33 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Hamirpur (Himachal Pradesh)', 'busname': 'HRTC - 566', 'price': '₹627', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '21:10', 'duration': '10h 20m', 'reaching_time': '07:30', 'star_rating': '4.4', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Hamirpur (Himachal Pradesh)', 'busname': 'HRTC - 16', 'price': '₹1,132', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '21:20', 'duration': '8h 30m', 'reaching_time': '05:50', 'star_rating': '4.2', 'seats_available': '12 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Delhi to Hamirpur (Himachal Pradesh)', 'busname': 'HRTC - 571', 'price': '₹662', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '21:40', 'duration': '10h 10m', 'reaching_time': '07:50', 'star_rating': '3.6', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Sunder Nagar (Himachal Pradesh) to Chandigarh', 'busname': 'Deltin Travels Pvt Ltd', 'price': '₹440', 'bustype': 'Volvo 9600 Multi Axle Semi-Sleeper (2+2)', 'departing_time': '21:00', 'duration': '3h 10m', 'reaching_time': '00:10', 'star_rating': '4.1', 'seats_available': '33 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Sunder Nagar (Himachal Pradesh) to Chandigarh', 'busname': 'City Land Travels', 'price': '₹709', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '21:00', 'duration': '3h 20m', 'reaching_time': '00:20', 'star_rating': '5', 'seats_available': '23 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Sunder Nagar (Himachal Pradesh) to Chandigarh', 'busname': 'Kashi Bus', 'price': '₹599', 'bustype': 'Volvo A/C B11R Multi Axle Semi Sleeper (2+2)', 'departing_time': '21:00', 'duration': '3h 5m', 'reaching_time': '00:05', 'star_rating': '4', 'seats_available': '43 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Sunder Nagar (Himachal Pradesh) to Chandigarh', 'busname': 'Subh Yatri Holidays', 'price': '₹630', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '21:40', 'duration': '4h 5m', 'reaching_time': '01:45', 'star_rating': '3.9', 'seats_available': '29 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Sunder Nagar (Himachal Pradesh) to Chandigarh', 'busname': 'DKN HOLIDAYS', 'price': '₹700', 'bustype': 'Scania Multi-Axle AC Semi Sleeper (2+2)', 'departing_time': '21:45', 'duration': '3h 45m', 'reaching_time': '01:30', 'star_rating': '3.3', 'seats_available': '33 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Sunder Nagar (Himachal Pradesh) to Chandigarh', 'busname': 'VT Bus', 'price': '₹540', 'bustype': 'Scania Multi-Axle AC Semi Sleeper (2+2)', 'departing_time': '21:45', 'duration': '2h 30m', 'reaching_time': '00:15', 'star_rating': '3.9', 'seats_available': '43 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Sunder Nagar (Himachal Pradesh) to Chandigarh', 'busname': 'City Land Travels', 'price': '₹583', 'bustype': 'Volvo A/C B11R Multi Axle Semi Sleeper (2+2)', 'departing_time': '21:50', 'duration': '3h 30m', 'reaching_time': '01:20', 'star_rating': '3.7', 'seats_available': '38 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Sunder Nagar (Himachal Pradesh) to Chandigarh', 'busname': 'Cab India Travels', 'price': '₹999', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '22:00', 'duration': '3h 15m', 'reaching_time': '01:15', 'star_rating': '3.8', 'seats_available': '41 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Sunder Nagar (Himachal Pradesh) to Chandigarh', 'busname': 'Ram Dalal Holidays Pvt Ltd', 'price': '₹688', 'bustype': 'Volvo 9600 Multi Axle Semi-Sleeper (2+2)', 'departing_time': '22:15', 'duration': '1h 40m', 'reaching_time': '23:55', 'star_rating': '4.4', 'seats_available': '22 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Sunder Nagar (Himachal Pradesh) to Chandigarh', 'busname': 'PAL Travel Lines', 'price': '₹679', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '22:15', 'duration': '3h', 'reaching_time': '01:15', 'star_rating': '3.2', 'seats_available': '33 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Sunder Nagar (Himachal Pradesh) to Chandigarh', 'busname': 'HRTC - 766', 'price': '₹315', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '18:00', 'duration': '5h', 'reaching_time': '23:00', 'star_rating': '3.8', 'seats_available': '27 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Sunder Nagar (Himachal Pradesh) to Chandigarh', 'busname': 'HRTC - 725', 'price': '₹305', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '19:00', 'duration': '3h 30m', 'reaching_time': '22:30', 'star_rating': '1.8', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Sunder Nagar (Himachal Pradesh) to Chandigarh', 'busname': 'HRTC - 776', 'price': '₹305', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '20:00', 'duration': '4h 10m', 'reaching_time': '00:10', 'star_rating': '3.8', 'seats_available': '29 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Sunder Nagar (Himachal Pradesh) to Chandigarh', 'busname': 'HRTC - 49', 'price': '₹609', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '20:30', 'duration': '5h 30m', 'reaching_time': '02:00', 'star_rating': '3.8', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Sunder Nagar (Himachal Pradesh) to Chandigarh', 'busname': 'HRTC - 736', 'price': '₹577', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '21:10', 'duration': '3h 50m', 'reaching_time': '01:00', 'star_rating': '3.8', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Sunder Nagar (Himachal Pradesh) to Chandigarh', 'busname': 'HRTC - 773', 'price': '₹290', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '21:40', 'duration': '4h 50m', 'reaching_time': '02:30', 'star_rating': '3.8', 'seats_available': '23 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Sunder Nagar (Himachal Pradesh) to Chandigarh', 'busname': 'HRTC - 705', 'price': '₹577', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '22:10', 'duration': '3h 50m', 'reaching_time': '02:00', 'star_rating': '3.8', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Sunder Nagar (Himachal Pradesh) to Chandigarh', 'busname': 'HRTC - 712', 'price': '₹318', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '23:00', 'duration': '6h 5m', 'reaching_time': '05:05', 'star_rating': '3.8', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Solan to Delhi', 'busname': 'RSRTC - 192348', 'price': '₹1,007', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '17:47', 'duration': '8h 43m', 'reaching_time': '02:30', 'star_rating': '3.1', 'seats_available': '34 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Solan to Delhi', 'busname': 'Laxmi holidays', 'price': '₹500', 'bustype': 'Scania Multi-Axle AC Semi Sleeper (2+2)', 'departing_time': '21:40', 'duration': '7h 5m', 'reaching_time': '04:45', 'star_rating': '4.6', 'seats_available': '38 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Solan to Delhi', 'busname': 'zingbus plus', 'price': '₹529', 'bustype': 'Bharat Benz A/C Semi Sleeper (2+2)', 'departing_time': '21:50', 'duration': '7h 15m', 'reaching_time': '05:05', 'star_rating': '4.3', 'seats_available': '30 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Solan to Delhi', 'busname': 'City Land Travels', 'price': '₹360', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '23:20', 'duration': '7h 10m', 'reaching_time': '06:30', 'star_rating': '4.7', 'seats_available': '22 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Solan to Delhi', 'busname': 'Ram Dalal Holidays Pvt Ltd', 'price': '₹538', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '23:20', 'duration': '7h 20m', 'reaching_time': '06:40', 'star_rating': '4.4', 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Solan to Delhi', 'busname': 'City Land Travels', 'price': '₹450', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '23:45', 'duration': '5h 45m', 'reaching_time': '05:30', 'star_rating': '4', 'seats_available': '17 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Solan to Delhi', 'busname': 'zingbus plus', 'price': '₹498', 'bustype': 'Bharat Benz A/C Semi Sleeper (2+2)', 'departing_time': '22:50', 'duration': '7h 25m', 'reaching_time': '06:15', 'star_rating': '4.6', 'seats_available': '31 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Solan to Delhi', 'busname': 'FlixBus', 'price': '₹428', 'bustype': 'Bharat Benz A/C Semi Sleeper (2+2)', 'departing_time': '22:35', 'duration': '7h 45m', 'reaching_time': '06:20', 'star_rating': '3.9', 'seats_available': '47 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Solan to Delhi', 'busname': 'FlixBus', 'price': '₹428', 'bustype': 'A/C Semi Sleeper (2+2)', 'departing_time': '22:05', 'duration': '7h 45m', 'reaching_time': '05:50', 'star_rating': '2.6', 'seats_available': '40 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Solan to Delhi', 'busname': 'Ram Dalal Holidays Pvt Ltd', 'price': '₹1,599', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '22:45', 'duration': '6h 45m', 'reaching_time': '05:30', 'star_rating': '3.9', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Solan to Delhi', 'busname': 'HRTC - 104', 'price': '₹543', 'bustype': 'A/C Executive (2+3)', 'departing_time': '19:31', 'duration': '8h 59m', 'reaching_time': '04:30', 'star_rating': '3.1', 'seats_available': '44 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Solan to Delhi', 'busname': 'HRTC - 188', 'price': '₹423', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '20:36', 'duration': '8h 21m', 'reaching_time': '04:57', 'star_rating': '4', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Solan to Delhi', 'busname': 'HRTC - 74', 'price': '₹806', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '21:10', 'duration': '7h 50m', 'reaching_time': '05:00', 'star_rating': '4.2', 'seats_available': '33 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Solan to Delhi', 'busname': 'HRTC - 3', 'price': '₹774', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '22:30', 'duration': '7h 30m', 'reaching_time': '06:00', 'star_rating': '4.5', 'seats_available': '22 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Solan to Delhi', 'busname': 'HRTC - 108', 'price': '₹543', 'bustype': 'A/C Executive (2+3)', 'departing_time': '22:31', 'duration': '8h 21m', 'reaching_time': '06:52', 'star_rating': '3.2', 'seats_available': '43 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Solan to Delhi', 'busname': 'HRTC - 130', 'price': '₹423', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '22:45', 'duration': '7h 30m', 'reaching_time': '06:15', 'star_rating': '3.4', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Solan to Delhi', 'busname': 'HRTC - 35', 'price': '₹774', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '23:00', 'duration': '7h 15m', 'reaching_time': '06:15', 'star_rating': '3.4', 'seats_available': '33 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Solan to Delhi', 'busname': 'HRTC - 19', 'price': '₹774', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '23:00', 'duration': '7h', 'reaching_time': '06:00', 'star_rating': '3.9', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Solan to Delhi', 'busname': 'HRTC - 92', 'price': '₹774', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '23:40', 'duration': '7h 40m', 'reaching_time': '07:20', 'star_rating': '4.7', 'seats_available': '28 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Dharamshala (Himachal Pradesh) to Chandigarh', 'busname': 'Laxmi holidays', 'price': '₹599', 'bustype': 'Volvo 9600 A/C Semi Sleeper (2+2)', 'departing_time': '18:00', 'duration': '6h 20m', 'reaching_time': '00:20', 'star_rating': '4.8', 'seats_available': '9 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Dharamshala (Himachal Pradesh) to Chandigarh', 'busname': 'Laxmi holidays', 'price': '₹616', 'bustype': 'Bharat Benz A/C Semi Sleeper (2+2)', 'departing_time': '18:30', 'duration': '5h 25m', 'reaching_time': '23:55', 'star_rating': '4.7', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Dharamshala (Himachal Pradesh) to Chandigarh', 'busname': 'zingbus plus', 'price': '₹500', 'bustype': 'Volvo 9600 A/C Semi Sleeper (2+2)', 'departing_time': '18:30', 'duration': '5h 55m', 'reaching_time': '00:25', 'star_rating': '4.8', 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Dharamshala (Himachal Pradesh) to Chandigarh', 'busname': 'zingbus Maxx', 'price': '₹1,027', 'bustype': 'Volvo 9600 Multi Axle Semi-Sleeper (2+2)', 'departing_time': '18:40', 'duration': '5h 45m', 'reaching_time': '00:25', 'star_rating': '4.4', 'seats_available': '32 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Dharamshala (Himachal Pradesh) to Chandigarh', 'busname': 'Ram Dalal Holidays Pvt Ltd', 'price': '₹600', 'bustype': 'Volvo 9600 Multi Axle Semi-Sleeper (2+2)', 'departing_time': '19:00', 'duration': '6h 10m', 'reaching_time': '01:10', 'star_rating': '4.8', 'seats_available': '34 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Dharamshala (Himachal Pradesh) to Chandigarh', 'busname': 'Bedi Travels', 'price': '₹444', 'bustype': 'Volvo 9600 Multi Axle Semi-Sleeper (2+2)', 'departing_time': '19:00', 'duration': '6h', 'reaching_time': '01:00', 'star_rating': '5', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Dharamshala (Himachal Pradesh) to Chandigarh', 'busname': 'zingbus plus', 'price': '₹279', 'bustype': 'Volvo Multi-Axle A/C Semisleeper (2+2)', 'departing_time': '21:40', 'duration': '4h 55m', 'reaching_time': '02:35', 'star_rating': '4.8', 'seats_available': '8 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Dharamshala (Himachal Pradesh) to Chandigarh', 'busname': 'FlixBus', 'price': '₹597', 'bustype': 'Volvo 9600 A/C Semi Sleeper (2+2)', 'departing_time': '19:15', 'duration': '6h 5m', 'reaching_time': '01:20', 'star_rating': '4.5', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Dharamshala (Himachal Pradesh) to Chandigarh', 'busname': 'zingbus Maxx', 'price': '₹872', 'bustype': 'Volvo 9600 Multi Axle Semi-Sleeper (2+2)', 'departing_time': '19:30', 'duration': '4h 55m', 'reaching_time': '00:25', 'star_rating': '4.8', 'seats_available': '32 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Dharamshala (Himachal Pradesh) to Chandigarh', 'busname': 'New Himalaya Travels', 'price': '₹550', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '19:34', 'duration': '5h 41m', 'reaching_time': '01:15', 'star_rating': '4.4', 'seats_available': '14 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Dharamshala (Himachal Pradesh) to Chandigarh', 'busname': 'HRTC - 904', 'price': '₹469', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '19:00', 'duration': '6h 30m', 'reaching_time': '01:30', 'star_rating': '3.3', 'seats_available': '41 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Dharamshala (Himachal Pradesh) to Chandigarh', 'busname': 'HRTC - 43', 'price': '₹923', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '19:35', 'duration': '5h 40m', 'reaching_time': '01:15', 'star_rating': '3.7', 'seats_available': '29 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Dharamshala (Himachal Pradesh) to Chandigarh', 'busname': 'HRTC - 13', 'price': '₹1,022', 'bustype': 'Himsuta AC Seater Volvo/Scania 2+2', 'departing_time': '20:05', 'duration': '6h 10m', 'reaching_time': '02:15', 'star_rating': '3.8', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Dharamshala (Himachal Pradesh) to Chandigarh', 'busname': 'HRTC - 905', 'price': '₹492', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '20:15', 'duration': '6h 15m', 'reaching_time': '02:30', 'star_rating': '3.8', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'HRTC', 'route': 'Dharamshala (Himachal Pradesh) to Chandigarh', 'busname': 'HRTC - 927', 'price': '₹512', 'bustype': 'Ordinary 3+2 Non AC Seater', 'departing_time': '21:00', 'duration': '7h 10m', 'reaching_time': '04:10', 'star_rating': '4.3', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Assam State Transport Corporation (ASTC)', 'route': 'Guwahati to Tezpur', 'busname': 'Kalpana Travels', 'price': '₹351', 'bustype': 'AC Seater (2+2)', 'departing_time': '16:50', 'duration': '3h 25m', 'reaching_time': '20:15', 'star_rating': '4.5', 'seats_available': '28 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Assam State Transport Corporation (ASTC)', 'route': 'Guwahati to Tezpur', 'busname': 'Shruti(North East Travels)', 'price': '₹340', 'bustype': 'Tata A/C Seater (2+2)', 'departing_time': '17:30', 'duration': '4h 30m', 'reaching_time': '22:00', 'star_rating': '4.4', 'seats_available': '27 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Assam State Transport Corporation (ASTC)', 'route': 'Guwahati to Tezpur', 'busname': 'Yatra Travels', 'price': '₹425', 'bustype': 'NON AC Seater/ Sleeper (2+1)', 'departing_time': '19:00', 'duration': '4h 45m', 'reaching_time': '23:45', 'star_rating': '3.8', 'seats_available': '28 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Assam State Transport Corporation (ASTC)', 'route': 'Guwahati to Tezpur', 'busname': 'Yatra Travels', 'price': '₹425', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '20:00', 'duration': '5h 30m', 'reaching_time': '01:30', 'star_rating': '4.3', 'seats_available': '27 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Assam State Transport Corporation (ASTC)', 'route': 'Guwahati to Tezpur', 'busname': 'Laxmi Travels', 'price': '₹450', 'bustype': 'A/C Seater (2+1)', 'departing_time': '20:15', 'duration': '4h 55m', 'reaching_time': '01:10', 'star_rating': '3.8', 'seats_available': '31 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Assam State Transport Corporation (ASTC)', 'route': 'Guwahati to Tezpur', 'busname': 'NORTH EAST TRAVELS(UNDER ASTC)', 'price': '₹500', 'bustype': 'NON A/C Seater (2+1)', 'departing_time': '20:15', 'duration': '4h', 'reaching_time': '00:15', 'star_rating': '3.7', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Assam State Transport Corporation (ASTC)', 'route': 'Guwahati to Tezpur', 'busname': 'Yatra Travels', 'price': '₹425', 'bustype': 'NON AC Seater/ Sleeper (2+1)', 'departing_time': '20:30', 'duration': '4h 30m', 'reaching_time': '01:00', 'star_rating': '4.9', 'seats_available': '30 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Assam State Transport Corporation (ASTC)', 'route': 'Guwahati to Tezpur', 'busname': 'Pranjit Travels (Under ASTC)', 'price': '₹500', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '20:45', 'duration': '5h 45m', 'reaching_time': '02:30', 'star_rating': '3.8', 'seats_available': '21 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Assam State Transport Corporation (ASTC)', 'route': 'Guwahati to Tezpur', 'busname': 'Yatra Travels', 'price': '₹425', 'bustype': 'NON A/C Seater/ Sleeper (2+1)', 'departing_time': '21:00', 'duration': '4h 30m', 'reaching_time': '01:30', 'star_rating': '3.6', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Assam State Transport Corporation (ASTC)', 'route': 'Guwahati to Tezpur', 'busname': 'Pranjit Travels (Under ASTC)', 'price': '₹450', 'bustype': 'NON A/C Seater (2+1)', 'departing_time': '21:15', 'duration': '4h', 'reaching_time': '01:15', 'star_rating': '4', 'seats_available': '22 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Assam State Transport Corporation (ASTC)', 'route': 'Guwahati to Nagaon (Assam)', 'busname': 'Shruti(North East Travels)', 'price': '₹255', 'bustype': 'Tata A/C Seater (2+2)', 'departing_time': '17:30', 'duration': '3h 15m', 'reaching_time': '20:45', 'star_rating': '3.5', 'seats_available': '28 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Assam State Transport Corporation (ASTC)', 'route': 'Guwahati to Nagaon (Assam)', 'busname': 'Rongpur(Ayaan)', 'price': '₹450', 'bustype': 'Bharat Benz A/C Seater (2+1)', 'departing_time': '19:15', 'duration': '2h 30m', 'reaching_time': '21:45', 'star_rating': '2.7', 'seats_available': '34 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Assam State Transport Corporation (ASTC)', 'route': 'Guwahati to Nagaon (Assam)', 'busname': 'Shopuris Travels', 'price': '₹600', 'bustype': 'NON A/C Seater (2+1)', 'departing_time': '19:25', 'duration': '2h 35m', 'reaching_time': '22:00', 'star_rating': '3.4', 'seats_available': '32 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Assam State Transport Corporation (ASTC)', 'route': 'Guwahati to Nagaon (Assam)', 'busname': 'Nishigandha(Divya Tours & Travels)', 'price': '₹579', 'bustype': 'A/C Seater (2+1)', 'departing_time': '21:15', 'duration': '2h 35m', 'reaching_time': '23:50', 'star_rating': '4.1', 'seats_available': '19 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Assam State Transport Corporation (ASTC)', 'route': 'Jorhat to North Lakhimpur', 'busname': 'Priyanka', 'price': '₹600', 'bustype': 'Non AC Seater (2+1)', 'departing_time': '19:00', 'duration': '9h 30m', 'reaching_time': '04:30', 'star_rating': '2.8', 'seats_available': '9 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Assam State Transport Corporation (ASTC)', 'route': 'North Lakhimpur to Sibsagar (Assam)', 'busname': 'Sylvan DEE Transport', 'price': '₹660', 'bustype': 'Non AC Seater (2+1)', 'departing_time': '18:00', 'duration': '11h 30m', 'reaching_time': '05:30', 'star_rating': '3.4', 'seats_available': '13 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Assam State Transport Corporation (ASTC)', 'route': 'Dhubri to Guwahati', 'busname': 'Yatra', 'price': '₹350', 'bustype': 'Non AC Seater (2+2)', 'departing_time': '20:15', 'duration': '7h 15m', 'reaching_time': '03:30', 'star_rating': '1.3', 'seats_available': '40 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Assam State Transport Corporation (ASTC)', 'route': 'North Lakhimpur to Jorhat', 'busname': 'Sylvan DEE Transport', 'price': '₹560', 'bustype': 'Non AC Seater (2+1)', 'departing_time': '18:00', 'duration': '10h 30m', 'reaching_time': '04:30', 'star_rating': '4', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Bareilly (Uttar Pradesh) to Delhi', 'busname': 'RS Yadav Smart Bus', 'price': '₹750', 'bustype': 'VE A/C Seater / Sleeper (2+1)', 'departing_time': '23:59', 'duration': '6h 26m', 'reaching_time': '06:25', 'star_rating': '3.9', 'seats_available': '27 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Bareilly (Uttar Pradesh) to Delhi', 'busname': 'New Khaira TPT', 'price': '₹760', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '23:45', 'duration': '5h', 'reaching_time': '04:45', 'star_rating': '4.7', 'seats_available': '13 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Bareilly (Uttar Pradesh) to Delhi', 'busname': 'Bajpai Travels', 'price': '₹1,890', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '22:00', 'duration': '7h', 'reaching_time': '05:00', 'star_rating': '2.1', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Bareilly (Uttar Pradesh) to Delhi', 'busname': 'HONEY TRAVELS', 'price': '₹1,090', 'bustype': 'AC Seater (2+3)', 'departing_time': '23:35', 'duration': '5h 15m', 'reaching_time': '04:50', 'star_rating': '1', 'seats_available': '5 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Bareilly (Uttar Pradesh) to Delhi', 'busname': 'Vijay Tour and Travels', 'price': '₹599', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '19:10', 'duration': '6h 10m', 'reaching_time': '01:20', 'star_rating': '3.3', 'seats_available': '13 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Bareilly (Uttar Pradesh) to Delhi', 'busname': 'JAI SHREE RAM TRAVELS', 'price': '₹750', 'bustype': 'A/C Seater (2+2)', 'departing_time': '23:50', 'duration': '5h', 'reaching_time': '04:50', 'star_rating': '3.3', 'seats_available': '21 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Bareilly (Uttar Pradesh) to Delhi', 'busname': 'Jaiswal Tour and Travels', 'price': '₹1,140', 'bustype': 'A/C Seater/Sleeper (2+1)', 'departing_time': '22:45', 'duration': '5h 45m', 'reaching_time': '04:30', 'star_rating': '2.2', 'seats_available': '5 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Bareilly (Uttar Pradesh) to Delhi', 'busname': 'Rahul Travels', 'price': '₹769', 'bustype': 'AshokLeyland Stile A/C', 'departing_time': '22:30', 'duration': '5h 30m', 'reaching_time': '04:00', 'star_rating': '2.3', 'seats_available': '8 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Bareilly (Uttar Pradesh) to Delhi', 'busname': 'JAI SHREE RAM TRAVELS', 'price': '₹500', 'bustype': 'A/C Seater (2+2)', 'departing_time': '23:55', 'duration': '6h 5m', 'reaching_time': '06:00', 'star_rating': '1.5', 'seats_available': '45 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Bareilly (Uttar Pradesh) to Delhi', 'busname': 'Jaiswal Holidays', 'price': '₹1,000', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '22:10', 'duration': '5h 50m', 'reaching_time': '04:00', 'star_rating': '2.8', 'seats_available': '4 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Bareilly (Uttar Pradesh) to Delhi', 'busname': 'UPSRTC - BSR0468', 'price': '₹404', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '17:46', 'duration': '3h 25m', 'reaching_time': '21:11', 'star_rating': None, 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Bareilly (Uttar Pradesh) to Delhi', 'busname': 'UPSRTC - SJP0476', 'price': '₹450', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '17:54', 'duration': '5h 39m', 'reaching_time': '23:33', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Bareilly (Uttar Pradesh) to Delhi', 'busname': 'UPSRTC - KSB0183', 'price': '₹410', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:00', 'duration': '5h 50m', 'reaching_time': '23:50', 'star_rating': '3.3', 'seats_available': '52 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Bareilly (Uttar Pradesh) to Delhi', 'busname': 'UPSRTC - BLY0836', 'price': '₹500', 'bustype': 'SHATABDI (2+3)', 'departing_time': '18:20', 'duration': '5h 30m', 'reaching_time': '23:50', 'star_rating': '3.1', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Bareilly (Uttar Pradesh) to Delhi', 'busname': 'UPSRTC - BSR0305', 'price': '₹424', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:22', 'duration': '4h 41m', 'reaching_time': '23:03', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Bareilly (Uttar Pradesh) to Delhi', 'busname': 'UPSRTC - KSB0186', 'price': '₹410', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:30', 'duration': '6h', 'reaching_time': '00:30', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Bareilly (Uttar Pradesh) to Delhi', 'busname': 'UPSRTC - KSB0223', 'price': '₹410', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:31', 'duration': '5h 20m', 'reaching_time': '23:51', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Bareilly (Uttar Pradesh) to Delhi', 'busname': 'UPSRTC - KSB0227', 'price': '₹410', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:33', 'duration': '6h', 'reaching_time': '00:33', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Bareilly (Uttar Pradesh) to Delhi', 'busname': 'UPSRTC - BLY0111', 'price': '₹500', 'bustype': 'Janrath AC Seater 2+2', 'departing_time': '19:01', 'duration': '4h 44m', 'reaching_time': '23:45', 'star_rating': '3.1', 'seats_available': '36 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Bareilly (Uttar Pradesh) to Delhi', 'busname': 'UPSRTC - HPR0139', 'price': '₹437', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:10', 'duration': '5h 55m', 'reaching_time': '01:05', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Aligarh (uttar pradesh) to Delhi', 'busname': 'UPSRTC - SKD0205', 'price': '₹257', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '17:45', 'duration': '3h 27m', 'reaching_time': '21:12', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Aligarh (uttar pradesh) to Delhi', 'busname': 'UPSRTC - BWR0123', 'price': '₹257', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '17:47', 'duration': '3h 43m', 'reaching_time': '21:30', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Aligarh (uttar pradesh) to Delhi', 'busname': 'UPSRTC - BSR1053', 'price': '₹230', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '17:50', 'duration': '5h 45m', 'reaching_time': '23:35', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Aligarh (uttar pradesh) to Delhi', 'busname': 'UPSRTC - SKD0199', 'price': '₹257', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '17:52', 'duration': '3h 19m', 'reaching_time': '21:11', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Aligarh (uttar pradesh) to Delhi', 'busname': 'UPSRTC - SKD0554', 'price': '₹230', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '17:55', 'duration': '4h 23m', 'reaching_time': '22:18', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Aligarh (uttar pradesh) to Delhi', 'busname': 'UPSRTC - SKD0591', 'price': '₹230', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '17:57', 'duration': '3h 22m', 'reaching_time': '21:19', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Aligarh (uttar pradesh) to Delhi', 'busname': 'UPSRTC - BDV0462', 'price': '₹259', 'bustype': 'Janrath AC Seater 2+2', 'departing_time': '17:59', 'duration': '3h 2m', 'reaching_time': '21:01', 'star_rating': '3.8', 'seats_available': '29 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Aligarh (uttar pradesh) to Delhi', 'busname': 'UPSRTC - BDV0092', 'price': '₹204', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '17:59', 'duration': '3h 19m', 'reaching_time': '21:18', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Aligarh (uttar pradesh) to Delhi', 'busname': 'UPSRTC - SKD0136', 'price': '₹230', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:00', 'duration': '3h 20m', 'reaching_time': '21:20', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Aligarh (uttar pradesh) to Delhi', 'busname': 'UPSRTC - VKS0212', 'price': '₹257', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:00', 'duration': '3h', 'reaching_time': '21:00', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Bareilly (Uttar Pradesh)', 'busname': 'RS Yadav Smart Bus', 'price': '₹799', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '21:40', 'duration': '6h 40m', 'reaching_time': '04:20', 'star_rating': '4', 'seats_available': '17 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Bareilly (Uttar Pradesh)', 'busname': 'RS Yadav Smart Bus', 'price': '₹799', 'bustype': 'VE A/C Seater / Sleeper (2+1)', 'departing_time': '20:30', 'duration': '6h 40m', 'reaching_time': '03:10', 'star_rating': '4.2', 'seats_available': '24 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Bareilly (Uttar Pradesh)', 'busname': 'New Khaira Transport', 'price': '₹494', 'bustype': 'A/C Seater / Sleeper (2+2)', 'departing_time': '21:00', 'duration': '6h', 'reaching_time': '03:00', 'star_rating': '2.1', 'seats_available': '7 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Bareilly (Uttar Pradesh)', 'busname': 'New Khaira TPT', 'price': '₹760', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '20:45', 'duration': '6h 45m', 'reaching_time': '03:30', 'star_rating': '1.8', 'seats_available': '19 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Bareilly (Uttar Pradesh)', 'busname': 'New Khaira TPT', 'price': '₹674', 'bustype': 'A/C Seater / Sleeper (2+2)', 'departing_time': '21:00', 'duration': '5h 45m', 'reaching_time': '02:45', 'star_rating': '2.4', 'seats_available': '19 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Bareilly (Uttar Pradesh)', 'busname': 'Rahul Travels', 'price': '₹666', 'bustype': 'AshokLeyland Stile A/C', 'departing_time': '21:30', 'duration': '5h', 'reaching_time': '02:30', 'star_rating': '3.6', 'seats_available': '8 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Bareilly (Uttar Pradesh)', 'busname': 'International Tourist Centre', 'price': '₹1,099', 'bustype': 'AC Seater (2+2)', 'departing_time': '23:00', 'duration': '7h', 'reaching_time': '06:00', 'star_rating': '4.3', 'seats_available': '4 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Bareilly (Uttar Pradesh)', 'busname': 'Jaiswal Tour and Travels', 'price': '₹1,425', 'bustype': 'A/C Seater/Sleeper (2+1)', 'departing_time': '20:30', 'duration': '6h', 'reaching_time': '02:30', 'star_rating': '2.5', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Bareilly (Uttar Pradesh)', 'busname': 'Jaiswal Tour and Travels', 'price': '₹1,282', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '20:45', 'duration': '6h', 'reaching_time': '02:45', 'star_rating': '2.5', 'seats_available': '21 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Bareilly (Uttar Pradesh)', 'busname': 'Jaiswal Tour and Travels', 'price': '₹1,425', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '21:00', 'duration': '6h', 'reaching_time': '03:00', 'star_rating': '2.5', 'seats_available': '24 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Bareilly (Uttar Pradesh)', 'busname': 'UPSRTC - RPD0088', 'price': '₹445', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '17:40', 'duration': '6h 20m', 'reaching_time': '00:00', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Bareilly (Uttar Pradesh)', 'busname': 'UPSRTC - RPD0039', 'price': '₹441', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '17:40', 'duration': '6h 50m', 'reaching_time': '00:30', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Bareilly (Uttar Pradesh)', 'busname': 'UPSRTC - KSB0338', 'price': '₹410', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:00', 'duration': '6h 22m', 'reaching_time': '00:22', 'star_rating': '3.3', 'seats_available': '52 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Bareilly (Uttar Pradesh)', 'busname': 'UPSRTC - KSB0340', 'price': '₹410', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:01', 'duration': '6h 24m', 'reaching_time': '00:25', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Bareilly (Uttar Pradesh)', 'busname': 'UPSRTC - SJP0642', 'price': '₹455', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:02', 'duration': '6h 49m', 'reaching_time': '00:51', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Bareilly (Uttar Pradesh)', 'busname': 'UPSRTC - HDD0082', 'price': '₹445', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:02', 'duration': '5h 13m', 'reaching_time': '23:15', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Bareilly (Uttar Pradesh)', 'busname': 'UPSRTC - RPD0092', 'price': '₹451', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:20', 'duration': '7h 30m', 'reaching_time': '01:50', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Bareilly (Uttar Pradesh)', 'busname': 'UPSRTC - RPD0041', 'price': '₹441', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:20', 'duration': '6h 50m', 'reaching_time': '01:10', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Bareilly (Uttar Pradesh)', 'busname': 'UPSRTC - HDD0006', 'price': '₹515', 'bustype': 'Janrath AC Seater 2+2', 'departing_time': '18:30', 'duration': '6h 30m', 'reaching_time': '01:00', 'star_rating': '3.3', 'seats_available': '34 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Bareilly (Uttar Pradesh)', 'busname': 'UPSRTC - HDD0113', 'price': '₹445', 'bustype': 'Rajdhani Non AC Seater 2+3', 'departing_time': '18:31', 'duration': '6h 29m', 'reaching_time': '01:00', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Aligarh (uttar pradesh)', 'busname': 'Vijay Tour and Travels', 'price': '₹665', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '21:10', 'duration': '3h 50m', 'reaching_time': '01:00', 'star_rating': '3.6', 'seats_available': '24 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Aligarh (uttar pradesh)', 'busname': 'UPSRTC - MNP0152', 'price': '₹257', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '17:45', 'duration': '2h 49m', 'reaching_time': '20:34', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Aligarh (uttar pradesh)', 'busname': 'UPSRTC - FKP0034', 'price': '₹257', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '17:50', 'duration': '2h 42m', 'reaching_time': '20:32', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Aligarh (uttar pradesh)', 'busname': 'UPSRTC - HDD0255', 'price': '₹257', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '17:58', 'duration': '2h 8m', 'reaching_time': '20:06', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Aligarh (uttar pradesh)', 'busname': 'UPSRTC - ETH1004', 'price': '₹257', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:00', 'duration': '3h 50m', 'reaching_time': '21:50', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Aligarh (uttar pradesh)', 'busname': 'UPSRTC - UNO0183', 'price': '₹252', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:01', 'duration': '3h 59m', 'reaching_time': '22:00', 'star_rating': '3.3', 'seats_available': '52 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Aligarh (uttar pradesh)', 'busname': 'UPSRTC - HDD0079', 'price': '₹257', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:05', 'duration': '2h 45m', 'reaching_time': '20:50', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Aligarh (uttar pradesh)', 'busname': 'UPSRTC - KSB0911', 'price': '₹230', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:05', 'duration': '3h 30m', 'reaching_time': '21:35', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Aligarh (uttar pradesh)', 'busname': 'UPSRTC - FZG0143', 'price': '₹257', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:05', 'duration': '3h 55m', 'reaching_time': '22:00', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Aligarh (uttar pradesh)', 'busname': 'UPSRTC - HDD0171', 'price': '₹257', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:06', 'duration': '3h 44m', 'reaching_time': '21:50', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Aligarh (uttar pradesh)', 'busname': 'UPSRTC - KRJ0064', 'price': '₹230', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:10', 'duration': '2h 49m', 'reaching_time': '20:59', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Prayagraj(Uttar Pradesh)', 'busname': 'Mahalaxmi Travels', 'price': '₹381', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '23:15', 'duration': '3h 14m', 'reaching_time': '02:29', 'star_rating': '3.5', 'seats_available': '22 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Prayagraj(Uttar Pradesh)', 'busname': 'Shakmbhari Travels', 'price': '₹5,000', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '22:30', 'duration': '4h 15m', 'reaching_time': '02:45', 'star_rating': '3.9', 'seats_available': '48 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Prayagraj(Uttar Pradesh)', 'busname': 'UPSRTC - PYD0155', 'price': '₹357', 'bustype': 'Janrath AC Seater 2+3', 'departing_time': '18:01', 'duration': '4h 30m', 'reaching_time': '22:31', 'star_rating': '3.7', 'seats_available': '48 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Prayagraj(Uttar Pradesh)', 'busname': 'UPSRTC - PYD0030', 'price': '₹541', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '18:30', 'duration': '4h 30m', 'reaching_time': '23:00', 'star_rating': '4.8', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Prayagraj(Uttar Pradesh)', 'busname': 'UPSRTC - RBL0010', 'price': '₹310', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:00', 'duration': '5h', 'reaching_time': '00:00', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Prayagraj(Uttar Pradesh)', 'busname': 'UPSRTC - PYD0157', 'price': '₹357', 'bustype': 'Janrath AC Seater 2+3', 'departing_time': '19:01', 'duration': '4h', 'reaching_time': '23:01', 'star_rating': '3.2', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Prayagraj(Uttar Pradesh)', 'busname': 'UPSRTC - PYD0037', 'price': '₹541', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '19:30', 'duration': '4h 29m', 'reaching_time': '23:59', 'star_rating': '4.5', 'seats_available': '38 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Prayagraj(Uttar Pradesh)', 'busname': 'UPSRTC - RBL0014', 'price': '₹310', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:40', 'duration': '5h', 'reaching_time': '00:40', 'star_rating': '2.1', 'seats_available': '52 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Prayagraj(Uttar Pradesh)', 'busname': 'UPSRTC - MZP0014', 'price': '₹310', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '20:00', 'duration': '5h 28m', 'reaching_time': '01:28', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Prayagraj(Uttar Pradesh)', 'busname': 'UPSRTC - MZP0110', 'price': '₹310', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '20:02', 'duration': '5h 11m', 'reaching_time': '01:13', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Prayagraj(Uttar Pradesh)', 'busname': 'UPSRTC - PYD0034', 'price': '₹541', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '20:30', 'duration': '4h 30m', 'reaching_time': '01:00', 'star_rating': '3.7', 'seats_available': '39 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Prayagraj(Uttar Pradesh)', 'busname': 'UPSRTC - RBL0018', 'price': '₹310', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '20:40', 'duration': '5h', 'reaching_time': '01:40', 'star_rating': '3.8', 'seats_available': '52 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Badaun to Delhi', 'busname': 'UPSRTC - SJP0474', 'price': '₹369', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '17:45', 'duration': '6h 18m', 'reaching_time': '00:03', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Badaun to Delhi', 'busname': 'UPSRTC - BSR0748', 'price': '₹343', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '17:48', 'duration': '5h 57m', 'reaching_time': '23:45', 'star_rating': '3.3', 'seats_available': '52 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Badaun to Delhi', 'busname': 'UPSRTC - SKD0061', 'price': '₹369', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '17:51', 'duration': '6h', 'reaching_time': '23:51', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Badaun to Delhi', 'busname': 'UPSRTC - SKD0415', 'price': '₹369', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '17:53', 'duration': '6h', 'reaching_time': '23:53', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Badaun to Delhi', 'busname': 'UPSRTC - SKD0408', 'price': '₹343', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '17:53', 'duration': '6h 3m', 'reaching_time': '23:56', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Badaun to Delhi', 'busname': 'UPSRTC - KRJ0443', 'price': '₹343', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '17:59', 'duration': '4h 31m', 'reaching_time': '22:30', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Badaun to Delhi', 'busname': 'UPSRTC - LNI0217', 'price': '₹369', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:00', 'duration': '4h 50m', 'reaching_time': '22:50', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Badaun to Delhi', 'busname': 'UPSRTC - BSR1080', 'price': '₹343', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:01', 'duration': '4h 48m', 'reaching_time': '22:49', 'star_rating': '3.3', 'seats_available': '52 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Badaun to Delhi', 'busname': 'UPSRTC - BDN0159', 'price': '₹411', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:01', 'duration': '14h 59m', 'reaching_time': '09:00', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Badaun to Delhi', 'busname': 'UPSRTC - SKD0062', 'price': '₹369', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:02', 'duration': '5h 57m', 'reaching_time': '23:59', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Badaun', 'busname': 'UPSRTC - SJP0622', 'price': '₹373', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:01', 'duration': '4h 49m', 'reaching_time': '22:50', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Badaun', 'busname': 'UPSRTC - HDD0299', 'price': '₹373', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:01', 'duration': '3h 53m', 'reaching_time': '21:54', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Badaun', 'busname': 'UPSRTC - SKD0082', 'price': '₹343', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:01', 'duration': '6h 1m', 'reaching_time': '00:02', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Badaun', 'busname': 'UPSRTC - SKD0486', 'price': '₹343', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:02', 'duration': '5h 57m', 'reaching_time': '23:59', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Badaun', 'busname': 'UPSRTC - BDN0107', 'price': '₹343', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:13', 'duration': '6h', 'reaching_time': '00:13', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Badaun', 'busname': 'UPSRTC - SJP0614', 'price': '₹369', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:31', 'duration': '4h 21m', 'reaching_time': '22:52', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Badaun', 'busname': 'UPSRTC - BDN0239', 'price': '₹343', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:36', 'duration': '4h 36m', 'reaching_time': '23:12', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Badaun', 'busname': 'UPSRTC - SKD0642', 'price': '₹343', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:42', 'duration': '4h 40m', 'reaching_time': '23:22', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Badaun', 'busname': 'UPSRTC - BDN0044', 'price': '₹411', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:00', 'duration': '6h 30m', 'reaching_time': '01:30', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Badaun', 'busname': 'UPSRTC - SKD0085', 'price': '₹343', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:00', 'duration': '4h 58m', 'reaching_time': '23:58', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Agra', 'busname': 'RSRTC - 177797', 'price': '₹891', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '21:00', 'duration': '5h 30m', 'reaching_time': '02:30', 'star_rating': '4.4', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Agra', 'busname': 'FlixBus', 'price': '₹1,053', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '21:40', 'duration': '5h', 'reaching_time': '02:40', 'star_rating': '4.6', 'seats_available': '5 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Agra', 'busname': 'IntrCity SmartBus', 'price': '₹439', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '23:30', 'duration': '5h 15m', 'reaching_time': '04:45', 'star_rating': '4.7', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Agra', 'busname': 'Sethi Yatra Company', 'price': '₹750', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '20:00', 'duration': '4h 30m', 'reaching_time': '00:30', 'star_rating': '4.4', 'seats_available': '8 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Agra', 'busname': 'PTC-SkyBus', 'price': '₹600', 'bustype': 'VE A/C Seater / Sleeper (2+1)', 'departing_time': '18:15', 'duration': '4h 35m', 'reaching_time': '22:50', 'star_rating': '4.7', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Agra', 'busname': 'FlixBus', 'price': '₹508', 'bustype': 'Bharat Benz A/C Semi Sleeper (2+2)', 'departing_time': '22:00', 'duration': '4h 50m', 'reaching_time': '02:50', 'star_rating': '4.6', 'seats_available': '47 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Agra', 'busname': 'Ashok Bus Service', 'price': '₹499', 'bustype': 'VE A/C Sleeper (2+1)', 'departing_time': '23:10', 'duration': '4h 20m', 'reaching_time': '03:30', 'star_rating': '4.2', 'seats_available': '30 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Agra', 'busname': 'FlixBus', 'price': '₹1,115', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '23:10', 'duration': '4h 25m', 'reaching_time': '03:35', 'star_rating': '4.3', 'seats_available': '4 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Agra', 'busname': 'Laxmi holidays', 'price': '₹364', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '23:00', 'duration': '5h 30m', 'reaching_time': '04:30', 'star_rating': '4.6', 'seats_available': '32 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Agra', 'busname': 'Raj Kalpana Travels Pvt.Ltd', 'price': '₹499', 'bustype': 'A/C Seater/Sleeper (2+1)', 'departing_time': '17:00', 'duration': '4h 5m', 'reaching_time': '21:05', 'star_rating': '4.1', 'seats_available': '6 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Agra', 'busname': 'UPSRTC - AZR0185', 'price': '₹516', 'bustype': 'Rajdhani Non AC Seater 2+3', 'departing_time': '17:52', 'duration': '6h 26m', 'reaching_time': '00:18', 'star_rating': None, 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Agra', 'busname': 'UPSRTC - CBG0054', 'price': '₹527', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:00', 'duration': '6h 45m', 'reaching_time': '00:45', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Agra', 'busname': 'UPSRTC - AWD0011', 'price': '₹644', 'bustype': 'Janrath AC Seater 2+2', 'departing_time': '18:00', 'duration': '6h 15m', 'reaching_time': '00:15', 'star_rating': '3.3', 'seats_available': '36 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Agra', 'busname': 'UPSRTC - HPR0176', 'price': '₹494', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:00', 'duration': '4h 30m', 'reaching_time': '22:30', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Agra', 'busname': 'UPSRTC - SNL0143', 'price': '₹491', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:12', 'duration': '6h 57m', 'reaching_time': '01:09', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Agra', 'busname': 'UPSRTC - SDN0040', 'price': '₹467', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:30', 'duration': '8h 28m', 'reaching_time': '02:58', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Agra', 'busname': 'UPSRTC - SNL0003', 'price': '₹491', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:40', 'duration': '5h 29m', 'reaching_time': '00:09', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Agra', 'busname': 'UPSRTC - AZS0120', 'price': '₹491', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:11', 'duration': '4h 23m', 'reaching_time': '23:34', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Agra', 'busname': 'UPSRTC - SDN0021', 'price': '₹429', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:19', 'duration': '7h 31m', 'reaching_time': '02:50', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Agra', 'busname': 'UPSRTC - SDN0031', 'price': '₹364', 'bustype': 'Ordinary Non AC Seater 2+3', 'departing_time': '19:26', 'duration': '8h 44m', 'reaching_time': '04:10', 'star_rating': '3.3', 'seats_available': '51 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Prayagraj(Uttar Pradesh) to Lucknow', 'busname': 'New Shree Mahaveer Travels (LKW)', 'price': '₹699', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '18:00', 'duration': '5h 30m', 'reaching_time': '23:30', 'star_rating': '4.4', 'seats_available': '19 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Prayagraj(Uttar Pradesh) to Lucknow', 'busname': 'Goyal Travels', 'price': '₹700', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '18:35', 'duration': '4h 50m', 'reaching_time': '23:25', 'star_rating': '3', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Prayagraj(Uttar Pradesh) to Lucknow', 'busname': 'IntrCity SmartBus', 'price': '₹419', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '20:15', 'duration': '4h 10m', 'reaching_time': '00:25', 'star_rating': '5', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Prayagraj(Uttar Pradesh) to Lucknow', 'busname': 'Sethi Yatra Company', 'price': '₹699', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '18:35', 'duration': '5h 5m', 'reaching_time': '23:40', 'star_rating': '2.2', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Prayagraj(Uttar Pradesh) to Lucknow', 'busname': 'Mahalaxmi Travels', 'price': '₹799', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '19:30', 'duration': '4h', 'reaching_time': '23:30', 'star_rating': '2.4', 'seats_available': '5 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Prayagraj(Uttar Pradesh) to Lucknow', 'busname': 'ARORA TRAVELS', 'price': '₹699', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '20:54', 'duration': '4h 45m', 'reaching_time': '01:39', 'star_rating': '2.6', 'seats_available': '21 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Prayagraj(Uttar Pradesh) to Lucknow', 'busname': 'Tiwari Travels', 'price': '₹3,800', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '21:00', 'duration': '4h', 'reaching_time': '01:00', 'star_rating': '2.7', 'seats_available': '36 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Prayagraj(Uttar Pradesh) to Lucknow', 'busname': 'UPSRTC - AWD0008', 'price': '₹388', 'bustype': 'Janrath AC Seater 2+2', 'departing_time': '18:00', 'duration': '4h 30m', 'reaching_time': '22:30', 'star_rating': '3.4', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Prayagraj(Uttar Pradesh) to Lucknow', 'busname': 'UPSRTC - ALM0024', 'price': '₹541', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '18:30', 'duration': '4h 30m', 'reaching_time': '23:00', 'star_rating': '3.8', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Prayagraj(Uttar Pradesh) to Lucknow', 'busname': 'UPSRTC - PYD0158', 'price': '₹357', 'bustype': 'Janrath AC Seater 2+3', 'departing_time': '19:01', 'duration': '4h 30m', 'reaching_time': '23:31', 'star_rating': '2.9', 'seats_available': '48 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Prayagraj(Uttar Pradesh) to Lucknow', 'busname': 'UPSRTC - PYD0409', 'price': '₹310', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:01', 'duration': '4h 58m', 'reaching_time': '23:59', 'star_rating': '3.3', 'seats_available': '52 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Prayagraj(Uttar Pradesh) to Lucknow', 'busname': 'UPSRTC - ALM0026', 'price': '₹541', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '19:30', 'duration': '4h 29m', 'reaching_time': '23:59', 'star_rating': '4.7', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Prayagraj(Uttar Pradesh) to Lucknow', 'busname': 'UPSRTC - PYD0411', 'price': '₹310', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '20:01', 'duration': '5h', 'reaching_time': '01:01', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Prayagraj(Uttar Pradesh) to Lucknow', 'busname': 'UPSRTC - ALM0060', 'price': '₹541', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '20:30', 'duration': '4h 30m', 'reaching_time': '01:00', 'star_rating': '4.4', 'seats_available': '39 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Prayagraj(Uttar Pradesh) to Lucknow', 'busname': 'UPSRTC - ALM0185', 'price': '₹541', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '21:00', 'duration': '4h', 'reaching_time': '01:00', 'star_rating': '4.5', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Prayagraj(Uttar Pradesh) to Lucknow', 'busname': 'UPSRTC - PYD0413', 'price': '₹310', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '21:01', 'duration': '5h', 'reaching_time': '02:01', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Prayagraj(Uttar Pradesh) to Lucknow', 'busname': 'UPSRTC - PYD0415', 'price': '₹310', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '22:01', 'duration': '5h', 'reaching_time': '03:01', 'star_rating': '3.3', 'seats_available': '52 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Bareilly (Uttar Pradesh)', 'busname': 'PTC-SkyBus', 'price': '₹399', 'bustype': 'A/C Seater/Sleeper (2+1)', 'departing_time': '22:45', 'duration': '5h 35m', 'reaching_time': '04:20', 'star_rating': '4.5', 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Bareilly (Uttar Pradesh)', 'busname': 'PTC-SkyBus', 'price': '₹599', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '23:15', 'duration': '4h 45m', 'reaching_time': '04:00', 'star_rating': '4.8', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Bareilly (Uttar Pradesh)', 'busname': 'New Shree Mahaveer Travels (LKW)', 'price': '₹499', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '22:45', 'duration': '5h', 'reaching_time': '03:45', 'star_rating': '4.3', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Bareilly (Uttar Pradesh)', 'busname': 'Samay Shatabdi Travels Pvt Ltd', 'price': '₹699', 'bustype': 'AC Sleeper (2+1)', 'departing_time': '22:00', 'duration': '4h 45m', 'reaching_time': '02:45', 'star_rating': '3.7', 'seats_available': '23 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Bareilly (Uttar Pradesh)', 'busname': 'Mahalaxmi Travels', 'price': '₹399', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '23:45', 'duration': '5h', 'reaching_time': '04:45', 'star_rating': '2.5', 'seats_available': '19 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Bareilly (Uttar Pradesh)', 'busname': 'Vijay Tour and Travels', 'price': '₹899', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '20:30', 'duration': '5h', 'reaching_time': '01:30', 'star_rating': '3.6', 'seats_available': '6 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Bareilly (Uttar Pradesh)', 'busname': 'Mahalaxmi Travels', 'price': '₹499', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '22:45', 'duration': '5h 5m', 'reaching_time': '03:50', 'star_rating': '3.3', 'seats_available': '7 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Bareilly (Uttar Pradesh)', 'busname': 'Shekhar Travels', 'price': '₹2,000', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '22:50', 'duration': '3h 30m', 'reaching_time': '02:20', 'star_rating': '3.3', 'seats_available': '7 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Bareilly (Uttar Pradesh)', 'busname': 'UPSRTC - HPR0094', 'price': '₹390', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:00', 'duration': '6h 20m', 'reaching_time': '00:20', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Bareilly (Uttar Pradesh)', 'busname': 'UPSRTC - HPR0130', 'price': '₹390', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:01', 'duration': '5h 44m', 'reaching_time': '23:45', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Bareilly (Uttar Pradesh)', 'busname': 'UPSRTC - HPR0095', 'price': '₹396', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:30', 'duration': '5h 50m', 'reaching_time': '00:20', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Bareilly (Uttar Pradesh)', 'busname': 'UPSRTC - HPR0131', 'price': '₹396', 'bustype': 'Ordinary Non AC Seater 2+3', 'departing_time': '18:31', 'duration': '5h 59m', 'reaching_time': '00:30', 'star_rating': '3.3', 'seats_available': '52 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Bareilly (Uttar Pradesh)', 'busname': 'UPSRTC - SDN0134', 'price': '₹392', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:37', 'duration': '5h 59m', 'reaching_time': '00:36', 'star_rating': None, 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Bareilly (Uttar Pradesh)', 'busname': 'UPSRTC - HPR0099', 'price': '₹396', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:00', 'duration': '6h 15m', 'reaching_time': '01:15', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Bareilly (Uttar Pradesh)', 'busname': 'UPSRTC - HPR0132', 'price': '₹396', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:01', 'duration': '6h 9m', 'reaching_time': '01:10', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Bareilly (Uttar Pradesh)', 'busname': 'UPSRTC - SBD0582', 'price': '₹483', 'bustype': 'Pink Express AC Seater 2+2', 'departing_time': '19:02', 'duration': '5h 43m', 'reaching_time': '00:45', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Bareilly (Uttar Pradesh)', 'busname': 'UPSRTC - HPR0107', 'price': '₹396', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:30', 'duration': '5h 30m', 'reaching_time': '01:00', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Bareilly (Uttar Pradesh)', 'busname': 'UPSRTC - HPR0133', 'price': '₹396', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:31', 'duration': '5h 49m', 'reaching_time': '01:20', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Farrukhabad (Uttar Pradesh)', 'busname': 'Abhay Travels (I.R.T.C)', 'price': '₹1,300', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '20:30', 'duration': '9h 30m', 'reaching_time': '06:00', 'star_rating': '2.5', 'seats_available': '30 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Farrukhabad (Uttar Pradesh)', 'busname': 'Shree Krishna Shtabadi Travels', 'price': '₹600', 'bustype': 'NON AC Seater / Sleeper 2+1', 'departing_time': '21:00', 'duration': '9h', 'reaching_time': '06:00', 'star_rating': '1.7', 'seats_available': '44 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Farrukhabad (Uttar Pradesh)', 'busname': 'Shivendra Travels', 'price': '₹3,500', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '21:00', 'duration': '9h', 'reaching_time': '06:00', 'star_rating': '2.2', 'seats_available': '1 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Farrukhabad (Uttar Pradesh)', 'busname': 'Shree Krishna Shtabadi Travels', 'price': '₹1,200', 'bustype': 'NON A/C Sleeper (2+1)', 'departing_time': '21:00', 'duration': '9h 30m', 'reaching_time': '06:30', 'star_rating': '1.7', 'seats_available': '32 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Farrukhabad (Uttar Pradesh)', 'busname': 'UPSRTC - FKP0034', 'price': '₹514', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '17:50', 'duration': '8h 55m', 'reaching_time': '02:45', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Farrukhabad (Uttar Pradesh)', 'busname': 'UPSRTC - HDD0255', 'price': '₹514', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '17:58', 'duration': '4h 22m', 'reaching_time': '22:20', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Farrukhabad (Uttar Pradesh)', 'busname': 'UPSRTC - HDD0079', 'price': '₹514', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:05', 'duration': '5h 25m', 'reaching_time': '23:30', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Farrukhabad (Uttar Pradesh)', 'busname': 'UPSRTC - HDD0171', 'price': '₹514', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:06', 'duration': '7h 44m', 'reaching_time': '01:50', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Farrukhabad (Uttar Pradesh)', 'busname': 'UPSRTC - HDD0035', 'price': '₹514', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:32', 'duration': '8h 3m', 'reaching_time': '02:35', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Farrukhabad (Uttar Pradesh)', 'busname': 'UPSRTC - HDD0292', 'price': '₹514', 'bustype': 'Rajdhani Non AC Seater 2+3', 'departing_time': '19:00', 'duration': '4h 55m', 'reaching_time': '23:55', 'star_rating': '3.3', 'seats_available': '51 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Farrukhabad (Uttar Pradesh)', 'busname': 'UPSRTC - FKP0036', 'price': '₹514', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:00', 'duration': '7h 40m', 'reaching_time': '02:40', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Farrukhabad (Uttar Pradesh)', 'busname': 'UPSRTC - KSB0264', 'price': '₹487', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:01', 'duration': '7h', 'reaching_time': '02:01', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Farrukhabad (Uttar Pradesh)', 'busname': 'UPSRTC - HDD0111', 'price': '₹514', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:03', 'duration': '10h 2m', 'reaching_time': '05:05', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Farrukhabad (Uttar Pradesh)', 'busname': 'UPSRTC - HDD0174', 'price': '₹514', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:06', 'duration': '7h 54m', 'reaching_time': '03:00', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Farrukhabad (Uttar Pradesh) to Delhi', 'busname': 'Chaturvedi Bus Service Pvt. Ltd.', 'price': '₹400', 'bustype': 'NON AC Seater / Sleeper 2+1', 'departing_time': '20:00', 'duration': '9h 20m', 'reaching_time': '05:20', 'star_rating': '2.4', 'seats_available': '30 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Farrukhabad (Uttar Pradesh) to Delhi', 'busname': 'Chaturvedi Bus Service Pvt. Ltd.', 'price': '₹450', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '19:30', 'duration': '9h 30m', 'reaching_time': '05:00', 'star_rating': '3.6', 'seats_available': '48 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Farrukhabad (Uttar Pradesh) to Delhi', 'busname': 'Abhay Travels (I.R.T.C)', 'price': '₹1,200', 'bustype': 'Non A/C Seater / Sleeper (2+1)', 'departing_time': '18:00', 'duration': '12h', 'reaching_time': '06:00', 'star_rating': '2.5', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Farrukhabad (Uttar Pradesh) to Delhi', 'busname': 'Shivendra Travels', 'price': '₹3,500', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '20:00', 'duration': '9h 30m', 'reaching_time': '05:30', 'star_rating': '2.2', 'seats_available': '1 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Farrukhabad (Uttar Pradesh) to Delhi', 'busname': 'Shree Krishna Shtabadi Travels', 'price': '₹600', 'bustype': 'NON AC Seater / Sleeper 2+1', 'departing_time': '18:30', 'duration': '10h 50m', 'reaching_time': '05:20', 'star_rating': '1.5', 'seats_available': '44 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Farrukhabad (Uttar Pradesh) to Delhi', 'busname': 'Shree Krishna Shtabadi Travels', 'price': '₹1,200', 'bustype': 'NON A/C Sleeper (2+1)', 'departing_time': '19:00', 'duration': '11h 30m', 'reaching_time': '06:30', 'star_rating': '1.5', 'seats_available': '32 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Farrukhabad (Uttar Pradesh) to Delhi', 'busname': 'UPSRTC - HPR0384', 'price': '₹514', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:00', 'duration': '8h', 'reaching_time': '02:00', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Farrukhabad (Uttar Pradesh) to Delhi', 'busname': 'UPSRTC - FKP0061', 'price': '₹514', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:00', 'duration': '8h 50m', 'reaching_time': '02:50', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Farrukhabad (Uttar Pradesh) to Delhi', 'busname': 'UPSRTC - KSB0989', 'price': '₹487', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:28', 'duration': '8h 30m', 'reaching_time': '02:58', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Farrukhabad (Uttar Pradesh) to Delhi', 'busname': 'UPSRTC - BSR0651', 'price': '₹487', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:29', 'duration': '7h 47m', 'reaching_time': '02:16', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Farrukhabad (Uttar Pradesh) to Delhi', 'busname': 'UPSRTC - KSB1106', 'price': '₹487', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:30', 'duration': '8h', 'reaching_time': '02:30', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Farrukhabad (Uttar Pradesh) to Delhi', 'busname': 'UPSRTC - LNI0049', 'price': '₹514', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:30', 'duration': '7h 35m', 'reaching_time': '02:05', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Farrukhabad (Uttar Pradesh) to Delhi', 'busname': 'UPSRTC - KSB1108', 'price': '₹487', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:31', 'duration': '8h', 'reaching_time': '02:31', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Farrukhabad (Uttar Pradesh) to Delhi', 'busname': 'UPSRTC - LNI0863', 'price': '₹514', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:31', 'duration': '8h 29m', 'reaching_time': '03:00', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Farrukhabad (Uttar Pradesh) to Delhi', 'busname': 'UPSRTC - BSR0559', 'price': '₹514', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:32', 'duration': '8h', 'reaching_time': '02:32', 'star_rating': None, 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Farrukhabad (Uttar Pradesh) to Delhi', 'busname': 'UPSRTC - BSR0059', 'price': '₹487', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:32', 'duration': '7h 28m', 'reaching_time': '02:00', 'star_rating': '3.3', 'seats_available': '52 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Agra to Lucknow', 'busname': 'RSRTC - 192993', 'price': '₹891', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '23:30', 'duration': '4h 50m', 'reaching_time': '04:20', 'star_rating': '3.4', 'seats_available': '29 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Agra to Lucknow', 'busname': 'Mahalaxmi Travels', 'price': '₹399', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '23:30', 'duration': '5h 9m', 'reaching_time': '04:39', 'star_rating': '4.2', 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Agra to Lucknow', 'busname': 'Laxmi Holidays Pvt Ltd', 'price': '₹452', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '22:30', 'duration': '4h 30m', 'reaching_time': '03:00', 'star_rating': '4.6', 'seats_available': '21 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Agra to Lucknow', 'busname': 'RS Yadav Smart Bus', 'price': '₹599', 'bustype': 'VE A/C Seater / Sleeper (2+2)', 'departing_time': '22:10', 'duration': '4h 30m', 'reaching_time': '02:40', 'star_rating': '4.3', 'seats_available': '2 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Agra to Lucknow', 'busname': 'Laxmi holidays', 'price': '₹419', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '23:15', 'duration': '4h 30m', 'reaching_time': '03:45', 'star_rating': '4', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Agra to Lucknow', 'busname': 'Raj Kalpana Travels Pvt.Ltd', 'price': '₹499', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '20:10', 'duration': '3h 49m', 'reaching_time': '23:59', 'star_rating': '4.2', 'seats_available': '3 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Agra to Lucknow', 'busname': 'Laxmi holidays', 'price': '₹407', 'bustype': 'VE A/C Seater / Sleeper (2+2)', 'departing_time': '23:30', 'duration': '4h 45m', 'reaching_time': '04:15', 'star_rating': '4.4', 'seats_available': '17 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Agra to Lucknow', 'busname': 'Anshi Raj Shree Travels', 'price': '₹599', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '19:00', 'duration': '4h 59m', 'reaching_time': '23:59', 'star_rating': '3.9', 'seats_available': '3 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Agra to Lucknow', 'busname': 'RS Yadav Smart Bus', 'price': '₹599', 'bustype': 'VE A/C Seater / Sleeper (2+1)', 'departing_time': '23:50', 'duration': '5h 20m', 'reaching_time': '05:10', 'star_rating': '4.2', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Agra to Lucknow', 'busname': 'Laxmi holidays', 'price': '₹340', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '21:20', 'duration': '4h 10m', 'reaching_time': '01:30', 'star_rating': '3.9', 'seats_available': '4 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Agra to Lucknow', 'busname': 'UPSRTC - SNL0010', 'price': '₹488', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '17:51', 'duration': '8h 47m', 'reaching_time': '02:38', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Agra to Lucknow', 'busname': 'UPSRTC - AWD0081', 'price': '₹644', 'bustype': 'Pink Express AC Seater 2+2', 'departing_time': '18:00', 'duration': '5h 59m', 'reaching_time': '23:59', 'star_rating': '3.1', 'seats_available': '41 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Agra to Lucknow', 'busname': 'UPSRTC - SDN0011', 'price': '₹467', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:30', 'duration': '8h 44m', 'reaching_time': '03:14', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Agra to Lucknow', 'busname': 'UPSRTC - SKB0034', 'price': '₹595', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:30', 'duration': '10h', 'reaching_time': '04:30', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Agra to Lucknow', 'busname': 'UPSRTC - CBG0443', 'price': '₹527', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:32', 'duration': '5h 58m', 'reaching_time': '00:30', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Agra to Lucknow', 'busname': 'UPSRTC - SNL0064', 'price': '₹488', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:51', 'duration': '8h 47m', 'reaching_time': '03:38', 'star_rating': None, 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Agra to Lucknow', 'busname': 'UPSRTC - TAJ0055', 'price': '₹644', 'bustype': 'Janrath AC Seater 2+2', 'departing_time': '19:01', 'duration': '5h 52m', 'reaching_time': '00:53', 'star_rating': '1', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Agra to Lucknow', 'busname': 'UPSRTC - TAJ0466', 'price': '₹644', 'bustype': 'Pink Express AC Seater 2+2', 'departing_time': '19:02', 'duration': '5h 53m', 'reaching_time': '00:55', 'star_rating': '3.3', 'seats_available': '41 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Agra to Lucknow', 'busname': 'UPSRTC - HPR0174', 'price': '₹494', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:31', 'duration': '6h 59m', 'reaching_time': '02:30', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Agra to Lucknow', 'busname': 'UPSRTC - AWD0010', 'price': '₹644', 'bustype': 'Janrath AC Seater 2+2', 'departing_time': '20:00', 'duration': '6h 13m', 'reaching_time': '02:13', 'star_rating': '3.7', 'seats_available': '36 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Sitapur (Uttar Pradesh)', 'busname': 'RS Yadav Smart Bus', 'price': '₹799', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '21:40', 'duration': '8h 55m', 'reaching_time': '06:35', 'star_rating': '4.2', 'seats_available': '17 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Sitapur (Uttar Pradesh)', 'busname': 'RS Yadav Smart Bus', 'price': '₹799', 'bustype': 'VE A/C Seater / Sleeper (2+1)', 'departing_time': '20:30', 'duration': '9h 30m', 'reaching_time': '06:00', 'star_rating': '4.2', 'seats_available': '24 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Sitapur (Uttar Pradesh)', 'busname': 'zingbus plus', 'price': '₹723', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '22:00', 'duration': '11h 55m', 'reaching_time': '09:55', 'star_rating': '3.7', 'seats_available': '19 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Sitapur (Uttar Pradesh)', 'busname': 'Banke Bihari Tour And Travels', 'price': '₹999', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '20:40', 'duration': '8h', 'reaching_time': '04:40', 'star_rating': None, 'seats_available': '42 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Sitapur (Uttar Pradesh)', 'busname': 'Maneesha Tour And Travels', 'price': '₹648', 'bustype': 'NON A/C Seater (2+3)', 'departing_time': '22:00', 'duration': '11h', 'reaching_time': '09:00', 'star_rating': None, 'seats_available': '55 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Sitapur (Uttar Pradesh)', 'busname': 'UPSRTC - HPR0368', 'price': '₹658', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:48', 'duration': '6h 7m', 'reaching_time': '00:55', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Sitapur (Uttar Pradesh)', 'busname': 'UPSRTC - HPR0370', 'price': '₹658', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:50', 'duration': '5h 20m', 'reaching_time': '00:10', 'star_rating': '3.3', 'seats_available': '52 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Sitapur (Uttar Pradesh)', 'busname': 'UPSRTC - STD0678', 'price': '₹658', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:00', 'duration': '11h', 'reaching_time': '06:00', 'star_rating': '3.3', 'seats_available': '52 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Sitapur (Uttar Pradesh)', 'busname': 'UPSRTC - HPR0091', 'price': '₹658', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:00', 'duration': '9h 30m', 'reaching_time': '04:30', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Sitapur (Uttar Pradesh)', 'busname': 'UPSRTC - SBD0110', 'price': '₹800', 'bustype': 'Pink Express AC Seater 2+2', 'departing_time': '19:01', 'duration': '9h 44m', 'reaching_time': '04:45', 'star_rating': '2.4', 'seats_available': '36 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Sitapur (Uttar Pradesh)', 'busname': 'UPSRTC - BSR1061', 'price': '₹658', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:29', 'duration': '3h 16m', 'reaching_time': '22:45', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Sitapur (Uttar Pradesh)', 'busname': 'UPSRTC - HPR0096', 'price': '₹658', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:30', 'duration': '7h 20m', 'reaching_time': '02:50', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Sitapur (Uttar Pradesh)', 'busname': 'UPSRTC - HPR0634', 'price': '₹658', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:33', 'duration': '4h 42m', 'reaching_time': '00:15', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Sitapur (Uttar Pradesh)', 'busname': 'UPSRTC - BSR0535', 'price': '₹658', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:44', 'duration': '5h 7m', 'reaching_time': '00:51', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Delhi to Sitapur (Uttar Pradesh)', 'busname': 'UPSRTC - NCL0322', 'price': '₹698', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:45', 'duration': '8h 45m', 'reaching_time': '04:30', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Mathura', 'busname': 'IntrCity SmartBus', 'price': '₹439', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '23:30', 'duration': '6h 5m', 'reaching_time': '05:35', 'star_rating': '4.7', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Mathura', 'busname': 'IntrCity SmartBus', 'price': '₹499', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '22:30', 'duration': '6h 10m', 'reaching_time': '04:40', 'star_rating': '4.8', 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Mathura', 'busname': 'Ashok Bus Service', 'price': '₹499', 'bustype': 'VE A/C Sleeper (2+1)', 'departing_time': '23:10', 'duration': '5h 10m', 'reaching_time': '04:20', 'star_rating': '4.6', 'seats_available': '29 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Mathura', 'busname': 'BT Travels', 'price': '₹404', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '21:50', 'duration': '5h 55m', 'reaching_time': '03:45', 'star_rating': '5', 'seats_available': '31 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Mathura', 'busname': 'Laxmi holidays', 'price': '₹405', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '22:00', 'duration': '7h 30m', 'reaching_time': '05:30', 'star_rating': '4.4', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Mathura', 'busname': 'Laxmi holidays', 'price': '₹364', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '23:00', 'duration': '6h 10m', 'reaching_time': '05:10', 'star_rating': '4.1', 'seats_available': '31 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Mathura', 'busname': 'PTC-SkyBus', 'price': '₹900', 'bustype': 'VE A/C Seater / Sleeper (2+1)', 'departing_time': '18:15', 'duration': '5h 25m', 'reaching_time': '23:40', 'star_rating': '4.8', 'seats_available': '8 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Mathura', 'busname': 'Laxmi holidays', 'price': '₹388', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '22:30', 'duration': '5h 15m', 'reaching_time': '03:45', 'star_rating': '4.9', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Mathura', 'busname': 'FlixBus', 'price': '₹1,517', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '21:25', 'duration': '5h 5m', 'reaching_time': '02:30', 'star_rating': '4.7', 'seats_available': '7 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Mathura', 'busname': 'FlixBus', 'price': '₹1,517', 'bustype': 'Bharat Benz A/C Sleeper (2+1)', 'departing_time': '23:10', 'duration': '5h 15m', 'reaching_time': '04:25', 'star_rating': '4.5', 'seats_available': '4 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Mathura', 'busname': 'UPSRTC - HPR0176', 'price': '₹588', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:00', 'duration': '5h 45m', 'reaching_time': '23:45', 'star_rating': '3.3', 'seats_available': '52 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Mathura', 'busname': 'UPSRTC - SNL0143', 'price': '₹589', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:12', 'duration': '7h 56m', 'reaching_time': '02:08', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Mathura', 'busname': 'UPSRTC - SNL0003', 'price': '₹589', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:40', 'duration': '6h 41m', 'reaching_time': '01:21', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Mathura', 'busname': 'UPSRTC - AZS0120', 'price': '₹590', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:11', 'duration': '5h 14m', 'reaching_time': '00:25', 'star_rating': '3.3', 'seats_available': '52 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Mathura', 'busname': 'UPSRTC - AWD0191', 'price': '₹718', 'bustype': 'Janrath AC Seater 2+2', 'departing_time': '19:30', 'duration': '5h 10m', 'reaching_time': '00:40', 'star_rating': '3.3', 'seats_available': '36 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Mathura', 'busname': 'UPSRTC - AWD0059', 'price': '₹734', 'bustype': 'Janrath AC Seater 2+2', 'departing_time': '20:00', 'duration': '8h', 'reaching_time': '04:00', 'star_rating': '4.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Mathura', 'busname': 'UPSRTC - AGF0070', 'price': '₹602', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '20:02', 'duration': '7h 28m', 'reaching_time': '03:30', 'star_rating': '3.8', 'seats_available': '52 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Mathura', 'busname': 'UPSRTC - SNL0007', 'price': '₹624', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '22:08', 'duration': '7h 42m', 'reaching_time': '05:50', 'star_rating': None, 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Mathura', 'busname': 'UPSRTC - AWD0056', 'price': '₹734', 'bustype': 'Janrath AC Seater 2+2', 'departing_time': '23:15', 'duration': '7h 45m', 'reaching_time': '07:00', 'star_rating': '4', 'seats_available': '31 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Gorakhpur (uttar pradesh) to Lucknow', 'busname': 'FlixBus', 'price': '₹972', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '19:00', 'duration': '6h', 'reaching_time': '01:00', 'star_rating': '4.5', 'seats_available': '7 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Gorakhpur (uttar pradesh) to Lucknow', 'busname': 'Samay Shatabdi Travels Pvt Ltd', 'price': '₹500', 'bustype': 'AC Sleeper (2+1)', 'departing_time': '22:00', 'duration': '5h 45m', 'reaching_time': '03:45', 'star_rating': '3.8', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Gorakhpur (uttar pradesh) to Lucknow', 'busname': 'Samay Shatabdi Travels Pvt Ltd', 'price': '₹500', 'bustype': 'AC Sleeper (2+1)', 'departing_time': '21:30', 'duration': '6h 14m', 'reaching_time': '03:44', 'star_rating': '3.6', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Gorakhpur (uttar pradesh) to Lucknow', 'busname': 'Gola Bus Service', 'price': '₹699', 'bustype': 'VE A/C Seater / Sleeper (2+1)', 'departing_time': '17:40', 'duration': '6h 10m', 'reaching_time': '23:50', 'star_rating': '4.9', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Gorakhpur (uttar pradesh) to Lucknow', 'busname': 'Laxmi Holidays Pvt Ltd', 'price': '₹995', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '17:30', 'duration': '6h 25m', 'reaching_time': '23:55', 'star_rating': '4.1', 'seats_available': '13 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Gorakhpur (uttar pradesh) to Lucknow', 'busname': 'Gola Bus Service', 'price': '₹699', 'bustype': 'VE A/C Seater / Sleeper (2+1)', 'departing_time': '21:00', 'duration': '5h', 'reaching_time': '02:00', 'star_rating': '4.8', 'seats_available': '21 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Gorakhpur (uttar pradesh) to Lucknow', 'busname': 'IntrCity SmartBus', 'price': '₹799', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '16:50', 'duration': '6h 20m', 'reaching_time': '23:10', 'star_rating': '4.4', 'seats_available': '1 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Gorakhpur (uttar pradesh) to Lucknow', 'busname': 'Gola Bus Service', 'price': '₹999', 'bustype': 'VE A/C Seater / Sleeper (2+1)', 'departing_time': '20:40', 'duration': '5h', 'reaching_time': '01:40', 'star_rating': '4.7', 'seats_available': '27 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Gorakhpur (uttar pradesh) to Lucknow', 'busname': 'Gola Bus Service', 'price': '₹999', 'bustype': 'AshokLeyland Stile A/C', 'departing_time': '21:30', 'duration': '5h 19m', 'reaching_time': '02:49', 'star_rating': '4.1', 'seats_available': '19 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Gorakhpur (uttar pradesh) to Lucknow', 'busname': 'Shatabdi Travels Gorakhpur', 'price': '₹525', 'bustype': 'AC Sleeper (2+1)', 'departing_time': '21:01', 'duration': '5h 30m', 'reaching_time': '02:31', 'star_rating': '3.9', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Gorakhpur (uttar pradesh) to Lucknow', 'busname': 'UPSRTC - UNO0064', 'price': '₹461', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:00', 'duration': '7h 20m', 'reaching_time': '01:20', 'star_rating': None, 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Gorakhpur (uttar pradesh) to Lucknow', 'busname': 'UPSRTC - CBG0221', 'price': '₹568', 'bustype': 'Janrath AC Seater 2+2', 'departing_time': '18:00', 'duration': '6h 30m', 'reaching_time': '00:30', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Gorakhpur (uttar pradesh) to Lucknow', 'busname': 'UPSRTC - CBG0444', 'price': '₹562', 'bustype': 'Janrath AC Seater 2+2', 'departing_time': '18:02', 'duration': '7h 28m', 'reaching_time': '01:30', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Gorakhpur (uttar pradesh) to Lucknow', 'busname': 'UPSRTC - NCL0227', 'price': '₹452', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:10', 'duration': '9h 14m', 'reaching_time': '03:24', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Gorakhpur (uttar pradesh) to Lucknow', 'busname': 'UPSRTC - SNL0063', 'price': '₹452', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:30', 'duration': '7h 53m', 'reaching_time': '02:23', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Gorakhpur (uttar pradesh) to Lucknow', 'busname': 'UPSRTC - ALM0154', 'price': '₹568', 'bustype': 'SHATABDI (2+3)', 'departing_time': '18:31', 'duration': '6h 31m', 'reaching_time': '01:02', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Gorakhpur (uttar pradesh) to Lucknow', 'busname': 'UPSRTC - DOR0157', 'price': '₹417', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:10', 'duration': '7h 20m', 'reaching_time': '02:30', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Gorakhpur (uttar pradesh) to Lucknow', 'busname': 'UPSRTC - DOR0022', 'price': '₹417', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:40', 'duration': '7h 20m', 'reaching_time': '03:00', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Gorakhpur (uttar pradesh) to Lucknow', 'busname': 'UPSRTC - CBG0217', 'price': '₹568', 'bustype': 'Janrath AC Seater 2+2', 'departing_time': '19:45', 'duration': '6h 45m', 'reaching_time': '02:30', 'star_rating': '3.3', 'seats_available': '36 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Gorakhpur (uttar pradesh) to Lucknow', 'busname': 'UPSRTC - FZG0046', 'price': '₹452', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '20:00', 'duration': '8h', 'reaching_time': '04:00', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Varanasi', 'busname': 'PTC-SkyBus', 'price': '₹400', 'bustype': 'A/C Seater/Sleeper (2+1)', 'departing_time': '22:30', 'duration': '7h', 'reaching_time': '05:30', 'star_rating': '4.6', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Varanasi', 'busname': 'Yolo Bus', 'price': '₹299', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '22:25', 'duration': '6h 45m', 'reaching_time': '05:10', 'star_rating': '4', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Varanasi', 'busname': 'Raj Kalpana Travels Pvt.Ltd', 'price': '₹499', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '23:59', 'duration': '6h 16m', 'reaching_time': '06:15', 'star_rating': '4', 'seats_available': '7 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Varanasi', 'busname': 'Mahalaxmi Travels', 'price': '₹381', 'bustype': 'Bharat Benz A/C Seater /Sleeper (2+1)', 'departing_time': '23:15', 'duration': '5h 46m', 'reaching_time': '05:01', 'star_rating': '3.5', 'seats_available': '22 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Varanasi', 'busname': 'Sethi Yatra Company', 'price': '₹249', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '22:00', 'duration': '8h 25m', 'reaching_time': '06:25', 'star_rating': '4.4', 'seats_available': '24 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Varanasi', 'busname': 'New Singh Bus Service', 'price': '₹449', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '22:30', 'duration': '4h 45m', 'reaching_time': '03:15', 'star_rating': '3.8', 'seats_available': '23 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Varanasi', 'busname': 'ARORA TRAVELS', 'price': '₹1,040', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '22:00', 'duration': '5h 1m', 'reaching_time': '03:01', 'star_rating': '2.5', 'seats_available': '21 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Varanasi', 'busname': 'New Singh Bus Service', 'price': '₹399', 'bustype': 'A/C Seater/Sleeper (2+1)', 'departing_time': '22:00', 'duration': '6h 25m', 'reaching_time': '04:25', 'star_rating': '3.6', 'seats_available': '28 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Varanasi', 'busname': 'Shakmbhari Travels', 'price': '₹5,000', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '22:30', 'duration': '7h', 'reaching_time': '05:30', 'star_rating': '3.9', 'seats_available': '48 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Varanasi', 'busname': 'Avadh Express', 'price': '₹999', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '22:00', 'duration': '7h 30m', 'reaching_time': '05:30', 'star_rating': '2', 'seats_available': '45 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Varanasi', 'busname': 'UPSRTC - CNT0138', 'price': '₹482', 'bustype': 'Ordinary Non AC Seater 2+3', 'departing_time': '18:31', 'duration': '7h 59m', 'reaching_time': '02:30', 'star_rating': '3.3', 'seats_available': '52 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Varanasi', 'busname': 'UPSRTC - CHL0019', 'price': '₹482', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:00', 'duration': '6h 45m', 'reaching_time': '01:45', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Varanasi', 'busname': 'UPSRTC - JNP0115', 'price': '₹482', 'bustype': 'Ordinary Non AC Seater 2+3', 'departing_time': '19:30', 'duration': '7h', 'reaching_time': '02:30', 'star_rating': '3.3', 'seats_available': '52 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Varanasi', 'busname': 'UPSRTC - CNT0141', 'price': '₹482', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:39', 'duration': '8h 1m', 'reaching_time': '03:40', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Varanasi', 'busname': 'UPSRTC - CNT0142', 'price': '₹482', 'bustype': 'Ordinary Non AC Seater 2+3', 'departing_time': '19:59', 'duration': '8h 1m', 'reaching_time': '04:00', 'star_rating': '3.3', 'seats_available': '52 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Varanasi', 'busname': 'UPSRTC - JNP0013', 'price': '₹482', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '20:20', 'duration': '6h 40m', 'reaching_time': '03:00', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Varanasi', 'busname': 'UPSRTC - CNT0089', 'price': '₹482', 'bustype': 'Ordinary Non AC Seater 2+3', 'departing_time': '20:29', 'duration': '8h 1m', 'reaching_time': '04:30', 'star_rating': '3.3', 'seats_available': '52 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Varanasi', 'busname': 'UPSRTC - JNP0012', 'price': '₹482', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '20:30', 'duration': '7h 30m', 'reaching_time': '04:00', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Varanasi', 'busname': 'UPSRTC - ALM0177', 'price': '₹591', 'bustype': 'Shatabdi AC Seater 2+2', 'departing_time': '20:47', 'duration': '6h 4m', 'reaching_time': '02:51', 'star_rating': '3.2', 'seats_available': '37 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Varanasi', 'busname': 'UPSRTC - CNT0098', 'price': '₹482', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '20:59', 'duration': '8h 1m', 'reaching_time': '05:00', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Aligarh (uttar pradesh)', 'busname': 'Royal Safari India Travels', 'price': '₹1,456', 'bustype': 'A/C Seater Push Back (2+2)', 'departing_time': '22:30', 'duration': '6h', 'reaching_time': '04:30', 'star_rating': '3.8', 'seats_available': '33 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Aligarh (uttar pradesh)', 'busname': 'UPSRTC - ALI0168', 'price': '₹656', 'bustype': 'Ordinary Non AC Seater 2+3', 'departing_time': '19:30', 'duration': '12h', 'reaching_time': '07:30', 'star_rating': '3.3', 'seats_available': '52 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Aligarh (uttar pradesh)', 'busname': 'UPSRTC - ALI0172', 'price': '₹527', 'bustype': 'Rajdhani Non AC Seater 2+3', 'departing_time': '21:30', 'duration': '9h 5m', 'reaching_time': '06:35', 'star_rating': '3.3', 'seats_available': '51 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Lucknow to Aligarh (uttar pradesh)', 'busname': 'UPSRTC - AWD0206', 'price': '₹802', 'bustype': 'Pink Express AC Seater 2+2', 'departing_time': '21:45', 'duration': '7h 45m', 'reaching_time': '05:30', 'star_rating': '3.9', 'seats_available': '38 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Shahjahanpur (Uttar Pradesh) to Delhi', 'busname': 'RS Yadav Smart Bus', 'price': '₹750', 'bustype': 'VE A/C Seater / Sleeper (2+1)', 'departing_time': '23:00', 'duration': '7h 25m', 'reaching_time': '06:25', 'star_rating': '4.6', 'seats_available': '30 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Shahjahanpur (Uttar Pradesh) to Delhi', 'busname': 'RS Yadav Smart Bus', 'price': '₹699', 'bustype': 'VE A/C Seater / Sleeper (2+1)', 'departing_time': '23:50', 'duration': '7h 40m', 'reaching_time': '07:30', 'star_rating': '4.3', 'seats_available': '29 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Shahjahanpur (Uttar Pradesh) to Delhi', 'busname': 'Bajpai Travels', 'price': '₹1,890', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '20:45', 'duration': '8h 15m', 'reaching_time': '05:00', 'star_rating': '3.3', 'seats_available': '13 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Shahjahanpur (Uttar Pradesh) to Delhi', 'busname': 'Rana Travels', 'price': '₹1,200', 'bustype': 'A/C Sleeper (2+1)', 'departing_time': '21:30', 'duration': '7h', 'reaching_time': '04:30', 'star_rating': '3.8', 'seats_available': '3 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Shahjahanpur (Uttar Pradesh) to Delhi', 'busname': 'Banke Bihari Tour And Travels', 'price': '₹1,399', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '23:00', 'duration': '6h', 'reaching_time': '05:00', 'star_rating': None, 'seats_available': '42 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Shahjahanpur (Uttar Pradesh) to Delhi', 'busname': 'UPSRTC - HDD0063', 'price': '₹581', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:00', 'duration': '8h 42m', 'reaching_time': '02:42', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Shahjahanpur (Uttar Pradesh) to Delhi', 'busname': 'UPSRTC - SJP0598', 'price': '₹571', 'bustype': 'Rajdhani Non AC Seater 2+3', 'departing_time': '18:03', 'duration': '8h', 'reaching_time': '02:03', 'star_rating': '3.3', 'seats_available': '51 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Shahjahanpur (Uttar Pradesh) to Delhi', 'busname': 'UPSRTC - SJP0610', 'price': '₹571', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:45', 'duration': '8h', 'reaching_time': '02:45', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Shahjahanpur (Uttar Pradesh) to Delhi', 'busname': 'UPSRTC - BSR0151', 'price': '₹537', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:12', 'duration': '5h 18m', 'reaching_time': '00:30', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Shahjahanpur (Uttar Pradesh) to Delhi', 'busname': 'UPSRTC - SJP0629', 'price': '₹556', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:23', 'duration': '7h 52m', 'reaching_time': '03:15', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Shahjahanpur (Uttar Pradesh) to Delhi', 'busname': 'UPSRTC - BSR1062', 'price': '₹524', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:35', 'duration': '6h 35m', 'reaching_time': '02:10', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Shahjahanpur (Uttar Pradesh) to Delhi', 'busname': 'UPSRTC - SJP0281', 'price': '₹571', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:45', 'duration': '7h 20m', 'reaching_time': '03:05', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Shahjahanpur (Uttar Pradesh) to Delhi', 'busname': 'UPSRTC - STD0163', 'price': '₹556', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '19:58', 'duration': '9h 27m', 'reaching_time': '05:25', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Shahjahanpur (Uttar Pradesh) to Delhi', 'busname': 'UPSRTC - SJP0637', 'price': '₹585', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '20:00', 'duration': '9h 32m', 'reaching_time': '05:32', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Shahjahanpur (Uttar Pradesh) to Delhi', 'busname': 'UPSRTC - STD0368', 'price': '₹555', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '20:01', 'duration': '8h 39m', 'reaching_time': '04:40', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Aligarh (uttar pradesh) to Greater noida (uttar pradesh)', 'busname': 'UPSRTC - SKD0199', 'price': '₹224', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '17:52', 'duration': '2h 43m', 'reaching_time': '20:35', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Aligarh (uttar pradesh) to Greater noida (uttar pradesh)', 'busname': 'UPSRTC - SKD0554', 'price': '₹224', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '17:55', 'duration': '4h 10m', 'reaching_time': '22:05', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Aligarh (uttar pradesh) to Greater noida (uttar pradesh)', 'busname': 'UPSRTC - SKD0591', 'price': '₹224', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '17:57', 'duration': '3h 13m', 'reaching_time': '21:10', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Aligarh (uttar pradesh) to Greater noida (uttar pradesh)', 'busname': 'UPSRTC - BDV0462', 'price': '₹214', 'bustype': 'Janrath AC Seater 2+2', 'departing_time': '17:59', 'duration': '2h 26m', 'reaching_time': '20:25', 'star_rating': '3.7', 'seats_available': '36 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Aligarh (uttar pradesh) to Greater noida (uttar pradesh)', 'busname': 'UPSRTC - BDV0092', 'price': '₹167', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '17:59', 'duration': '2h 44m', 'reaching_time': '20:43', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Aligarh (uttar pradesh) to Greater noida (uttar pradesh)', 'busname': 'UPSRTC - VKS0212', 'price': '₹224', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:00', 'duration': '2h 15m', 'reaching_time': '20:15', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Aligarh (uttar pradesh) to Greater noida (uttar pradesh)', 'busname': 'UPSRTC - SKD0136', 'price': '₹224', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:00', 'duration': '3h 5m', 'reaching_time': '21:05', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Aligarh (uttar pradesh) to Greater noida (uttar pradesh)', 'busname': 'UPSRTC - VKS0305', 'price': '₹224', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:01', 'duration': '3h 19m', 'reaching_time': '21:20', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Aligarh (uttar pradesh) to Greater noida (uttar pradesh)', 'busname': 'UPSRTC - SKD0734', 'price': '₹224', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:01', 'duration': '2h 59m', 'reaching_time': '21:00', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'Uttar Pradesh State Road Transport Corporation UPSRTC', 'route': 'Aligarh (uttar pradesh) to Greater noida (uttar pradesh)', 'busname': 'UPSRTC - HMR0307', 'price': '₹228', 'bustype': 'Ordinary Express Non AC Seater 2+2', 'departing_time': '18:09', 'duration': '2h 40m', 'reaching_time': '20:49', 'star_rating': '3.3', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Kolkata to Digha', 'busname': 'WBTC (CTC) HABRA-DIGHA via Esplanade - 176|23:10', 'price': '₹145', 'bustype': 'Non AC Seater (2+3)', 'departing_time': '23:10', 'duration': '4h 15m', 'reaching_time': '03:25', 'star_rating': '4.1', 'seats_available': '22 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Kolkata to Digha', 'busname': 'Raipur Cruiser', 'price': '₹300', 'bustype': 'Volvo 9600 Multi-Axle A/C Seater (2+2)', 'departing_time': '23:30', 'duration': '5h', 'reaching_time': '04:30', 'star_rating': '4.7', 'seats_available': '32 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Kolkata to Digha', 'busname': 'Raipur Cruiser', 'price': '₹299', 'bustype': 'Volvo 9600 Multi-Axle A/C Seater (2+2)', 'departing_time': '23:55', 'duration': '5h 5m', 'reaching_time': '05:00', 'star_rating': '4.7', 'seats_available': '51 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Kolkata to Digha', 'busname': 'Shyamoli Paribahan Pvt Ltd', 'price': '₹349', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '23:45', 'duration': '5h', 'reaching_time': '04:45', 'star_rating': '4.3', 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Kolkata to Digha', 'busname': 'Shyamoli Paribahan Pvt Ltd', 'price': '₹300', 'bustype': 'VE A/C Seater (2+2)', 'departing_time': '23:30', 'duration': '5h', 'reaching_time': '04:30', 'star_rating': '4.1', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Kolkata to Digha', 'busname': 'Bhorpet Travels', 'price': '₹300', 'bustype': 'A/C Seater/Sleeper (2+1)', 'departing_time': '23:50', 'duration': '4h 10m', 'reaching_time': '04:00', 'star_rating': '4.5', 'seats_available': '35 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Kolkata to Digha', 'busname': 'SANTOSH BUS SERVICE', 'price': '₹249', 'bustype': 'A/C Seater / Sleeper (2+2)', 'departing_time': '00:29', 'duration': '4h 36m', 'reaching_time': '05:05', 'star_rating': '3.9', 'seats_available': '49 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Kolkata to Digha', 'busname': 'Parameswar Travels', 'price': '₹289', 'bustype': 'A/C Seater Push Back (2+3)', 'departing_time': '23:59', 'duration': '5h 1m', 'reaching_time': '05:00', 'star_rating': '3.4', 'seats_available': '56 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Kolkata to Digha', 'busname': 'Parameswar Travels', 'price': '₹281', 'bustype': 'A/C Seater Push Back (2+3)', 'departing_time': '23:55', 'duration': '5h 5m', 'reaching_time': '05:00', 'star_rating': '3.5', 'seats_available': '47 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Kolkata to Digha', 'busname': 'Maa Chandi Travels', 'price': '₹306', 'bustype': 'A/C Seater / Sleeper (2+2)', 'departing_time': '23:57', 'duration': '5h 13m', 'reaching_time': '05:10', 'star_rating': '3.9', 'seats_available': '21 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Kolkata to Digha', 'busname': 'SBSTC-KOLKATA-DIGHA 23:30 (DIG DEPOT) - KOLKATA-N/S-D/23-00', 'price': '₹155', 'bustype': 'Express(Non AC Seater)', 'departing_time': '23:30', 'duration': '4h 59m', 'reaching_time': '04:29', 'star_rating': '3.9', 'seats_available': '32 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Kolkata to Digha', 'busname': 'SBSTC-GARIA-DIGHA -22:30 (DIGHA DEPOT) - DIGHA-3 N/H-D/07-00', 'price': '₹162', 'bustype': 'Express(Non AC Seater)', 'departing_time': '23:45', 'duration': '5h', 'reaching_time': '04:45', 'star_rating': '3.8', 'seats_available': '27 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Digha to Kolkata', 'busname': 'Satya Paribahan', 'price': '₹315', 'bustype': 'A/C Seater (2+3)', 'departing_time': '23:00', 'duration': '4h 45m', 'reaching_time': '03:45', 'star_rating': '3', 'seats_available': '33 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Digha to Kolkata', 'busname': 'Snemita Paribahan (Shinjini)', 'price': '₹275', 'bustype': 'AC Seater (2+3)', 'departing_time': '23:35', 'duration': '4h 40m', 'reaching_time': '04:15', 'star_rating': '2.9', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Digha to Kolkata', 'busname': 'Parameswar Travels', 'price': '₹285', 'bustype': 'A/C Seater Push Back (2+3)', 'departing_time': '21:00', 'duration': '6h', 'reaching_time': '03:00', 'star_rating': '2.9', 'seats_available': '51 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Digha to Kolkata', 'busname': 'Parameswar Travels', 'price': '₹285', 'bustype': 'A/C Seater Push Back (2+3)', 'departing_time': '17:50', 'duration': '5h 10m', 'reaching_time': '23:00', 'star_rating': '2.3', 'seats_available': '29 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Digha to Kolkata', 'busname': 'Aradhana Travel', 'price': '₹450', 'bustype': 'A/C Seater / Sleeper (2+2)', 'departing_time': '18:35', 'duration': '4h 25m', 'reaching_time': '23:00', 'star_rating': '1', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Digha to Kolkata', 'busname': 'Bhorpet Travels', 'price': '₹270', 'bustype': 'A/C Seater Push Back (2+2)', 'departing_time': '17:35', 'duration': '4h 55m', 'reaching_time': '22:30', 'star_rating': '3.9', 'seats_available': '29 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Digha to Kolkata', 'busname': 'Aradhana Bus Service', 'price': '₹630', 'bustype': 'A/C Seater / Sleeper (2+2)', 'departing_time': '22:55', 'duration': '5h 5m', 'reaching_time': '04:00', 'star_rating': '3.9', 'seats_available': '48 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Digha to Kolkata', 'busname': 'The Joy Ride', 'price': '₹650', 'bustype': 'A/C Seater / Sleeper (2+2)', 'departing_time': '21:15', 'duration': '5h 15m', 'reaching_time': '02:30', 'star_rating': None, 'seats_available': '44 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Digha to Kolkata', 'busname': 'Riya Bus', 'price': '₹450', 'bustype': 'Non AC Seater (2+3)', 'departing_time': '20:20', 'duration': '10h 10m', 'reaching_time': '06:30', 'star_rating': None, 'seats_available': '20 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Digha to Kolkata', 'busname': 'City Express (Shinjini)', 'price': '₹333', 'bustype': 'AC Seater (2+3)', 'departing_time': '22:45', 'duration': '5h 25m', 'reaching_time': '04:10', 'star_rating': '3.1', 'seats_available': '26 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Digha to Barasat (West Bengal)', 'busname': 'Aradhana Bus Service', 'price': '₹405', 'bustype': 'Non AC Seater (2+3)', 'departing_time': '16:55', 'duration': '6h 35m', 'reaching_time': '23:30', 'star_rating': '3', 'seats_available': '54 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Digha to Barasat (West Bengal)', 'busname': 'Satya Paribahan', 'price': '₹315', 'bustype': 'A/C Seater (2+3)', 'departing_time': '23:00', 'duration': '6h 10m', 'reaching_time': '05:10', 'star_rating': '3.5', 'seats_available': '33 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Digha to Barasat (West Bengal)', 'busname': 'Aradhana Travel', 'price': '₹450', 'bustype': 'Non AC Seater (2+3)', 'departing_time': '21:45', 'duration': '6h 55m', 'reaching_time': '04:40', 'star_rating': '2.7', 'seats_available': '11 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Digha to Barasat (West Bengal)', 'busname': 'The Joy Ride', 'price': '₹650', 'bustype': 'A/C Seater / Sleeper (2+2)', 'departing_time': '21:15', 'duration': '6h 20m', 'reaching_time': '03:35', 'star_rating': None, 'seats_available': '44 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Digha to Barasat (West Bengal)', 'busname': 'Aradhana Bus Service', 'price': '₹630', 'bustype': 'A/C Seater / Sleeper (2+2)', 'departing_time': '22:55', 'duration': '6h', 'reaching_time': '04:55', 'star_rating': '3', 'seats_available': '48 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Digha to Barasat (West Bengal)', 'busname': 'Aradhana Travel', 'price': '₹540', 'bustype': 'A/C Seater / Sleeper (2+2)', 'departing_time': '18:35', 'duration': '5h 20m', 'reaching_time': '23:55', 'star_rating': '2.7', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Barasat (West Bengal) to Digha', 'busname': 'WBTC (CTC) HABRA-DIGHA via Esplanade - 176|21:15', 'price': '₹165', 'bustype': 'Non AC Seater (2+3)', 'departing_time': '21:15', 'duration': '6h 10m', 'reaching_time': '03:25', 'star_rating': '4.3', 'seats_available': '23 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Barasat (West Bengal) to Digha', 'busname': 'Sethu Express', 'price': '₹399', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '00:21', 'duration': '5h 9m', 'reaching_time': '05:30', 'star_rating': '3.9', 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Barasat (West Bengal) to Digha', 'busname': 'Rohit Travels', 'price': '₹360', 'bustype': 'A/C Seater / Sleeper (2+2)', 'departing_time': '23:30', 'duration': '5h 30m', 'reaching_time': '05:00', 'star_rating': '2.3', 'seats_available': '51 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Durgapur (West Bengal) to Kolkata', 'busname': 'SHYAMOLI Pvt. Ltd.(Karunamoyee)', 'price': '₹486', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '16:55', 'duration': '4h', 'reaching_time': '20:55', 'star_rating': '4', 'seats_available': '9 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Durgapur (West Bengal) to Kolkata', 'busname': 'Greenline', 'price': '₹486', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '17:10', 'duration': '3h 50m', 'reaching_time': '21:00', 'star_rating': '4.5', 'seats_available': '15 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Durgapur (West Bengal) to Kolkata', 'busname': 'Greenline (Karunamoyee)', 'price': '₹486', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '17:30', 'duration': '3h 50m', 'reaching_time': '21:20', 'star_rating': '4.2', 'seats_available': '7 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Durgapur (West Bengal) to Kolkata', 'busname': 'SHYAMOLI Pvt. Ltd.(Karunamoyee)', 'price': '₹486', 'bustype': 'Scania Multi-Axle AC Semi Sleeper (2+2)', 'departing_time': '18:00', 'duration': '3h 45m', 'reaching_time': '21:45', 'star_rating': '3.8', 'seats_available': '24 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Durgapur (West Bengal) to Kolkata', 'busname': 'Express Line', 'price': '₹486', 'bustype': 'Volvo 9600 A/C Semi Sleeper (2+2)', 'departing_time': '18:45', 'duration': '4h 5m', 'reaching_time': '22:50', 'star_rating': '4.5', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'SHYAMOLI Pvt. Ltd.(Karunamoyee)', 'price': '₹486', 'bustype': 'Scania Multi-Axle AC Semi Sleeper (2+2)', 'departing_time': '17:00', 'duration': '3h', 'reaching_time': '20:00', 'star_rating': '4.2', 'seats_available': '31 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'Greenline (Karunamoyee)', 'price': '₹486', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '17:00', 'duration': '3h 45m', 'reaching_time': '20:45', 'star_rating': '4.3', 'seats_available': '16 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'Shyamoli Paribahan Pvt Ltd', 'price': '₹486', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '17:05', 'duration': '3h 35m', 'reaching_time': '20:40', 'star_rating': '4', 'seats_available': '8 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'SHYAMOLI Pvt. Ltd.(Karunamoyee)', 'price': '₹486', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '17:30', 'duration': '4h', 'reaching_time': '21:30', 'star_rating': '4.3', 'seats_available': '30 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'Express Line', 'price': '₹486', 'bustype': 'Volvo A/C Seater (2+2)', 'departing_time': '17:20', 'duration': '4h', 'reaching_time': '21:20', 'star_rating': '4.6', 'seats_available': '33 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'Greenline', 'price': '₹486', 'bustype': 'Volvo A/C B11R Multi Axle Semi Sleeper (2+2)', 'departing_time': '18:05', 'duration': '4h', 'reaching_time': '22:05', 'star_rating': '4.4', 'seats_available': '39 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'Greenline (Karunamoyee)', 'price': '₹486', 'bustype': 'Volvo A/C Semi Sleeper (2+2)', 'departing_time': '18:15', 'duration': '4h', 'reaching_time': '22:15', 'star_rating': '4.1', 'seats_available': '28 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'Royal Cruiser', 'price': '₹350', 'bustype': 'Scania Multi-Axle AC Semi Sleeper (2+2)', 'departing_time': '19:30', 'duration': '3h 45m', 'reaching_time': '23:15', 'star_rating': '3.3', 'seats_available': '47 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'Greenline', 'price': '₹600', 'bustype': 'Volvo 9600 A/C Seater/Sleeper (2+1)', 'departing_time': '20:00', 'duration': '3h 58m', 'reaching_time': '23:58', 'star_rating': '4.5', 'seats_available': '36 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'Aitiana Airwings', 'price': '₹446', 'bustype': 'A/C Seater/Sleeper (2+1)', 'departing_time': '20:30', 'duration': '3h 50m', 'reaching_time': '00:20', 'star_rating': '4', 'seats_available': '39 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'Maharani Express', 'price': '₹675', 'bustype': 'NON A/C Seater / Sleeper (2+2)', 'departing_time': '17:00', 'duration': '3h 30m', 'reaching_time': '20:30', 'star_rating': '1.9', 'seats_available': '10 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'Pariwartan Bus Service', 'price': '₹500', 'bustype': 'NON A/C Seater / Sleeper (2+2)', 'departing_time': '18:30', 'duration': '4h 30m', 'reaching_time': '23:00', 'star_rating': None, 'seats_available': '54 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'Pammi Travels', 'price': '₹900', 'bustype': 'Volvo 9600 Multi-Axle A/C Sleeper (2+1)', 'departing_time': '21:00', 'duration': '5h 15m', 'reaching_time': '02:15', 'star_rating': '3.3', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'Lokenath Bus Service', 'price': '₹1,199', 'bustype': 'Volvo Multi Axle B11R AC Seater\\Sleeper (2+1)', 'departing_time': '18:00', 'duration': '3h 30m', 'reaching_time': '21:30', 'star_rating': '2.9', 'seats_available': '45 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'Maharani Travels', 'price': '₹810', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '21:05', 'duration': '4h 25m', 'reaching_time': '01:30', 'star_rating': '2.4', 'seats_available': '18 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'SBSTC-KOLKATA-JAMURIA VIA-DURGAPUR-17:30 (HOW DEPOT) - JAMURIA 1-U/17:30', 'price': '₹150', 'bustype': 'Express(Non AC Seater)', 'departing_time': '17:30', 'duration': '3h 30m', 'reaching_time': '21:00', 'star_rating': '3.9', 'seats_available': '52 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'SBSTC-KARUNAMOYEE-DURGAPUR 17:30 (DGP DEPOT) - L-63-D/17-30', 'price': '₹155', 'bustype': 'Express(Non AC Seater)', 'departing_time': '17:30', 'duration': '3h 59m', 'reaching_time': '21:29', 'star_rating': '3.5', 'seats_available': '44 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Kolkata to Durgapur (West Bengal)', 'busname': 'SBSTC-KOLKATA-DURGAPUR 18:30 (DGP DEPOT) - DGPKOLE-D/17-45', 'price': '₹150', 'bustype': 'Express(Non AC Seater)', 'departing_time': '18:30', 'duration': '4h', 'reaching_time': '22:30', 'star_rating': '5', 'seats_available': '49 Seats', 'type': 'Government', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Barasat (West Bengal) to Contai (Kanthi)', 'busname': 'Sethu Express', 'price': '₹399', 'bustype': 'A/C Seater / Sleeper (2+1)', 'departing_time': '00:21', 'duration': '3h 54m', 'reaching_time': '04:15', 'star_rating': None, 'seats_available': '12 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Barasat (West Bengal) to Contai (Kanthi)', 'busname': 'Rohit Travels', 'price': '₹650', 'bustype': 'A/C Seater / Sleeper (2+2)', 'departing_time': '23:30', 'duration': '4h 30m', 'reaching_time': '04:00', 'star_rating': '3.6', 'seats_available': '50 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Kolkata to Suri', 'busname': 'SBSTC-KOLKATA - SURI - 18:00 - (SURI DEPOT) - KOLKATA 6-D/18-00', 'price': '₹190', 'bustype': 'Express(Non AC Seater)', 'departing_time': '18:00', 'duration': '5h', 'reaching_time': '23:00', 'star_rating': '3.8', 'seats_available': '45 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}
{'state': 'WBTC (CTC)', 'route': 'Habra to Digha', 'busname': 'WBTC (CTC) HABRA-DIGHA via Esplanade - 176|20:15', 'price': '₹179', 'bustype': 'Non AC Seater (2+3)', 'departing_time': '20:15', 'duration': '7h 10m', 'reaching_time': '03:25', 'star_rating': '4.7', 'seats_available': '23 Seats', 'type': 'Private', 'date_of_scrape': '02-08-2025'}